In [1]:
### - IHVN DHIS2 API
# -----------------------------------------------------------------------------------------
# Purpose: Fetch and process data from the IHVN DHIS2 API, transforming it into structured DataFrames.
#          The script handles multiple report types (e.g., Report Rate, ART, PMTCT), maps data element IDs
#          to human-readable names or descriptions, and organizes organizational units (OUs) with their
#          hierarchies and cluster mappings for Local Government Areas (LGAs) in Anambra, Nigeria.

import requests        # Library for making HTTP requests to interact with DHIS2 API endpoints
from requests.auth import HTTPBasicAuth  # Module for HTTP Basic Authentication to secure API requests
from requests.exceptions import HTTPError, RequestException  # Exceptions to handle HTTP and network errors
import pandas as pd    # Library for data manipulation, used to create and process DataFrames
import re              # Library for regular expressions, used to parse URL parameters (e.g., data elements)
import time            # Library for time-related functions, used for retry delays during API failures
import socket          # Library for network operations, used to check internet connectivity
import ipywidgets as widgets  # Library for Jupyter interactive widgets, used for real-time progress display
from IPython.display import display, clear_output  # Functions for managing output in Jupyter notebooks

# Dictionary of named URLs for DHIS2 reports (shortened for brevity)
# Keys are report names; values are URL-encoded DHIS2 API endpoints specifying dimensions:
# - dx: Data elements (indicators or metrics)
# - ou: Organizational units (e.g., facilities, LGAs)
# - pe: Periods (e.g., months in YYYYMM format)
# URL encoding uses %3A for ':' and %3B for ';'. Parameters like showHierarchy and includeMetadataDetails
# ensure metadata (e.g., names, hierarchies) is included in API responses.
named_urls = {
    "Report Rate Facility": "https://ihvn.dhistance.com/api/analytics.json?dimension=pe%3A202501&dimension=ou%3AKH62ia35VIZ%3Bum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&dimension=dx%3AZ7E9RxXmwxG.REPORTING_RATE%3BVmGwLcfPS2N.REPORTING_RATE%3BYFnIy7lATQL.REPORTING_RATE%3BNkuV7xoThHV.REPORTING_RATE%3BHwfLR3npibF.REPORTING_RATE%3BvN9rk5ChByM.REPORTING_RATE%3BoxUN7AXSF8r.REPORTING_RATE&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false",
    "Report Rate LGA": "https://ihvn.dhistance.com/api/analytics.json?dimension=pe%3A202501&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-lmSTo2yxNsA&dimension=dx%3AZ7E9RxXmwxG.REPORTING_RATE%3BVmGwLcfPS2N.REPORTING_RATE%3BYFnIy7lATQL.REPORTING_RATE%3BNkuV7xoThHV.REPORTING_RATE%3BHwfLR3npibF.REPORTING_RATE%3BvN9rk5ChByM.REPORTING_RATE%3BoxUN7AXSF8r.REPORTING_RATE&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "AGYW MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AQ1KKjeS4seJ%3BzhvcKIWKvEX%3BpgsZWQLbTvw%3BdPefeXOI0MT%3Bb62rwfvBP13%3Bechn4uCHBhF%3BeoSKe92wBYa%3BIYJciZgP6Yt%3BbbSqZH1OAxk%3By5mYZFQbMe6%3BSSa2P2O1keL%3BrtjZkt3ImND%3BDhw4lcmA8i5%3BAMs19im8mm7%3BTNTPcRPC3jV%3BWWaSbjutZof%3BcspEoTIBnOB%3BQ1ntMoY7ZrP%3BjQ51vKvy1SN%3BKj0TobAENyg%3BKRf4sYxv9KG&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "ART MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AEVJnWv5UQ2I%3BaDFx7U0OSNp%3BuTAVBA24Qgg%3BVNvZaoEcS8M%3BdJpKA2CL66w%3BwUlUHsYzh80%3BmaTBR3htwav%3BN4skK4jJVnm%3BcMpe2pMLJed%3BmJ3Af4Qg0wV%3BxFsrvhyu0Wx%3BCVIR5mDSrr0%3BqO3FSAwqg15%3BE8J56tfMIEa%3BBiqkhMdFIwy%3BfVpRSB8jy9Q%3BudXxeZhT8Fd%3BE20mRpvl5jK%3Bgcg9I4dagWN%3BfBJzc5QIP1b%3BgDuNCzc5liq%3BngCJ4UZOCme%3BEtg6BPVX548%3BiTJ2VvKOWHG%3BrJB5XXrF5zx%3BFRMmrIYSRfz%3BPhUxFwPj2US%3BKtMzH6OTxXL%3Bu0W1SpovSd3%3Bxz9C4uZwMuB%3BLqdahCtUzSX%3Bqb3YzC5X9Lo%3BOU60086uSKx%3Bs2d4Hk231P9%3BmnbibCKaDNb%3BlEdQdPdK5KL&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "ART MSF_tb screening": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AaDFx7U0OSNp&dimension=sbKiaUuaHpX%3AmyDWsEb0XMg%3BnsnGDTrtxbw&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "HTS MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AsxXB2RokZrt%3BhxYcq0LyXh6%3Bi3I5wU8vm0U%3BwSLnU5ihgb3%3BVz0ZjXCnFtX%3BK7Bkwdae7X1%3BRBxwgKGZYpv%3BjNodLWC4U4d%3Bq6bey1tg06I%3BAZSZngrMzj6%3BpS0Cjik4WEH%3BBrVqJd9MHSA%3BXKu24OQK64R%3BpgiqhOhD95N%3BYL87jqnIzQA%3BmN5TqnUQRgq%3BEvB0bzvxMq0%3BgWVJbVYOT1A%3Bd8lMQuQpLPe%3BT6f8BA7M9Q0%3BSVpx1fVD3bh%3BpJH2bCApdTe%3BuGUbRjQvCeN%3BE3bZgIIG5qQ%3Bd8pnVkpyGlU%3BDn4PJOJJASf%3BQV3wq0WsLXe%3Bs9ksVidbnRG%3BOFj0H56KbKp%3Bh9SMVFhNbBN%3BrJD9ER08iT4%3BJLOnLNuLOva%3BZztEVzPBwwl%3BX6wyczqYmJN%3Bvhhrj8rTq83%3ByR5JEOs2t7q%3BCnSrjlY5Siw%3BUMPLBlGYjWM%3BiE5kXuUJoMC%3BuMqKjodBP76%3Bu9gp0VFbnHh%3BzJuaAQbUlq8%3BQXNyy0dbIpd%3BomaL0XteWO8%3BKtzvesDYABJ%3BHdTQzlvMtjU%3Bqznyc2H90Ay%3Bdtri8UEwZFV%3BFB1EZfVSmYi%3BXzghRlcLMqB%3BOk6fJzfTk75%3BomujB6405jI%3BenPRkAVrHKi%3BE6wddhqJVIN%3BjWajcYCyiDE%3BLSQuUgoTH7o%3Bilp3JWW1qt5%3BURgQ4d3y8WF%3Bxc40kwzTBVG%3BmCD2Qhbd6CY%3BnQtTVOdEMqV%3BKQAEkHXQe7N%3BxBr9Sgyk4cO%3BlI7YzdC7wEd%3BegRRIwWxnJs%3BPPxR0HhCKQR%3BkmBNFDE8duJ%3BGTUzO3HGLWA%3Bh8HFl5EeHHl%3BuXwAyGT9eqW%3BDahOUj6bRk0%3BT8G2KNNZ4eI%3BD7ygy6yCHFs%3BuSQJcqAEvHg%3BU74jSwLCoA1%3BZheiLjTrqRZ%3BjlFslOk3bkU%3BOxBKckVqp29%3BJWHc7D4J132%3Bq1XlBEcB1PE%3BNb95zNUKf29%3BevlfYhoKrjI%3BU8xSE6OneYl%3BF2JEmiJt3Yl%3BPZWrrb7VyCj%3BXxpFcHK1S89%3BsICfdv3or5G%3Bg5HvgkCKSSU%3Bh7XmbTNyTUi%3BUi7DiLwSgqm%3BHIHQHXPOGKl%3Bl3aXhanFimZ%3BSl6d3hqzq2C%3BpIsmPa1GjFs%3BOwrvPMKq0pQ%3BV7hcDYvuPMY%3BDWb8URoPRym%3BL7l30ySaQDy%3BnYrdLtwDlV0%3BifE9LKaLqUm%3BuMkZMIHVVjV%3BcCLFgRUkIww%3BHUWYU2ruloN%3BoXsckqTpzhN%3Bm4CzDuc50Jn%3BLPyxdv2eBEj%3BZLARI5LYBOL%3BPYL5GdQGPfI%3BmQZ4z94jzak%3BRkwT0w0mPNj%3BEC5iN37lZ61%3BS0QI1IcESjq%3BnR3ZklnEuBy%3BgxZKoTwyLnn%3BIp75Jb7o8Au%3BVPMZR303TWP%3BjqP0tN3v5hJ%3BuoBeVy413Mj%3BPn1GooQL0I4%3BEh4GHGcZV1U%3BJr0gixBBrvT&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "HTS MSF_hivst_approach": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AT8G2KNNZ4eI&dimension=tBdRxXi3Dxr%3AALL_ITEMS&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "HTS MSF_hivst_response_classification": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AU74jSwLCoA1&dimension=srck01HTxTQ%3ADvPd0vf2shT%3Bf1mGCK98z8a%3Brui70AoUB3b&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "NSP MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AxesSKTdzhPF%3BuIx4sChOeMC%3BkJSLUKrBKez%3BJylie7CPg63%3BTV5DhhOgF7s%3BL0X8cbiEfrC%3BoSDqDrfRpb3%3BIQYpNyC6lF2%3BlOVVI8B4Ag6%3BRxQv56EIuWs%3BqyV24hEIgM5%3BObJbRGQ3QPI%3BptgY5CK3GUc%3BC16TIH6Zxju%3BKY2gb90cvTu%3BO3g2Lq9fpUU%3Bkk3hEztWa48%3BxvfqoSSOIOL%3BcLSlAzBug6Y%3BayTk1t8sjFN%3BjY2SLdSlMug%3Bv3sfwf9O1R9%3BxrHzg7SORIt&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "PMTCT MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3ABWLsXE490Yw%3BAtzuY8wFIaP%3BmjoPYWLowE6%3BhWkEco9hG0R%3BgybVHn9Y3SQ%3BkNEHSaIxOja%3BSWsoIgeQBKd%3Bym1WjkXu8ol%3BHDVcSJrN68V%3BIY3zDXL1t0b%3BCrv4RYmAJot%3Bn1MZAkrNWlM%3BGDFB54Uuepj%3BPaXizDbvXcE%3BhnqKzrB2hX0%3BZv0Go6hoDxZ%3BZOzgBnzWLOK%3Bbj8jc0HujI2%3BVlhKdJD8Mav%3BqO3KHhkVM38%3BkSHVwcO9iYZ%3BWnEhg2SiMjF%3BFR9ibOgyNKg%3BbyiZy4Xo7Rx%3BvpEeS23uOzB%3BzoSht7Xk3OI%3BUMFtzIDX9nE%3BxkQqd3C5vd8%3BCAO8uuth3q6%3BRATzsmd7JfL%3BDPFGLfkzZTF%3BZZEXn1RzLwL%3BuMATgKm1ZEV%3ByShXhGc00TY%3Bdc0p7yc1jPy%3BJsyAmKQMqK1%3BlIJwQ20CgPE%3BbxeHOqH0s2V%3BOum5ofYWexC%3BadqiOXDSRbU%3BHFq8NihMhgw%3BcibdW4TLOJ5%3BMJHhWCCOy1q%3BPQHBZmMVWE3%3Bw9NSJbXp5Ub%3BCyrYJsUOe7Q%3BER9cFdvluIM%3BUP3mRp6Zr69%3Bgl5kyYu85zX%3BrIY8zpZJxtv%3BaPtwtDwhOeL%3BuQEZL5j8ZHS%3BSpuoRhODIuZ%3Bvu5TP1CuesM%3BeZ4kUaPlQWC%3BSVDAcLT6IKS%3BFLt9JLhBWlK%3Bt9NJ4JhsXVl%3BnOpah0JpIzI%3BDuFJekp1ymh%3BTJ5BREHjZnu%3BWzAmqotjxV7%3BwRRoicp2S4Y%3Bzq6Tvgk5GFl%3BLMrI0jXhjP4%3BTDLVNVgHUvf%3BakVDx9ew0y8%3ByDAbvAlPR69%3Bi6rCuewXarq%3BzKYuhIKhMKM%3BGQ4kUNGMf7i%3Bv2e5LaziiBr%3BsLWmlnDQzgt%3BrKzvhBgkpe5%3BpyUW87YFbY5%3BssvxYtnGXpp%3BlY6wzD8718l%3BeLZuFwiv4rQ%3BDRFLRwv7f8F%3BLgbHzzQILZY%3BWt8jhQxqq9L%3BTdqRbqoS3Uj%3BcfUpkarx8og%3BWbRBSkKaTE8%3BEvZz6jNSRMX%3BYGZr2K5Y0Yq%3BJGj4bfhtzCk%3BkKSgWbeTAL1%3Bo9zPtZI4c9T%3BcmkwuM6ZOaN%3BcGPBuUrZ5Oi%3BN4WysYNBRFk%3Bd0izXgJhn2P%3BxUYwPZH7ulJ%3BykZcKOVoJeH%3Bj3ycCFq6vzR%3BK2lEmtE4xjz%3Bw4jlyeHkVTD%3BDbj5r3zevDF%3BivG6D3HaMKs%3BCOsmPB8utmg%3BuQKHVsCIm4y%3BZYmUM2TtV80%3BYfvO0pN0SVl%3BwbplLMxCVSo%3BEJ31P0DyQaY&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "KP Prev MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3ArJFps9PGgYI%3BpFrm4E6yE0i%3Bwld8ChYjUUS%3Bhnqs8zD2RBz%3BFdogfvZ9Es5%3BUWnjd2hDMpZ%3BmM3TZYEkMaF%3BK93mvkTXpbl%3BtIZvjax5TD5%3BN05zqacXFsD%3BcVHdvMKuwse%3BBFSM6D4KWg4%3ByHBgGkS0dPr%3BEw3ZRxdYY7g%3BihRnS748lOC%3BlIu2vmR2zHU%3BfbxvYGgVsrK%3BTdATue0b5th%3BFBBKTzFqBVR%3BF27CfSgCmSk%3BdecdNlpZy4B%3BRVuw3Ny3ysw%3BhN5ZT3zziqA%3BKngVqiOqAdp%3Bdfznt6rhrYg%3BbxK7Uh32HMK%3BduzVXts1QUh%3BXpDOj1a7ZdC%3BogXBJLCw1Qe%3BAKQTKPoUHt6%3BrClFr1FdYVY%3BdTWbSB6HZg6%3BGZo5a5CwzV7%3BdIElcrfEgSB%3BP8oCCg2mUcG%3BZc7dmnNEQwj%3BmWQsKiSyv7J%3BQVOL72RTOZR%3Bri9iXOK3wxX%3Bpqp8Tyvsdc1%3BNat3LtmEWXD%3BPm78aQ5UVh8%3BWaI8nE16Ab0%3BfHlNG1CMqYX%3BGwxEq7jrcSc%3BiqqfsLcdWiK%3BeF7Yi0cLTXP%3BVnDrAREzUE2%3BSSxprjJB8vd%3BL5pAPbPCrNF%3BPrdKtyHsOE8%3BLNQtCiSCfnC%3Bb3JIlAZUjNJ%3BeK3YlBsHGkB%3BTz9WrAjxVY9%3BMkl91koS5cn%3BdAQlguHfLF8%3BBdgtqOw1P33%3BhZzpfxOQo4b%3BJBoik13RWdP%3BhAVQ3sTq3nJ%3BGKiJWrIeEJu%3BqqFvQly7I9h%3BJtOlLSk4yTF%3BUfMiT3LdXYc%3BdLj7PgakBi5%3BUaOdcjrcI59%3BssQZNWweFrL%3BhHkzZLsAqmI%3BkYAhnVk8fYp%3BkTIGrSkqbrG%3Bl4ZD3aaxH7j%3BWAZm2HXm3xI%3BSECmJH5Lfer%3BKHnHX7Rb29d%3BVE0Z777wXB2%3Bzgqte2mbKxT%3BDbavrmvoWPG%3BJgDiY0dv2RX%3BS8WFXWTFvai%3BBBwzjs2JQ1Z%3BxRJrWUdpAxm%3BYS8QAyBakNp%3BJfPSsLKaeDV%3BkmsvoBtZ4oa%3BOzGjpuSQHV9%3BVmFeyHAUguh%3BUNoJvdMUZHp%3BfeKbAOjRqQT&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "KP Prev MSF_access_type_mhs": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3APrdKtyHsOE8&dimension=p0cjhizcn4a%3AowMnEudwpzU%3BoeF9HvF2R9X&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "KP Prev MSF_access_type_msh_diagnose": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3ASSxprjJB8vd&dimension=p0cjhizcn4a%3AowMnEudwpzU%3BoeF9HvF2R9X&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "KP Prev MSF_access_type_msh_support": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3Al4ZD3aaxH7j&dimension=p0cjhizcn4a%3AowMnEudwpzU%3BoeF9HvF2R9X&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "KP Prev MSF_PrEP_eligible_for_pk_at_risk": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AJgDiY0dv2RX&dimension=oDMnfmxGwgK%3AmoMYeaLyS23%3Bpej3KLkwxLD%3BXHJmw3SqFA3%3BhJnA2pWsV1M%3BX5siefU1Ouk%3BTTSbymL6EhP%3Bgw134Hv72Ea%3BBTZ4X4h6Gj1&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "KP Prev MSF_PrEP_received_for_pk_at_risk": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AVE0Z777wXB2&dimension=oDMnfmxGwgK%3AmoMYeaLyS23%3Bpej3KLkwxLD%3BXHJmw3SqFA3%3BhJnA2pWsV1M%3BX5siefU1Ouk%3BTTSbymL6EhP%3Bgw134Hv72Ea%3BBTZ4X4h6Gj1&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "KP Prev MSF_PrEP_returned_with_retesting_negetive_for_pk_at_risk": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3Azgqte2mbKxT&dimension=oDMnfmxGwgK%3AmoMYeaLyS23%3Bpej3KLkwxLD%3BXHJmw3SqFA3%3BhJnA2pWsV1M%3BX5siefU1Ouk%3BTTSbymL6EhP%3Bgw134Hv72Ea%3BBTZ4X4h6Gj1&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "KP Prev MSF_PrEP_returned_with_retesting_positive_for_pk_at_risk": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3ADbavrmvoWPG&dimension=oDMnfmxGwgK%3AmoMYeaLyS23%3Bpej3KLkwxLD%3BXHJmw3SqFA3%3BhJnA2pWsV1M%3BX5siefU1Ouk%3BTTSbymL6EhP%3Bgw134Hv72Ea%3BBTZ4X4h6Gj1&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "KP Prev MSF_PrEP_discountined_for_pk_at_risk": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AKHnHX7Rb29d&dimension=oDMnfmxGwgK%3AmoMYeaLyS23%3Bpej3KLkwxLD%3BXHJmw3SqFA3%3BhJnA2pWsV1M%3BX5siefU1Ouk%3BTTSbymL6EhP%3Bgw134Hv72Ea%3BBTZ4X4h6Gj1&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "PMTCT MSF_sdp": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AN4WysYNBRFk&dimension=brKpOJgkKa0%3AOLcx26MiJia%3BQ9RcEpV6443%3BEGHRBByvqFu&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "PMTCT MSF_sdp_pos": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AykZcKOVoJeH&dimension=brKpOJgkKa0%3AOLcx26MiJia%3BQ9RcEpV6443%3BEGHRBByvqFu&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "PMTCT MSF_sd<72_in-outside": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3Av2e5LaziiBr&dimension=FtBUOVZVrC6%3AE8fr3yVA0mn%3BS0QLh0UvPbm&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
    "PMTCT MSF_sd>72_in-outside": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AGQ4kUNGMf7i&dimension=FtBUOVZVrC6%3AE8fr3yVA0mn%3BS0QLh0UvPbm&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID"
}   

def fetch_and_process_DHIS2_data(username, password, start_period, end_period, named_urls=named_urls):
    """
    Fetch and process DHIS2 data from named URLs with a specified period range, returning a dictionary of processed DataFrames.
    Ensures all requested data elements or dimension values appear as columns, even if they have no data. For Report Rate URLs, 
    uses metadata names for dx columns; for others, maps data element IDs to descriptions with fallback to metadata names or dimension names.
    
    Args:
        username (str): DHIS2 username for authentication
        password (str): DHIS2 password for authentication
        start_period (str): Start period in YYYYMM format (e.g., '202501')
        end_period (str): End period in YYYYMM format (e.g., '202503')
        named_urls (dict, optional): Dictionary with names as keys and DHIS2 API URLs as values
                                    Defaults to the predefined named_urls dictionary
    
    Returns:
        dict: Dictionary with URL names as keys and processed DataFrames as values
    """
    # -- Step 1: Initialize separator and output widget
    separator_line = '-' * 43  # Create a line of 43 dashes for formatting console output
    success_output = widgets.Output()  # Initialize Jupyter widget to display progress and success messages
    display(success_output)  # Display the widget in Jupyter notebook for real-time updates

    # -- Step 2: Extract unique data element IDs (dx) from URLs
    all_dx_ids = set()  # Initialize empty set to store unique data element IDs
    try:  # Handle potential errors during URL parsing
        for url in named_urls.values():  # Iterate through all URLs in the dictionary
            dx_matches = re.findall(r'dx%3A([^&]+)', url)  # Extract dx (data element) parameters using regex
            for match in dx_matches:  # Process each dx parameter found
                ids = [id.split('.')[0] for id in match.split('%3B')]  # Split IDs and remove .REPORTING_RATE suffix
                all_dx_ids.update(ids)  # Add unique IDs to the set
    except Exception as e:  # Catch any parsing errors
        print(f"{separator_line}\n⦸ Error: Failed to extract data element IDs from URLs: {str(e)}\n{separator_line}")
        return {}  # Return empty dict if extraction fails

    # -- Step 3: Fetch data element descriptions from DHIS2 API
    base_url = 'https://ihvn.dhistance.com/api/dataElements'  # Base URL for DHIS2 data elements endpoint
    dx_filter = f"id:in:[{','.join(all_dx_ids)}]"  # Create filter for specific data element IDs
    params = {  # Define query parameters for API request
        'fields': 'id,name,description',  # Request ID, name, and description fields
        'filter': dx_filter,  # Filter to only include specified data element IDs
        'paging': 'false'  # Disable pagination to retrieve all results in one request
    }
    max_retries = 3  # Maximum number of retry attempts for API requests
    retry_delay = 5  # Delay in seconds between retry attempts
    dataelement_to_description = {}  # Dictionary to store data element ID-to-description mappings

    for attempt in range(1, max_retries + 1):  # Attempt API call up to max_retries times
        try:
            socket.create_connection(("8.8.8.8", 53), timeout=5)  # Test internet connectivity using Google's DNS
            response = requests.get(
                base_url,
                auth=HTTPBasicAuth(username, password),  # Authenticate with provided credentials
                params=params,
                timeout=30  # Set timeout to 30 seconds
            )
            response.raise_for_status()  # Raise exception for HTTP errors
            data_elements = response.json().get('dataElements', [])  # Extract data elements from response
            with success_output:  # Update Jupyter widget with success message
                print(f"Fetched {len(data_elements)} data elements.")
            dataelement_to_description = {
                de['id']: de.get('description', de['name']) for de in data_elements  # Map IDs to descriptions or names
            }
            break  # Exit retry loop on success
        except HTTPError as e:  # Handle HTTP errors (e.g., 401, 404)
            print(f"\n⦸ HTTP Error (Attempt {attempt}/{max_retries}): {str(e)}")
            if response.status_code == 401:  # Check for unauthorized access
                print(f"⦸ Error: Invalid DHIS2 login credentials.")
                return {}  # Return empty dict for invalid credentials
            if attempt == max_retries:  # Check if max retries reached
                print(f"⦸ Error: Max retries reached. Unable to fetch data elements.")
                return {}  # Return empty dict after max retries
            time.sleep(retry_delay)  # Wait before retrying
        except (ConnectionError, socket.gaierror) as e:  # Handle network connectivity issues
            print(f"\n⦸ Network Error (Attempt {attempt}/{max_retries}): Unable to connect. Please check your internet connection.")
            if attempt == max_retries:  # Check if max retries reached
                print(f"⦸ Error: Max retries reached. Please verify your network connection.")
                return {}  # Return empty dict after max retries
            print(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)  # Wait before retrying
        except RequestException as e:  # Handle other request-related errors
            print(f"\n⦸ Request Error (Attempt {attempt}/{max_retries}): Failed to fetch data elements: {str(e)}.")
            if attempt == max_retries:  # Check if max retries reached
                print(f"⦸ Error: Max retries reached. Unable to fetch data elements.")
                return {}  # Return empty dict after max retries
            time.sleep(retry_delay)  # Wait before retrying

    # -- Step 4: Validate and parse period inputs
    try:  # Validate start_period and end_period formats
        for period in [start_period, end_period]:  # Check both periods
            if not (isinstance(period, str) and len(period) == 6 and period.isdigit() and 1 <= int(period[4:]) <= 12):
                raise ValueError("Invalid period format. Use YYYYMM (e.g., '202501').")  # Ensure YYYYMM format and valid month
        start_year = int(start_period[:4])  # Extract year from start period
        start_month = int(start_period[4:])  # Extract month from start period
        end_year = int(end_period[:4])  # Extract year from end period
        end_month = int(end_period[4:])  # Extract month from end period
    except ValueError as e:  # Handle invalid period format
        print(f"\n⦸ Error: {str(e)}.")
        return {}  # Return empty dict if period validation fails

    # -- Step 5: Print processing start message
    with success_output:  # Display processing message in Jupyter widget
        print(f"Data processing...\n{separator_line}")

    # -- Step 6: Generate list of periods between start and end
    periods = []  # Initialize list to store period strings
    current_year, current_month = start_year, start_month  # Start from provided year and month
    while (current_year < end_year) or (current_year == end_year and current_month <= end_month):  # Loop until end period
        periods.append(f"{current_year}{current_month:02d}")  # Add period in YYYYMM format
        current_month += 1  # Increment month
        if current_month > 12:  # If month exceeds 12, reset to 1 and increment year
            current_month = 1
            current_year += 1

    # -- Step 7: Encode periods for URL
    period_string = "%3B".join(periods)  # Join periods with URL-encoded semicolon (%3B)
    period_param = f"dimension=pe%3A{period_string}"  # Create period dimension parameter for API URLs

    # -- Step 8: Initialize data storage
    processed_data = {}  # Dictionary to store processed DataFrames for each URL
    success_count = 0  # Counter for successfully processed URLs
    total_urls = len(named_urls)  # Total number of URLs to process

    # -- Step 9: Define cluster mapping for LGAs
    cluster = {
        "an Aguata": "Aguata",  # Map Anambra LGAs to their respective clusters
        "an Anaocha": "Aguata",
        "an Orumba North": "Aguata",
        "an Orumba South": "Aguata",
        "an Awka North": "Awka",
        "an Awka South": "Awka",
        "an Dunukofia": "Awka",
        "an Idemili North": "Awka",
        "an Idemili South": "Awka",
        "an Njikoka": "Awka",
        "an Ekwusigo": "Nnewi",
        "an Ihiala": "Nnewi",
        "an Nnewi North": "Nnewi",
        "an Nnewi South": "Nnewi",
        "an Anambra East": "Omambala",
        "an Anambra West": "Omambala",
        "an Ayamelum": "Omambala",
        "an Oyi": "Omambala",
        "an Ogbaru": "Onitsha",
        "an Onitsha North": "Onitsha",
        "an Onitsha South": "Onitsha"
    }  # Maps LGA names to cluster groups for reporting

    # -- Step 10: Process each named URL
    for url_name, url in named_urls.items():  # Iterate through each named URL
        try:
            # -- Update URL with new period range
            if "dimension=pe%3A" in url:  # Check if period dimension exists in URL
                start_idx = url.find("dimension=pe%3A")  # Find start of period parameter
                end_idx = url.find("&", start_idx) if url.find("&", start_idx) != -1 else len(url)  # Find end of parameter
                url = url[:start_idx] + period_param + url[end_idx:]  # Replace period parameter
            else:  # If no period dimension, append it
                url = url + "&" + period_param if "?" in url else url + "?" + period_param  # Add period parameter

            # -- Fetch data from DHIS2 API
            response = requests.get(url, auth=HTTPBasicAuth(username, password))  # Make API request with authentication
            response.raise_for_status()  # Raise exception for HTTP errors
            data = response.json()  # Parse JSON response

            # -- Extract table structure from response
            headers = [header['name'] for header in data.get('headers', [])]  # Extract column headers from response
            df = pd.DataFrame(data.get('rows', []), columns=headers)  # Create DataFrame from rows
            df = df.rename(columns={'dx': 'dataElement', 'ou': 'orgUnit', 'pe': 'period'})  # Rename columns for consistency

            # -- Extract metadata from response
            meta = data.get('metaData', {})  # Get metadata dictionary
            ou_hierarchy = meta.get('ouHierarchy', {})  # Get organizational unit hierarchy
            items = meta.get('items', {}).copy()  # Copy items metadata to avoid modifying original

            # -- Extract requested data elements from URL
            dx_matches = re.findall(r'dx%3A([^&]+)', url)  # Extract dx parameters using regex
            requested_dx_ids = []  # Initialize list for requested data element IDs
            if dx_matches:  # If dx parameters found
                requested_dx_ids = [id.split('.')[0] for id in dx_matches[0].split('%3B')]  # Clean IDs by removing suffixes

            # -- Handle special dimension URLs (e.g., TB screening, HIV self-testing)
            dimension_id = None  # Initialize dimension ID for specific URLs
            if url_name == 'ART MSF_tb screening':  # Set dimension for TB screening
                dimension_id = 'sbKiaUuaHpX'
            elif url_name == 'HTS MSF_hivst_approach':  # Set dimension for HIV self-testing approach
                dimension_id = 'tBdRxXi3Dxr'
            elif url_name in ['PMTCT MSF_sdp', 'PMTCT MSF_sdp_pos']:  # Set dimension for PMTCT service delivery points
                dimension_id = 'brKpOJgkKa0'
            elif url_name in ['PMTCT MSF_sd<72_in-outside', 'PMTCT MSF_sd>72_in-outside']:  # Set dimension for PMTCT timing
                dimension_id = 'FtBUOVZVrC6'
            elif url_name == 'HTS MSF_hivst_response_classification':  # Set dimension for HIV self-testing response
                dimension_id = 'srck01HTxTQ'
            elif url_name in ['KP Prev MSF_access_type_mhs', 'KP Prev MSF_access_type_msh_diagnose', 'KP Prev MSF_access_type_msh_support']:
                dimension_id = 'p0cjhizcn4a'
            elif url_name in [
                'KP Prev MSF_PrEP_eligible_for_pk_at_risk', 'KP Prev MSF_PrEP_received_for_pk_at_risk', 'KP Prev MSF_PrEP_discountined_for_pk_at_risk',
                'KP Prev MSF_PrEP_returned_with_retesting_negetive_for_pk_at_risk', 'KP Prev MSF_PrEP_returned_with_retesting_positive_for_pk_at_risk'
                ]:
                dimension_id = 'oDMnfmxGwgK'
            if dimension_id:  # If a special dimension is specified
                dimension_values = set(df[dimension_id].unique())  # Get unique dimension values from DataFrame
                missing_dimensions = [dv for dv in dimension_values if dv not in items]  # Identify missing dimension metadata
                if missing_dimensions:  # If dimensions are missing from metadata
                    dimension_url = 'https://ihvn.dhistance.com/api/categoryOptions'  # URL for category options endpoint
                    dimension_filter = f"id:in:[{','.join(missing_dimensions)}]"  # Filter for missing dimension IDs
                    dimension_params = {'fields': 'id,name', 'filter': dimension_filter, 'paging': 'false'}  # Query parameters
                    try:
                        dimension_response = requests.get(dimension_url, auth=HTTPBasicAuth(username, password), params=dimension_params)
                        dimension_response.raise_for_status()  # Raise exception for HTTP errors
                        dimensions = dimension_response.json().get('categoryOptions', [])  # Extract category options
                        for dim in dimensions:  # Update items with dimension names
                            items[dim['id']] = {'name': dim['name']}
                    except RequestException as e:  # Handle errors in fetching dimension metadata
                        print(f"⦸ Warning: Failed to fetch dimension names for {url_name}: {str(e)}")

            # -- Create organizational unit mappings
            if url_name == "Report Rate LGA":  # For LGA-level reports, map org units to themselves
                orgunit_to_level = {ou: ou for ou in df['orgUnit'].unique()}
            else:  # For other reports, map org units to their parent level in hierarchy
                orgunit_to_level = {
                    ou: ou_hierarchy.get(ou, '').split('/')[1] if '/' in ou_hierarchy.get(ou, '') else ou
                    for ou in df['orgUnit'].unique()
                }

            # -- Create name mappings for organizational units and data elements
            level_to_name = {  # Map organizational unit levels to their names
                org_id: items[org_id]['name']
                for org_id in set(orgunit_to_level.values()) if org_id in items
            }
            orgunit_to_name = {  # Map org unit IDs to their names
                ou: items[ou]['name']
                for ou in df['orgUnit'].unique() if ou in items
            }
            dataelement_to_name = {  # Map data element IDs to their names from metadata
                de: items[de]['name']
                for de in requested_dx_ids if de in items
            }

            # -- Combine mappings: Prefer dataelement_to_description, fall back to dataelement_to_name
            combined_dataelement_mapping = {}  # Initialize mapping for data element column names
            for de in requested_dx_ids:  # Iterate over requested data elements
                if de in dataelement_to_description:  # Prefer description from data elements API
                    combined_dataelement_mapping[de] = dataelement_to_description[de]
                elif de in dataelement_to_name:  # Fallback to name from metadata
                    combined_dataelement_mapping[de] = dataelement_to_name[de]
                else:  # If no mapping found, use raw ID
                    combined_dataelement_mapping[de] = de

            # -- Pivot DataFrame based on URL type
            if dimension_id:  # For URLs with special dimensions, pivot on dimension_id
                if dimension_id not in df.columns:  # Check if dimension exists in DataFrame
                    print(f"⦸ Warning: Dimension {dimension_id} missing for {url_name}.")
                    continue  # Skip to next URL if dimension is missing
                dimension_matches = re.findall(rf'{dimension_id}%3A([^&]+)', url)  # Extract dimension values from URL
                requested_dimension_ids = []  # Initialize list for requested dimension IDs
                if dimension_matches:  # If dimension values found
                    requested_dimension_ids = dimension_matches[0].split('%3B')  # Split into list
                pivoted_df = df.pivot(
                    index=['period', 'orgUnit'],  # Pivot on period and orgUnit
                    columns=dimension_id,  # Use dimension as columns
                    values='value'  # Use value column for pivot values
                ).reset_index()
                for dim_id in requested_dimension_ids:  # Ensure all requested dimension values are columns
                    if dim_id not in pivoted_df.columns:
                        pivoted_df[dim_id] = 0  # Add missing dimension columns with zero values
            else:  # For standard URLs, pivot on dataElement
                pivoted_df = df.pivot(
                    index=['period', 'orgUnit'],  # Pivot on period and orgUnit
                    columns='dataElement',  # Use dataElement as columns
                    values='value'  # Use value column for pivot values
                ).reset_index()
                for dx_id in requested_dx_ids:  # Ensure all requested data elements are columns
                    if dx_id not in pivoted_df.columns:
                        pivoted_df[dx_id] = 0  # Add missing data element columns with zero values
            pivoted_df.columns.name = None  # Remove column name index for cleaner output

            # -- Format period column to 'Mon-YY'
            pivoted_df['period'] = pd.to_datetime(pivoted_df['period'], format='%Y%m').dt.strftime('%b-%y')  # Convert YYYYMM to Mon-YY

            # -- Add organizational hierarchy information
            pivoted_df['orgunitlevel'] = pivoted_df['orgUnit'].map(orgunit_to_level)  # Add parent org unit level
            pivoted_df['LGA'] = pivoted_df['orgunitlevel'].map(level_to_name)  # Map parent level to LGA name
            pivoted_df['Cluster'] = pivoted_df['LGA'].map(cluster)  # Map LGA to cluster
            pivoted_df['orgUnit'] = pivoted_df['orgUnit'].map(orgunit_to_name)  # Map orgUnit to facility name

            # -- Rename columns for readability
            if url_name in ['Report Rate Facility', 'Report Rate LGA']:  # For reporting rate URLs
                rename_dict = {
                    **{col: dataelement_to_name.get(col, col) for col in pivoted_df.columns if col in dataelement_to_name},  # Use metadata names
                    'period': 'ReportPeriod',  # Rename period column
                    'orgUnit': 'FacilityName'  # Rename orgUnit column
                }
            elif dimension_id:  # For URLs with special dimensions
                rename_dict = {
                    **{col: items.get(col, {}).get('name', col) for col in pivoted_df.columns if col in items},  # Use dimension names
                    'period': 'ReportPeriod',  # Rename period column
                    'orgUnit': 'FacilityName'  # Rename orgUnit column
                }
            else:  # For standard URLs
                rename_dict = {
                    **{col: combined_dataelement_mapping.get(col, col) for col in pivoted_df.columns if col in combined_dataelement_mapping},  # Use combined mappings
                    'period': 'ReportPeriod',  # Rename period column
                    'orgUnit': 'FacilityName'  # Rename orgUnit column
                }
            pivoted_df.rename(columns=rename_dict, inplace=True)  # Apply column renaming

            # -- Shorten FacilityName for display
            pivoted_df['FacilityName'] = pivoted_df['FacilityName'].apply(
                lambda x: x[:34] + '...' if isinstance(x, str) and len(x) > 33 else x  # Truncate names longer than 33 characters
            )

            # -- Handle NaN values based on URL type
            if url_name in ['Report Rate Facility', 'Report Rate LGA']:  # For reporting rates
                pivoted_df.fillna('', inplace=True)  # Replace NaN with empty strings
            else:  # For other data
                pivoted_df.fillna(0, inplace=True)  # Replace NaN with zeros

            # -- Store processed DataFrame
            pivoted_df = pivoted_df.reset_index(drop=True)  # Reset index for clean DataFrame
            processed_data[url_name] = pivoted_df  # Store DataFrame in result dictionary
            success_count += 1  # Increment success counter

            # -- Update success message in Jupyter widget
            with success_output:
                clear_output(wait=True)  # Clear previous output
                print(
                    f"Fetched {len(data_elements)} data elements\n"
                    f"Data processing...\n{separator_line}\n"
                    f"Retrieving: {url_name}\n"
                    f"Retrieved : {success_count}/{total_urls} data files"  # Show progress
                )

        except RequestException as e:  # Handle API request errors
            print(f"\n⦸ Error: Failed to fetch data for '{url_name}': {str(e)}.")
            continue  # Skip to next URL
        except Exception as e:  # Handle unexpected errors
            print(f"\n⦸ Error: Unexpected issue processing '{url_name}': {str(e)}.")
            continue  # Skip to next URL

    # -- Step 11: Display processing summary
    report_period_display = (
        f"✔️ Data fetched successfully! ({success_count}/{total_urls})\n{separator_line}"  # Summary message
    )
    if processed_data:  # If at least one DataFrame was processed
        print(report_period_display)
        with success_output:
            clear_output()  # Clear Jupyter widget
    else:  # If no data was processed
        print(f"\n⦸ Failed:\nReport extracts: (0/{total_urls})\nInvalid DHIS2 login credentials or no data retrieved.\n{separator_line}")
        with success_output:
            clear_output()  # Clear Jupyter widget

    return processed_data  # Return dictionary of processed DataFrames
# End of the function ---------------------------------------------------------------------

In [2]:
### - GET DATA
# -----------------------------------------------------------------------------------------
from datetime import datetime                               # Library for date/time operations, used for period formatting

def fetch_dhis2_data_interactive_jupyter_mode():
    """
    Interactive function to collect period inputs with DatePicker (MM-YY display) and fetch/process DHIS2 data using static credentials.
    Args:
        None
    Returns:
        dict: DHIS2_data fetched from the DHIS2 server (accessible globally after submission)
    """
    # -- GLOBALS
    global DHIS2_data, load_dhis2_config                    # Declare global variables for data and config function

    # -- Step 1: Define constants and output widget
    separator_line = '-' * 43                              # Define separator line for output formatting
    output = widgets.Output()                              # Create output widget for Jupyter UI display

    # -- Step 2: Static credentials for DHIS2 login
    username = "Promise_Uzondu"                            # Set static username for DHIS2 login
    password = "P@ssword118"                               # Set static password for DHIS2 login

    # -- Step 3: Create calendar widgets for period selection
    start_period_picker = widgets.DatePicker(              # Create DatePicker for start period
        description="Start Period:",                       # Label for start period picker
        value=datetime.now().replace(day=1),               # Default to first day of current month
        layout={'width': '300px'}                          # Set picker width to 300px
    )
    end_period_picker = widgets.DatePicker(                # Create DatePicker for end period
        description="End Period:",                         # Label for end period picker
        value=datetime.now().replace(day=1),               # Default to first day of current month
        layout={'width': '300px'}                          # Set picker width to 300px
    )
    submit_periods = widgets.Button(description="Submit")   # Create submit button for period selection
    periods_box = widgets.VBox([                           # Create vertical box for period widgets
        start_period_picker,                               # Include start period picker
        end_period_picker,                                 # Include end period picker
        submit_periods                                     # Include submit button
    ])

    # -- Step 4: Store collected period inputs
    periods = [None, None]                                 # Initialize list to store periods in YYYYMM

    # -- Step 5: Define formatting functions for display
    def format_credentials(username, password):             # Define function to format credentials
        """
        Format login credentials with masked password.
        Args:
            username (str): DHIS2 username
            password (str): DHIS2 password
        Returns:
            str: Formatted string with masked password
        """
        username_line = f"{'Username: ':<{43 - len(username)}}{username}"  # Format username line
        password_line = f"{'Passkey: ':<{43 - len(password)}}{'*' * len(password)}"  # Format masked password line
        return f"{username_line}\n{password_line}"         # Return formatted credentials string

    def format_report_period(start_period, end_period):     # Define function to format periods
        """
        Format report periods in Mon-YYYY format (e.g., Feb-2025).
        Args:
            start_period (str): Start period in YYYYMM
            end_period (str): End period in YYYYMM
        Returns:
            str: Formatted period string
        """
        start_date = datetime.strptime(start_period, "%Y%m")  # Parse start period to datetime
        end_date = datetime.strptime(end_period, "%Y%m")   # Parse end period to datetime
        start_mon_yyyy = start_date.strftime("%b-%Y")      # Format start as Mon-YYYY
        end_mon_yyyy = end_date.strftime("%b-%Y")          # Format end as Mon-YYYY
        start_period_line = f"{'Period Start Date: ':<{43 - len(start_mon_yyyy)}}{start_mon_yyyy}"  # Format start period line
        end_period_line = f"{'Period End Date: ':<{43 - len(end_mon_yyyy)}}{end_mon_yyyy}"  # Format end period line
        return f"{start_period_line}\n{end_period_line}"   # Return formatted period string

    def display_information(credentials_display, report_display):  # Define function to display info
        """
        Display formatted credentials and periods.
        Args:
            credentials_display (str): Formatted credentials string
            report_display (str): Formatted period string
        """
        with output:                                       # Use output widget for display
            clear_output()                                 # Clear previous output
            print("IHVN DHIS2 login credentials:")         # Print credentials header
            print(separator_line)                          # Print separator line
            print(credentials_display)                     # Print formatted credentials
            print(separator_line)                          # Print separator line
            print()                                        # Print blank line
            print('Selected report periods:')              # Print periods header
            print(separator_line)                          # Print separator line
            print(report_display)                          # Print formatted periods
            print(separator_line)                          # Print separator line
            print()                                        # Print blank line

    # -- Step 6: Define period submission handler
    def on_submit_periods(b):                              # Define handler for submit button
        """
        Handle period submission and fetch data.
        Args:
            b: Button click event (ignored)
        """
        global DHIS2_data                                  # Access global DHIS2_data
        if start_period_picker.value and end_period_picker.value:  # Check if periods are selected
            periods[0] = start_period_picker.value.strftime("%Y%m")  # Convert start to YYYYMM
            periods[1] = end_period_picker.value.strftime("%Y%m")  # Convert end to YYYYMM
        else:                                              # Handle missing period selection
            with output:                                   # Use output widget
                clear_output()                             # Clear previous output
                print("Error: Please select both start and end periods.")  # Print error message
                display(periods_box)                       # Redisplay period selection box
            return                                         # Exit handler
        if int(periods[1]) < int(periods[0]):              # Validate end period not before start
            with output:                                   # Use output widget
                clear_output()                             # Clear previous output
                print("Error: End period cannot be before start period.")  # Print error message
                display(periods_box)                       # Redisplay period selection box
            return                                         # Exit handler
        credentials_display = format_credentials(username, password)  # Format credentials
        report_display = format_report_period(periods[0], periods[1])  # Format periods
        display_information(credentials_display, report_display)  # Display formatted info
        with output:                                       # Use output widget for data fetch
            DHIS2_data = fetch_and_process_DHIS2_data(username, password, periods[0], periods[1])  # Fetch DHIS2 data
            if "Report Rate LGA" in DHIS2_data and "LGA" in DHIS2_data["Report Rate LGA"].columns:  # Check for LGA data
                available_lgas = sorted(DHIS2_data["Report Rate LGA"]["LGA"].dropna().unique())  # Get unique LGAs
                lga_filter_widget = widgets.SelectMultiple(options=available_lgas, rows=6)  # Create LGA filter widget
                apply_filter_button = widgets.Button(description="Apply Filter")  # Create filter apply button
                def on_apply_filter_clicked(b):                # Define handler for filter button
                    """
                    Handle LGA filter application.
                    Args:
                        b: Button click event (ignored)
                    """
                    selected_lgas = list(lga_filter_widget.value)  # Get selected LGAs
                    with output:                               # Use output widget
                        if not selected_lgas:                  # If no LGAs selected
                            load_dhis2_config()                # Load config for all data
                            print("✔️ State level data ready")  # Print success message
                        else:                                  # If LGAs selected
                            for key, df in DHIS2_data.items():  # Iterate DHIS2_data
                                if isinstance(df, pd.DataFrame) and "LGA" in df.columns:  # Check for LGA column
                                    DHIS2_data[key] = df[df["LGA"].isin(selected_lgas)].copy()  # Filter by LGAs
                            load_dhis2_config()                # Load config for filtered data
                            print(f"✔️ LGA level data ready for {selected_lgas}")  # Print success message
                apply_filter_button.on_click(on_apply_filter_clicked)  # Link filter button to handler
                print(f"\nSelect report level - LGA")      # Print LGA filter header
                print(separator_line)                      # Print separator line
                display(widgets.VBox([lga_filter_widget, apply_filter_button]))  # Display LGA filter widgets
                print(separator_line)                      # Print separator line

    # -- Step 7: Link submit button to handler
    submit_periods.on_click(on_submit_periods)             # Link submit button to period handler

    # -- Step 8: Display the initial interface
    with output:                                           # Use output widget for initial display
        print("IHVN DHIS2 login credentials:")             # Print credentials header
        print(separator_line)                              # Print separator line
        print(format_credentials(username, password))      # Print formatted credentials
        print(separator_line)                              # Print separator line
        print()                                            # Print blank line
        print('Select report periods:')                    # Print periods header
        print(separator_line)                              # Print separator line
        display(periods_box)                               # Display period selection box
    display(output)                                        # Display output widget
# End of the function ---------------------------------------------------------------------


In [ ]:
### - VARIBLES AND FUNCTIONS - for web
import os
import pandas as pd
import textwrap
import dataframe_image as dfi
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment
from docx import Document
from docx.shared import Inches, Pt, RGBColor
import numpy as np
import re
import ipywidgets as widgets
from IPython.display import display, HTML
import io
import gc
import os  # For path joining, kept for compatibility with path construction
import zipfile                      # Library for creating ZIP archives

def load_dhis2_config(base_prefix="reports"):
    """
    Load DHIS2 configuration, functions, and data for reporting, storing files in report_archive for ZIP.
    Optimized for low memory usage. Uses a configurable base prefix for path construction.
    Args:
        base_prefix (str): Base prefix for report_archive keys (default: 'reports').
    Returns:
        dict: report_archive containing file paths and contents, or None on error.
    """
    separator_line = '-' * 43

    try:
        # -- Step 1: Declare global variables and functions
        global report_name, report_name_rate, report_name_outlier, report_name_period
        global report_name_period_name, report_period_name_folder, sub_folder_image_file, sub_folder_doc_file
        global sub_folder2_image_file_report_rate, sub_folder2_image_file_msf_outlier
        global doc_file_report_rate_xlsx, doc_file_msf_outlier_docx, doc_file_msf_outlier_xlsx
        global highlight_red_list, MSF_hierarchy
        global outlier_red_report_rate, outlier_green_report_rate, outlier_red, outlier_yellow
        global outlier_red_LT0, outlier_yellow_LT0, outlier_red_GT0, outlier_yellow_GT0
        global export_df_to_doc_image_excel, filter_gap_and_check_empty_df, prepare_and_convert_df
        global wrap_column_headers, wrap_column_headers2, widget_display_df
        global Pre_HTS_MSF_positive, Pre_MSF_positives_all
        global report_archive

        # -- Step 2: Initialize report_archive
        report_archive = {}

        # -- Step 3: Create report name, dynamic period, and joined report period name
        try:
            global DHIS2_data
            if 'Report Rate Facility' not in DHIS2_data:
                raise KeyError("Report Rate Facility not found in DHIS2_data")
            report_name = "anso msf report"
            report_name_rate = "anso msf report rate"
            report_name_outlier = "anso msf outlier"
            report_name_period = DHIS2_data["Report Rate Facility"].ReportPeriod.iloc[0]
            report_name_period_name = f"{report_name_period} {report_name}"
        except (KeyError, AttributeError, IndexError) as e:
            print(f"⦸ Error: Failed to access DHIS2_data or ReportPeriod: {str(e)}")
            return None

        # -- Step 4: Define folder structure for report_archive keys
        report_period_name_folder = f"{base_prefix}/{report_name_period_name}"
        sub_folder_image_file = f"{report_period_name_folder}/image file"
        sub_folder_doc_file = f"{report_period_name_folder}/document file"
        sub_folder2_image_file_report_rate = f"{sub_folder_image_file}/{report_name_rate}"
        sub_folder2_image_file_msf_outlier = f"{sub_folder_image_file}/{report_name_outlier}"
        doc_file_report_rate_xlsx = f"{sub_folder_doc_file}/{report_name_period} {report_name_rate}.xlsx"
        doc_file_msf_outlier_docx = f"{sub_folder_doc_file}/{report_name_period_name}.docx"
        doc_file_msf_outlier_xlsx = f"{sub_folder_doc_file}/{report_name_period_name}.xlsx"

        # -- Step 5: Define highlight_red_list
        highlight_red_list = [
            'Community', 'Walk-In', 'Community & Walk-In', 'subset of 4',
            'Self, Spouse, Sexual Partner, Children, Social Network, Others',
            'FSW, MSM, PWID, TG, Others', 'Testing frequency', 'Outreach',
            'Outreach-Pregnant', 'Outreach-Others', 'Excluding community testing',
            'Excluding previously known', 'IPV', 'ANC', 'L&D', '<72hrs PP',
            '<72 hrs', '>72 hrs - < 6 months', '>6 - 12 months',
            'ANC, L&D, <72hrs Post Partum', 'Facility', 'Outside Facility',
            'Within and outside the facility', 'within 72 hrs of birth',
            'between >72 hrs - <2 months of birth', 'All regimens', 'Regimen Lines',
            'MMD', 'DSD', 'excludes ART transfer-in', 'ART Addendum-2'
        ]

        # -- Step 6: Define MSF hierarchy
        MSF_hierarchy = ['ReportPeriod', 'Cluster', 'LGA', 'FacilityName']

        # -- Step 7: Define wrap_column_headers function
        def wrap_column_headers(df, max_width=20):
            try:
                wrapped_columns = []
                for col in df.columns:
                    if len(str(col)) > max_width:
                        wrapped = '\n'.join(textwrap.wrap(str(col), max_width, break_long_words=True))
                        wrapped_columns.append(wrapped)
                    else:
                        wrapped_columns.append(col)
                df.columns = wrapped_columns
                return df
            except Exception as e:
                print(f"⦸ Error: Failed in wrap_column_headers: {str(e)}")
                return df

        # -- Step 8: Define wrap_column_headers2 function
        def wrap_column_headers2(columns, max_width=20):
            try:
                wrapped_columns = []
                for col in columns:
                    if len(str(col)) > max_width:
                        wrapped = '\n'.join(textwrap.wrap(str(col), max_width, break_long_words=True))
                        wrapped_columns.append(wrapped)
                    else:
                        wrapped_columns.append(col)
                return wrapped_columns
            except Exception as e:
                print(f"⦸ Error: Failed in wrap_column_headers2: {str(e)}")
                return columns

        # -- Step 9: Define outlier_red_report_rate function
        def outlier_red_report_rate(val):
            try:
                condition = (
                    ((isinstance(val, (int, float)) and val < 100) or
                     (isinstance(val, str) and val != '100' and val != '')) or
                    (not (isinstance(val, (int, float)) and val < 100) and
                     (isinstance(val, (int, float)) and val != 0))
                )
                if condition:
                    return 'background-color: lightcoral; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
            except Exception as e:
                print(f"⦸ Error: Failed in outlier_red_report_rate: {str(e)}")
                return None

        # -- Step 10: Define outlier_green_report_rate function
        def outlier_green_report_rate(val):
            try:
                if (isinstance(val, (int, float)) and val == 100) or (isinstance(val, str) and val == '100' and val != ''):
                    return 'background-color: lightgreen; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
            except Exception as e:
                print(f"⦸ Error: Failed in outlier_green_report_rate: {str(e)}")
                return None

        # -- Step 11: Define outlier_red function
        def outlier_red(val):
            try:
                if isinstance(val, (int, float)) and val != 0:
                    return 'background-color: lightcoral; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
            except Exception as e:
                print(f"⦸ Error: Failed in outlier_red: {str(e)}")
                return None

        # -- Step 12: Define outlier_yellow function
        def outlier_yellow(val):
            try:
                if isinstance(val, (int, float)) and val != 0:
                    return 'background-color: #fff59d; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
            except Exception as e:
                print(f"⦸ Error: Failed in outlier_yellow: {str(e)}")
                return None

        # -- Step 13: Define outlier_red_LT0 function
        def outlier_red_LT0(val):
            try:
                if isinstance(val, (int, float)) and val < 0:
                    return 'background-color: lightcoral; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
            except Exception as e:
                print(f"⦸ Error: Failed in outlier_red_LT0: {str(e)}")
                return None

        # -- Step 14: Define outlier_yellow_LT0 function
        def outlier_yellow_LT0(val):
            try:
                if isinstance(val, (int, float)) and val < 0:
                    return 'background-color: #fff59d; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
            except Exception as e:
                print(f"⦸ Error: Failed in outlier_yellow_LT0: {str(e)}")
                return None

        # -- Step 15: Define outlier_red_GT0 function
        def outlier_red_GT0(val):
            try:
                if isinstance(val, (int, float)) and val > 0:
                    return 'background-color: lightcoral; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
            except Exception as e:
                print(f"⦸ Error: Failed in outlier_red_GT0: {str(e)}")
                return None

        # -- Step 16: Define outlier_yellow_GT0 function
        def outlier_yellow_GT0(val):
            try:
                if isinstance(val, (int, float)) and val > 0:
                    return 'background-color: #fff59d; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
            except Exception as e:
                print(f"⦸ Error: Failed in outlier_yellow_GT0: {str(e)}")
                return None

        # -- Step 17: Define export_df_to_doc_image_excel function
        def export_df_to_doc_image_excel(
            report_name=None, df_style=None, img_file_name=None, img_file_path=None,
            doc_description=None, doc_indicators_to_italicize=None, doc_indicators_to_underline=None,
            doc_file_path=doc_file_msf_outlier_docx, xlm_file_path=None, xlm_sheet_name=None,
            highlight_red_list=highlight_red_list
        ):
            try:
                image_path = None

                # -- Step 17.1: Process image export
                if all([df_style is not None, img_file_name is not None, img_file_path is not None]):
                    try:
                        styled_df = df_style.set_properties(**{'text-align': 'right', 'font-size': '10pt'})
                        image_buffer = io.BytesIO()
                        dfi.export(styled_df, image_buffer, table_conversion='matplotlib',
                                  max_rows=-1, max_cols=-1, fontsize=10, dpi=150)
                        image_buffer.seek(0)
                        image_path = f"{img_file_path}/{img_file_name}"
                        report_archive[image_path] = image_buffer.getvalue()
                        image_buffer.close()
                        del styled_df
                        gc.collect()
                    except Exception as e:
                        print(f"⦸ Error: Failed to export image for {img_file_name}: {str(e)}")
                        return None

                # -- Step 17.2: Process Excel export
                if all([df_style is not None, xlm_file_path is not None, xlm_sheet_name is not None]):
                    try:
                        xlm_sheet_name = xlm_sheet_name[:31]
                        excel_buffer = io.BytesIO()
                        with pd.ExcelWriter(excel_buffer, engine='openpyxl') as writer:
                            df_style.to_excel(writer, sheet_name=xlm_sheet_name, index=False)
                        excel_buffer.seek(0)
                        wb = load_workbook(excel_buffer)
                        ws = wb[xlm_sheet_name]
                        for cell in ws[1]:
                            if cell.value and isinstance(cell.value, str):
                                protected_map = {}
                                for phrase in highlight_red_list or []:
                                    token = f"@@PROTECT_{abs(hash(phrase))}@@"
                                    protected_map[token] = phrase
                                    cell.value = cell.value.replace(phrase, token)
                                cell.value = re.sub(r"<.*?>", "", cell.value)
                                for token, phrase in protected_map.items():
                                    cell.value = cell.value.replace(token, phrase)
                                cell.value = cell.value.strip()
                        font_style = Font(name='Calibri', size=8)
                        header_font = Font(name='Calibri', size=8, bold=True)
                        header_alignment = Alignment(horizontal="left", vertical="bottom", wrap_text=True)
                        for row in ws.iter_rows():
                            for cell in row:
                                cell.font = font_style
                        for cell in ws[1]:
                            cell.alignment = header_alignment
                            cell.font = header_font
                        for col in ws.iter_cols(min_col=1, max_col=4):
                            max_length = max((len(str(cell.value)) if cell.value else 0) for cell in col)
                            ws.column_dimensions[col[0].column_letter].width = max_length
                        ws.auto_filter.ref = ws.dimensions
                        excel_output_buffer = io.BytesIO()
                        wb.save(excel_output_buffer)
                        excel_output_buffer.seek(0)
                        report_archive[xlm_file_path] = excel_output_buffer.getvalue()
                        excel_buffer.close()
                        excel_output_buffer.close()
                        del wb, ws
                        gc.collect()
                    except Exception as e:
                        print(f"⦸ Error: Failed to export Excel for {xlm_sheet_name}: {str(e)}")
                        return None

                # -- Step 17.3: Create or append to Word document
                if all([doc_file_path is not None, image_path is not None, doc_indicators_to_italicize is not None, doc_indicators_to_underline is not None]):
                    try:
                        doc = Document()
                        style = doc.styles['Normal']
                        style.font.name = 'Calibri'
                        style.font.size = Pt(9.5)
                        for section in doc.sections:
                            section.left_margin = Inches(0.5)
                            section.right_margin = Inches(0.5)
                            section.top_margin = Inches(1)
                            section.bottom_margin = Inches(1)
                        if doc_description:
                            title_paragraph = doc.add_heading(report_name, level=2)
                            title_run = title_paragraph.runs[0]
                            title_run.font.size = Pt(10)
                            title_run.font.color.rgb = RGBColor(0, 0, 0)
                            paragraph = doc.add_paragraph()
                            paragraph.paragraph_format.space_after = Pt(0)
                            phrases_to_bold = [
                                "REPORT ONLY 2025 LIVE BIRTHS BY PPW",
                                "REPORT ONLY HEI ARVs FOR 2025 LIVE BIRTHS BY PPW",
                                "REPORT ONLY EID SAMPLE COLLECTION FOR 2025 LIVE BIRTHS BY PPW",
                                "REPORT ONLY EID PCR RESULTS FOR 2025 LIVE BIRTHS BY PPW",
                                "Report Name:", "should not be greater than",
                                "should not be lesser than", "should not be equal to",
                                "should be greater than", "should be lesser than",
                                "should be equal to", "plus", "Note", "OR"
                            ]
                            all_phrases = phrases_to_bold + (doc_indicators_to_italicize or []) + (doc_indicators_to_underline or [])
                            pattern = r'|'.join(re.escape(phrase) for phrase in all_phrases)
                            matches = list(re.finditer(pattern, doc_description))
                            last_index = 0
                            for match in matches:
                                start, end = match.start(), match.end()
                                paragraph.add_run(doc_description[last_index:start])
                                run = paragraph.add_run(doc_description[start:end])
                                if match.group(0) in phrases_to_bold:
                                    run.bold = True
                                if match.group(0) in doc_indicators_to_italicize:
                                    run.italic = True
                                if match.group(0) in doc_indicators_to_underline:
                                    run.underline = True
                                last_index = end
                            paragraph.add_run(doc_description[last_index:])
                        image_buffer = io.BytesIO(report_archive[image_path])
                        doc.add_picture(image_buffer, width=Inches(7))
                        image_buffer.close()
                        section = doc.sections[-1]
                        footer = section.footer.paragraphs[0]
                        footer.text = "This is an auto-generated report. Ensure all data is reviewed before any update is made."
                        footer.runs[0].font.size = Pt(7.5)
                        footer.runs[0].font.color.rgb = RGBColor(100, 100, 100)
                        doc_buffer = io.BytesIO()
                        doc.save(doc_buffer)
                        doc_buffer.seek(0)
                        report_archive[doc_file_path] = doc_buffer.getvalue()
                        doc_buffer.close()
                        del doc
                        gc.collect()
                    except Exception as e:
                        print(f"⦸ Error: Failed to create Word document at {doc_file_path}: {str(e)}")
                        return None

                # -- Step 17.4: Generate and print success messages
                if all([report_name, img_file_name, img_file_path, xlm_file_path, xlm_sheet_name]):
                    try:
                        img_file_path_name = img_file_path
                        xlm_file_path_name = xlm_file_path
                        image_success_print = rf"IMG: '{img_file_name}' in {img_file_path_name}"
                        excel_success_print = rf"XLS: '{xlm_sheet_name}' in {xlm_file_path_name}"
                        messages = [image_success_print, excel_success_print]
                        if doc_description:
                            doc_file_path_name = doc_file_path
                            doc_success_print = rf"DOC: '{report_name}' in {doc_file_path_name}"
                            messages.append(doc_success_print)
                        separator_length = max(len(msg) for msg in messages)
                        print(f"✔️ {report_name}")
                        print('-' * separator_length)
                        print('\n'.join(messages))
                        print('-' * separator_length)
                    except Exception as e:
                        print(f"⦸ Error: Failed to print success messages: {str(e)}")
                        return image_path if image_path else None

                return image_path if image_path else None

            except Exception as e:
                print(f"⦸ Error: Failed in export_df_to_doc_image_excel: {str(e)}")
                return None

        # -- Step 18: Define prepare_and_convert_df function
        def prepare_and_convert_df(DHIS2_data_key=None, hierarchy_columns=None, data_columns=None):
            try:
                if DHIS2_data_key not in DHIS2_data:
                    print(f"⦸ Error: '{DHIS2_data_key}' not found in DHIS2_data")
                    return None
                df_raw = DHIS2_data[DHIS2_data_key]
                if hierarchy_columns is None:
                    hierarchy_columns = []
                available_columns = [col for col in data_columns if col in df_raw.columns]
                missing_columns = [col for col in data_columns if col not in df_raw.columns]
                for col in missing_columns:
                    print(f"⦸ Warning: Column '{col}' not found in '{DHIS2_data_key}'")
                if not available_columns:
                    print(f"⦸ Warning: None of the requested columns found in '{DHIS2_data_key}'. Missing: {missing_columns}")
                df = df_raw[hierarchy_columns + available_columns].copy()
                df.sort_values(by=hierarchy_columns, inplace=True, ignore_index=True)
                for col in available_columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
                for col in data_columns:
                    if col not in df.columns:
                        df[col] = 0
                df = df[hierarchy_columns + data_columns]
                del df_raw
                gc.collect()
                return df
            except Exception as e:
                print(f"⦸ Error: Failed to prepare DataFrame for '{DHIS2_data_key}': {str(e)}")
                return None

        # -- Step 19: Define filter_gap_and_check_empty_df function
        def filter_gap_and_check_empty_df(
            df=None, msg=None, opNonZero=None, opNeg=None, opPos=None,
            opNonPos=None, opNonNeg=None, opZero=None, opLT100=None
        ):
            try:
                if df is None or df.empty:
                    print(f"⦸ Error: Input DataFrame is None or empty")
                    return None
                if not msg:
                    print(f"⦸ Error: No gap message provided for empty result")
                    return None
                operator_map = {
                    'opNonZero': lambda x: x != 0,
                    'opNeg': lambda x: x < 0,
                    'opPos': lambda x: x > 0,
                    'opZero': lambda x: x == 0,
                    'opLT100': lambda x: x < 100
                }
                conditions = []
                for arg, cols in {
                    'opNonZero': opNonZero, 'opNeg': opNeg, 'opPos': opPos,
                    'opNonPos': opNonPos, 'opNonNeg': opNonNeg, 'opZero': opZero, 'opLT100': opLT100
                }.items():
                    if cols:
                        for col in cols:
                            if col not in df.columns:
                                print(f"⦸ Error: Column '{col}' not found in DataFrame")
                                return None
                            numeric_series = pd.to_numeric(df[col], errors='coerce').fillna(0)
                            conditions.append(operator_map[arg](numeric_series))
                if not conditions:
                    print(f"⦸ Warning: No filtering conditions provided, returning original DataFrame")
                    return df
                combined_condition = pd.DataFrame(conditions).T.any(axis=1)
                df_filtered = df[combined_condition]
                if df_filtered.empty:
                    print("✋🏿Checked:")
                    print("-" * len(msg))
                    print(msg)
                    print("-" * len(msg))
                    return None
                return df_filtered
            except Exception as e:
                print(f"⦸ Error: Failed to filter DataFrame: {str(e)}")
                return None

        # -- Step 20: Define widget_display_df function
        def widget_display_df(styler, widget=None):
            try:
                base_styles = [
                    {'selector': 'table', 'props': [('background-color', 'white'), ('border-collapse', 'collapse')]},
                    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'right'), ('padding', '5px')]},
                    {'selector': 'td', 'props': [('text-align', 'right'), ('padding', '5px')]},
                    {'selector': 'tr:hover', 'props': [('background-color', '#e0f7fa')]},
                    {'selector': 'tr:hover td', 'props': [('font-weight', 'bold')]}
                ]
                gap_col_styles = []
                for idx, col in enumerate(styler.columns):
                    if "gap" in col.lower():
                        gap_col_styles.append({'selector': f'th.col{idx}', 'props': [('background-color', '#ffe6e6')]})
                table_styles = base_styles + gap_col_styles
                styled = styler.set_table_styles(table_styles)
                html_output = f"""
                <div style="overflow-x: auto; border: 1px solid #ccc; padding: 5px; max-width: 100%;">
                    {styled.to_html()}
                </div>
                """
                if widget is None:
                    widget = widgets.Output()
                elif not isinstance(widget, widgets.Output):
                    raise TypeError("⦸ Error: 'widget' must be an ipywidgets.Output object.")
                with widget:
                    display(HTML(html_output))
                display(widget)
                return widget
            except Exception as e:
                print(f"⦸ Error: Failed to display DataFrame in widget: {str(e)}")
                return None

        # -- Step 21: Constants Initialization for Positive Data Processing
        HTS_cols = {
            "positive": [
                "Number of people who tested HIV positive and received results (Inpatient)",
                "Number of people who tested HIV positive and received results (Outpatient)",
                "Number of people who tested HIV positive and received results (Standalone)",
                "Number of people who tested HIV positive and received results  (Community)"
            ],
            "known_positive": [
                "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Inpatient)",
                "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Outpatient)",
                "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Standalone)",
                "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Community)"
            ]
        }
        AGYW_cols = [
            "Number of AGYW who tested HIV Positive during the reporting period (Community)",
            "Number of AGYW who tested HIV Positive during the reporting period (Walk-In)"
        ]
        PMTCT_col = ["Number of pregnant women tested HIV positive"]
        KP_cols = [
            "HTS-3a Number of MSM that have received an HIV test during the reporting period in KP-specific programs and received HIV Positive results",
            "HTS-3b Number of TG that have received an HIV test during the reporting period in KP-specific programs and HIV positive results",
            "HTS-3c Number of sex workers that have received an HIV test during the reporting period in KP-specific programs and received HIV-positive results",
            "HTS-3d Number of people who inject drugs (PWID) that have received an HIV test during the reporting period in KP-specific programs and received HIV positive results",
            "HTS-3e Number of other vulnerable populations (OVP) that have received an HIV test during the reporting period and received HIV-positive results",
            "HTS-3f Number of people in prisons and other closed settings that have received an HIV test during the reporting period and received HIV-positive results"
        ]

        # -- Step 22: Prepare and process HTS data
        try:
            Pre_HTS_MSF_positive = prepare_and_convert_df("HTS MSF", MSF_hierarchy, HTS_cols["positive"] + HTS_cols["known_positive"])
            if Pre_HTS_MSF_positive is None:
                print(f"⦸ Error: Failed to prepare HTS MSF data")
                return None
            Pre_HTS_MSF_positive["HTS total tested - positive"] = Pre_HTS_MSF_positive[HTS_cols["positive"]].sum(axis=1)
            Pre_HTS_MSF_positive["HTS total tested - previously known positive"] = Pre_HTS_MSF_positive[HTS_cols["known_positive"]].sum(axis=1)
            Pre_HTS_MSF_positive["HTS total tested - new positive (excluding previously known)"] = np.where(
                Pre_HTS_MSF_positive["HTS total tested - previously known positive"] > 0,
                Pre_HTS_MSF_positive["HTS total tested - positive"] - Pre_HTS_MSF_positive["HTS total tested - previously known positive"],
                Pre_HTS_MSF_positive["HTS total tested - positive"]
            )
            gc.collect()
        except Exception as e:
            print(f"⦸ Error: Failed to process HTS MSF data: {str(e)}")
            return None

        # -- Step 23: Prepare AGYW data
        try:
            Pre_AGYW_MSF_positive = prepare_and_convert_df("AGYW MSF", MSF_hierarchy, AGYW_cols)
            if Pre_AGYW_MSF_positive is None:
                print(f"⦸ Error: Failed to prepare AGYW MSF data")
                return None
            Pre_AGYW_MSF_positive["AGYW total tested - new positive"] = Pre_AGYW_MSF_positive[AGYW_cols].sum(axis=1)
            gc.collect()
        except Exception as e:
            print(f"⦸ Error: Failed to process AGYW MSF data: {str(e)}")
            return None

        # -- Step 24: Prepare PMTCT data
        try:
            Pre_PMTCT_MSF_positive = prepare_and_convert_df("PMTCT MSF", MSF_hierarchy, PMTCT_col)
            if Pre_PMTCT_MSF_positive is None:
                print(f"⦸ Error: Failed to prepare PMTCT MSF data")
                return None
            Pre_PMTCT_MSF_positive.rename(columns={PMTCT_col[0]: "PMTCT total tested - new positive"}, inplace=True)
            gc.collect()
        except Exception as e:
            print(f"⦸ Error: Failed to process PMTCT MSF data: {str(e)}")
            return None

        # -- Step 25: Prepare KP data
        try:
            Pre_KP_MSF_positive = prepare_and_convert_df("KP Prev MSF", MSF_hierarchy, KP_cols)
            if Pre_KP_MSF_positive is None:
                print(f"⦸ Error: Failed to prepare KP Prev MSF data")
                return None
            Pre_KP_MSF_positive["KP_Prev total tested - new positive"] = Pre_KP_MSF_positive[KP_cols].sum(axis=1)
            gc.collect()
        except Exception as e:
            print(f"⦸ Error: Failed to process KP Prev MSF data: {str(e)}")
            return None

        # -- Step 26: Merge all DataFrames sequentially
        try:
            base_cols = ["ReportPeriod", "Cluster", "LGA", "FacilityName"]
            if "Report Rate Facility" not in DHIS2_data:
                print(f"⦸ Error: 'Report Rate Facility' not found in DHIS2_data")
                return None
            Pre_MSF_positives_all = DHIS2_data["Report Rate Facility"][base_cols].copy()
            for df in [
                Pre_HTS_MSF_positive[base_cols + ["HTS total tested - new positive (excluding previously known)"]],
                Pre_AGYW_MSF_positive[base_cols + ["AGYW total tested - new positive"]],
                Pre_PMTCT_MSF_positive[base_cols + ["PMTCT total tested - new positive"]],
                Pre_KP_MSF_positive[base_cols + ["KP_Prev total tested - new positive"]]
            ]:
                Pre_MSF_positives_all = Pre_MSF_positives_all.merge(df, on=base_cols, how="left")
                gc.collect()
            for col in [
                "HTS total tested - new positive (excluding previously known)",
                "AGYW total tested - new positive",
                "PMTCT total tested - new positive",
                "KP_Prev total tested - new positive"
            ]:
                Pre_MSF_positives_all[col] = pd.to_numeric(Pre_MSF_positives_all[col], errors='coerce').fillna(0).astype(int)
            Pre_MSF_positives_all["Total new positive"] = Pre_MSF_positives_all[[
                "HTS total tested - new positive (excluding previously known)",
                "AGYW total tested - new positive",
                "PMTCT total tested - new positive",
                "KP_Prev total tested - new positive"
            ]].sum(axis=1)
            gc.collect()
        except Exception as e:
            print(f"⦸ Error: Failed to merge DataFrames: {str(e)}")
            return None

        print(f"✔️ Configuration, functions, and data loaded successfully")
        return report_archive

    except Exception as e:
        print(f"⦸ Error: Failed to load DHIS2 configuration: {str(e)}")
        return None                         # Return archive

def create_report_zip(zip_name="ANSO MSF Report.zip"):
    """
    Creates a ZIP archive from report_archive and returns a FileLink for Jupyter
    or a BytesIO buffer for web apps.
    Args:
        zip_name (str): Name of the ZIP file
    Returns:
        FileLink or io.BytesIO: Download link for Jupyter or buffer for web
    """
    global report_archive                           # Access report_archive
    try:                                            # Handle ZIP creation
        zip_buffer = io.BytesIO()                   # Create ZIP buffer
        with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:  # Open ZIP
            for file_path, file_content in report_archive.items():  # Iterate archive
                zip_file.writestr(file_path, file_content)  # Write to ZIP
        zip_buffer.seek(0)                          # Reset buffer

        # Save ZIP for FileLink in Jupyter
        with open(zip_name, 'wb') as f:             # Write ZIP to disk
            f.write(zip_buffer.getvalue())          # Save ZIP content
        #print(f"ZIP file saved as {zip_name}")      # Confirm save

        # Return FileLink for Jupyter
        return FileLink(zip_name, result_html_prefix="Download ")  # Return link
    except Exception as e:                          # Catch ZIP errors
        print(f"Error creating ZIP file: {str(e)}")  # Print error
        return None                                 # Return None on error
# End of the function ---------------------------------------------------------------------

In [4]:
## - MSF REPORTING RATE
### - Reporting rate gap for LGAs
# -----------------------------------------------------------------------------------------
# -- Define the main function to process LGA report rate gap
def process_lga_report_rate_gap(display_output=None):
    """
    # -- Function description: Process LGA report rate gap, exporting results as image and Excel files.
    # -- Caches the styled DataFrame and its shape for faster display in subsequent calls.
    # -- Reprocesses if the DataFrame shape changes.
    
    Args:
        display_output (bool, optional): # -- If True, displays the styled DataFrame for LGAs with gaps.
            # -- Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        MSF_report_rate_columns = [  # -- List of human-readable column names for MSF report rates
            'AGYW Monthly Summary Form - Reporting rate',  # Report rate for Adolescent Girls and Young Women
            'ART MSF - Reporting rate', # Report rate for Antiretroviral Therapy
            'Care & Support MSF - Reporting rate', # Report rate for Care & Support
            'HTS Summary Form - Reporting rate', # Report rate for HIV Testing Services
            'NSP Summary Form - Reporting rate', # Report rate for Needle and Syringe Program
            'PMTCT MSF - Reporting rate', # Report rate for Prevention of Mother-to-Child Transmission
            'Prevention Summary Form - Reporting rate' # Report rate for Prevention Summary
        ]  # -- Defines columns to process for report rate gaps
        msf_naming = [  # -- List of DHIS2 data element IDs corresponding to report rate columns
            'Z7E9RxXmwxG.REPORTING_RATE', 
            'VmGwLcfPS2N.REPORTING_RATE',
            'YFnIy7lATQL.REPORTING_RATE',
            'NkuV7xoThHV.REPORTING_RATE',
            'HwfLR3npibF.REPORTING_RATE',
            'vN9rk5ChByM.REPORTING_RATE',
            'oxUN7AXSF8r.REPORTING_RATE'
        ]  # -- Maps to MSF_report_rate_columns for data retrieval
        report_name = "ANSO MSF Report Rate Gap"  # -- Name of the report for file naming and display
        MSF_report_rate_msg = f"No {report_name}"  # -- Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # -- Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # -- Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # -- Combined string for display header

        # -- Step 2: Prepare data
        df_Report_Rate_LGA = prepare_and_convert_df(  # -- Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key='Report Rate LGA',  # -- Specify key to fetch LGA-level report rate data
            hierarchy_columns=MSF_hierarchy,  # -- Use predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=msf_naming  # -- Include specified data element IDs for report rates
        )  # -- Returns processed DataFrame or None if failed
        if df_Report_Rate_LGA is None:  # -- Check if data preparation failed or returned empty
            return  # -- Exit function if no valid data is retrieved
        
        # -- Drop the 'FacilityName' column if it exists
        df_Report_Rate_LGA = df_Report_Rate_LGA.drop(columns='FacilityName')  # -- Remove FacilityName as focus is on LGA-level data

        df_Report_Rate_LGA = df_Report_Rate_LGA.rename(  # -- Rename columns for readability
            columns=dict(zip(msf_naming, MSF_report_rate_columns))  # -- Map DHIS2 IDs to human-readable names
        )  # -- Updates DataFrame with descriptive column names
        
        wrap_column_headers(df_Report_Rate_LGA)  # -- Apply wrapping to column headers (assumed function for formatting)
        MSF_report_rate_columns2 = wrap_column_headers2(MSF_report_rate_columns)  # -- Wrap report rate column names for display

        # -- Step 3: Set export variables
        report_month = df_Report_Rate_LGA['ReportPeriod'].iloc[0]  # -- Extract first report period (e.g., 'Jan-25') for file naming
        report_sheet_name = "All LGAs"  # -- Define name for Excel sheet
        report_image_name = f"{report_month}_{report_name}.png"  # -- Create image file name using report month and name

        # -- Step 4: Check and display cached styled DataFrame
        if display_output:  # -- Check if user requested to display output
            if hasattr(process_lga_report_rate_gap, 'cached_style'):  # -- Check if a cached styled DataFrame exists
                cached_shape = getattr(process_lga_report_rate_gap, 'cached_shape', None)  # -- Retrieve cached DataFrame shape
                current_shape = df_Report_Rate_LGA.shape  # -- Get current DataFrame shape
                if cached_shape == current_shape:  # -- If shapes match, use cached version
                    display = process_lga_report_rate_gap.cached_style  # -- Retrieve cached styled DataFrame
                    print(print_display_name)  # -- Print display name with separators
                    widget_display_df(display)  # -- Display cached styled DataFrame (assumed Jupyter widget function)
                    return  # -- Exit function to avoid reprocessing

        # -- Step 5: Filter for gaps
        df_Report_Rate_LGA_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for rows with report rate gaps
            df=df_Report_Rate_LGA,  # -- Input DataFrame to filter
            msg=MSF_report_rate_msg,  # -- Message to display if no gaps are found
            opNonZero=MSF_report_rate_columns2,  # -- Columns to check for non-zero values
            opNeg=None,  # -- No negative value filter applied
            opPos=None,  # -- No positive value filter applied
            opZero=None,  # -- No zero value filter applied
            opLT100=MSF_report_rate_columns2  # -- Filter for report rates less than 100
        )  # -- Returns filtered DataFrame or None if no gaps

        if df_Report_Rate_LGA_gap is None:  # -- Check if filtering returned no gaps
            if hasattr(process_lga_report_rate_gap, 'cached_style'):  # -- Check if cache exists
                del process_lga_report_rate_gap.cached_style  # -- Clear cached styled DataFrame
            if hasattr(process_lga_report_rate_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_lga_report_rate_gap.cached_shape  # -- Clear cached DataFrame shape
            return  # -- Exit function if no gaps found

        # -- Step 6: Style the DataFrame
        df_Report_Rate_LGA_style = (  # -- Apply styling to filtered DataFrame
            df_Report_Rate_LGA_gap.style  # -- Create style object from filtered DataFrame
            .hide(axis='index')  # -- Hide row index for cleaner display
            .map(outlier_red_report_rate, subset=MSF_report_rate_columns2)  # -- Highlight outliers in red for report rate columns (assumed function)
            .map(outlier_green_report_rate)  # -- Apply green outlier styling (assumed function, applied globally)
        )  # -- Creates styled DataFrame for display and export

        # -- Step 7: Cache styled DataFrame and shape
        process_lga_report_rate_gap.cached_style = df_Report_Rate_LGA_style  # -- Store styled DataFrame in function attribute
        process_lga_report_rate_gap.cached_shape = df_Report_Rate_LGA.shape  # -- Store original DataFrame shape for future checks

        # -- Step 8: Export results
        if not display_output:  # -- Check if export is required (no display requested)
            export_df_to_doc_image_excel(  # -- Export styled DataFrame to image and Excel files
                report_name=report_name,  # -- Pass report name for file naming
                df_style=df_Report_Rate_LGA_style,  # -- Pass styled DataFrame for export
                img_file_name=report_image_name,  # -- Pass image file name
                img_file_path=sub_folder2_image_file_report_rate,  # -- Pass path for image file (assumed defined elsewhere)
                doc_description=None,  # -- No document description provided
                doc_indicators_to_italicize=None,  # -- No indicators to italicize
                doc_indicators_to_underline=None,  # -- No indicators to underline
                xlm_file_path=doc_file_report_rate_xlsx,  # -- Pass path for Excel file (assumed defined elsewhere)
                xlm_sheet_name=report_sheet_name  # -- Pass Excel sheet name
            )  # -- Exports results to specified formats

        # -- Step 9: Optionally display styled DataFrame
        if display_output:  # -- Check if display is requested
            print(print_display_name)  # -- Print display header
            widget_display_df(df_Report_Rate_LGA_style)  # -- Display styled DataFrame using assumed widget function

    except Exception as e:  # -- Catch any unexpected errors during processing
        print(f"⦸ Error processing LGA report rate gap: {str(e)}")  # -- Print error message
        if hasattr(process_lga_report_rate_gap, 'cached_style'):  # -- Check if cache exists
            del process_lga_report_rate_gap.cached_style  # -- Clear cached styled DataFrame
        if hasattr(process_lga_report_rate_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_lga_report_rate_gap.cached_shape  # -- Clear cached DataFrame shape
# End of the function ---------------------------------------------------------------------

### - Reporting rate gap for facilities
# ----------------------------------------------------------------------------------------- 
# Define the main function to process facility report rate gap
def process_facility_report_rate_gap(display_output=None):
    """
    # Function description: Process facility report rate gaps for each LGA, exporting results as images and Excel files.
    # Caches styled DataFrames for each LGA and displays them on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): # If True, displays the styled DataFrame images for each LGA with gaps.
            # Defaults to None (will treat as False unless explicitly True).
    """
    try:
        # Step 1: Initialize constants
        MSF_report_rate_columns = [  # List of human-readable column names for MSF report rates
            'AGYW Monthly Summary Form - Reporting rate',  # Report rate for Adolescent Girls and Young Women
            'ART MSF - Reporting rate',  # Report rate for Antiretroviral Therapy
            'Care & Support MSF - Reporting rate',  # Report rate for Care & Support
            'HTS Summary Form - Reporting rate',  # Report rate for HIV Testing Services
            'NSP Summary Form - Reporting rate',  # Report rate for Needle and Syringe Program
            'PMTCT MSF - Reporting rate',  # Report rate for Prevention of Mother-to-Child Transmission
            'Prevention Summary Form - Reporting rate'  # Report rate for Prevention Summary
        ]  # Defines columns to process for report rate gaps
        msf_naming = [  # List of DHIS2 data element IDs corresponding to report rate columns
            'Z7E9RxXmwxG.REPORTING_RATE', 
            'VmGwLcfPS2N.REPORTING_RATE',
            'YFnIy7lATQL.REPORTING_RATE',
            'NkuV7xoThHV.REPORTING_RATE',
            'HwfLR3npibF.REPORTING_RATE',
            'vN9rk5ChByM.REPORTING_RATE',
            'oxUN7AXSF8r.REPORTING_RATE'
        ]  # Maps to MSF_report_rate_columns for data retrieval

        # Step 2: Prepare data
        df_Report_Rate_Facility = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="Report Rate Facility",  # Specify key to fetch facility-level report rate data
            hierarchy_columns=MSF_hierarchy,  # Use predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=msf_naming  # Include specified data element IDs for report rates
        )  # Returns processed DataFrame or None if failed
        if df_Report_Rate_Facility is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data is retrieved
        
        df_Report_Rate_Facility = df_Report_Rate_Facility.rename(  # Rename columns for readability
            columns=dict(zip(msf_naming, MSF_report_rate_columns))  # Map DHIS2 IDs to human-readable names
        )  # Updates DataFrame with descriptive column names
        
        wrap_column_headers(df_Report_Rate_Facility)  # Apply wrapping to column headers (assumed function for formatting)
        MSF_report_rate_columns2 = wrap_column_headers2(MSF_report_rate_columns)  # Wrap report rate column names for display

        # Step 3: Check and display cached styled DataFrames
        if display_output:  # Check if user requested to display output
            if hasattr(process_facility_report_rate_gap, 'cached_styles'):  # Check if cached styled DataFrames exist
                cached_shape = getattr(process_facility_report_rate_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_Report_Rate_Facility.shape  # Get current DataFrame shape
                if cached_shape == current_shape:  # If shapes match, use cached versions
                    for lga, style in process_facility_report_rate_gap.cached_styles.items():  # Iterate over cached styles per LGA
                        LGA_name = lga.replace("an ", "").replace(" ", "_")  # Replace 'an' in LGA name with nothing for cleaner display
                        report_name = f"{LGA_name} Facilities Report Rate Gap"  # Define report name for current LGA
                        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
                        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length
                        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header
                        print(print_display_name)  # Print display header
                        widget_display_df(style)  # Display cached styled DataFrame for LGA (assumed Jupyter display function)
                    return  # Exit function to avoid reprocessing

        # Step 4: Initialize cache
        if not hasattr(process_facility_report_rate_gap, 'cached_styles'):  # Check if cache attribute exists
            process_facility_report_rate_gap.cached_styles = {}  # Initialize dictionary to store styled DataFrames per LGA

        # Step 5: Identify unique LGAs
        lga_list = pd.Series(df_Report_Rate_Facility['LGA'].unique())  # Extract unique LGA names as a pandas Series

        # Step 6: Process each LGA for report rate gaps
        for current_lga in lga_list:  # Iterate over each unique LGA
            # Step 6.1: Filter DataFrame for current LGA
            lga_filtered = df_Report_Rate_Facility[df_Report_Rate_Facility['LGA'] == current_lga]  # Filter DataFrame to current LGA

            LGA_name = current_lga.replace("an ", "").replace(" ", "_")  # Replace 'an' in LGA name with nothing for cleaner display
            report_name = f"{LGA_name} Facilities Report Rate Gap"  # Define report name for current LGA
            MSF_report_rate_msg = f"No {report_name}"  # Define message for no gaps in current LGA
            display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
            display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length
            print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

            # Step 6.2: Apply filtering for gaps
            lga_filtered_gap = filter_gap_and_check_empty_df(  # Filter LGA-specific subset for report rate gaps
                df=lga_filtered,  # Input LGA-filtered DataFrame
                msg=MSF_report_rate_msg,  # Message to display if no gaps are found
                opNonZero=MSF_report_rate_columns2,  # Columns to check for non-zero values
                opNeg=None,  # No negative value filter applied
                opPos=None,  # No positive value filter applied
                opZero=None,  # No zero value filter applied
                opLT100=MSF_report_rate_columns2  # Filter for report rates less than 100
            )  # Returns filtered DataFrame or None if no gaps

            if lga_filtered_gap is None:  # Check if no gaps found for this LGA
                if current_lga in process_facility_report_rate_gap.cached_styles:  # Check if LGA is in cache
                    del process_facility_report_rate_gap.cached_styles[current_lga]  # Remove LGA from cache
                continue  # Skip to next LGA

            # Step 6.3: Style the DataFrame
            lga_filtered_style = (  # Apply styling to filtered LGA DataFrame
                lga_filtered_gap.style  # Create style object from filtered DataFrame
                .hide(axis='index')  # Hide row index for cleaner display
                .map(outlier_red_report_rate, subset=MSF_report_rate_columns2)  # Highlight outliers in red for report rate columns (assumed function)
                .map(outlier_green_report_rate)  # Apply green outlier styling (assumed function, applied globally)
            )  # Creates styled DataFrame for display and export

            # Step 6.4: Cache the styled DataFrame for this LGA
            process_facility_report_rate_gap.cached_styles[current_lga] = lga_filtered_style  # Store styled DataFrame in cache

            # Step 6.5: Define export variables
            report_image_name = f"{LGA_name}.png"  # Define image file name using LGA name
            report_sheet_name = f"{LGA_name}"  # Define Excel sheet name using LGA name

            # Step 6.6: Export results
            if not display_output:  # Check if user requested to export results
                export_df_to_doc_image_excel(  # Export LGA-specific DataFrame to image and Excel
                    report_name=report_name,  # Pass report name for file naming
                    df_style=lga_filtered_style,  # Pass styled LGA DataFrame for export
                    img_file_name=report_image_name,  # Pass image file name
                    img_file_path=sub_folder2_image_file_report_rate,  # Pass path for image file (assumed defined elsewhere)
                    doc_description=None,  # No document description provided
                    doc_indicators_to_italicize=None,  # No indicators to italicize
                    doc_indicators_to_underline=None,  # No indicators to underline
                    xlm_file_path=doc_file_report_rate_xlsx,  # Pass path for Excel file (assumed defined elsewhere)
                    xlm_sheet_name=report_sheet_name  # Pass Excel sheet name
                )  # Exports results to specified formats
    
            # Step 6.7: Optionally display styled DataFrame
            if display_output:  # Check if display is requested
                print(print_display_name)  # Print display header
                widget_display_df(lga_filtered_style)  # Display styled DataFrame using assumed widget function 
                print()   

        # Step 7: Cache overall unfiltered DataFrame shape
        process_facility_report_rate_gap.cached_shape = df_Report_Rate_Facility.shape  # Store original DataFrame shape for future checks

    except Exception as e:  # Catch any unexpected errors during processing
        print(f"⦸ Error processing facility report rate gaps: {str(e)}")  # Print error message
        if hasattr(process_facility_report_rate_gap, 'cached_styles'):  # Check if cache exists
            process_facility_report_rate_gap.cached_styles.clear()  # Clear cached styles dictionary
        if hasattr(process_facility_report_rate_gap, 'cached_shape'):  # Check if cached shape exists
            del process_facility_report_rate_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit function on error
# End of the function ---------------------------------------------------------------------

## - AGYW MSF
### - AGYW HTS gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process AGYW HTS gap
def process_AGYW_HTS_gap(display_output=None):
    """
    Process AGYW HTS gap for each LGA, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the DataFrame for LGAs with gaps.
            Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        AGYW_HTS_columns = [  # List of human-readable column names for AGYW HTS metrics in desired order
            "Number of AGYW reached with HIV Prevention Program - defined package of service during the reporting period (Community)",  # Community-based prevention reach
            "Number of AGYW reached with HIV Prevention Program - defined package of service during the reporting (Walk-In)",  # Walk-in prevention reach
            "Number of AGYW that received an HIV test during the reporting period and know their status (Community)",  # Community-based testing
            "Number of AGYW that received an HIV test during the reporting period and know their status (Walk-In)"  # Walk-in testing
        ]  # Defines columns to process for HTS gaps
        name = "AGYW HTS Gap"  # General name for the report
        AGYW_HTS_gap_columns = ['AGYW HTS gap']  # Name for the calculated gap column
        report_name = f"{name}3"  # Report name with suffix for uniqueness
        AGYW_HTS_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_AGYW_HTS = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key='AGYW MSF',  # Specify key to fetch AGYW MSF data
            hierarchy_columns=MSF_hierarchy,  # Use predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=AGYW_HTS_columns  # Include specified AGYW HTS columns
        )  # Returns processed DataFrame or None if failed
        if df_AGYW_HTS is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data is retrieved

        wrap_column_headers(df_AGYW_HTS)  # Apply wrapping to DataFrame column headers (assumed function for formatting)
        AGYW_HTS_columns2 = wrap_column_headers2(AGYW_HTS_columns)  # Wrap AGYW HTS column names for display

        # -- Step 3: Check and display cached styled DataFrame
        if display_output:  # Check if user requested to display output
            if hasattr(process_AGYW_HTS_gap, 'cached_style'):  # Check if a cached styled DataFrame exists
                cached_shape = getattr(process_AGYW_HTS_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_AGYW_HTS.shape  # Get current DataFrame shape
                if cached_shape == current_shape:  # If shapes match, use cached version
                    print(print_display_name)  # Print display name with separators
                    display = process_AGYW_HTS_gap.cached_style  # Retrieve cached styled DataFrame
                    widget_display_df(display)  # Display cached styled DataFrame (assumed Jupyter widget function)
                    return  # Exit function to avoid reprocessing

        # -- Step 4: Calculate derived metrics
        # -- Step 4.1: Total AGYW reached with HIV Prevention
        df_AGYW_HTS["Total AGYW reached with HIV Prevention"] = (  # Calculate total reached
            df_AGYW_HTS[AGYW_HTS_columns2[0]] + df_AGYW_HTS[AGYW_HTS_columns2[1]]  # Sum Community and Walk-In prevention reach
        )  # Adds new column with total prevention reach
        # -- Step 4.2: Total AGYW received HIV test & know status
        df_AGYW_HTS["Total AGYW received HIV test & know status"] = (  # Calculate total tested
            df_AGYW_HTS[AGYW_HTS_columns2[2]] + df_AGYW_HTS[AGYW_HTS_columns2[3]]  # Sum Community and Walk-In testing
        )  # Adds new column with total tested and status-known
        # -- Step 4.3: AGYW HTS gap
        df_AGYW_HTS[AGYW_HTS_gap_columns[0]] = np.where(  # Calculate gap (assumes numpy as np)
            df_AGYW_HTS["Total AGYW received HIV test & know status"] > df_AGYW_HTS["Total AGYW reached with HIV Prevention"],  # Condition: tested exceeds reached
            df_AGYW_HTS["Total AGYW received HIV test & know status"] - df_AGYW_HTS["Total AGYW reached with HIV Prevention"],  # Calculate positive gap
            0  # Set to 0 if no gap (tested <= reached)
        )  # Adds gap column with calculated values

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_AGYW_HTS)  # Reapply wrapping to DataFrame headers after adding new columns
        gap_columns_wrap = wrap_column_headers2(AGYW_HTS_gap_columns)  # Wrap gap column name for display

        # -- Step 6: Filter and validate gaps
        df_AGYW_HTS_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_AGYW_HTS,  # Input DataFrame with calculated gaps
            msg=AGYW_HTS_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero values in gap column
            opNeg=None,  # No negative value filter applied
            opPos=None,  # No positive value filter applied
            opZero=None,  # No zero value filter applied
            opLT100=None  # No less-than-100 filter applied
        )  # Returns filtered DataFrame or None if no gaps
        if df_AGYW_HTS_gap is None:  # Check if filtering returned no gaps
            if hasattr(process_AGYW_HTS_gap, 'cached_style'):  # Check if cache exists
                del process_AGYW_HTS_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_AGYW_HTS_gap, 'cached_shape'):  # Check if cached shape exists
                del process_AGYW_HTS_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps found

        # -- Step 7: Style the DataFrame
        df_AGYW_HTS_gap_style = (  # Apply styling to filtered DataFrame
            df_AGYW_HTS_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight outliers in gap column in red (assumed function)
        )  # Creates styled DataFrame for display and export

        # -- Step 8: Cache styled DataFrame and shape
        process_AGYW_HTS_gap.cached_style = df_AGYW_HTS_gap_style  # Store styled DataFrame in function attribute
        process_AGYW_HTS_gap.cached_shape = df_AGYW_HTS.shape  # Store original DataFrame shape for future checks

        # -- Step 9: Define export variables
        report_month = df_AGYW_HTS['ReportPeriod'].iloc[0]  # Extract first report period (e.g., 'Jan-25') for file naming
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name using report month and name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined elsewhere)
        report_sheet_name = report_name  # Define Excel sheet name using report name

        # -- Step 10: Create description for Word document
        if (df_AGYW_HTS[AGYW_HTS_gap_columns[0]] != 0).any():  # Check if any non-zero gaps exist
            report_description = (  # Define description for Word document
                f"Report Name: {gap_columns_wrap[0]}"  # Include gap column name
                f"\n{AGYW_HTS_columns[2]}\nplus {AGYW_HTS_columns[3]}"  # Describe testing metrics
                f"\nshould not be greater than"  # Explain expected relationship
                f"\n{AGYW_HTS_columns[0]}\nplus {AGYW_HTS_columns[1]}"  # Describe prevention metrics
            )  # Creates multi-line description for clarity

        # -- Step 11: Export results
        if not display:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word formats
                report_name=report_name,  # Pass report name for file naming
                df_style=df_AGYW_HTS_gap_style,  # Pass styled DataFrame for export
                img_file_name=report_image_name,  # Pass image file name
                img_file_path=report_image_path,  # Pass image file path
                doc_file_path=doc_file_msf_outlier_docx,  # Pass Word document path (assumed defined elsewhere)
                doc_description=report_description,  # Pass Word document description
                doc_indicators_to_italicize=AGYW_HTS_columns,  # Italicize AGYW HTS column names in Word doc
                doc_indicators_to_underline=AGYW_HTS_gap_columns,  # Underline gap column name in Word doc
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Pass Excel file path (assumed defined elsewhere)
                xlm_sheet_name=report_sheet_name  # Pass Excel sheet name
            )  # Exports results to specified formats

        # -- Step 12: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_AGYW_HTS_gap_style)  # Display styled DataFrame using assumed widget function

    except Exception as e:  # Catch any unexpected errors during processing
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error message with report name
        if hasattr(process_AGYW_HTS_gap, 'cached_style'):  # Check if cache exists
            del process_AGYW_HTS_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_AGYW_HTS_gap, 'cached_shape'):  # Check if cached shape exists
            del process_AGYW_HTS_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit function on error
# End of the function ---------------------------------------------------------------------

### - AGYW Positive gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process AGYW Positive gap
def process_AGYW_Positive_gap(display_output=None):
    """
    Process AGYW Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the DataFrame for gaps.
            Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        AGYW_Positive_columns = [  # List of human-readable column names for AGYW Positive metrics in desired order
            "Number of AGYW that received an HIV test during the reporting period and know their status (Community)",  # Community-based testing
            "Number of AGYW that received an HIV test during the reporting period and know their status (Walk-In)",  # Walk-in testing
            "Number of AGYW who tested HIV Positive during the reporting period (Community)",  # Community-based positive cases
            "Number of AGYW who tested HIV Positive during the reporting period (Walk-In)"  # Walk-in positive cases
        ]  # Defines columns to process for positive gaps
        name = "AGYW Positive Gap"  # General name for the report
        AGYW_Positive_gap_columns = ["AGYW tested positive gap"]  # Name for the calculated gap column
        report_name = f"{name}4"  # Report name with suffix for uniqueness
        AGYW_Positive_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_AGYW_Positive = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key='AGYW MSF',  # Specify key to fetch AGYW MSF data
            hierarchy_columns=MSF_hierarchy,  # Use predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=AGYW_Positive_columns  # Include specified AGYW Positive columns
        )  # Returns processed DataFrame or None if failed
        if df_AGYW_Positive is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data is retrieved

        wrap_column_headers(df_AGYW_Positive)  # Apply wrapping to DataFrame column headers (assumed function for formatting)
        AGYW_Positive_columns2 = wrap_column_headers2(AGYW_Positive_columns)  # Wrap AGYW Positive column names for display

        # -- Step 3: Check and display cached styled DataFrame
        if display_output:  # Check if user requested to display output
            if hasattr(process_AGYW_Positive_gap, 'cached_style'):  # Check if a cached styled DataFrame exists
                cached_shape = getattr(process_AGYW_Positive_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_AGYW_Positive.shape  # Get current DataFrame shape
                if cached_shape == current_shape:  # If shapes match, use cached version
                    display = process_AGYW_Positive_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached styled DataFrame (assumed Jupyter widget function)
                    return  # Exit function to avoid reprocessing

        # -- Step 4: Calculate derived metrics
        # -- Step 4.1: Total AGYW tested
        df_AGYW_Positive["Total AGYW tested"] = (  # Calculate total tested
            df_AGYW_Positive[AGYW_Positive_columns2[0]] +  # Add Community tested
            df_AGYW_Positive[AGYW_Positive_columns2[1]]  # Add Walk-In tested
        )  # Adds new column with total tested
        # -- Step 4.2: Total AGYW tested positive
        df_AGYW_Positive["Total AGYW tested positive"] = (  # Calculate total tested positive
            df_AGYW_Positive[AGYW_Positive_columns2[2]] +  # Add Community positive
            df_AGYW_Positive[AGYW_Positive_columns2[3]]  # Add Walk-In positive
        )  # Adds new column with total positive cases
        # -- Step 4.3: AGYW tested Positive gap
        df_AGYW_Positive[AGYW_Positive_gap_columns[0]] = np.where(  # Calculate gap (assumes numpy as np)
            df_AGYW_Positive["Total AGYW tested positive"] > df_AGYW_Positive["Total AGYW tested"],  # Condition: positive cases exceed tested
            df_AGYW_Positive["Total AGYW tested positive"] - df_AGYW_Positive["Total AGYW tested"],  # Calculate positive gap
            0  # Set to 0 if no gap (positive <= tested)
        )  # Adds gap column with calculated values

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_AGYW_Positive)  # Reapply wrapping to DataFrame headers after adding new columns
        gap_columns_wrap = wrap_column_headers2(AGYW_Positive_gap_columns)  # Wrap gap column name for display

        # -- Step 6: Filter and validate gaps
        df_AGYW_Positive_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_AGYW_Positive,  # Input DataFrame with calculated gaps
            msg=AGYW_Positive_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero values in gap column
            opNeg=None,  # No negative value filter applied
            opPos=None,  # No positive value filter applied
            opZero=None,  # No zero value filter applied
            opLT100=None  # No less-than-100 filter applied
        )  # Returns filtered DataFrame or None if no gaps
        
        if df_AGYW_Positive_gap is None:  # Check if filtering returned no gaps
            if hasattr(process_AGYW_Positive_gap, 'cached_style'):  # Check if cache exists
                del process_AGYW_Positive_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_AGYW_Positive_gap, 'cached_shape'):  # Check if cached shape exists
                del process_AGYW_Positive_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps found

        # -- Step 7: Style the DataFrame
        df_AGYW_Positive_gap_style = (  # Apply styling to filtered DataFrame
            df_AGYW_Positive_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight outliers in gap column in red (assumed function)
        )  # Creates styled DataFrame for display and export

        # -- Step 8: Cache styled DataFrame and shape
        process_AGYW_Positive_gap.cached_style = df_AGYW_Positive_gap_style  # Store styled DataFrame in function attribute
        process_AGYW_Positive_gap.cached_shape = df_AGYW_Positive.shape  # Store original DataFrame shape for future checks

        # -- Step 9: Define export variables
        report_month = df_AGYW_Positive_gap['ReportPeriod'].iloc[0]  # Extract first report period from filtered DataFrame (e.g., 'Jan-25')
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name using report month and name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined elsewhere)
        report_sheet_name = report_name  # Define Excel sheet name using report name

        # -- Step 10: Create description for Word document
        if (df_AGYW_Positive_gap[gap_columns_wrap[0]] != 0).any():  # Check if any non-zero gaps exist
            report_description = (  # Define description for Word document
                f"Report Name: {gap_columns_wrap[0]}"  # Include gap column name
                f"\n{AGYW_Positive_columns[2]}\nplus {AGYW_Positive_columns[3]}"  # Describe positive case metrics
                f"\nshould not be greater than"  # Explain expected relationship
                f"\n{AGYW_Positive_columns[0]}\nplus {AGYW_Positive_columns[1]}"  # Describe tested metrics
            )  # Creates multi-line description for clarity

        # -- Step 11: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word formats
                report_name=report_name,  # Pass report name for file naming
                df_style=df_AGYW_Positive_gap_style,  # Pass styled DataFrame for export
                img_file_name=report_image_name,  # Pass image file name
                img_file_path=report_image_path,  # Pass image file path
                doc_file_path=doc_file_msf_outlier_docx,  # Pass Word document path (assumed defined elsewhere)
                doc_description=report_description,  # Pass Word document description
                doc_indicators_to_italicize=AGYW_Positive_columns,  # Italicize AGYW Positive column names in Word doc
                doc_indicators_to_underline=AGYW_Positive_gap_columns,  # Underline gap column name in Word doc
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Pass Excel file path (assumed defined elsewhere)
                xlm_sheet_name=report_sheet_name  # Pass Excel sheet name
            )  # Exports results to specified formats

        # -- Step 12: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_AGYW_Positive_gap_style)  # Display styled DataFrame using assumed widget function

    except Exception as e:  # Catch any unexpected errors during processing
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error message with report name
        if hasattr(process_AGYW_Positive_gap, 'cached_style'):  # Check if cache exists
            del process_AGYW_Positive_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_AGYW_Positive_gap, 'cached_shape'):  # Check if cached shape exists
            del process_AGYW_Positive_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit function on error
# End of the function ---------------------------------------------------------------------

### - AGYW Positive Linkage gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process AGYW Positive Linkage gap
def process_AGYW_Positive_Linkage_gap(display_output=None):
    """
    Process AGYW Positive Linkage gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        AGYW_Positive_Linkage_columns = [  # List of column names for AGYW Positive Linkage metrics
            "Number of AGYW who tested HIV Positive during the reporting period (Community)",  # Community-based positive cases
            "Number of AGYW who tested HIV Positive during the reporting period (Walk-In)",  # Walk-In positive cases
            "Total number of AGYW who tested HIV Positive and are successfully linked to treatment during the reporting period (Community & Walk-In)",  # Total linked to treatment
            "Linked/Referred for treatment to GF supported site (subset of 4)",  # Linked to GF-supported sites
            "Linked/Referred for treatment to non-GF supported site (subset of 4)",  # Linked to non-GF-supported sites
            "Number of AGYW newly started on ART during the reporting period"  # Newly started on ART
        ]  # Defines columns for linkage gap analysis
        name = "AGYW Positive Linkage Gap"  # Base name for the report
        AGYW_Positive_Linkage_gap_columns = [  # Names for calculated gap columns
            "AGYW positive linked to treatment gap",  # Gap for linkage to treatment
            "AGYW positive linkage to GF/non-GF supported site gap",  # Gap for GF/non-GF site linkage
            "AGYW newly started on ART gap"  # Gap for ART initiation
        ]  # Defines gap column names
        report_name = f"{name}5"  # Report name with suffix for uniqueness
        AGYW_Positive_Linkage_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_AGYW_Positive_Linkage = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key='AGYW MSF',  # Key to fetch AGYW MSF data
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=AGYW_Positive_Linkage_columns  # Include AGYW Positive Linkage columns
        )  # Returns processed DataFrame or None if failed
        if df_AGYW_Positive_Linkage is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        wrap_column_headers(df_AGYW_Positive_Linkage)  # Format DataFrame column headers (assumed function)
        AGYW_Positive_Linkage_columns2 = wrap_column_headers2(AGYW_Positive_Linkage_columns)  # Wrap column names for display

        # -- Step 3: Calculate derived metrics
        # -- Step 3.1: Total AGYW Tested Positive
        df_AGYW_Positive_Linkage["Total AGYW Tested Positive"] = (  # Calculate total tested positive
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[0]] +  # Add Community positive cases
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[1]]  # Add Walk-In positive cases
        )  # Adds column for total positive cases
        # -- Step 3.2: AGYW positive linked to treatment gap
        df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_gap_columns[0]] = np.where(  # Calculate linkage gap (requires numpy as np)
            df_AGYW_Positive_Linkage["Total AGYW Tested Positive"] != df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[2]],  # Check if total positive differs from linked
            df_AGYW_Positive_Linkage["Total AGYW Tested Positive"] - df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[2]],  # Compute gap (positive - linked)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for treatment linkage
        # -- Step 3.3: AGYW positive linkage to GF/non-GF supported site gap
        total_linked_to_sites = (  # Calculate total linked to GF and non-GF sites
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[3]] +  # Add GF-supported site linkages
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[4]]  # Add non-GF-supported site linkages
        )  # Temporary variable for total site linkages
        df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_gap_columns[1]] = np.where(  # Calculate site linkage gap
            total_linked_to_sites != df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[2]],  # Check if site linkages differ from total linked
            total_linked_to_sites - df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[2]],  # Compute gap (site linkages - total linked)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for GF/non-GF site linkage
        # -- Step 3.4: AGYW newly started on ART gap
        df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_gap_columns[2]] = np.where(  # Calculate ART initiation gap
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[2]] != df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[5]],  # Check if total linked differs from ART started
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[2]] - df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[5]],  # Compute gap (linked - ART started)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for ART initiation

        # -- Step 4: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_AGYW_Positive_Linkage_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_AGYW_Positive_Linkage.shape  # Get current DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_AGYW_Positive_Linkage_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                    return  # Exit to avoid reprocessing

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_AGYW_Positive_Linkage)  # Reapply header wrapping after adding columns
        gap_columns_wrap = wrap_column_headers2(AGYW_Positive_Linkage_gap_columns)  # Wrap gap column names

        # -- Step 6: Filter and validate gaps
        df_AGYW_Positive_Linkage_gap = filter_gap_and_check_empty_df(  # Filter for rows with non-zero gaps
            df=df_AGYW_Positive_Linkage,  # Input DataFrame with gaps
            msg=AGYW_Positive_Linkage_msg,  # Message if no gaps found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_AGYW_Positive_Linkage_gap is None:  # Check if no gaps were found
            if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_style'):  # Check for cached style
                del process_AGYW_Positive_Linkage_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_shape'):  # Check for cached shape
                del process_AGYW_Positive_Linkage_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the DataFrame
        df_AGYW_Positive_Linkage_gap_style = (  # Apply styling to filtered DataFrame
            df_AGYW_Positive_Linkage_gap.style  # Create style object
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight gap column outliers in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_AGYW_Positive_Linkage_gap.cached_style = df_AGYW_Positive_Linkage_gap_style  # Store styled DataFrame
        process_AGYW_Positive_Linkage_gap.cached_shape = df_AGYW_Positive_Linkage.shape  # Store original DataFrame shape

        # -- Step 9: Define export variables
        report_month = df_AGYW_Positive_Linkage_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Set Excel sheet name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # List to collect Word document descriptions
        # -- Step 10.1: Description for AGYW Positive Linkage gap
        if (df_AGYW_Positive_Linkage_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero linkage gap
            report_description.append(  # Add linkage gap description
                f"Report Name: {AGYW_Positive_Linkage_gap_columns[0]}"  # Gap column name
                f"\n{AGYW_Positive_Linkage_columns[0]}\nplus {AGYW_Positive_Linkage_columns[1]}"  # Positive case metrics
                f"\nshould be equal to {AGYW_Positive_Linkage_columns2[2]}"  # Expected equality
                f"\nNote: Where this AGYW linkage gap is true, please ignore the outlier."  # Outlier note
            )  # Append linkage gap description
        # -- Step 10.2: Description for AGYW Positive Linkage to GF/non-GF supported site gap
        if (df_AGYW_Positive_Linkage_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero site linkage gap
            report_description.append(  # Add site linkage gap description
                f"Report Name: {AGYW_Positive_Linkage_gap_columns[1]}"  # Gap column name
                f"\n{AGYW_Positive_Linkage_columns[3]}\nplus {AGYW_Positive_Linkage_columns[4]}"  # Site linkage metrics
                f"\nshould be equal to {AGYW_Positive_Linkage_columns2[2]}"  # Expected equality
            )  # Append site linkage gap description
        # -- Step 10.3: Description for AGYW newly started on ART gap
        if (df_AGYW_Positive_Linkage_gap[gap_columns_wrap[2]] != 0).any():  # Check for non-zero ART gap
            report_description.append(  # Add ART gap description
                f"Report Name: {AGYW_Positive_Linkage_gap_columns[2]}"  # Gap column name
                f"\n{AGYW_Positive_Linkage_columns[5]}"  # ART initiation metric
                f"\nshould be equal to {AGYW_Positive_Linkage_columns[2]}"  # Expected equality
            )  # Append ART gap description
        # -- Step 10.4: Join all descriptions
        report_description = "\n\n".join(report_description)  # Combine descriptions with double newlines

        # -- Step 11: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_AGYW_Positive_Linkage_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_file_path=doc_file_msf_outlier_docx,  # Word document path (assumed defined)
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=AGYW_Positive_Linkage_columns,  # Italicize linkage columns
                doc_indicators_to_underline=AGYW_Positive_Linkage_gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)   # Print display name with separators
            widget_display_df(df_AGYW_Positive_Linkage_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_style'):  # Check for cached style
            del process_AGYW_Positive_Linkage_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_shape'):  # Check for cached shape
            del process_AGYW_Positive_Linkage_gap.cached_shape  # Clear cached shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - AGYW TB Screening gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process AGYW TB Screening gap
def process_AGYW_TB_Screening_gap(display_output=None):
    """
    Process AGYW TB Screening gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        AGYW_TB_Screening_columns = [  # List of column names for AGYW TB Screening metrics
            "Number of AGYW newly started on ART during the reporting period",  # AGYW newly started on ART
            "Number of AGYW screened for TB amongst those newly started on ART during the reporting period"  # AGYW screened for TB
        ]  # Defines columns for TB screening gap analysis
        name = "AGYW TB Screening Gap"  # Base name for the report
        AGYW_TB_Screening_gap_columns = ["AGYW TB screening gap"]  # Name for the calculated gap column
        report_name = f"{name}6"  # Report name with suffix for uniqueness
        AGYW_TB_Screening_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_AGYW_TB_Screening = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key='AGYW MSF',  # Key to fetch AGYW MSF data
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=AGYW_TB_Screening_columns  # Include AGYW TB Screening columns
        )  # Returns processed DataFrame or None if failed
        if df_AGYW_TB_Screening is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        wrap_column_headers(df_AGYW_TB_Screening)  # Format DataFrame column headers (assumed function)
        AGYW_TB_Screening_columns2 = wrap_column_headers2(AGYW_TB_Screening_columns)  # Wrap column names for display

        # -- Step 3: Calculate derived metrics
        # -- Step 3.1: AGYW TB Screening gap
        df_AGYW_TB_Screening[AGYW_TB_Screening_gap_columns[0]] = np.where(  # Calculate TB screening gap (requires numpy as np)
            df_AGYW_TB_Screening[AGYW_TB_Screening_columns2[1]] != df_AGYW_TB_Screening[AGYW_TB_Screening_columns2[0]],  # Check if screened differs from started on ART
            df_AGYW_TB_Screening[AGYW_TB_Screening_columns2[1]] - df_AGYW_TB_Screening[AGYW_TB_Screening_columns2[0]],  # Compute gap (screened - started)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for TB screening

        # -- Step 4: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_AGYW_TB_Screening_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_AGYW_TB_Screening_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_AGYW_TB_Screening.shape  # Get current DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_AGYW_TB_Screening_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                    return  # Exit to avoid reprocessing

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_AGYW_TB_Screening)  # Reapply header wrapping after adding gap column
        gap_columns_wrap = wrap_column_headers2(AGYW_TB_Screening_gap_columns)  # Wrap gap column name

        # -- Step 6: Filter and validate gaps
        df_AGYW_TB_Screening_gap = filter_gap_and_check_empty_df(  # Filter for rows with non-zero gaps
            df=df_AGYW_TB_Screening,  # Input DataFrame with gap
            msg=AGYW_TB_Screening_msg,  # Message if no gaps found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_AGYW_TB_Screening_gap is None:  # Check if no gaps were found
            if hasattr(process_AGYW_TB_Screening_gap, 'cached_style'):  # Check for cached style
                del process_AGYW_TB_Screening_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_AGYW_TB_Screening_gap, 'cached_shape'):  # Check for cached shape
                del process_AGYW_TB_Screening_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the DataFrame
        df_AGYW_TB_Screening_gap_style = (  # Apply styling to filtered DataFrame
            df_AGYW_TB_Screening_gap.style  # Create style object
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight gap column outliers in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_AGYW_TB_Screening_gap.cached_style = df_AGYW_TB_Screening_gap_style  # Store styled DataFrame
        process_AGYW_TB_Screening_gap.cached_shape = df_AGYW_TB_Screening.shape  # Store original DataFrame shape

        # -- Step 9: Define export variables
        report_month = df_AGYW_TB_Screening_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Set Excel sheet name

        # -- Step 10: Create descriptions for Word document
        # -- Step 10.1: Add description for AGYW TB Screening gap
        if (df_AGYW_TB_Screening_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero TB screening gap
            report_description = (  # Create description for TB screening gap
                f"Report Name: {AGYW_TB_Screening_gap_columns[0]}"  # Gap column name
                f"\n{AGYW_TB_Screening_columns[1]}\nshould be equal to {AGYW_TB_Screening_columns[0]}"  # Expected equality
            )  # Define Word document description

        # -- Step 11: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_AGYW_TB_Screening_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_file_path=doc_file_msf_outlier_docx,  # Word document path (assumed defined)
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=AGYW_TB_Screening_columns,  # Italicize TB screening columns
                doc_indicators_to_underline=AGYW_TB_Screening_gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_AGYW_TB_Screening_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_AGYW_TB_Screening_gap, 'cached_style'):  # Check for cached style
            del process_AGYW_TB_Screening_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_AGYW_TB_Screening_gap, 'cached_shape'):  # Check for cached shape
            del process_AGYW_TB_Screening_gap.cached_shape  # Clear cached shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

## - ART MSF
### - ART Linkage gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process ART positive and enrollment gap
def process_ART_PosEnrolment_gap(display_output=None):
    """
    Process ART Positive and Enrolment gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        ART_PosEnrolment_columns = [  # List of column names for ART positive and enrollment metrics
            "ART 1: Number of HIV positive persons newly enrolled in clinical care during the month",  # Newly enrolled in care
            "ART 2: Number of people living with HIV newly started on ART during the month (excludes ART transfer-in)"  # Newly started on ART
        ]  # Defines columns for ART gap analysis
        ART_PosEnrolment_columns_desrpt = ART_PosEnrolment_columns + ["Total Tested Positive"]  # Extended list including total positive for descriptions
        name = "ART Positive-Enrolment Gap"  # Base name for the report
        ART_PosEnrolment_gap_columns = ["ART enrolment gap", "ART linkage gap"]  # Names for calculated gap columns
        report_name = f"{name}7"  # Report name with suffix for uniqueness

        # -- Step 2: Prepare data
        df_ART_PosEnrolment = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key='ART MSF',  # Key to fetch ART MSF data
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=ART_PosEnrolment_columns  # Include ART positive and enrollment columns
        )  # Returns processed DataFrame or None if failed

        # -- Step 3: Merge with external DataFrame
        df_ART_PosEnrolment = Pre_MSF_positives_all.merge(  # Merge with external positives data (assumed defined elsewhere)
            df_ART_PosEnrolment,  # Target DataFrame for merging
            on=MSF_hierarchy,  # Columns to merge on
            how="left"  # Keep all rows from df_ART_PosEnrolment
        )  # Merges to include total positive data
        df_ART_PosEnrolment = df_ART_PosEnrolment.fillna(0)  # Fill NaN values with 0
        float_columns = df_ART_PosEnrolment.select_dtypes(include=['float64', 'float32']).columns  # Identify float-type columns
        for col in float_columns:  # Iterate over float columns
            df_ART_PosEnrolment[col] = df_ART_PosEnrolment[col].astype(int)  # Convert float columns to integers

        wrap_column_headers(df_ART_PosEnrolment)  # Format DataFrame column headers (assumed function)
        ART_PosEnrolment_columns2 = wrap_column_headers2(ART_PosEnrolment_columns)  # Wrap column names for display

        # -- Step 4: Calculate derived metrics
        # -- Step 4.1: ART enrolment gap
        df_ART_PosEnrolment[ART_PosEnrolment_gap_columns[0]] = np.where(  # Calculate enrolment gap (requires numpy as np)
            df_ART_PosEnrolment[ART_PosEnrolment_columns2[0]] != df_ART_PosEnrolment["Total new positive"],  # Check if enrolled differs from total positive
            df_ART_PosEnrolment[ART_PosEnrolment_columns2[0]] - df_ART_PosEnrolment["Total new positive"],  # Compute gap (enrolled - total positive)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for enrolment
        # -- Step 4.2: ART linkage gap
        df_ART_PosEnrolment[ART_PosEnrolment_gap_columns[1]] = np.where(  # Calculate linkage gap
            df_ART_PosEnrolment[ART_PosEnrolment_columns2[1]] != df_ART_PosEnrolment[ART_PosEnrolment_columns2[0]],  # Check if started on ART differs from enrolled
            df_ART_PosEnrolment[ART_PosEnrolment_columns2[1]] - df_ART_PosEnrolment[ART_PosEnrolment_columns2[0]],  # Compute gap (started - enrolled)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for linkage

        # -- Step 5: Check and display cached styled DataFrames
        if display_output:  # Check if display output is requested
            if hasattr(process_ART_PosEnrolment_gap, 'cached_styles'):  # Check if cached styles exist
                cached_shape = getattr(process_ART_PosEnrolment_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_ART_PosEnrolment.shape  # Get current DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    for cluster, style in process_ART_PosEnrolment_gap.cached_styles.items():  # Iterate over cached styles
                        display_name = f"✔️ Displaying {cluster} {report_name}"  # Formatted display name for output
                        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
                        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header
                        print(print_display_name)  # Print display name with separators
                        widget_display_df(style)  # Display styled DataFrame (assumed widget function)tyle)  # Display cached styled DataFrame (assumed widget function)
                    return  # Exit to avoid reprocessing

        # -- Step 6: Initialize cache
        if not hasattr(process_ART_PosEnrolment_gap, 'cached_styles'):  # Check if cache attribute exists
            process_ART_PosEnrolment_gap.cached_styles = {}  # Initialize empty dictionary for cached styles

        # -- Step 7: Identify unique clusters
        cluster_list = pd.Series(df_ART_PosEnrolment['Cluster'].unique())  # Extract unique cluster values (requires pandas as pd)

        # -- Step 8: Process each cluster
        for current_cluster in cluster_list:  # Iterate over each unique cluster
            cluster_filtered = df_ART_PosEnrolment[df_ART_PosEnrolment['Cluster'] == current_cluster]  # Filter DataFrame for current cluster
            
            ART_PosEnrolment_msg = f"No {current_cluster} {report_name}"  # Define message for no gaps in cluster
            display_name = f"✔️ Displaying {current_cluster} {report_name}"  # Formatted display name for output
            display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
            print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

            cluster_filtered_gap = filter_gap_and_check_empty_df(  # Filter for rows with non-zero gaps
                df=cluster_filtered,  # Input cluster-filtered DataFrame
                msg=ART_PosEnrolment_msg,  # Message if no gaps found
                opNonZero=ART_PosEnrolment_gap_columns,  # Filter for non-zero gap values
                opNeg=None,  # No negative value filter
                opPos=None,  # No positive value filter
                opZero=None,  # No zero value filter
                opLT100=None  # No less-than-100 filter
            )  # Returns filtered DataFrame or None if no gaps

            if cluster_filtered_gap is None:  # Check if no gaps were found for cluster
                if current_cluster in process_ART_PosEnrolment_gap.cached_styles:  # Check if cluster in cache
                    del process_ART_PosEnrolment_gap.cached_styles[current_cluster]  # Remove cluster from cache
                continue  # Skip to next cluster

            cluster_filtered_style = (  # Apply styling to filtered cluster DataFrame
                cluster_filtered_gap.style  # Create style object
                .hide(axis='index')  # Hide row index for cleaner display
                .map(outlier_red, subset=ART_PosEnrolment_gap_columns)  # Highlight gap column outliers in red (assumed function)
            )  # Creates styled DataFrame for display/export

            process_ART_PosEnrolment_gap.cached_styles[current_cluster] = cluster_filtered_style  # Cache styled DataFrame for cluster

            # -- Step 8.1: Define export variables
            report_name_cluster = f"{current_cluster}_{report_name}"  # Create cluster-specific report name
            report_month = cluster_filtered_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
            report_image_name = f"{report_month}_{report_name_cluster}.png"  # Create image file name
            #report_image_path = rf"{sub_folder2_image_file_msf_outlier}\{report_image_name}"  # Define image file path (commented out)
            report_sheet_name = f"{current_cluster}_{report_name}"  # Define cluster-specific Excel sheet name

            # -- Step 9: Create descriptions for Word document
            report_description = []  # Initialize list for Word document descriptions
            if (cluster_filtered_gap[ART_PosEnrolment_gap_columns[0]] != 0).any():  # Check for non-zero enrolment gap
                report_description.append(  # Add enrolment gap description
                    f"Report Name: {ART_PosEnrolment_gap_columns[0]}\n"  # Gap column name
                    f"{ART_PosEnrolment_columns_desrpt[0]}\nshould be equal to {ART_PosEnrolment_columns_desrpt[2]}\n"  # Expected equality
                    f"Note: Where this ART enrolment gap is true, please ignore the outlier."  # Outlier note
                )  # Append enrolment gap description
            if (cluster_filtered_gap[ART_PosEnrolment_gap_columns[1]] != 0).any():  # Check for non-zero linkage gap
                report_description.append(  # Add linkage gap description
                    f"Report Name: {ART_PosEnrolment_gap_columns[1]}\n"  # Gap column name
                    f"{ART_PosEnrolment_columns_desrpt[1]}\nshould be equal to {ART_PosEnrolment_columns_desrpt[0]}\n"  # Expected equality
                    f"Note: Where this ART linkage gap is true, please ignore the outlier."  # Outlier note
                )  # Append linkage gap description
            report_description = "\n\n".join(report_description)  # Combine descriptions with double newlines

            # -- Step 10: Export results
            if not display_output:  # Check if user requested to export results
                export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                    report_name=report_name_cluster,  # Cluster-specific report name
                    df_style=cluster_filtered_style,  # Styled DataFrame for export
                    img_file_name=report_image_name,  # Image file name
                    img_file_path=sub_folder2_image_file_msf_outlier,  # Image file path (assumed defined)
                    doc_description=report_description,  # Word document description
                    doc_indicators_to_italicize=ART_PosEnrolment_columns_desrpt,  # Italicize ART columns for description
                    doc_indicators_to_underline=ART_PosEnrolment_gap_columns,  # Underline gap columns
                    xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                    xlm_sheet_name=report_sheet_name  # Excel sheet name
                )  # Exports to specified formats

            if display_output:  # Check if display is requested
                print(print_display_name)  # Print display name with separators
                widget_display_df(cluster_filtered_style)  # Display styled DataFrame (assumed widget function)

        # -- Step 11: Cache overall unfiltered DataFrame shape
        process_ART_PosEnrolment_gap.cached_shape = df_ART_PosEnrolment.shape  # Store original DataFrame shape

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_ART_PosEnrolment_gap, 'cached_styles'):  # Check for cached styles
            process_ART_PosEnrolment_gap.cached_styles.clear()  # Clear cached styles
        if hasattr(process_ART_PosEnrolment_gap, 'cached_shape'):  # Check for cached shape
            del process_ART_PosEnrolment_gap.cached_shape  # Clear cached shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - ART Regimen Line, MMD and DSD gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process ART regimen line, MMD and DSD gap
def process_ART_RegimentLine_MMD_DSD_gap(display_output=None):
    """
    Process ART Regimen line, MMD and DSD gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        ART_RegimenLine_MMD_DSD_columns = [  # List of column names for ART regimen, MMD, and DSD metrics
            "ART 3: Number of people living with HIV who are currently receiving ART during the month (All regimens)",  # Total ART recipients
            "Number of people living with HIV who are currently receiving ART during the month (All regimens): By Regimen Line",  # ART by regimen line
            "Number of people living with HIV who are currently receiving ART during the month (All regimens) - Multi-Month Dispensing",  # ART with MMD
            "Number of people living with HIV who are currently receiving ART during the month (All regimens) - DSD Model"  # ART with DSD
        ]  # Defines columns for ART gap analysis
        name = "ART Regimen-Line MMD DSD Gap"  # Base name for the report
        ART_RegimenLine_MMD_DSD_gap_columns = ["ART Regimen Line gap", "ART MMD gap", "ART DSD gap"]  # Names for calculated gap columns
        report_name = f"{name}8"  # Report name with suffix for uniqueness
        ART_RegimenLine_MMD_DSD_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_ART_RegimenLine_MMD_DSD = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="ART MSF",  # Key to fetch ART MSF data
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=ART_RegimenLine_MMD_DSD_columns  # Include ART regimen, MMD, and DSD columns
        )  # Returns processed DataFrame or None if failed
        if df_ART_RegimenLine_MMD_DSD is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        wrap_column_headers(df_ART_RegimenLine_MMD_DSD)  # Format DataFrame column headers (assumed function)
        ART_RegimenLine_MMD_DSD_columns2 = wrap_column_headers2(ART_RegimenLine_MMD_DSD_columns)  # Wrap column names for display

        # -- Step 3: Calculate derived metrics
        # -- Step 3.1: ART regimen line gap
        df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_gap_columns[0]] = np.where(  # Calculate regimen line gap (requires numpy as np)
            df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[0]] != df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[1]],  # Check if total ART differs from regimen line
            df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[0]] - df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[1]],  # Compute gap (total - regimen line)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for regimen line
        # -- Step 3.2: ART MMD gap
        df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_gap_columns[1]] = np.where(  # Calculate MMD gap
            df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[0]] < df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[2]],  # Check if total ART is less than MMD
            df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[0]] - df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[2]],  # Compute gap (total - MMD)
            0  # Set to 0 if no gap (total not less than MMD)
        )  # Adds gap column for MMD
        # -- Step 3.3: ART DSD gap
        df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_gap_columns[2]] = np.where(  # Calculate DSD gap
            df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[0]] < df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[3]],  # Check if total ART is less than DSD
            df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[0]] - df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[3]],  # Compute gap (total - DSD)
            0  # Set to 0 if no gap (total not less than DSD)
        )  # Adds gap column for DSD

        # -- Step 4: Check and display cached styled DataFrames
        if display_output:  # Check if display output is requested
            if hasattr(process_ART_RegimentLine_MMD_DSD_gap, 'cached_style'):  # Check if cached style exists
                cached_shape = getattr(process_ART_RegimentLine_MMD_DSD_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_ART_RegimenLine_MMD_DSD.shape  # Get current DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_ART_RegimentLine_MMD_DSD_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 5: Filter and validate gaps
        df_ART_RegimenLine_MMD_DSD_gap = filter_gap_and_check_empty_df(  # Filter for rows with non-zero gaps
                df=df_ART_RegimenLine_MMD_DSD,  # Input DataFrame with gaps
                msg=ART_RegimenLine_MMD_DSD_msg,  # Message if no gaps found
                opNonZero=ART_RegimenLine_MMD_DSD_gap_columns,  # Filter for non-zero gap values
                opNeg=None,  # No negative value filter
                opPos=None,  # No positive value filter
                opZero=None,  # No zero value filter
                opLT100=None  # No less-than-100 filter
            )  # Returns filtered DataFrame or None if no gaps

        if df_ART_RegimenLine_MMD_DSD_gap is None:  # Check if no gaps were found
            if hasattr(process_ART_RegimentLine_MMD_DSD_gap, 'cached_style'):  # Check for cached style
                del process_ART_RegimentLine_MMD_DSD_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_ART_RegimentLine_MMD_DSD_gap, 'cached_shape'):  # Check for cached shape
                del process_ART_RegimentLine_MMD_DSD_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 6: Style the DataFrame
        df_ART_RegimenLine_MMD_DSD_gap_style = (  # Apply styling to filtered DataFrame
            df_ART_RegimenLine_MMD_DSD_gap.style  # Create style object
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=ART_RegimenLine_MMD_DSD_gap_columns)  # Highlight gap column outliers in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 7: Cache styled DataFrame and shape
        process_ART_RegimentLine_MMD_DSD_gap.cached_style = df_ART_RegimenLine_MMD_DSD_gap_style  # Store styled DataFrame
        process_ART_RegimentLine_MMD_DSD_gap.cached_shape = df_ART_RegimenLine_MMD_DSD.shape  # Store original DataFrame shape

        # -- Step 8: Define export variables
        report_month = df_ART_RegimenLine_MMD_DSD_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = f"{report_name}"  # Define Excel sheet name

        # -- Step 9: Create descriptions for Word document
        report_description = []  # Initialize list for Word document descriptions
        if (df_ART_RegimenLine_MMD_DSD_gap[ART_RegimenLine_MMD_DSD_gap_columns[0]] != 0).any():  # Check for non-zero regimen line gap
            report_description.append(  # Add regimen line gap description
                f"Report Name: {ART_RegimenLine_MMD_DSD_gap_columns[0]}\n"  # Gap column name
                f"{ART_RegimenLine_MMD_DSD_columns[0]}\nshould be equal to {ART_RegimenLine_MMD_DSD_columns[1]}"  # Expected equality
            )  # Append regimen line gap description
        if (df_ART_RegimenLine_MMD_DSD_gap[ART_RegimenLine_MMD_DSD_gap_columns[1]] != 0).any():  # Check for non-zero MMD gap
            report_description.append(  # Add MMD gap description
                f"Report Name: {ART_RegimenLine_MMD_DSD_gap_columns[1]}\n"  # Gap column name
                f"{ART_RegimenLine_MMD_DSD_columns[0]}\nshould be greater than {ART_RegimenLine_MMD_DSD_columns[2]}\n"  # Expected relation
            )  # Append MMD gap description
        if (df_ART_RegimenLine_MMD_DSD_gap[ART_RegimenLine_MMD_DSD_gap_columns[2]] != 0).any():  # Check for non-zero DSD gap
            report_description.append(  # Add DSD gap description
                f"Report Name: {ART_RegimenLine_MMD_DSD_gap_columns[2]}\n"  # Gap column name
                f"{ART_RegimenLine_MMD_DSD_columns[0]}\nshould be greater than {ART_RegimenLine_MMD_DSD_columns[3]}\n"  # Expected relation
            )  # Append DSD gap description
        report_description = "\n\n".join(report_description)  # Combine descriptions with double newlines

        # -- Step 10: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_ART_RegimenLine_MMD_DSD_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=ART_RegimenLine_MMD_DSD_columns,  # Italicize ART columns
                doc_indicators_to_underline=ART_RegimenLine_MMD_DSD_gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 11: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_ART_RegimenLine_MMD_DSD_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_ART_RegimentLine_MMD_DSD_gap, 'cached_styles'):  # Check for cached styles
            process_ART_RegimentLine_MMD_DSD_gap.cached_style.clear()  # Clear cached styles (note: should be cached_styles)
        if hasattr(process_ART_RegimentLine_MMD_DSD_gap, 'cached_shape'):  # Check for cached shape
            del process_ART_RegimentLine_MMD_DSD_gap.cached_shape  # Clear cached shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - ART TB Screening gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process ART TB Screening gap
def process_ART_TB_Screening_gap(display_output=None):
    """
    Process ART Regimen line, MMD and DSD gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        ART_TB_Screening_columns = ["ART 2: Number of people living with HIV newly started on ART during the month (excludes ART transfer-in)"]  # Column for newly started on ART
        place_holder = ["ART 10: Number of PLHIV on ART (Including PMTCT) who were Clinically Screened for TB in HIV Treatment Settings"]  # Placeholder for TB screening column
        ART_TB_Screening_columns_new_old = ['Newly on ART', 'Already on ART']  # Column names for new/old ART TB screening
        ART_TB_Screening_columns_desrpt = [  # Extended list for Word document descriptions
            ART_TB_Screening_columns[0],  # Newly started on ART
            place_holder[0] + ' - ' + ART_TB_Screening_columns_new_old[0],  # TB screening for newly on ART
            place_holder[0] + ' - ' + ART_TB_Screening_columns_new_old[1]  # TB screening for already on ART
        ]  # Defines columns for description
        name = "ART Tx_New TB Screening Gap"  # Base name for the report
        ART_TB_Screening_gap_columns = ['ART Tx New TB screening gap']  # Name for the calculated gap column
        report_name = f"{name}9"  # Report name with suffix for uniqueness
        ART_RegimenLine_MMD_DSD_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_ART_TB_Screening = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="ART MSF",  # Key to fetch ART MSF data
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=ART_TB_Screening_columns  # Include ART TB screening column
        )  # Returns processed DataFrame or None if failed
        if df_ART_TB_Screening is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 2.1: Prepare TB screening data
        df_ART_TB_Screening_new_old = prepare_and_convert_df(  # Fetch and prepare TB screening DataFrame
            DHIS2_data_key="ART MSF_tb screening",  # Key to fetch TB screening data
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns
            data_columns=ART_TB_Screening_columns_new_old  # Include new/old TB screening columns
        )  # Returns processed DataFrame or None if failed
        if df_ART_TB_Screening_new_old is None:  # Check if TB screening data preparation failed
            return  # Exit function if no valid data
        
        # -- Step 3: Merge with external DataFrame
        df_ART_TB_Screening = df_ART_TB_Screening.merge(  # Merge ART data with TB screening data
            df_ART_TB_Screening_new_old,  # Target DataFrame for merging
            on=MSF_hierarchy,  # Columns to merge on
            how="left"  # Keep all rows from df_ART_TB_Screening
        )  # Combines ART and TB screening data
        df_ART_TB_Screening = df_ART_TB_Screening.fillna(0)  # Fill NaN values with 0
        float_columns = df_ART_TB_Screening.select_dtypes(include=['float64', 'float32']).columns  # Identify float-type columns
        for col in float_columns:  # Iterate over float columns
            df_ART_TB_Screening[col] = df_ART_TB_Screening[col].astype(int)  # Convert float columns to integers
        
        # -- Step 3.1: Rename columns
        df_ART_TB_Screening = df_ART_TB_Screening.rename(columns={  # Rename TB screening columns for clarity
            'Newly on ART': place_holder[0] + ' - ' + ART_TB_Screening_columns_new_old[0],  # Rename new ART screening
            'Already on ART': place_holder[0] + ' - ' + ART_TB_Screening_columns_new_old[1]  # Rename existing ART screening
        })  # Updates column names in DataFrame

        wrap_column_headers(df_ART_TB_Screening)  # Format DataFrame column headers (assumed function)
        ART_TB_Screening_columns2 = wrap_column_headers2(ART_TB_Screening_columns_desrpt)  # Wrap description column names for display

        # -- Step 4: Calculate derived metrics
        df_ART_TB_Screening[ART_TB_Screening_gap_columns[0]] = np.where(  # Calculate TB screening gap (requires numpy as np)
            df_ART_TB_Screening[ART_TB_Screening_columns2[0]] != df_ART_TB_Screening[ART_TB_Screening_columns2[1]],  # Check if newly started differs from screened
            df_ART_TB_Screening[ART_TB_Screening_columns2[0]] - df_ART_TB_Screening[ART_TB_Screening_columns2[1]],  # Compute gap (newly started - screened)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for TB screening

        # -- Step 5: Check and display cached styled DataFrames
        if display_output:  # Check if display output is requested
            if hasattr(process_ART_TB_Screening_gap, 'cached_style'):  # Check if cached style exists
                cached_shape = getattr(process_ART_TB_Screening_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_ART_TB_Screening.shape  # Get current DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_ART_TB_Screening_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing
                    
        # -- Step 6: Filter and validate gaps
        df_ART_TB_Screening_gap = filter_gap_and_check_empty_df(  # Filter for rows with non-zero gaps
                df=df_ART_TB_Screening,  # Input DataFrame with gap
                msg=ART_RegimenLine_MMD_DSD_msg,  # Message if no gaps found
                opNonZero=ART_TB_Screening_gap_columns,  # Filter for non-zero gap values
                opNeg=None,  # No negative value filter
                opPos=None,  # No positive value filter
                opZero=None,  # No zero value filter
                opLT100=None  # No less-than-100 filter
            )  # Returns filtered DataFrame or None if no gaps

        if df_ART_TB_Screening_gap is None:  # Check if no gaps were found
            if hasattr(process_ART_TB_Screening_gap, 'cached_style'):  # Check for cached style
                del process_ART_TB_Screening_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_ART_TB_Screening_gap, 'cached_shape'):  # Check for cached shape
                del process_ART_TB_Screening_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps                       

        # -- Step 7: Style the DataFrame
        df_ART_TB_Screening_gap_style = (  # Apply styling to filtered DataFrame
            df_ART_TB_Screening_gap.style  # Create style object
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=ART_TB_Screening_gap_columns)  # Highlight gap column outliers in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_ART_TB_Screening_gap.cached_style = df_ART_TB_Screening_gap_style  # Store styled DataFrame
        process_ART_TB_Screening_gap.cached_shape = df_ART_TB_Screening.shape  # Store original DataFrame shape

        # -- Step 9: Define export variables
        report_month = df_ART_TB_Screening_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = f"{report_name}"  # Define Excel sheet name

        # -- Step 10: Create descriptions for Word document
        if (df_ART_TB_Screening_gap[ART_TB_Screening_gap_columns[0]] != 0).any():  # Check for non-zero TB screening gap
            report_description = (  # Create description for TB screening gap
                f"Report Name: {ART_TB_Screening_gap_columns[0]}\n"  # Gap column name
                f"{ART_TB_Screening_columns_desrpt[0]}\nshould be equal to {ART_TB_Screening_columns_desrpt[1]}"  # Expected equality
            )  # Define Word document description

        # -- Step 11: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_ART_TB_Screening_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=ART_TB_Screening_columns_desrpt,  # Italicize description columns
                doc_indicators_to_underline=ART_TB_Screening_gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            widget_display_df(df_ART_TB_Screening_gap_style)  # Display styled DataFrame (assumed widget function)
            
    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_ART_TB_Screening_gap, 'cached_styles'):  # Check for cached styles
            process_ART_TB_Screening_gap.cached_style.clear()  # Clear cached styles (note: should be cached_styles)
        if hasattr(process_ART_TB_Screening_gap, 'cached_shape'):  # Check for cached shape
            del process_ART_TB_Screening_gap.cached_shape  # Clear cached shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - ART TB Presumptive Test gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process ART TB Presumptive Test gap
def process_ART_TB_Presumptive_Test_gap(display_output=None):
    """
    Process ART Presumptive Test gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        ART_TB_Presumptive_Test_columns = [  # List of column names for ART TB Presumptive Test metrics
            "ART 11: Number of PLHIV on ART with Presumptive TB during the month",  # PLHIV with presumptive TB
            "ART 12: Number of PLHIV on ART with Presumptive TB and Tested for TB during the month"  # PLHIV tested for TB
        ]  # Defines columns for TB presumptive test gap analysis
        name = "ART TB Presumptive Test Gap"  # Base name for the report
        ART_TB_Presumptive_Test_gap_columns = ["ART TB presumptive test gap"]  # Name for the calculated gap column
        report_name = f"{name}10"  # Report name with suffix for uniqueness
        ART_TB_Presumptive_Test_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_ART_TB_Presumptive_Test = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="ART MSF",  # Key to fetch ART MSF data
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=ART_TB_Presumptive_Test_columns  # Include ART TB presumptive test columns
        )  # Returns processed DataFrame or None if failed
        if df_ART_TB_Presumptive_Test is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        wrap_column_headers(df_ART_TB_Presumptive_Test)  # Format DataFrame column headers (assumed function)
        ART_TB_Presumptive_Test_columns2 = wrap_column_headers2(ART_TB_Presumptive_Test_columns)  # Wrap column names for display

        # -- Step 3: Calculate derived metrics
        # -- Step 3.1: ART TB Presumptive Test gap
        df_ART_TB_Presumptive_Test[ART_TB_Presumptive_Test_gap_columns[0]] = np.where(  # Calculate TB presumptive test gap (requires numpy as np)
            df_ART_TB_Presumptive_Test[ART_TB_Presumptive_Test_columns2[0]] != df_ART_TB_Presumptive_Test[ART_TB_Presumptive_Test_columns2[1]],  # Check if presumptive TB differs from tested
            df_ART_TB_Presumptive_Test[ART_TB_Presumptive_Test_columns2[0]] - df_ART_TB_Presumptive_Test[ART_TB_Presumptive_Test_columns2[1]],  # Compute gap (presumptive - tested)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for TB presumptive test

        # -- Step 4: Check and display cached styled DataFrames
        if display_output:  # Check if display output is requested
            if hasattr(process_ART_TB_Presumptive_Test_gap, 'cached_style'):  # Check if cached style exists
                cached_shape = getattr(process_ART_TB_Presumptive_Test_gap, 'cached_shape', None)  # Retrieve Retrieve cached DataFrame shape
                current_shape = df_ART_TB_Presumptive_Test.shape  # Get current DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_ART_TB_Presumptive_Test_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing
                    
        # -- Step 5: Filter and validate gaps
        df_ART_TB_Presumptive_Test_gap = filter_gap_and_check_empty_df(  # Filter for rows with non-zero gaps
            df=df_ART_TB_Presumptive_Test,  # Input DataFrame with gap
            msg=ART_TB_Presumptive_Test_msg,  # Message if no gaps found
            opNonZero=ART_TB_Presumptive_Test_gap_columns,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps

        if df_ART_TB_Presumptive_Test_gap is None:  # Check if no gaps were found
            if hasattr(process_ART_TB_Presumptive_Test_gap, 'cached_style'):  # Check for cached style
                del process_ART_TB_Presumptive_Test_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_ART_TB_Presumptive_Test_gap, 'cached_shape'):  # Check for cached shape
                del process_ART_TB_Presumptive_Test_gap.cached_shape  # Clear cached shape
            return  # Exit function if no gaps

        # -- Step 6: Style the DataFrame
        df_ART_TB_Presumptive_Test_gap_style = (  # Apply styling to filtered DataFrame
            df_ART_TB_Presumptive_Test_gap.style  # Create style object
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=ART_TB_Presumptive_Test_gap_columns)  # Highlight gap column outliers in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 7: Cache styled DataFrame and shape
        process_ART_TB_Presumptive_Test_gap.cached_style = df_ART_TB_Presumptive_Test_gap_style  # Store styled DataFrame
        process_ART_TB_Presumptive_Test_gap.cached_shape = df_ART_TB_Presumptive_Test.shape  # Store original DataFrame shape

        # -- Step 8: Define export variables
        report_month = df_ART_TB_Presumptive_Test_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = f"{report_name}"  # Define Excel sheet name

        # -- Step 9: Create descriptions for Word document
        if (df_ART_TB_Presumptive_Test_gap[ART_TB_Presumptive_Test_gap_columns[0]] != 0).any():  # Check for non-zero TB presumptive test gap
            report_description = (  # Create description for TB presumptive Test gap
                f"Report Name: {ART_TB_Presumptive_Test_gap_columns[0]}\n"
                f"{ART_TB_Presumptive_Test_columns[0]}\nshould be equal to {ART_TB_Presumptive_Test_columns[1]}"
            )  # Define Word document description

        # -- Step 10: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_ART_TB_Presumptive_Test_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=ART_TB_Presumptive_Test_columns,  # Italicize TB presumptive test columns
                doc_indicators_to_underline=ART_TB_Presumptive_Test_gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 11: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_ART_TB_Presumptive_Test_gap_style)  # Display styled DataFrame (assumed widget function)
            
    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_ART_TB_Presumptive_Test_gap, 'cached_styles'):  # Check for cached styles
            process_ART_TB_Presumptive_Test_gap.cached_style.clear()  # Clear cached styles (note: should be cached_styles)
        if hasattr(process_ART_TB_Presumptive_Test_gap, 'cached_shape'):  # Check for cached shape
            del process_ART_TB_Presumptive_Test_gap.cached_shape  # Clear cached shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - ART TB Treatment Test gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process ART TB Treatment gap
def process_ART_TB_Treatment_gap(display_output=None):
    """
    Process ART TB Treatment gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        ART_TB_Treatment_columns = [  # List of column names for ART TB Treatment metrics
            "ART 13: Number of of PLHIV on ART who have Active TB Disease",  # PLHIV with active TB
            "ART 14: Number of PLHIV on ART with active TB disease who initiated TB treatment"  # PLHIV initiated on TB treatment
        ]  # Defines columns for TB treatment gap analysis
        name = "ART TB Treatment Gap"  # Base name for the report
        ART_TB_Treatment_gap_columns = ["ART TB treatment gap"]  # Name for the calculated gap column
        report_name = f"{name}11"  # Report name with suffix for uniqueness
        ART_TB_Treatment_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_ART_TB_Treatment = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="ART MSF",  # Key to fetch ART MSF data
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=ART_TB_Treatment_columns  # Include ART TB treatment columns
        )  # Returns processed DataFrame or None if failed
        if df_ART_TB_Treatment is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        wrap_column_headers(df_ART_TB_Treatment)  # Format DataFrame column headers (assumed function)
        ART_TB_Treatment_columns_wrap = wrap_column_headers2(ART_TB_Treatment_columns)  # Wrap column names for display

        # -- Step 3: Calculate derived metrics
        # -- Step 3.1: ART TB Treatment gap
        df_ART_TB_Treatment[ART_TB_Treatment_gap_columns[0]] = np.where(  # Calculate TB treatment gap (requires numpy as np)
            df_ART_TB_Treatment[ART_TB_Treatment_columns_wrap[0]] != df_ART_TB_Treatment[ART_TB_Treatment_columns_wrap[1]],  # Check if active TB differs from treated
            df_ART_TB_Treatment[ART_TB_Treatment_columns_wrap[0]] - df_ART_TB_Treatment[ART_TB_Treatment_columns_wrap[1]],  # Compute gap (active TB - treated)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for TB treatment

        # -- Step 4: Check and display cached styled DataFrames
        if display_output:  # Check if display output is requested
            if hasattr(process_ART_TB_Treatment_gap, 'cached_style'):  # Check if cached style exists
                cached_shape = getattr(process_ART_TB_Treatment_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_ART_TB_Treatment.shape  # Get current DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_ART_TB_Treatment_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing
                    
        # -- Step 5: Filter and validate gaps
        df_ART_TB_Treatment_gap = filter_gap_and_check_empty_df(  # Filter for rows with non-zero gaps
            df=df_ART_TB_Treatment,  # Input DataFrame with gap
            msg=ART_TB_Treatment_msg,  # Message if no gaps found
            opNonZero=ART_TB_Treatment_gap_columns,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps

        if df_ART_TB_Treatment_gap is None:  # Check if no gaps were found
            if hasattr(process_ART_TB_Treatment_gap, 'cached_style'):  # Check for cached style
                del process_ART_TB_Treatment_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_ART_TB_Treatment_gap, 'cached_shape'):  # Check for cached shape
                del process_ART_TB_Treatment_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 6: Style the DataFrame
        df_ART_TB_Treatment_gap_style = (  # Apply styling to filtered DataFrame
            df_ART_TB_Treatment_gap.style  # Create style object
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=ART_TB_Treatment_gap_columns)  # Highlight gap column outliers in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 7: Cache styled DataFrame and shape
        process_ART_TB_Treatment_gap.cached_style = df_ART_TB_Treatment_gap_style  # Store styled DataFrame
        process_ART_TB_Treatment_gap.cached_shape = df_ART_TB_Treatment.shape  # Store original DataFrame shape

        # -- Step 8: Define export variables
        report_month = df_ART_TB_Treatment_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = f"{report_name}"  # Define Excel sheet name

        # -- Step 9: Create descriptions for Word document
        if (df_ART_TB_Treatment_gap[ART_TB_Treatment_gap_columns[0]] != 0).any():  # Check for non-zero TB treatment gap
            report_description = (  # Create description for TB treatment gap
                f"Report Name: {ART_TB_Treatment_gap_columns[0]}\n"  # Gap column name
                f"{ART_TB_Treatment_columns[1]}\nshould be equal to {ART_TB_Treatment_columns[0]}"  # Expected equality (note: order reversed in description)
            )  # Define Word document description

        # -- Step 10: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_ART_TB_Treatment_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=ART_TB_Treatment_columns,  # Italicize TB treatment columns
                doc_indicators_to_underline=ART_TB_Treatment_gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 11: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_ART_TB_Treatment_gap_style)  # Display styled DataFrame (assumed widget function)
            
    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_ART_TB_Treatment_gap, 'cached_styles'):  # Check for cached styles
            process_ART_TB_Treatment_gap.cached_style.clear()  # Clear cached styles (note: should be cached_styles)
        if hasattr(process_ART_TB_Treatment_gap, 'cached_shape'):  # Check for cached shape
            del process_ART_TB_Treatment_gap.cached_shape  # Clear cached shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - ART Viral Load Supression gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process ART Viral Load Suppression gap
def process_ART_Viral_Load_Suppression_gap(display_output=None):
    """
    Process ART Viral Load Suppression gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        ART_Viral_Load_Suppression_columns = [  # List of column names for ART Viral Load Suppression metrics
            "ART 6: Number of PLHIV on ART for at least 6 months with a VL test result during the month: Routine",  # Routine VL test results
            "Number of PLHIV on ART for at least 6 months with a VL test result during the month: Targeted",  # Targeted VL test results
            "ART 7: Number of PLHIV on ART (for at least 6 months) who have virologic suppression (<1000 copies/ml) during the month: Routine",  # Routine VL suppression
            "ART 7: Number of PLHIV on ART (for at least 6 months) who have virologic suppression (<1000 copies/ml) during the month: Targeted"  # Targeted VL suppression
        ]  # Defines columns for viral load suppression gap analysis
        ART_Viral_Load_Suppression_columns_cal = [  # Calculated column names for aggregated metrics
            "ART 6: Number of PLHIV on ART for at least 6 months with a VL test result during the month - Routine and Targeted",  # Total VL test results
            "ART 7: Number of PLHIV on ART (for at least 6 months) who have virologic suppression (<1000 copies/ml) during the month - Routine and Targeted"  # Total VL suppression
        ]  # Defines columns for calculated totals
        name = "ART Viral Load Suppression Gap"  # Base name for the report
        ART_Viral_Load_Suppression_gap_columns = ["ART viral load suppression gap"]  # Name for the calculated gap column
        report_name = f"{name}12"  # Report name with suffix for uniqueness
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_ART_Viral_Load_Suppression = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="ART MSF",  # Key to fetch ART MSF data
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=ART_Viral_Load_Suppression_columns  # Include ART viral load suppression columns
        )  # Returns processed DataFrame or None if failed
        if df_ART_Viral_Load_Suppression is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate derived metrics
        # -- Step 3.1: Calculate total viral load results received
        df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns_cal[0]] = (  # Sum Routine and Targeted VL test results
            df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns[0]] + df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns[1]]
        )  # Adds column for total VL test results

        # -- Step 3.2: Calculate total viral load results suppressed
        df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns_cal[1]] = (  # Sum Routine and Targeted VL suppression
            df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns[2]] + df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns[3]]
        )  # Adds column for total VL suppression

        # -- Step 3.3: Drop the original Routine and Targeted columns
        df_ART_Viral_Load_Suppression = df_ART_Viral_Load_Suppression.drop(  # Remove original columns to focus on aggregated metrics
            columns=[
                ART_Viral_Load_Suppression_columns[0],  # Routine VL test results
                ART_Viral_Load_Suppression_columns[1],  # Targeted VL test results
                ART_Viral_Load_Suppression_columns[2],  # Routine VL suppression
                ART_Viral_Load_Suppression_columns[3]   # Targeted VL suppression
            ]
        )  # Updates DataFrame with only aggregated columns

        wrap_column_headers(df_ART_Viral_Load_Suppression)  # Format DataFrame column headers (assumed function)
        ART_Viral_Load_Suppression_columns_wrap = wrap_column_headers2(ART_Viral_Load_Suppression_columns_cal)  # Wrap calculated column names for display

        # -- Step 3.4: Calculate viral load suppression gap
        df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_gap_columns[0]] = np.where(  # Calculate gap (requires numpy as np)
            df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns_wrap[1]] > df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns_wrap[0]],  # Check if suppressed exceeds tested
            df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns_wrap[1]] - df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns_wrap[0]],  # Compute gap (suppressed - tested)
            0  # Set to 0 if no gap (suppressed not greater than tested)
        )  # Adds gap column for viral load suppression

        # -- Step 4: Check and display cached styled DataFrames
        if display_output:  # Check if display output is requested
            if hasattr(process_ART_Viral_Load_Suppression_gap, 'cached_style'):  # Check if cached style exists
                cached_shape = getattr(process_ART_Viral_Load_Suppression_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_ART_Viral_Load_Suppression.shape  # Get current DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_ART_Viral_Load_Suppression_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing
                    
        # -- Step 5: Filter and validate gaps
        df_ART_Viral_Load_Suppression_gap = filter_gap_and_check_empty_df(  # Filter for rows with non-zero gaps
            df=df_ART_Viral_Load_Suppression,  # Input DataFrame with gap
            msg=No_gap_msg,  # Message if no gaps found
            opNonZero=ART_Viral_Load_Suppression_gap_columns,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps

        if df_ART_Viral_Load_Suppression_gap is None:  # Check if no gaps were found
            if hasattr(process_ART_Viral_Load_Suppression_gap, 'cached_style'):  # Check for cached style
                del process_ART_Viral_Load_Suppression_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_ART_Viral_Load_Suppression_gap, 'cached_shape'):  # Check for cached shape
                del process_ART_Viral_Load_Suppression_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 6: Style the DataFrame
        df_ART_Viral_Load_Suppression_gap_style = (  # Apply styling to filtered DataFrame
            df_ART_Viral_Load_Suppression_gap.style  # Create style object
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=ART_Viral_Load_Suppression_gap_columns)  # Highlight gap column outliers in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 7: Cache styled DataFrame and shape
        process_ART_Viral_Load_Suppression_gap.cached_style = df_ART_Viral_Load_Suppression_gap_style  # Store styled DataFrame
        process_ART_Viral_Load_Suppression_gap.cached_shape = df_ART_Viral_Load_Suppression.shape  # Store original DataFrame shape

        # -- Step 8: Define export variables
        report_month = df_ART_Viral_Load_Suppression_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = f"{report_name}"  # Define Excel sheet name

        # -- Step 9: Create descriptions for Word document
        if (df_ART_Viral_Load_Suppression_gap[ART_Viral_Load_Suppression_gap_columns[0]] != 0).any():  # Check for non-zero viral load suppression gap
            report_description = (  # Create description for viral load suppression gap
                f"Report Name: {ART_Viral_Load_Suppression_gap_columns[0]}\n"  # Gap column name
                f"{ART_Viral_Load_Suppression_columns[2]}\nplus{ART_Viral_Load_Suppression_columns[3]}\n"  # Routine + Targeted suppression
                f"should not be greater than {ART_Viral_Load_Suppression_columns[0]}\nplus{ART_Viral_Load_Suppression_columns[1]}"  # Routine + Targeted test results
            )  # Define Word document description

        # -- Step 10: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_ART_Viral_Load_Suppression_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=ART_Viral_Load_Suppression_columns,  # Italicize viral load suppression columns
                doc_indicators_to_underline=ART_Viral_Load_Suppression_gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 11: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_ART_Viral_Load_Suppression_gap_style)  # Display styled DataFrame (assumed widget function)
            
    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_ART_Viral_Load_Suppression_gap, 'cached_styles'):  # Check for cached styles
            process_ART_Viral_Load_Suppression_gap.cached_style.clear()  # Clear cached styles (note: should be cached_styles)
        if hasattr(process_ART_Viral_Load_Suppression_gap, 'cached_shape'):  # Check for cached shape
            del process_ART_Viral_Load_Suppression_gap.cached_shape  # Clear cached shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

## - HTS MSF
### - HTS New Positive gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process HTS New Positive gap
def process_HTS_New_Positive_gap(display_output=None):
    """
    Process HTS New Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        HTS_New_Positive_columns = [  # List of column names for HTS New Positive metrics
            "Number of people who tested HIV positive and received results (Inpatient, Outpatient, Standalone)",  # Total positive results (facility-based)
            "Number of people who tested HIV positive and received results  (Community)",  # Total positive results (community-based)
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling (Inpatient, Outpatient, Standalone)",  # Known positives (facility-based)
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Community)",  # Known positives (community-based)
            "HTS total tested - positive",  # Total tested positive
            "HTS total tested - previously known positive",  # Total previously known positives
            "HTS total tested - new positive (excluding previously known)",  # New positives excluding known
            "Number of people who tested HIV positive and received results (Inpatient)",  # Inpatient positive results
            "Number of people who tested HIV positive and received results (Outpatient)",  # Outpatient positive results
            "Number of people who tested HIV positive and received results (Standalone)",  # Standalone positive results
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Inpatient)",  # Inpatient known positives
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Outpatient)",  # Outpatient known positives
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Standalone)"  # Standalone known positives
        ]  # Defines columns for HTS New Positive gap analysis

        HTS_New_Positive_columns_order = [  # Desired order of columns for output
            "Number of people who tested HIV positive and received results (Inpatient, Outpatient, Standalone)",  # Facility-based positive results
            "Number of people who tested HIV positive and received results  (Community)",  # Community-based positive results
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling (Inpatient, Outpatient, Standalone)",  # Facility-based known positives
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Community)",  # Community-based known positives
            "HTS total tested - positive",  # Total tested positive
            "HTS total tested - previously known positive",  # Total previously known positives
            "HTS total tested - new positive (excluding previously known)"  # New positives excluding known
        ]  # Specifies column order for DataFrame
        Gap_title_special = ["Community testing gap"]  # Special title for community testing gap
        name = "HTS New Positive Gap"  # Base name for the report
        HTS_New_Positive_gap_columns = ["HTS total tested - new positive (excluding previously known)"]  # Name for the calculated gap column
        report_name = f"{name}13"  # Report name with suffix for uniqueness
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_HTS_New_Positive = Pre_HTS_MSF_positive.copy()  # Create a copy of the predefined HTS total positives DataFrame (assumed defined elsewhere)

        # -- Step 3: Calculate total HTS New Positive results
        df_HTS_New_Positive[HTS_New_Positive_columns[0]] = (  # Sum facility-based positive results (Inpatient, Outpatient, Standalone)
            df_HTS_New_Positive.iloc[:, 4:7].sum(axis=1)  # Sum columns 4 to 6 (indices for Inpatient, Outpatient, Standalone positives)
        )  # Adds column for total facility-based positive results

        # -- Step 4: Calculate total HTS New Positive results previously known
        df_HTS_New_Positive[HTS_New_Positive_columns[2]] = (  # Sum facility-based known positives (Inpatient, Outpatient, Standalone)
            df_HTS_New_Positive.iloc[:, 8:11].sum(axis=1)  # Sum columns 8 to 10 (indices for Inpatient, Outpatient, Standalone known positives)
        )  # Adds column for total facility-based known positives

        # -- Step 5: Drop the original HTS SDP columns
        df_HTS_New_Positive = df_HTS_New_Positive.drop(  # Remove specific service delivery point (SDP) columns to simplify DataFrame
            columns=[
                HTS_New_Positive_columns[7],  # Inpatient positive results
                HTS_New_Positive_columns[8],  # Outpatient positive results
                HTS_New_Positive_columns[9],  # Standalone positive results
                HTS_New_Positive_columns[10],  # Inpatient known positives
                HTS_New_Positive_columns[11],  # Outpatient known positives
                HTS_New_Positive_columns[12]  # Standalone known positives
            ]
        )  # Updates DataFrame with only aggregated columns

        # -- Step 6: Reorder the columns to match the desired order
        df_HTS_New_Positive = df_HTS_New_Positive[MSF_hierarchy + HTS_New_Positive_columns_order]  # Reorder columns to include hierarchy and specified order (MSF_hierarchy assumed defined)

        wrap_column_headers(df_HTS_New_Positive)  # Format DataFrame column headers for readability (assumed function)
        HTS_New_Positive_columns_wrap = wrap_column_headers2(HTS_New_Positive_columns_order)  # Wrap ordered column names for display
        HTS_New_Positive_gap_columns_wrap = wrap_column_headers2(HTS_New_Positive_gap_columns)  # Wrap gap column names for display

        # -- Step 7: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_HTS_New_Positive_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_HTS_New_Positive_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_HTS_New_Positive.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_HTS_New_Positive_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)   
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 8: Filter and validate gaps
        df_HTS_New_Positive_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with gaps
            df=df_HTS_New_Positive,  # Input DataFrame with calculated metrics
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=None,  # No non-zero value filter
            opNeg=HTS_New_Positive_columns_wrap,  # Filter for negative values in specified columns
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        
        if df_HTS_New_Positive_gap is None:  # Check if no gaps were found
            if hasattr(process_HTS_New_Positive_gap, 'cached_style'):  # Check for cached style
                del process_HTS_New_Positive_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_HTS_New_Positive_gap, 'cached_shape'):  # Check for cached shape
                del process_HTS_New_Positive_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 9: Style the DataFrame
        df_HTS_New_Positive_style = (  # Apply styling to filtered DataFrame
            df_HTS_New_Positive_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red_LT0, subset=HTS_New_Positive_columns_wrap)  # Highlight negative outliers in specified columns (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 10: Cache styled DataFrame and shape
        process_HTS_New_Positive_gap.cached_style = df_HTS_New_Positive_style  # Store styled DataFrame
        process_HTS_New_Positive_gap.cached_shape = df_HTS_New_Positive.shape  # Store original unfiltered DataFrame shape

        # -- Step 11: Define export variables
        report_month = df_HTS_New_Positive['ReportPeriod'].iloc[0]  # Extract report period from unfiltered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 12: Create descriptions for Word document
        report_description = []  # Initialize empty list for descriptions
        if (df_HTS_New_Positive[HTS_New_Positive_gap_columns_wrap[0]] < 0).any():  # Check for negative new positive gaps
            report_description.append(  # Add description for new positive gap
                f"Report Name: {HTS_New_Positive_gap_columns[0]}\n"  # Gap column name
                f"{HTS_New_Positive_columns[6]}\nshould not be less than 0"  # New positives should be non-negative
            )  # Append description to list
        if (df_HTS_New_Positive[HTS_New_Positive_columns_wrap[1]] != 0).any() or (df_HTS_New_Positive[HTS_New_Positive_columns_wrap[3]] != 0).any():  # Check for non-zero community testing results
            report_description.append(  # Add description for community testing gap
                f"Report Name: {Gap_title_special[0]}\n"  # Special community gap title
                f"{HTS_New_Positive_columns[1]}\n"  # Community positive results
                f"plus {HTS_New_Positive_columns[3]} should not be greater than 0"  # Community known positives
            )  # Append description to list
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

        # -- Step 13: Export results
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_HTS_New_Positive_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_file_path=doc_file_msf_outlier_docx,  # Word document path (assumed defined)
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=HTS_New_Positive_columns,  # Italicize HTS columns
                doc_indicators_to_underline=HTS_New_Positive_gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 14: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)
            widget_display_df(df_HTS_New_Positive_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_HTS_New_Positive_gap, 'cached_style'):  # Check for cached style
            del process_HTS_New_Positive_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_HTS_New_Positive_gap, 'cached_shape'):  # Check for cached shape
            del process_HTS_New_Positive_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - HTS TB Screening gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process HTS TB Screening gap
def process_HTS_TB_Screening_gap(display_output=None): 
    """
    Process HTS TB Screening gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        HTS_TB_Screening_columns = [  # List of column names for HTS TB Screening metrics
            "Number of people who tested HIV negative and received their results. (Inpatient)",  # HIV negative results (Inpatient)
            "Number of people who tested HIV negative and received their results. (Outpatient)",  # HIV negative results (Outpatient)
            "Number of people who tested HIV negative and received their results. (Standalone)",  # HIV negative results (Standalone)
            "Number of people who tested HIV negative and received their results. (Community)",  # HIV negative results (Community)
            "Number of people who tested HIV positive and received results (Inpatient)",  # HIV positive results (Inpatient)
            "Number of people who tested HIV positive and received results (Outpatient)",  # HIV positive results (Outpatient)
            "Number of people who tested HIV positive and received results (Standalone)",  # HIV positive results (Standalone)
            "Number of people who tested HIV positive and received results  (Community)",  # HIV positive results (Community)
            "Number of HTS clients clinically screened for TB (Inpatient)",  # TB screening (Inpatient)
            "Number of HTS clients clinically screened for TB (Outpatient)",  # TB screening (Outpatient)
            "Number of HTS clients clinically screened for TB (Standalone)",  # TB screening (Standalone)
            "Number of HTS clients clinically screened for TB (Community)"  # TB screening (Community)
        ]  # Defines columns for HTS TB Screening gap analysis
        HTS_TB_Screening_columns_spec = [  # Specific columns for summary in report description
            "Number of people who tested HIV negative and received their results. (Inpatient, Outpatient, Standalone)",  # Aggregated HIV negative results
            "Number of HTS clients clinically screened for TB (Inpatient, Outpatient, Standalone)"  # Aggregated TB screening
        ]  # Defines summary columns for documentation
        name = "HTS TB Screening Gap"  # Base name for the report
        Gap_columns = ["HTS TB screening gap"]  # Name for the calculated gap column
        report_name = f"{name}14"  # Report name with suffix for uniqueness
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_HTS_TB_Screening = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF data
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=HTS_TB_Screening_columns  # Include HTS TB Screening columns
        )  # Returns processed DataFrame or None if failed
        if df_HTS_TB_Screening is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate derived metrics
        # -- Step 3.1: Calculate total HTS testing
        df_HTS_TB_Screening["HTS total tested"] = (  # Sum total HIV tested (positive and negative across all settings)
            df_HTS_TB_Screening.iloc[:, 4:12].sum(axis=1)  # Sum columns 4 to 11 (HIV positive and negative results)
        )  # Adds column for total HTS tested individuals

        # -- Step 3.2: Calculate total HTS TB screening
        df_HTS_TB_Screening["HTS TB total screened"] = (  # Sum total TB screenings across all settings
            df_HTS_TB_Screening.iloc[:, 12:16].sum(axis=1)  # Sum columns 12 to 15 (TB screening columns)
        )  # Adds column for total TB screened individuals

        # -- Step 3.3: Calculate HTS TB Screening gap
        df_HTS_TB_Screening[Gap_columns[0]] = np.where(  # Calculate TB screening gap (requires numpy as np)
            df_HTS_TB_Screening["HTS total tested"] != df_HTS_TB_Screening["HTS TB total screened"],  # Check if tested differs from screened
            df_HTS_TB_Screening["HTS TB total screened"] - df_HTS_TB_Screening["HTS total tested"],  # Compute gap (screened - tested)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for TB screening

        # -- Step 4: Drop original columns
        df_HTS_TB_Screening = df_HTS_TB_Screening.drop(  # Remove original detailed columns to focus on derived metrics
            columns=HTS_TB_Screening_columns  # Drop all original HTS TB Screening columns
        )  # Updates DataFrame with only hierarchy, totals, and gap columns

        wrap_column_headers(df_HTS_TB_Screening)  # Format DataFrame column headers for readability (assumed function)

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_HTS_TB_Screening_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_HTS_TB_Screening_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_HTS_TB_Screening.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_HTS_TB_Screening_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_HTS_TB_Screening_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_HTS_TB_Screening,  # Input DataFrame with gap
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=Gap_columns,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_HTS_TB_Screening_gap is None:  # Check if no gaps were found
            if hasattr(process_HTS_TB_Screening_gap, 'cached_style'):  # Check for cached style
                del process_HTS_TB_Screening_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_HTS_TB_Screening_gap, 'cached_shape'):  # Check for cached shape
                del process_HTS_TB_Screening_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the DataFrame
        df_HTS_TB_Screening_gap_style = (  # Apply styling to filtered DataFrame
            df_HTS_TB_Screening_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=Gap_columns)  # Highlight outliers in gap column (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_HTS_TB_Screening_gap.cached_style = df_HTS_TB_Screening_gap_style  # Store styled DataFrame
        process_HTS_TB_Screening_gap.cached_shape = df_HTS_TB_Screening.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Define export variables
        report_month = df_HTS_TB_Screening_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_HTS_TB_Screening_gap[Gap_columns[0]] != 0).any():  # Check for non-zero TB screening gaps
            report_description = (  # Create description for TB screening gap
                f"Report Name: {Gap_columns[0]}\n"  # Gap column name
                f"{HTS_TB_Screening_columns_spec[1]}\n"  # Aggregated TB screening
                f"should be equal to {HTS_TB_Screening_columns_spec[0]}"  # Aggregated HIV negative results
            )  # Define Word document description

        # -- Step 11: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_HTS_TB_Screening_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=HTS_TB_Screening_columns_spec,  # Italicize specific summary columns
                doc_indicators_to_underline=Gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)   # Print display name with separators
            widget_display_df(df_HTS_TB_Screening_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_HTS_TB_Screening_gap, 'cached_style'):  # Check for cached style
            del process_HTS_TB_Screening_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_HTS_TB_Screening_gap, 'cached_shape'):  # Check for cached shape
            del process_HTS_TB_Screening_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - HTS Positive Enrolment gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process HTS Enrolment gap
def process_HTS_Enrolment_gap(display_output=None):
    """
    Process HTS Enrolment gap, exporting results as image, Excel, and Word files.
    Iterates over each cluster, caches styled DataFrames, and displays them on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        HTS_Enrolment_columns = [  # List of column names for HTS Enrolment metrics
            "Number of people tested HIV positive who are successfully enrolled in HIV Care (Inpatient)",  # Enrolled in HIV care (Inpatient)
            "Number of people tested HIV positive who are successfully enrolled in HIV Care (Outpatient)",  # Enrolled in HIV care (Outpatient)
            "Number of people tested HIV positive who are successfully enrolled in HIV Care (Standalone)",  # Enrolled in HIV care (Standalone)
            "Number of people tested HIV positive who are successfully enrolled in HIV Care (Community)"  # Enrolled in HIV care (Community)
        ]  # Defines columns for HTS Enrolment gap analysis
        HTS_Enrolment_columns_spec = [  # Specific columns for summary in report description
            "Number of people tested HIV positive who are successfully enrolled in HIV Care (Inpatient, Outpatient, Standalone)",  # Aggregated enrolment
            "HTS total tested - new positive (excluding previously known)"  # New positives excluding known
        ]  # Defines summary columns for documentation
        columns_to_keep = MSF_hierarchy + ["HTS total tested - new positive (excluding previously known)"]  # Columns to retain from positives data
        Pre_MSF_positives_subset = Pre_MSF_positives_all[columns_to_keep]  # Subset of predefined positives DataFrame (assumed defined elsewhere)
        name = "HTS Enrolment Gap"  # Base name for the report
        Gap_columns = ["HTS enrolment gap"]  # Name for the calculated gap column
        report_name = f"{name}15"  # Report name with suffix for uniqueness

        # -- Step 2: Prepare data
        df_HTS_Enrolment = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF data
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=HTS_Enrolment_columns  # Include HTS Enrolment columns
        )  # Returns processed DataFrame or None if failed
        if df_HTS_Enrolment is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Merge with Pre_MSF_positives_all subset
        df_HTS_Enrolment = Pre_MSF_positives_subset.merge(  # Merge enrolment data with positives subset
            df_HTS_Enrolment,  # Target DataFrame for merge
            on=MSF_hierarchy,  # Merge on hierarchy columns
            how="left"  # Keep all rows from Pre_MSF_positives_subset
        )  # Updates DataFrame with merged data

        # -- Step 4: Sort and clean data
        df_HTS_Enrolment.sort_values(by=MSF_hierarchy, inplace=True, ignore_index=True)  # Sort DataFrame by hierarchy columns and reset index
        df_HTS_Enrolment = df_HTS_Enrolment.fillna(0)  # Replace NaN values with 0
        float_columns = df_HTS_Enrolment.select_dtypes(include=['float64', 'float32']).columns  # Identify columns with float data types
        for col in float_columns:  # Iterate over float columns
            df_HTS_Enrolment[col] = df_HTS_Enrolment[col].astype(int)  # Convert float columns to integers

        # -- Step 5: Calculate derived metrics
        # -- Step 5.1: Calculate total enrolment
        df_HTS_Enrolment[HTS_Enrolment_columns_spec[0]] = (  # Sum enrolment across all settings
            df_HTS_Enrolment[HTS_Enrolment_columns].sum(axis=1)  # Sum specified enrolment columns
        )  # Adds column for total enrolment

        # -- Step 5.2: Calculate enrolment gap
        df_HTS_Enrolment[Gap_columns[0]] = np.where(  # Calculate enrolment gap (requires numpy as np)
            df_HTS_Enrolment[HTS_Enrolment_columns_spec[0]] != df_HTS_Enrolment[HTS_Enrolment_columns_spec[1]],  # Check if enrolled differs from new positives
            df_HTS_Enrolment[HTS_Enrolment_columns_spec[0]] - df_HTS_Enrolment[HTS_Enrolment_columns_spec[1]],  # Compute gap (enrolled - new positives)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for enrolment

        # -- Step 6: Drop original columns
        df_HTS_Enrolment = df_HTS_Enrolment.drop(  # Remove original detailed enrolment columns
            columns=HTS_Enrolment_columns  # Drop specified enrolment columns
        )  # Updates DataFrame with hierarchy, totals, and gap columns

        # -- Step 7: Wrap column headers for better readability
        wrap_column_headers(df_HTS_Enrolment)  # Format DataFrame column headers (assumed function)

        # -- Step 8: Check and display cached styled DataFrames
        if display_output:  # Check if display output is requested
            if hasattr(process_HTS_Enrolment_gap, 'cached_styles'):  # Check if cached styles dictionary exists
                cached_shape = getattr(process_HTS_Enrolment_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_HTS_Enrolment.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    for cluster, style in process_HTS_Enrolment_gap.cached_styles.items():  # Iterate over cached styles by cluster
                        display_name = f"✔️ Displaying {cluster} {report_name}"  # Formatted display name for output
                        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
                        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header
                        print(print_display_name)  # Print display name with separators
                        widget_display_df(style)  # Display cached styled DataFrame (assumed display function)
                    return  # Exit to avoid reprocessing

        # -- Step 9: Initialize cache
        if not hasattr(process_HTS_Enrolment_gap, 'cached_styles'):  # Check if cached styles dictionary is initialized
            process_HTS_Enrolment_gap.cached_styles = {}  # Initialize empty dictionary for caching styles

        # -- Step 10: Identify unique clusters
        cluster_list = pd.Series(df_HTS_Enrolment['Cluster'].unique())  # Extract unique cluster values from DataFrame (requires pandas as pd)

        # -- Step 11: Process each cluster
        for current_cluster in cluster_list:  # Iterate over each unique cluster
            cluster_filtered = df_HTS_Enrolment[df_HTS_Enrolment['Cluster'] == current_cluster]  # Filter DataFrame for current cluster
            
            HTS_Enrolment_msg = f"No {current_cluster} {report_name}"  # Define cluster-specific message for no gaps
            display_name = f"✔️ Displaying {current_cluster} {report_name}"  # Formatted display name for output
            display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
            print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

            cluster_filtered_gap = filter_gap_and_check_empty_df(  # Filter for rows with non-zero gaps
                df=cluster_filtered,  # Input filtered cluster DataFrame
                msg=HTS_Enrolment_msg,  # Message if no gaps found
                opNonZero=Gap_columns,  # Filter for non-zero gap values
                opNeg=None,  # No negative value filter
                opPos=None,  # No positive value filter
                opZero=None,  # No zero value filter
                opLT100=None  # No less-than-100 filter
            )  # Returns filtered DataFrame or None if no gaps

            if cluster_filtered_gap is None:  # Check if no gaps were found for cluster
                if current_cluster in process_HTS_Enrolment_gap.cached_styles:  # Check if cluster is in cache
                    del process_HTS_Enrolment_gap.cached_styles[current_cluster]  # Remove cluster from cache
                continue  # Skip to next cluster

            cluster_filtered_style = (  # Apply styling to filtered cluster DataFrame
                cluster_filtered_gap.style  # Create style object from filtered DataFrame
                .hide(axis='index')  # Hide row index for cleaner display
                .map(outlier_red, subset=Gap_columns)  # Highlight outliers in gap column (assumed function)
            )  # Creates styled DataFrame for display/export

            process_HTS_Enrolment_gap.cached_styles[current_cluster] = cluster_filtered_style  # Store styled DataFrame in cache by cluster

            # -- Step 12: Define export variables
            report_name_cluster = f"{current_cluster}_{report_name}"  # Create cluster-specific report name
            report_month = cluster_filtered_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered cluster DataFrame
            report_image_name = f"{report_month}_{report_name_cluster}.png"  # Create image file name with report month and cluster
            report_sheet_name = f"{current_cluster}_{report_name}"  # Define Excel sheet name with cluster

            # -- Step 13: Create descriptions
            report_description = []  # Initialize empty list for descriptions
            if (cluster_filtered_gap[Gap_columns[0]] != 0).any():  # Check for non-zero enrolment gaps
                report_description.append(  # Add description for enrolment gap
                    f"Report Name: {Gap_columns[0]}\n"  # Gap column name
                    f"{HTS_Enrolment_columns_spec[1]}\n"  # New positives excluding known
                    f"should be equal to {HTS_Enrolment_columns_spec[0]}"  # Aggregated enrolment
                )  # Append description to list
            report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

            # -- Step 14: Export results
            if not display_output:  # Check if user requested to export results
                export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                    report_name=report_name_cluster,  # Cluster-specific report name
                    df_style=cluster_filtered_style,  # Styled DataFrame for export
                    img_file_name=report_image_name,  # Image file name
                    img_file_path=sub_folder2_image_file_msf_outlier,  # Image file path (assumed defined)
                    doc_description=report_description,  # Word document description
                    doc_indicators_to_italicize=HTS_Enrolment_columns_spec,  # Italicize specific summary columns
                    doc_indicators_to_underline=Gap_columns,  # Underline gap column
                    xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                    xlm_sheet_name=report_sheet_name  # Excel sheet name
                )  # Exports to specified formats

            if display_output:  # Check if display is requested
                print(print_display_name)  # Print display name with separators
                widget_display_df(cluster_filtered_style)  # Display styled DataFrame (assumed widget function)

        # -- Step 15: Cache overall unfiltered DataFrame shape
        process_HTS_Enrolment_gap.cached_shape = df_HTS_Enrolment.shape  # Store original unfiltered DataFrame shape

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_HTS_Enrolment_gap, 'cached_styles'):  # Check for cached styles dictionary
            process_HTS_Enrolment_gap.cached_styles.clear()  # Clear cached styles
        if hasattr(process_HTS_Enrolment_gap, 'cached_shape'):  # Check for cached shape
            del process_HTS_Enrolment_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - HTS Couple Counselling gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process HTS Couple Counselling gap
def process_HTS_Couple_Counselling_gap(display_output=None):
    """
    Process HTS Couple Counselling gap, exporting results as image, Excel, and Word files.
    Iterates over data, caches styled DataFrames, and displays them on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the DataFrame for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        HTS_Couple_Counselling_columns = [  # List of column names for HTS Couple Counselling metrics
            "No of couples counselled, tested for HIV and received result (Inpatient)",  # Couples tested in Inpatient setting
            "No of couples counselled, tested for HIV and received result (Outpatient)",  # Couples tested in Outpatient setting
            "No of couples counselled, tested for HIV and received result (Standalone)",  # Couples tested in Standalone setting
            "No of couples counselled, tested for HIV and received result (Community)",  # Couples tested in Community setting
            "No of couples counselled, tested for HIV and received discordant result (Inpatient)",  # Discordant results in Inpatient setting
            "No of couples counselled, tested for HIV and received discordant result (Outpatient)",  # Discordant results in Outpatient setting
            "No of couples counselled, tested for HIV and received discordant result (Standalone)",  # Discordant results in Standalone setting
            "No of couples counselled, tested for HIV and received discordant result (Community)"  # Discordant results in Community setting
        ]  # Defines columns for HTS Couple Counselling gap analysis
        HTS_Couple_Counselling_columns_spec = [  # Specific columns for summary in report description
            "No of couples counselled, tested for HIV and received result (Inpatient, Outpatient, Standalone)",  # Aggregated couples tested
            "No of couples counselled, tested for HIV and received discordant result (Inpatient, Outpatient, Standalone)"  # Aggregated discordant results
        ]  # Defines summary columns for documentation
        name = "HTS Discordant Couple Test Gap"  # Base name for the report
        Gap_columns = ["HTS Discordant Couple Test gap"]  # Name for the calculated gap column
        report_name = f"{name}16"  # Report name with suffix for uniqueness
        No_gap_msg = f"No {report_name}"  # Message to display when no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_HTS_Couple_Counselling = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF data
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=HTS_Couple_Counselling_columns  # Include HTS Couple Counselling columns
        )  # Returns processed DataFrame or None if failed
        if df_HTS_Couple_Counselling is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Sort and clean data
        df_HTS_Couple_Counselling.sort_values(by=MSF_hierarchy, inplace=True, ignore_index=True)  # Sort DataFrame by hierarchy columns and reset index
        df_HTS_Couple_Counselling = df_HTS_Couple_Counselling.fillna(0)  # Replace NaN values with 0
        float_columns = df_HTS_Couple_Counselling.select_dtypes(include=['float64', 'float32']).columns  # Identify columns with float data types
        for col in float_columns:  # Iterate over float columns
            df_HTS_Couple_Counselling[col] = df_HTS_Couple_Counselling[col].astype(int)  # Convert float columns to integers

        # -- Step 4: Calculate derived metrics
        # -- Step 4.1: Calculate total couples counselled and tested
        df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns_spec[0]] = (  # Sum couples tested across specified settings
            df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns[:4]].sum(axis=1)  # Sum first four columns (Inpatient, Outpatient, Standalone, Community)
        )  # Adds column for total couples tested

        # -- Step 4.2: Calculate total discordant results
        df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns_spec[1]] = (  # Sum discordant results across specified settings
            df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns[4:]].sum(axis=1)  # Sum last four columns (discordant results)
        )  # Adds column for total discordant results

        # -- Step 4.3: Calculate discordant couple test gap
        df_HTS_Couple_Counselling[Gap_columns[0]] = np.where(  # Calculate gap (requires numpy as np)
            df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns_spec[1]] > df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns_spec[0]],  # Check if discordant results exceed total tested
            df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns_spec[0]] - df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns_spec[1]],  # Compute gap (tested - discordant)
            0  # Set to 0 if no gap (discordant <= tested)
        )  # Adds gap column for discordant results

        # -- Step 5: Drop original columns
        df_HTS_Couple_Counselling = df_HTS_Couple_Counselling.drop(  # Remove original detailed columns
            columns=HTS_Couple_Counselling_columns  # Drop specified counselling columns
        )  # Updates DataFrame with hierarchy, totals, and gap columns

        # -- Step 6: Wrap column headers for better readability
        wrap_column_headers(df_HTS_Couple_Counselling)  # Format DataFrame column headers (assumed function)
        Gap_columns_wrap = wrap_column_headers2(Gap_columns)  # Format gap column headers (assumed function)

        # -- Step 7: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_HTS_Couple_Counselling_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_HTS_Couple_Counselling_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_HTS_Couple_Counselling.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(process_HTS_Couple_Counselling_gap.cached_style)  # Display cached styled DataFrame (assumed display function)
                    return  # Exit to avoid reprocessing

        # -- Step 8: Filter and validate gaps
        df_HTS_Couple_Counselling_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for non-zero gaps
            df=df_HTS_Couple_Counselling,  # Input DataFrame
            msg=No_gap_msg,  # Message if no gaps found
            opNonZero=Gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_HTS_Couple_Counselling_gap is None:  # Check if no gaps were found
            if hasattr(process_HTS_Couple_Counselling_gap, 'cached_style'):  # Check if cache exists
                del process_HTS_Couple_Counselling_gap.cached_style  # Clear cached style
            if hasattr(process_HTS_Couple_Counselling_gap, 'cached_shape'):  # Check if cached shape exists
                del process_HTS_Couple_Counselling_gap.cached_shape  # Clear cached shape
            return  # Exit function

        # -- Step 9: Style the DataFrame
        df_HTS_Couple_Counselling_gap_style = (  # Apply styling to filtered DataFrame
            df_HTS_Couple_Counselling_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=Gap_columns)  # Highlight outliers in gap column (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 10: Cache styled DataFrame and shape
        process_HTS_Couple_Counselling_gap.cached_style = df_HTS_Couple_Counselling_gap_style  # Store styled DataFrame in cache
        process_HTS_Couple_Counselling_gap.cached_shape = df_HTS_Couple_Counselling.shape  # Store original unfiltered DataFrame shape

        # -- Step 11: Define export variables
        report_month = df_HTS_Couple_Counselling_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = sub_folder2_image_file_msf_outlier  # Image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name

        # -- Step 12: Create descriptions for Word document
        report_description = []  # Initialize empty list for descriptions
        if (df_HTS_Couple_Counselling_gap[Gap_columns[0]] != 0).any():  # Check for non-zero gaps
            report_description.append(  # Add description for discordant couple test gap
                f"Report Name: {Gap_columns[0]}\n"  # Gap column name
                f"{HTS_Couple_Counselling_columns_spec[1]}\n"  # Total discordant results
                f"should be less than or equal to {HTS_Couple_Counselling_columns_spec[0]}"  # Total couples tested
            )  # Append description to list
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

        # -- Step 13: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name
                df_style=df_HTS_Couple_Counselling_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=HTS_Couple_Counselling_columns_spec,  # Italicize specific summary columns
                doc_indicators_to_underline=Gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 14: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_HTS_Couple_Counselling_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_HTS_Couple_Counselling_gap, 'cached_style'):  # Check for cached style
            del process_HTS_Couple_Counselling_gap.cached_style  # Clear cached style
        if hasattr(process_HTS_Couple_Counselling_gap, 'cached_shape'):  # Check for cached shape
            del process_HTS_Couple_Counselling_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - HTS CD4 Test gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process HTS CD4 gap
def process_HTS_CD4_gap(display_output=None):
    """
    Process HTS CD4 gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        HTS_CD4_columns = [  # List of column names for HTS CD4 metrics
            "Number of newly diagnosed PLHIV who received CD4 <200 cells/mm3 test during the reporting period\xa0\xa0(Inpatient)",  # CD4 <200 (Inpatient)
            "Number of newly diagnosed PLHIV who received CD4 <200 cells/mm3 test during the reporting period\xa0\xa0(Outpatient)",  # CD4 <200 (Outpatient)
            "Number of newly diagnosed PLHIV who received CD4 <200 cells/mm3 test during the reporting period\xa0\xa0 (Standalone)",  # CD4 <200 (Standalone)
            "Number of newly diagnosed PLHIV who received CD4 <200 cells/mm3 test during the reporting period\xa0\xa0(Community)",  # CD4 <200 (Community)
            "Number of newly diagnosed PLHIV who received CD4 >200 cells/mm3 test during the reporting period\xa0\xa0(Inpatient)",  # CD4 >200 (Inpatient)
            "Number of newly diagnosed PLHIV who received CD4 >200 cells/mm3 test during the reporting period\xa0\xa0(Outpatient)",  # CD4 >200 (Outpatient)
            "Number of newly diagnosed PLHIV who received CD4 >200 cells/mm3 test during the reporting period\xa0\xa0(Standalone)",  # CD4 >200 (Standalone)
            "Number of newly diagnosed PLHIV who received CD4 >200 cells/mm3 test during the reporting period\xa0\xa0(Community)"  # CD4 >200 (Community)
        ]  # Defines columns for HTS CD4 gap analysis
        HTS_CD4_columns_spec = [  # Specific columns for summary in report description
            "Number of newly diagnosed PLHIV who received CD4 <200 cells/mm3 test during the reporting period  (Inpatient, Outpatient, Standalone)",  # Aggregated CD4 <200
            "Number of newly diagnosed PLHIV who received CD4 >200 cells/mm3 test during the reporting period  (Inpatient, Outpatient, Standalone)",  # Aggregated CD4 >200
            "HTS total tested - new positive (excluding previously known)",  # New positives excluding known
            "Total Number of newly diagnosed PLHIV who received CD4 (<200 and >200) cells/mm3"  # Total CD4 tested
        ]  # Defines summary columns for documentation
        columns_to_keep = MSF_hierarchy + ["HTS total tested - new positive (excluding previously known)"]  # Columns to retain from positives data
        Pre_MSF_positives_subset = Pre_MSF_positives_all[columns_to_keep]  # Subset of predefined positives DataFrame (assumed defined elsewhere)
        name = "HTS CD4 Gap"  # Base name for the report
        Gap_columns = ["HTS CD4 gap"]  # Name for the calculated gap column
        report_name = f"{name}17"  # Report name with suffix for uniqueness

        # -- Step 2: Prepare data
        df_HTS_CD4 = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF data
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=HTS_CD4_columns  # Include HTS CD4 columns
        )  # Returns processed DataFrame or None if failed
        if df_HTS_CD4 is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Merge with Pre_MSF_positives_all subset
        df_HTS_CD4 = Pre_MSF_positives_subset.merge(  # Merge CD4 data with positives subset
            df_HTS_CD4,  # Target DataFrame for merge
            on=MSF_hierarchy,  # Merge on hierarchy columns
            how="left"  # Keep all rows from Pre_MSF_positives_subset
        )  # Updates DataFrame with merged data

        # -- Step 4: Sort and clean data
        df_HTS_CD4.sort_values(by=MSF_hierarchy, inplace=True, ignore_index=True)  # Sort DataFrame by hierarchy columns and reset index
        df_HTS_CD4 = df_HTS_CD4.fillna(0)  # Replace NaN values with 0
        float_columns = df_HTS_CD4.select_dtypes(include=['float64', 'float32']).columns  # Identify columns with float data types
        for col in float_columns:  # Iterate over float columns
            df_HTS_CD4[col] = df_HTS_CD4[col].astype(int)  # Convert float columns to integers

        # -- Step 5: Calculate derived metrics
        # -- Step 5.1: Calculate total CD4 <200
        df_HTS_CD4[HTS_CD4_columns_spec[0]] = (  # Sum total CD4 <200 tests across facility settings
            df_HTS_CD4.iloc[:, 5:9].sum(axis=1)  # Sum columns 5 to 8 (Inpatient, Outpatient, Standalone, Community for CD4 <200)
        )  # Adds column for total CD4 <200 tests

        # -- Step 5.2: Calculate total CD4 >200
        df_HTS_CD4[HTS_CD4_columns_spec[1]] = (  # Sum total CD4 >200 tests across facility settings
            df_HTS_CD4.iloc[:, 9:13].sum(axis=1)  # Sum columns 9 to 12 (Inpatient, Outpatient, Standalone, Community for CD4 >200)
        )  # Adds column for total CD4 >200 tests

        # -- Step 5.3: Calculate total CD4 <200 and >200
        df_HTS_CD4[HTS_CD4_columns_spec[3]] = (  # Sum total CD4 tests (both <200 and >200)
            df_HTS_CD4[HTS_CD4_columns_spec[0]] + df_HTS_CD4[HTS_CD4_columns_spec[1]]  # Add CD4 <200 and >200 totals
        )  # Adds column for total CD4 tests

        # -- Step 5.4: Calculate CD4 gap
        df_HTS_CD4[Gap_columns[0]] = np.where(  # Calculate CD4 gap (requires numpy as np)
            df_HTS_CD4[HTS_CD4_columns_spec[2]] != df_HTS_CD4[HTS_CD4_columns_spec[3]],  # Check if new positives differ from total CD4 tested
            df_HTS_CD4[HTS_CD4_columns_spec[2]] - df_HTS_CD4[HTS_CD4_columns_spec[3]],  # Compute gap (new positives - total CD4 tested)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for CD4 testing

        # -- Step 6: Drop original columns
        columns_to_drop = HTS_CD4_columns + [HTS_CD4_columns_spec[0], HTS_CD4_columns_spec[1]]  # Combine original and aggregated CD4 columns to drop
        df_HTS_CD4 = df_HTS_CD4.drop(  # Remove specified columns to focus on key metrics
            columns=columns_to_drop  # Drop original and intermediate CD4 columns
        )  # Updates DataFrame with hierarchy, new positives, total CD4, and gap columns

        # -- Step 7: Wrap column headers for better readability
        wrap_column_headers(df_HTS_CD4)  # Format DataFrame column headers (assumed function)

        # -- Step 8: Check and display cached styled DataFrames
        if display_output:  # Check if display output is requested
            if hasattr(process_HTS_CD4_gap, 'cached_styles'):  # Check if cached styles dictionary exists
                cached_shape = getattr(process_HTS_CD4_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_HTS_CD4.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    for cluster, style in process_HTS_CD4_gap.cached_styles.items():  # Iterate over cached styles by cluster
                        display_name = f"✔️ Displaying {cluster} {report_name}"  # Formatted display name for output
                        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
                        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header
                        print(print_display_name)  # Print display name with separators
                        widget_display_df(style)  # Display cached styled DataFrame (assumed widget function)
                    return  # Exit to avoid reprocessing

        # -- Step 9: Initialize cache
        if not hasattr(process_HTS_CD4_gap, 'cached_styles'):  # Check if cached styles dictionary is initialized
            process_HTS_CD4_gap.cached_styles = {}  # Initialize empty dictionary for caching styles

        # -- Step 10: Identify unique clusters
        cluster_list = pd.Series(df_HTS_CD4['Cluster'].unique())  # Extract unique cluster values from DataFrame (requires pandas as pd)

        # -- Step 11: Process each cluster
        for current_cluster in cluster_list:  # Iterate over each unique cluster
            cluster_filtered = df_HTS_CD4[df_HTS_CD4['Cluster'] == current_cluster]  # Filter DataFrame for current cluster
            
            HTS_CD4_msg = f"No {current_cluster} {report_name}"  # Define cluster-specific message for no gaps
            display_name = f"✔️ Displaying {current_cluster} {report_name}"  # Formatted display name for output
            display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
            print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

            cluster_filtered_gap = filter_gap_and_check_empty_df(  # Filter for rows with non-zero gaps
                df=cluster_filtered,  # Input filtered cluster DataFrame
                msg=HTS_CD4_msg,  # Message if no gaps found
                opNonZero=Gap_columns,  # Filter for non-zero gap values
                opNeg=None,  # No negative value filter
                opPos=None,  # No positive value filter
                opZero=None,  # No zero value filter
                opLT100=None  # No less-than-100 filter
            )  # Returns filtered DataFrame or None if no gaps

            if cluster_filtered_gap is None:  # Check if no gaps were found for cluster
                if current_cluster in process_HTS_CD4_gap.cached_styles:  # Check if cluster is in cache
                    del process_HTS_CD4_gap.cached_styles[current_cluster]  # Remove cluster from cache
                continue  # Skip to next cluster

            cluster_filtered_style = (  # Apply styling to filtered cluster DataFrame
                cluster_filtered_gap.style  # Create style object from filtered DataFrame
                .hide(axis='index')  # Hide row index for cleaner display
                .map(outlier_red, subset=Gap_columns)  # Highlight outliers in gap column (assumed function)
            )  # Creates styled DataFrame for display/export

            process_HTS_CD4_gap.cached_styles[current_cluster] = cluster_filtered_style  # Store styled DataFrame in cache by cluster

            # -- Step 12: Define export variables
            report_name_cluster = f"{current_cluster}_{report_name}"  # Create cluster-specific report name
            report_month = cluster_filtered_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered cluster DataFrame
            report_image_name = f"{report_month}_{report_name_cluster}.png"  # Create image file name with report month and cluster
            report_sheet_name = f"{current_cluster}_{report_name}"  # Define Excel sheet name with cluster

            # -- Step 13: Create descriptions
            if (cluster_filtered_gap[Gap_columns[0]] != 0).any():  # Check for non-zero CD4 gaps
                report_description = (  # Add description for CD4 gap
                    f"Report Name: {Gap_columns[0]}\n"  # Gap column name
                    f"{HTS_CD4_columns_spec[2]}\nshould be equal to {HTS_CD4_columns_spec[3]}\n"  # New positives vs. total CD4 tested
                    f"Note: Where this CD4 gap is true, please ignore the outlier."  # Instruction to ignore outlier
                )  

            # -- Step 14: Export results
            if not display_output:  # Check if user requested to export results
                export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                    report_name=report_name_cluster,  # Cluster-specific report name
                    df_style=cluster_filtered_style,  # Styled DataFrame for export
                    img_file_name=report_image_name,  # Image file name
                    img_file_path=sub_folder2_image_file_msf_outlier,  # Image file path (assumed defined)
                    doc_description=report_description,  # Word document description
                    doc_indicators_to_italicize=HTS_CD4_columns_spec,  # Italicize specific summary columns
                    doc_indicators_to_underline=Gap_columns,  # Underline gap column
                    xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                    xlm_sheet_name=report_sheet_name  # Excel sheet name
                )  # Exports to specified formats

            if display_output:  # Check if display is requested
                print(print_display_name)  # Print display name with separators
                widget_display_df(cluster_filtered_style)  # Display styled DataFrame (assumed widget function)

        # -- Step 15: Cache overall unfiltered DataFrame shape
        process_HTS_CD4_gap.cached_shape = df_HTS_CD4.shape  # Store original unfiltered DataFrame shape

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_HTS_CD4_gap, 'cached_styles'):  # Check for cached styles dictionary
            process_HTS_CD4_gap.cached_styles.clear()  # Clear cached styles
        if hasattr(process_HTS_CD4_gap, 'cached_shape'):  # Check for cached shape
            del process_HTS_CD4_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

## - HIVST MSF
### - HIVST Distribution Mode gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process HIVST Distribution Mode gap
def process_HIVST_Distr_Mode_gap(display_output=None):
    """
    Process HIVST Distribution Mode gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for HIVST distribution by mode
            "Number of individual HIVST kits distributed - Assisted (Distribution By)",  # Assisted distribution by specific recipients
            "Number of individual HIVST kits distributed - Uassisted (Distribution By)"  # Unassisted distribution by specific recipients
        ]  # Defines primary HIVST distribution columns
        df_columns_spec = [  # Specific column names for summary and comparison
            "Number of individual HIVST kits distributed - Assisted",  # Total assisted distribution
            "Number of individual HIVST kits distributed - Uassisted",  # Total unassisted distribution
            "Number of individual HIVST kits distributed - Assisted (Distribution By - Self, Spouse, Sexual Partner, Children, Social Network, Other)",  # Assisted distribution by recipients
            "Number of individual HIVST kits distributed - Uassisted (Distribution By - Self, Spouse, Sexual Partner, Children, Social Network, Other)"  # Unassisted distribution by recipients
        ]  # Defines columns for gap calculation and reporting
        columns_to_keep = MSF_hierarchy + ["Assisted", "Unassisted"]  # Columns to retain from HIVST approach dataset
        name = "HIVST Distribution Mode Gap"  # Base name for the report
        gap_columns = ["Assisted distribution mode gap", "Uassisted distribution mode gap"]  # Names for calculated gap columns
        report_name = f"{name}18"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_HIVST_Distri_Mode = prepare_and_convert_df(  # Fetch and prepare primary DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include HIVST distribution columns
        )  # Returns processed DataFrame or None if failed
        if df_HIVST_Distri_Mode is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Fetch and prepare additional HIVST mode data
        df_HIVST_Mode_extra = prepare_and_convert_df(  # Fetch and prepare DataFrame for HIVST approach
            DHIS2_data_key="HTS MSF_hivst_approach",  # Key to fetch HIVST approach dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns
            data_columns=["Assisted", "Unassisted"]  # Include total assisted and unassisted columns
        )  # Returns processed DataFrame or None if failed
        if df_HIVST_Mode_extra is None:  # Check if extra data preparation failed or returned empty
            return  # Exit function if no valid extra data

        df_HIVST_Mode_extra = df_HIVST_Mode_extra[columns_to_keep]  # Subset to retain only hierarchy and total assisted/unassisted columns

        # -- Step 4: Merge datasets
        df_HIVST_Distri_Mode = df_HIVST_Mode_extra.merge(  # Merge extra HIVST mode data with primary distribution data
            df_HIVST_Distri_Mode,  # Target DataFrame for merge
            on=MSF_hierarchy,  # Merge on hierarchy columns
            how="right"  # Right join to keep all rows from primary distribution data
        )  # Updates DataFrame with merged data

        # -- Step 5: Rename columns for consistency
        df_HIVST_Distri_Mode = df_HIVST_Distri_Mode.rename(columns={
            "Assisted": f"{df_columns_spec[0]}",  # Rename to total assisted distribution
            "Unassisted": f"{df_columns_spec[1]}",  # Rename to total unassisted distribution
            "Number of individual HIVST kits distributed - Assisted (Distribution By)": f"{df_columns_spec[2]}",  # Rename to assisted by recipients
            "Number of individual HIVST kits distributed - Uassisted (Distribution By)": f"{df_columns_spec[3]}"  # Rename to unassisted by recipients
        })  # Updates column names to align with specified naming

        # -- Step 6: Clean and format data
        df_HIVST_Distri_Mode.sort_values(by=MSF_hierarchy, inplace=True, ignore_index=True)  # Sort DataFrame by hierarchy columns and reset index
        df_HIVST_Distri_Mode = df_HIVST_Distri_Mode.fillna(0)  # Replace NaN values with 0
        float_columns = df_HIVST_Distri_Mode.select_dtypes(include=['float64', 'float32']).columns  # Identify columns with float data types
        for col in float_columns:  # Iterate over float columns
            df_HIVST_Distri_Mode[col] = df_HIVST_Distri_Mode[col].astype(int)  # Convert float columns to integers

        # -- Step 7: Calculate gaps
        df_HIVST_Distri_Mode[gap_columns[0]] = np.where(  # Calculate gap for assisted distribution (requires numpy as np)
            df_HIVST_Distri_Mode[df_columns_spec[2]] != df_HIVST_Distri_Mode[df_columns_spec[0]],  # Check if assisted by recipients differs from total assisted
            df_HIVST_Distri_Mode[df_columns_spec[2]] - df_HIVST_Distri_Mode[df_columns_spec[0]],  # Compute gap (by recipients - total)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for assisted distribution
        df_HIVST_Distri_Mode[gap_columns[1]] = np.where(  # Calculate gap for unassisted distribution
            df_HIVST_Distri_Mode[df_columns_spec[3]] != df_HIVST_Distri_Mode[df_columns_spec[1]],  # Check if unassisted by recipients differs from total unassisted
            df_HIVST_Distri_Mode[df_columns_spec[3]] - df_HIVST_Distri_Mode[df_columns_spec[1]],  # Compute gap (by recipients - total)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for unassisted distribution

        # -- Step 8: Reorder columns for output
        reorder_columns = MSF_hierarchy + [
            df_columns_spec[0], df_columns_spec[2], gap_columns[0], 
            df_columns_spec[1], df_columns_spec[3], gap_columns[1]
        ]  # Define column order: hierarchy, assisted pair, assisted gap, unassisted pair, unassisted gap
        df_HIVST_Distri_Mode = df_HIVST_Distri_Mode[reorder_columns]  # Reorder DataFrame columns for consistent output

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_HIVST_Distri_Mode)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_HIVST_Distr_Mode_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_HIVST_Distr_Mode_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_HIVST_Distri_Mode.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_HIVST_Distr_Mode_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)   
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 11: Filter and validate gaps
        df_HIVST_Distri_Mode_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_HIVST_Distri_Mode,  # Input DataFrame with gaps
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_HIVST_Distri_Mode_gap is None:  # Check if no gaps were found
            if hasattr(process_HIVST_Distr_Mode_gap, 'cached_style'):  # Check for cached style
                del process_HIVST_Distr_Mode_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_HIVST_Distr_Mode_gap, 'cached_shape'):  # Check for cached shape
                del process_HIVST_Distr_Mode_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 12: Style the filtered DataFrame
        df_HIVST_Distri_Mode_gap_style = (  # Apply styling to filtered DataFrame
            df_HIVST_Distri_Mode_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 13: Cache styled DataFrame and shape
        process_HIVST_Distr_Mode_gap.cached_style = df_HIVST_Distri_Mode_gap_style  # Store styled DataFrame
        process_HIVST_Distr_Mode_gap.cached_shape = df_HIVST_Distri_Mode.shape  # Store original unfiltered DataFrame shape

        # -- Step 14: Prepare export variables
        report_month = df_HIVST_Distri_Mode_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 15: Create descriptions for Word document
        report_description = []  # Initialize empty list for descriptions
        if (df_HIVST_Distri_Mode_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero assisted distribution gaps
            report_description.append(  # Add description for assisted gap
                f"Report Name: {gap_columns[0]}\n"  # Assisted gap column name
                f"{df_columns_spec[1]}\n"  # Total unassisted (incorrect reference, preserved as-is)
                f"should be equal to {df_columns_spec[0]}"  # Total assisted
            )  # Append description to list
        if (df_HIVST_Distri_Mode_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero unassisted distribution gaps
            report_description.append(  # Add description for unassisted gap
                f"Report Name: {gap_columns[1]}\n"  # Unassisted gap column name
                f"{df_columns_spec[3]}\n"  # Unassisted by recipients
                f"should be equal to {df_columns_spec[1]}"  # Total unassisted
            )  # Append description to list
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

        # -- Step 16: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_HIVST_Distri_Mode_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns_spec,  # Italicize specified summary columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 17: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)
            widget_display_df(df_HIVST_Distri_Mode_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_HIVST_Distr_Mode_gap, 'cached_style'):  # Check for cached style
            del process_HIVST_Distr_Mode_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_HIVST_Distr_Mode_gap, 'cached_shape'):  # Check for cached shape
            del process_HIVST_Distr_Mode_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - HIVST Testing Frequency gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process HIVST Testing Frequency gap
def process_HIVST_Test_Freq_gap(display_output=None):
    """
    Process HIVST Testing Frequency gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for HIVST testing frequency metrics
            "Number of individual HIVST kits distributed (Directly Assisted & Unassisted)",  # Total kits distributed
            "Number of individual HIVST kits distributed (Testing Frequency)"  # Kits by testing frequency
        ]  # Defines columns for HIVST testing frequency gap analysis
        name = "HIVST Testing Frequency Gap"  # Base name for the report
        gap_columns = ["HIVST Testing Frequency gap"]  # Name for the calculated gap column
        report_name = f"{name}19"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_HIVST_Test_Freq = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include HIVST testing frequency columns
        )  # Returns processed DataFrame or None if failed
        if df_HIVST_Test_Freq is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate HIVST testing frequency gap
        df_HIVST_Test_Freq[gap_columns[0]] = np.where(  # Calculate gap (requires numpy as np)
            df_HIVST_Test_Freq[df_columns[1]] > df_HIVST_Test_Freq[df_columns[0]],  # Check if testing frequency exceeds total distributed
            df_HIVST_Test_Freq[df_columns[1]] - df_HIVST_Test_Freq[df_columns[0]],  # Compute gap (frequency - total)
            0  # Set to 0 if no gap (frequency not greater than total)
        )  # Adds gap column for testing frequency

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_HIVST_Test_Freq)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_HIVST_Test_Freq_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_HIVST_Test_Freq_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_HIVST_Test_Freq.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_HIVST_Test_Freq_gap.cached_style # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_HIVST_Test_Freq_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_HIVST_Test_Freq,  # Input DataFrame with gap
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_HIVST_Test_Freq_gap is None:  # Check if no gaps were found
            if hasattr(process_HIVST_Test_Freq_gap, 'cached_style'):  # Check for cached style
                del process_HIVST_Test_Freq_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_HIVST_Test_Freq_gap, 'cached_shape'):  # Check for cached shape
                del process_HIVST_Test_Freq_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the DataFrame
        df_HIVST_Test_Freq_gap_style = (  # Apply styling to filtered DataFrame
            df_HIVST_Test_Freq_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_HIVST_Test_Freq_gap.cached_style = df_HIVST_Test_Freq_gap_style  # Store styled DataFrame
        process_HIVST_Test_Freq_gap.cached_shape = df_HIVST_Test_Freq.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Define export variables
        report_month = df_HIVST_Test_Freq_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_HIVST_Test_Freq_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero testing frequency gaps
            report_description = (  # Create description for testing frequency gap
                f"Report Name: {gap_columns[0]}\n"  # Gap column name
                f"{df_columns[1]}\n"  # Testing frequency
                f"should be equal to {df_columns[0]}"  # Total distributed
            )  # Define Word document description

        # -- Step 11: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_HIVST_Test_Freq_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            widget_display_df(df_HIVST_Test_Freq_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_HIVST_Test_Freq_gap, 'cached_style'):  # Check for cached style
            del process_HIVST_Test_Freq_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_HIVST_Test_Freq_gap, 'cached_shape'):  # Check for cached shape
            del process_HIVST_Test_Freq_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - HIVST Result gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process HIVST Result gap
def process_HIVST_Result_gap(display_output=None):
    """
    Process HIVST Result gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for HIVST result metrics
            "Number of individual HIVST kits distributed (Directly Assisted & Unassisted)",  # Total kits distributed
            "Number of individual reporting HIVST results"  # Kits with reported results
        ]  # Defines columns for HIVST result gap analysis
        name = "HIVST Result Gap"  # Base name for the report
        gap_columns = ["HIVST Result gap"]  # Name for the calculated gap column
        report_name = f"{name}20"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_HIVST_Result = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include HIVST result columns
        )  # Returns processed DataFrame or None if failed
        if df_HIVST_Result is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate HIVST result gap
        df_HIVST_Result[gap_columns[0]] = np.where(  # Calculate gap (requires numpy as np)
            df_HIVST_Result[df_columns[1]] != df_HIVST_Result[df_columns[0]],  # Check if reported results differ from total distributed
            df_HIVST_Result[df_columns[0]] - df_HIVST_Result[df_columns[1]],  # Compute gap (distributed - reported)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for HIVST results

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_HIVST_Result)  # Format DataFrame column headers (assumed function)

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_HIVST_Result_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_HIVST_Result_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_HIVST_Result.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_HIVST_Result_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_HIVST_Test_Freq_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_HIVST_Result,  # Input DataFrame with gap
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_HIVST_Test_Freq_gap is None:  # Check if no gaps were found
            if hasattr(process_HIVST_Result_gap, 'cached_style'):  # Check for cached style
                del process_HIVST_Result_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_HIVST_Result_gap, 'cached_shape'):  # Check for cached shape
                del process_HIVST_Result_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the DataFrame
        df_HIVST_Test_Freq_gap_style = (  # Apply styling to filtered DataFrame
            df_HIVST_Test_Freq_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_yellow, subset=gap_columns)  # Highlight non-zero gaps in yellow (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_HIVST_Result_gap.cached_style = df_HIVST_Test_Freq_gap_style  # Store styled DataFrame
        process_HIVST_Result_gap.cached_shape = df_HIVST_Result.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Define export variables
        report_month = df_HIVST_Test_Freq_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_HIVST_Test_Freq_gap[gap_columns[0]] != 0).any():  # Check for non-zero result gaps
            report_description = (  # Create description for result gap
                f"Report Name: {gap_columns[0]}\n"  # Gap column name
                f"{df_columns[1]}\n"  # Reported results
                f"should be equal to {df_columns[0]}"  # Total distributed
                f"Note: Where this report is correct, please ignore the gap - only review."  # Instruction to review gap
            )  # Define Word document description

        # -- Step 11: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_HIVST_Test_Freq_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)
            widget_display_df(df_HIVST_Test_Freq_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_HIVST_Result_gap, 'cached_style'):  # Check for cached style
            del process_HIVST_Result_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_HIVST_Result_gap, 'cached_shape'):  # Check for cached shape
            del process_HIVST_Result_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - HIVST Reactive Confirmation and Linkage gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process HIVST Reactive and Linkage gap
def process_HIVST_Reactive_Link_gap(display_output=None):
    """
    Process HIVST Reactive and Linkaget gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for HIVST reactive and linkage metrics
            "Number of individuals reporting reactive HIVST results referred for confirmatory test (HTS)",  # Referred for confirmatory test
            "Number of individuals reporting reactive HIVST results referred for confirmatory test(HTS) who received HIV positive test results.",  # Confirmed positive
            "Number of individuals reporting reactive HIVST results referred for confirmatory test(HTS) who received HIV negative test results.",  # Confirmed negative
            "Number of individuals with confirmed HIV-positive results who are successfully linked with HIV care and treatment"  # Linked to care
        ]  # Defines primary HIVST columns for gap analysis
        df_columns_spec = [  # Specific column names for summary and reporting
            "Number of individual reporting HIVST results - Reactive",  # Total reactive results
            "Number of individuals reporting reactive HIVST results referred for confirmatory test (HTS)",  # Referred for confirmatory test
            "Number of individuals reporting reactive HIVST results referred for confirmatory test(HTS) who received HIV positive test results.",  # Confirmed positive
            "Number of individuals reporting reactive HIVST results referred for confirmatory test(HTS) who received HIV negative test results.",  # Confirmed negative
            "Number of individuals with confirmed HIV-positive results who are successfully linked with HIV care and treatment"  # Linked to care
        ]  # Defines columns for gap calculation and documentation
        df_columns2 = ['Reactive']  # Column for additional reactive data from second dataset
        name = "HIVST Reactive Linkage Gap"  # Base name for the report
        gap_columns = [  # Names for calculated gap columns
            "HIVST reactive referral for confirmatory gap", 
            "HIVST confirmatory test result gap",
            "HIVST confirmed positive linkage gap"
        ]  # Defines gap column names
        report_name = f"{name}21"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_HIVST_Reactive_Link = prepare_and_convert_df(  # Fetch and prepare primary DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include HIVST reactive and linkage columns
        )  # Returns processed DataFrame or None if failed
        if df_HIVST_Reactive_Link is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Fetch and prepare additional HIVST mode data
        df_main2 = prepare_and_convert_df(  # Fetch and prepare DataFrame for HIVST response classification
            DHIS2_data_key="HTS MSF_hivst_response_classification",  # Key to fetch HIVST response dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns
            data_columns=df_columns2  # Include reactive column
        )  # Returns processed DataFrame or None if failed
        if df_main2 is None:  # Check if extra data preparation failed or returned empty
            return  # Exit function if no valid extra data

        # -- Step 4: Merge datasets
        df_HIVST_Reactive_Link = df_HIVST_Reactive_Link.merge(  # Merge primary data with additional reactive data
            df_main2,  # Target DataFrame for merge
            on=MSF_hierarchy,  # Merge on hierarchy columns
            how="left"  # Left join to keep all rows from primary data
        )  # Updates DataFrame with merged data
        
        # -- Step 5: Rename columns for consistency
        df_HIVST_Reactive_Link = df_HIVST_Reactive_Link.rename(columns={  # Rename columns to align with specified names
            f"{df_columns2[0]}": f"{df_columns_spec[0]}"  # Rename 'Reactive' to total reactive results
        })  # Updates column names in DataFrame

        # -- Step 6: Clean and format data
        df_HIVST_Reactive_Link.sort_values(by=MSF_hierarchy, inplace=True, ignore_index=True)  # Sort DataFrame by hierarchy columns and reset index
        df_HIVST_Reactive_Link = df_HIVST_Reactive_Link.fillna(0)  # Replace NaN values with 0
        float_columns = df_HIVST_Reactive_Link.select_dtypes(include=['float64', 'float32']).columns  # Identify columns with float data types
        for col in float_columns:  # Iterate over float columns
            df_HIVST_Reactive_Link[col] = df_HIVST_Reactive_Link[col].astype(int)  # Convert float columns to integers
        df_HIVST_Reactive_Link = df_HIVST_Reactive_Link[MSF_hierarchy + df_columns_spec]  # Reorder DataFrame to include hierarchy and specified columns

        # -- Step 7: Calculate gaps
        df_HIVST_Reactive_Link[gap_columns[0]] = np.where(  # Calculate referral for confirmatory test gap (requires numpy as np)
            df_HIVST_Reactive_Link[df_columns_spec[1]] != df_HIVST_Reactive_Link[df_columns_spec[0]],  # Check if referred differs from total reactive
            df_HIVST_Reactive_Link[df_columns_spec[1]] - df_HIVST_Reactive_Link[df_columns_spec[0]],  # Compute gap (referred - reactive)
            0  # Set to 0 if no gap (values equal)
        )  # Adds referral gap column
        df_HIVST_Reactive_Link[gap_columns[1]] = np.where(  # Calculate confirmatory test result gap
            df_HIVST_Reactive_Link[df_columns_spec[2:4]].sum(axis=1) != df_HIVST_Reactive_Link[df_columns_spec[1]],  # Check if sum of positive and negative results differs from referred
            df_HIVST_Reactive_Link[df_columns_spec[2:4]].sum(axis=1) - df_HIVST_Reactive_Link[df_columns_spec[1]],  # Compute gap (results sum - referred)
            0  # Set to 0 if no gap (values equal)
        )  # Adds confirmatory test gap column
        df_HIVST_Reactive_Link[gap_columns[2]] = np.where(  # Calculate confirmed positive linkage gap
            df_HIVST_Reactive_Link[df_columns_spec[2]] != df_HIVST_Reactive_Link[df_columns_spec[4]],  # Check if linked differs from confirmed positive
            df_HIVST_Reactive_Link[df_columns_spec[4]] - df_HIVST_Reactive_Link[df_columns_spec[2]],  # Compute gap (linked - confirmed positive)
            0  # Set to 0 if no gap (values equal)
        )  # Adds linkage gap column

        # -- Step 8: Reorder columns for output
        reorder_columns = MSF_hierarchy + [  # Define column order: hierarchy, reactive pair, referral gap, test results pair, test gap, linkage pair
            df_columns_spec[0], df_columns_spec[1], gap_columns[0], 
            df_columns_spec[2], df_columns_spec[3], gap_columns[1],
            df_columns_spec[4], gap_columns[2]
        ]  # Specifies ordered columns for DataFrame
        df_HIVST_Reactive_Link = df_HIVST_Reactive_Link[reorder_columns]  # Reorder DataFrame columns for consistent output

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_HIVST_Reactive_Link)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_HIVST_Reactive_Link_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_HIVST_Reactive_Link_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_HIVST_Reactive_Link.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_HIVST_Reactive_Link_gap.cached_style # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 11: Filter and validate gaps
        df_HIVST_Reactive_Link_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_HIVST_Reactive_Link,  # Input DataFrame with gaps
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_HIVST_Reactive_Link_gap is None:  # Check if no gaps were found
            if hasattr(process_HIVST_Reactive_Link_gap, 'cached_style'):  # Check for cached style
                del process_HIVST_Reactive_Link_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_HIVST_Reactive_Link_gap, 'cached_shape'):  # Check for cached shape
                del process_HIVST_Reactive_Link_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 12: Style the DataFrame
        df_HIVST_Reactive_Link_gap_style = (  # Apply styling to filtered DataFrame
            df_HIVST_Reactive_Link_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 13: Cache styled DataFrame and shape
        process_HIVST_Reactive_Link_gap.cached_style = df_HIVST_Reactive_Link_gap_style  # Store styled DataFrame
        process_HIVST_Reactive_Link_gap.cached_shape = df_HIVST_Reactive_Link.shape  # Store original unfiltered DataFrame shape

        # -- Step 14: Define export variables
        report_month = df_HIVST_Reactive_Link_gap['ReportPeriod'].iloc[0]  # Extract report month from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 15: Create descriptions for Word document
        report_description = []  # Initialize empty list for descriptions
        if (df_HIVST_Reactive_Link_gap[gap_columns[0]] != 0).any():  # Check for non-zero referral gaps
            report_description.append(  # Add description for referral gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns_spec[1]}\n"
                f"should be equal to {df_columns_spec[0]}"
            )  # Append description to list
        if (df_HIVST_Reactive_Link_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero confirmatory test gaps
            report_description.append(  # Add description for confirmatory test gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[2]}\nplus {df_columns_spec[3]}\n"
                f"should be equal to {df_columns_spec[1]}"
            )  # Append description to list
        if (df_HIVST_Reactive_Link_gap[gap_columns_wrap[2]] != 0).any():  # Check for non-zero linkage gaps
            report_description.append(  # Add description for linkage gap
                f"Report Name: {gap_columns[2]}\n"
                f"{df_columns_spec[4]}\n"
                f"should be equal to {df_columns_spec[2]}"
            )  # Append description to list
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

        # -- Step 16: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_HIVST_Reactive_Link_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns_spec,  # Italicize specified summary columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 17: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)   # Print display name with separators
            widget_display_df(df_HIVST_Reactive_Link_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_HIVST_Reactive_Link_gap, 'cached_style'):  # Check for cached style
            del process_HIVST_Reactive_Link_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_HIVST_Reactive_Link_gap, 'cached_shape'):  # Check for cached shape
            del process_HIVST_Reactive_Link_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - HIVST Prevention Service gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process HIVST Prevention Service gap
def process_HIVST_Prevention_Serv_gap(display_output=None):
    """
    Process HIVST Prevention Service gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for HIVST prevention service metrics
            "Number of individual reporting HIVST results",  # Total individuals reporting HIVST results
            "Number of individuals reporting non reactive HIVST results that referred prevention services.",  # Referred to prevention services
            "Number of individuals reporting non reactive HIVST results that accessed prevention services"  # Accessed prevention services
        ]  # Defines columns for HIVST prevention service gap analysis
        name = "HIVST Prevention Service Gap"  # Base name for the report
        gap_columns = [  # Names for calculated gap columns
            "HIVST non-reactive referral for prevention services gap", 
            "HIVST non-reactive client that accessed for prevention services gap"
        ]  # Defines gap column names
        report_name = f"{name}22"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_HIVST_Prevention_Serv = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include HIVST prevention service columns
        )  # Returns processed DataFrame or None if failed
        if df_HIVST_Prevention_Serv is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_HIVST_Prevention_Serv[gap_columns[0]] = np.where(  # Calculate referral for prevention services gap (requires numpy as np)
            df_HIVST_Prevention_Serv[df_columns[1]] > df_HIVST_Prevention_Serv[df_columns[0]],  # Check if referred exceeds total reporting
            df_HIVST_Prevention_Serv[df_columns[1]] - df_HIVST_Prevention_Serv[df_columns[0]],  # Compute gap (referred - total reporting)
            0  # Set to 0 if no gap (referred not greater than total)
        )  # Adds referral gap column
        df_HIVST_Prevention_Serv[gap_columns[1]] = np.where(  # Calculate access to prevention services gap
            df_HIVST_Prevention_Serv[df_columns[2]] != df_HIVST_Prevention_Serv[df_columns[1]],  # Check if accessed differs from referred
            df_HIVST_Prevention_Serv[df_columns[2]] - df_HIVST_Prevention_Serv[df_columns[1]],  # Compute gap (accessed - referred)
            0  # Set to 0 if no gap (values equal)
        )  # Adds access gap column

        # -- Step 4: Reorder columns for output
        reorder_columns = MSF_hierarchy + [  # Define column order: hierarchy, total reporting, referred, referral gap, accessed, access gap
            df_columns[0], df_columns[1], gap_columns[0], 
            df_columns[2], gap_columns[1]
        ]  # Specifies ordered columns for DataFrame
        df_HIVST_Prevention_Serv = df_HIVST_Prevention_Serv[reorder_columns]  # Reorder DataFrame columns for consistent output

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_HIVST_Prevention_Serv)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 6: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_HIVST_Prevention_Serv_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_HIVST_Prevention_Serv_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_HIVST_Prevention_Serv.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_HIVST_Prevention_Serv_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 7: Filter and validate gaps
        df_HIVST_Prevention_Serv_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_HIVST_Prevention_Serv,  # Input DataFrame with gaps
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_HIVST_Prevention_Serv_gap is None:  # Check if no gaps were found
            if hasattr(process_HIVST_Prevention_Serv_gap, 'cached_style'):  # Check for cached style
                del process_HIVST_Prevention_Serv_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_HIVST_Prevention_Serv_gap, 'cached_shape'):  # Check for cached shape
                del process_HIVST_Prevention_Serv_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 8: Style the DataFrame
        df_HIVST_Prevention_Serv_gap_style = (  # Apply styling to filtered DataFrame
            df_HIVST_Prevention_Serv_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap[0])  # Highlight first gap (referral) in red (assumed function)
            .map(outlier_yellow, subset=gap_columns_wrap[1])  # Highlight second gap (access) in yellow (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 9: Cache styled DataFrame and shape
        process_HIVST_Prevention_Serv_gap.cached_style = df_HIVST_Prevention_Serv_gap_style  # Store styled DataFrame
        process_HIVST_Prevention_Serv_gap.cached_shape = df_HIVST_Prevention_Serv.shape  # Store original unfiltered DataFrame shape

        # -- Step 10: Define export variables
        report_month = df_HIVST_Prevention_Serv_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 11: Create descriptions for Word document
        report_description = []  # Initialize empty list for descriptions
        if (df_HIVST_Prevention_Serv_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero referral gaps
            report_description.append(  # Add description for referral gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be too greater than {df_columns[0]}"  # Add description for referral gap
            )  # Append description to list
        if (df_HIVST_Prevention_Serv_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero access gaps
            report_description.append(  # Add description for access gap
                f"Report Name: {gap_columns[0]} \n"
                f"{df_columns[2]}\n"
                f"should be equal to {df_columns[1]}"  # Report description for access gap
                f"Note: Access gap should be reviewed."
            )  # Append description to list
        report_description = "\n\n".join(report_description)  # -- Join descriptions with double newlines

        # -- Step 12: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_HIVST_Prevention_Serv_gap_style, # -- Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # -- Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # -- Italicize specified columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # # -- Excel file path
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 13: Optionally display styled DataFrame
        if display_output:  # -- Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_HIVST_Prevention_Serv_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # -- Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_HIVST_Prevention_Serv_gap, 'cached_style'):  # Check for cached style
            del process_HIVST_Prevention_Serv_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_HIVST_Prevention_Serv_gap, 'cached_shape'):  # -- Check for cached shape
            del process_HIVST_Prevention_Serv_gap.cached_shape  # -- Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - HIVST Partner Screening gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process HIVST Partner Screening gap
def process_HIVST_Partner_Screening_gap(display_output=None):
    """
    Process HIVST Partner Screening  gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for HIVST partner screening metrics
            "Number of partners of people living with HIV screened with HIVST kit (confirmed during follow up)",  # Partners screened
            "Number of partners of people living with HIV reporting HIVST results"  # Partners reporting results
        ]  # Defines columns for HIVST partner screening gap analysis
        name = "HIVST Partner Screening Gap"  # Base name for the report
        gap_columns = ["HIVST partner screening gap"]  # Name for the calculated gap column
        report_name = f"{name}22"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_HIVST_Partner_Screening = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include HIVST partner screening columns
        )  # Returns processed DataFrame or None if failed
        if df_HIVST_Partner_Screening is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate HIVST partner screening gap
        df_HIVST_Partner_Screening[gap_columns[0]] = np.where(  # Calculate gap (requires numpy as np)
            df_HIVST_Partner_Screening[df_columns[1]] != df_HIVST_Partner_Screening[df_columns[0]],  # Check if reported results differ from screened partners
            df_HIVST_Partner_Screening[df_columns[1]] - df_HIVST_Partner_Screening[df_columns[0]],  # Compute gap (reported - screened)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for partner screening

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_HIVST_Partner_Screening)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_HIVST_Partner_Screening_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_HIVST_Partner_Screening_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_HIVST_Partner_Screening.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_HIVST_Partner_Screening_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing  

        # -- Step 6: Filter and validate gaps
        df_HIVST_Partner_Screening_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_HIVST_Partner_Screening,  # Input DataFrame with gap
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_HIVST_Partner_Screening_gap is None:  # Check if no gaps were found
            if hasattr(process_HIVST_Partner_Screening_gap, 'cached_style'):  # Check for cached style
                del process_HIVST_Partner_Screening_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_HIVST_Partner_Screening_gap, 'cached_shape'):  # Check for cached shape
                del process_HIVST_Partner_Screening_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the DataFrame
        df_HIVST_Partner_Screening_gap_style = (  # Apply styling to filtered DataFrame
            df_HIVST_Partner_Screening_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_yellow, subset=gap_columns_wrap)  # Highlight gap in yellow (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_HIVST_Partner_Screening_gap.cached_style = df_HIVST_Partner_Screening_gap_style  # Store styled DataFrame
        process_HIVST_Partner_Screening_gap.cached_shape = df_HIVST_Partner_Screening.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Define export variables
        report_month = df_HIVST_Partner_Screening_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_HIVST_Partner_Screening_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero screening gaps
            report_description = (  # Create description for screening gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"
                f"Note: Where this report is correct, please ignore the gap - only review."
            )  # Define Word document description

        # -- Step 11: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_HIVST_Partner_Screening_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_HIVST_Partner_Screening_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_HIVST_Partner_Screening_gap, 'cached_style'):  # Check for cached style
            del process_HIVST_Partner_Screening_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_HIVST_Partner_Screening_gap, 'cached_shape'):  # Check for cached shape
            del process_HIVST_Partner_Screening_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

## ICT MSF
### - ICT Offering gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process ICT Index Acceptance gap
def process_ICT_Index_Acceptance_gap(display_output=None):
    """
    Process ICT Index Acceptance  gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for ICT index acceptance metrics
            "Number of HIV Positive Clients Offered Index Testing",  # Clients offered index testing
            "Number of HIV Positive Clients Accepting Index Testing"  # Clients accepting index testing
        ]  # Defines columns for ICT index acceptance gap analysis
        name = "ICT Index Acceptance Gap"  # Base name for the report
        gap_columns = ["ICT index acceptance gap"]  # Name for the calculated gap column
        report_name = f"{name}24"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header
        

        # -- Step 2: Prepare data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include ICT index acceptance columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate ICT index acceptance gap
        df_main[gap_columns[0]] = np.where(  # Calculate gap (requires numpy as np)
            df_main[df_columns[1]] > df_main[df_columns[0]],  # Check if acceptances exceed offers
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Compute gap (acceptances - offers)
            0  # Set to 0 if no gap (acceptances not greater than offers)
        )  # Adds gap column for index acceptance

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_ICT_Index_Acceptance_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_ICT_Index_Acceptance_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_ICT_Index_Acceptance_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_mine_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_mine_gap is None:  # Check if no gaps were found
            if hasattr(process_ICT_Index_Acceptance_gap, 'cached_style'):  # Check for cached style
                del process_ICT_Index_Acceptance_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_ICT_Index_Acceptance_gap, 'cached_shape'):  # Check for cached shape
                del process_ICT_Index_Acceptance_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the DataFrame
        df_mine_gap_style = (  # Apply styling to filtered DataFrame
            df_mine_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight gap in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_ICT_Index_Acceptance_gap.cached_style = df_mine_gap_style  # Store styled DataFrame
        process_ICT_Index_Acceptance_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Define export variables
        report_month = df_mine_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_mine_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero acceptance gaps
            report_description = (  # Create description for acceptance gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"
            )  # Define Word document description

        # -- Step 11: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_mine_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_mine_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_ICT_Index_Acceptance_gap, 'cached_style'):  # Check for cached style
            del process_ICT_Index_Acceptance_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_ICT_Index_Acceptance_gap, 'cached_shape'):  # Check for cached shape
            del process_ICT_Index_Acceptance_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

## ICT MSF
### - ICT Contact gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process ICT Contact gap
def process_ICT_Contact_gap(display_output=None):  # Function to process ICT Contact gaps
    """
    Process ICT Contact gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the DataFrame for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for ICT contact metrics
            "Number of HIV Positive Clients Accepting Index Testing",  # Clients accepting index testing
            "Number of Children enumerated and Partners elicited from index client"  # Contacts enumerated
        ]  # Defines columns for ICT contact gap analysis
        name = "ICT Contact Gap"  # Base name for the report
        gap_columns = ["ICT contact gap"]  # Name for the calculated gap column
        report_name = f"{name}25"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include ICT contact columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate ICT contact gap
        df_main[gap_columns[0]] = np.where(  # Calculate gap (requires numpy as np)
            df_main[df_columns[1]] < df_main[df_columns[0]],  # Check if enumerated contacts are less than clients accepting index testing
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Compute gap (contacts - acceptances)
            0  # Set to 0 if no gap (contacts not less than acceptances)
        )  # Adds gap column for ICT contacts

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_ICT_Contact_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_ICT_Contact_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_ICT_Contact_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_mine_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_mine_gap is None:  # Check if no gaps were found
            if hasattr(process_ICT_Contact_gap, 'cached_style'):  # Check for cached style
                del process_ICT_Contact_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_ICT_Contact_gap, 'cached_shape'):  # Check for cached shape
                del process_ICT_Contact_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the DataFrame
        df_mine_gap_style = (  # Apply styling to filtered DataFrame
            df_mine_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_yellow, subset=gap_columns_wrap)  # Highlight gap in yellow (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_ICT_Contact_gap.cached_style = df_mine_gap_style  # Store styled DataFrame
        process_ICT_Contact_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Define export variables
        report_month = df_mine_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_mine_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero contact gaps
            report_description = (  # Create description for contact gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be greater than {df_columns[0]}"
                f"Note: Where this report is correct, please ignore the gap - only review."
            )  # Define Word document description

        # -- Step 11: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_mine_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_mine_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_ICT_Contact_gap, 'cached_style'):  # Check for cached style
            del process_ICT_Contact_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_ICT_Contact_gap, 'cached_shape'):  # Check for cached shape
            del process_ICT_Contact_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - ICT HTS gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process ICT HTS gap
def process_ICT_HTS_gap(display_output=None):
    """
    Process ICT HTS gap, exporting results as image, Excel, and Word files.
    Iterates over each cluster, caches styled DataFrames, and displays them on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the DataFrame for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for ICT HTS metrics
            "Number of Children enumerated and Partners elicited from index client",  # Enumerated contacts
            "Number of contacts of index clients tested HIV Positive",  # Contacts tested positive
            "Number of contacts of index clients tested HIV Negative"  # Contacts tested negative
        ]  # Defines columns for ICT HTS gap analysis
        name = "ICT Contact Testing Gap"  # Base name for the report
        gap_columns = ["ICT contact testing gap"]  # Name for the calculated gap column
        report_name = f"{name}26"  # Report name with unique suffix

        # -- Step 2: Prepare data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include ICT HTS columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Clean and convert data types
        df_main = df_main.fillna(0)  # Replace NaN values with 0
        # Explicitly convert relevant columns to numeric, coercing errors to 0
        for col in df_columns:  # Iterate over data columns
            df_main[col] = pd.to_numeric(df_main[col], errors='coerce').fillna(0).astype(int)  # Convert to numeric, handle errors, and cast to integer

        # -- Step 4: Calculate ICT HTS gap
        df_main[gap_columns[0]] = np.where(  # Calculate gap (requires numpy as np)
            df_main[df_columns[1:3]].sum(axis=1) != df_main[df_columns[0]],  # Check if sum of tested (positive + negative) differs from enumerated contacts
            df_main[df_columns[1:3]].sum(axis=1) - df_main[df_columns[0]],  # Compute gap (tested sum - enumerated)
            0  # Set to 0 if no gap (values equal)
        ).astype(int)  # Ensure gap column is integer type

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 6: Check and display cached styled DataFrames
        if display_output:  # Check if display output is requested
            if hasattr(process_ICT_HTS_gap, 'cached_styles'):  # Check if cached styles dictionary exists
                cached_shape = getattr(process_ICT_HTS_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    for cluster, style in process_ICT_HTS_gap.cached_styles.items():  # Iterate over cached cluster styles
                        display_name = f"✔️ Displaying {cluster} {report_name}"  # Formatted display name for output
                        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
                        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header
                        print(print_display_name)  # Print display name with separators
                        widget_display_df(style)  # Display cached styled DataFrame (assumed widget function)
                    return  # Exit to avoid reprocessing

        # -- Step 7: Initialize cache
        if not hasattr(process_ICT_HTS_gap, 'cached_styles'):  # Check if cache dictionary is not initialized
            process_ICT_HTS_gap.cached_styles = {}  # Initialize empty dictionary for caching styled DataFrames

        # -- Step 8: Identify unique clusters
        cluster_list = pd.Series(df_main['Cluster'].unique())  # Extract unique cluster values from DataFrame (requires pandas as pd)

        # -- Step 9: Process each cluster
        for current_cluster in cluster_list:  # Iterate over each unique cluster
            cluster_filtered = df_main[df_main['Cluster'] == current_cluster]  # Filter DataFrame for current cluster
            
            ICT_HTS_msg = f"No {current_cluster} {report_name}"  # Define message for no gaps in current cluster
            display_name = f"✔️ Displaying {current_cluster} {report_name}"  # Formatted display name for current cluster
            display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length
            print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

            cluster_filtered_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
                df=cluster_filtered,  # Input cluster-specific DataFrame
                msg=ICT_HTS_msg,  # Message to display if no gaps
                opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
                opNeg=None,  # No negative value filter
                opPos=None,  # No positive value filter
                opZero=None,  # No zero value filter
                opLT100=None  # No less-than-100 filter
            )  # Returns filtered DataFrame or None if no gaps

            if cluster_filtered_gap is None:  # Check if no gaps were found for cluster
                if current_cluster in process_ICT_HTS_gap.cached_styles:  # Check if cluster is in cache
                    del process_ICT_HTS_gap.cached_styles[current_cluster]  # Remove cluster from cache
                continue  # Skip to next cluster

            cluster_filtered_style = (  # Apply styling to filtered cluster DataFrame
                cluster_filtered_gap.style  # Create style object from filtered DataFrame
                .hide(axis='index')  # Hide row index for cleaner display
                .map(outlier_yellow, subset=gap_columns_wrap)  # Highlight gap in yellow (assumed function)
            )  # Creates styled DataFrame for cluster

            process_ICT_HTS_gap.cached_styles[current_cluster] = cluster_filtered_style  # Store styled DataFrame in cache for cluster

            # -- Step 10: Define export variables
            report_name_cluster = f"{current_cluster}_{report_name}"  # Create cluster-specific report name
            report_month = cluster_filtered_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
            report_image_name = f"{report_month}_{report_name_cluster}.png"  # Create image file name with report month and cluster
            report_sheet_name = f"{current_cluster}_{report_name}"  # Define Excel sheet name with cluster

            # -- Step 11: Create descriptions
            report_description = []  # Initialize empty list for descriptions
            if (cluster_filtered_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero testing gaps
                report_description.append(  # Add description for testing gap
                    f"Report Name: {gap_columns[0]}\n"
                    f"{df_columns[1]}\nplus {df_columns[2]}\n"  # Describe expected equality
                    f"should be equal to {df_columns[0]}\n"
                    f"Note: Where this report is correct, please ignore the gap - only review."
                )  # Append description to list
            report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

            # -- Step 12: Export results
            if not display_output:  # Check if user requested to export results
                export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                    report_name=report_name_cluster,  # Cluster-specific report name
                    df_style=cluster_filtered_style,  # Styled DataFrame for export
                    img_file_name=report_image_name,  # Image file name
                    img_file_path=sub_folder2_image_file_msf_outlier,  # Image file path (assumed defined)
                    doc_description=report_description,  # Word document description
                    doc_indicators_to_italicize=df_columns,  # Italicize input columns
                    doc_indicators_to_underline=gap_columns,  # Underline gap column
                    xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                    xlm_sheet_name=report_sheet_name  # Excel sheet name
                )  # Exports to specified formats

            if display_output:  # Check if display is requested
                print(print_display_name)  # Print display name with separators
                widget_display_df(cluster_filtered_style)  # Display styled DataFrame for cluster (assumed widget function)

        # -- Step 13: Cache overall unfiltered DataFrame shape
        process_ICT_HTS_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

    except Exception as e:  # Catch any processing errors
        print(f"⦔ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_ICT_HTS_gap, 'cached_styles'):  # Check for cached styles dictionary
            process_ICT_HTS_gap.cached_styles.clear()  # Clear all cached styles
        if hasattr(process_ICT_HTS_gap, 'cached_shape'):  # Check for cached shape
            del process_ICT_HTS_gap.cached_shape  # -- Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - ICT Positive Linkage gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process ICT Positive Linkage gap
def process_ICT_Positive_Link_gap(display_output=None):
    """
    Process ICT Positive Linkage gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the DataFrame for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for ICT Positive Linkage metrics
            "Number of contacts of index clients tested HIV Positive",  # Contacts tested positive
            "Number of contacts of index clients linked to ART"  # Contacts linked to ART
        ]  # Defines columns for ICT linkage gap analysis
        name = "ICT Contact Testing Gap"  # Base name for report (misnamed, preserved as-is)
        gap_columns = ["ICT contact testing gap"]  # Name for the calculated gap column (misnamed, preserved as-is)
        report_name = f"{name}27"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Prepare data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="HTS MSF",  # Key to fetch HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include ICT linkage columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate ICT Positive Linkage gap
        df_main[gap_columns[0]] = np.where(  # Calculate gap (requires numpy as np)
            df_main[df_columns[1]] != df_main[df_columns[0]],  # Check if linked contacts differ from tested positive
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Compute gap (linked - tested positive)
            0  # Set to 0 if no gap (values equal)
        )  # Adds linkage gap column

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column name for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_ICT_Positive_Link_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_ICT_Positive_Link_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_ICT_Positive_Link_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_mine_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_mine_gap is None:  # Check if no gaps were found
            if hasattr(process_ICT_Positive_Link_gap, 'cached_style'):  # Check for cached style
                del process_ICT_Positive_Link_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_ICT_Positive_Link_gap, 'cached_shape'):  # Check for cached shape
                del process_ICT_Positive_Link_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the DataFrame
        df_mine_gap_style = (  # Apply styling to filtered DataFrame
            df_mine_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight gap in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_ICT_Positive_Link_gap.cached_style = df_mine_gap_style  # Store styled DataFrame
        process_ICT_Positive_Link_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Define export variables
        report_month = df_mine_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_mine_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero linkage gaps
            report_description = (  # Create description for linkage gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"
                f"Note: Where this report is correct, please ignore the gap - only review."
            )  # Define Word document description

        # -- Step 11: Export results
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_mine_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Optionally display styled DataFrame
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_mine_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_ICT_Positive_Link_gap, 'cached_style'):  # Check for cached style
            del process_ICT_Positive_Link_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_ICT_Positive_Link_gap, 'cached_shape'):  # Check for cached shape
            del process_ICT_Positive_Link_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

## PMCTC MSF
### - PMTCT New ANC Testing gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PMTCT New ANC HTS gap
def process_PMTCT_ANC_Optmz_gap(display_output=None):
    """
    Process PMTCT New ANC HTS gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of primary column names for PMTCT ANC metrics
            "Number of new ANC Clients",  # New ANC clients
            "Number of pregnant women with previously known HIV positive infection"  # Known HIV positives
        ]  # Defines primary columns for initial data fetch
        df_columns_spec = [  # Comprehensive list of all relevant columns
            'Number of new ANC Clients',  # New ANC clients
            'Number of pregnant women with previously known HIV positive infection',  # Known HIV positives
            'Number of pregnant women HIV tested and received results (ANC)',  # Tested at ANC
            'Number of pregnant women HIV tested and received results (L&D)',  # Tested at Labor & Delivery
            'Number of pregnant women HIV tested and received results (<72hrs Post Partum)'  # Tested <72hrs postpartum
        ]  # Defines all columns for analysis and reporting
        df_columns2 = ['ANC', 'L&D', '<72hrs Post Partum']  # Service delivery point columns from secondary dataset
        name = "PMTCT New ANC HTS Optimization Gap"  # Base name for report
        gap_columns = ["PMTCT new ANC HTS gap"]  # Name for the calculated gap column
        report_name = f"{name}28"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare primary DataFrame from DHIS2
            DHIS2_data_key="PMTCT MSF",  # Key to fetch PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include primary PMTCT ANC columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Fetch and prepare additional HIVST mode data
        df_main2 = prepare_and_convert_df(  # Fetch and prepare secondary DataFrame from DHIS2
            DHIS2_data_key="PMTCT MSF_sdp",  # Key to fetch PMTCT MSF service delivery point dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns
            data_columns=df_columns2  # Include service delivery point columns
        )  # Returns processed DataFrame or None if failed
        if df_main2 is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 4: Merge datasets
        df_main = df_main.merge(  # Merge primary and secondary DataFrames
            df_main2,  # Secondary DataFrame with service delivery point data
            on=MSF_hierarchy,  # Merge on MSF hierarchy columns
            how="left"  # Left join to keep all rows from primary data
        )  # Updates df_main with merged data

        # -- Step 5: Rename columns for consistency
        df_main = df_main.rename(columns={  # Rename service delivery point columns to match df_columns_spec
            f"{df_columns2[0]}": f"{df_columns_spec[2]}",  # Rename ANC to ANC tested results
            f"{df_columns2[1]}": f"{df_columns_spec[3]}",  # Rename L&D to L&D tested results
            f"{df_columns2[2]}": f"{df_columns_spec[4]}"  # Rename <72hrs Post Partum to postpartum tested results
        })  # Apply renamed columns to DataFrame

        # -- Step 6: Clean and format data
        df_main.sort_values(by=MSF_hierarchy, inplace=True, ignore_index=True)  # Sort DataFrame by hierarchy columns
        df_main = df_main.fillna(0)  # Replace NaN values with 0
        float_columns = df_main.select_dtypes(include=['float64', 'float32']).columns  # Identify float-type columns
        for col in float_columns:  # Convert float columns to integers
            df_main[col] = df_main[col].astype(int)  # Apply integer conversion

        # -- Step 7: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate PMTCT ANC HTS gap (requires numpy as np)
            df_main[df_columns_spec[1:3]].sum(axis=1) != df_main[df_columns[0]],  # Check if sum of known positives and ANC tested differs from new ANC clients
            df_main[df_columns_spec[1:3]].sum(axis=1) - df_main[df_columns[0]],  # Compute gap (sum - new ANC clients)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for PMTCT ANC HTS

        # -- Step 8: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column name for display

        # -- Step 9: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_PMTCT_ANC_Optmz_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_PMTCT_ANC_Optmz_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_PMTCT_ANC_Optmz_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 10: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_PMTCT_ANC_Optmz_gap, 'cached_style'):  # Check for cached style
                del process_PMTCT_ANC_Optmz_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_PMTCT_ANC_Optmz_gap, 'cached_shape'):  # Check for cached shape
                del process_PMTCT_ANC_Optmz_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 11: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight gap in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 12: Cache styled DataFrame and shape
        process_PMTCT_ANC_Optmz_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_PMTCT_ANC_Optmz_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 13: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 14: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero HTS gaps
            report_description = (  # Create description for HTS gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns_spec[1]}\npluse {df_columns_spec[2]}\n"  # Describe expected equality (typo preserved)
                f"should be equal to {df_columns_spec[0]}"  # Specify expected relationship
            )  # Define Word document description

        # -- Step 15: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns_spec,  # Italicize specific columns
                doc_indicators_to_underline=gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 16: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_PMTCT_ANC_Optmz_gap, 'cached_style'):  # Check for cached style
            del process_PMTCT_ANC_Optmz_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_PMTCT_ANC_Optmz_gap, 'cached_shape'):  # Check for cached shape
            del process_PMTCT_ANC_Optmz_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PMTCT Positive gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PMTCT Positive gap
def process_PMTCT_Positive_gap(display_output=None):
    """
    Process PMTCT Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PMTCT Positive metrics
            "Number of pregnant women HIV tested and received results (ANC)",  # Tested at ANC
            "Number of pregnant women HIV tested and received results (L&D)",  # Tested at Labor & Delivery
            "Number of pregnant women HIV tested and received results (<72hrs Post Partum)",  # Tested <72hrs postpartum
            "Number of pregnant women tested HIV positive (ANC)",  # HIV positive at ANC
            "Number of pregnant women tested HIV positive (L&D)",  # HIV positive at Labor & Delivery
            "Number of pregnant women tested HIV positive (<72hrs Post Partum)"  # HIV positive <72hrs postpartum
        ]  # Defines columns for PMTCT positive gap analysis
        df_columns2 = ['ANC', 'L&D', '<72hrs Post Partum']  # Service delivery point columns from datasets
        name = "PMTCT Positive Gap"  # Base name for report
        gap_columns = [  # Names for calculated gap columns
            "PMTCT positive (ANC) gap",  # ANC positive gap
            "PMTCT positive (L&D) gap",  # L&D positive gap
            "PMTCT positive (<72hrs Post Partum) gap"  # <72hrs Post Partum positive gap
        ]  # Defines gap columns
        report_name = f"{name}29"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare primary DataFrame from DHIS2
            DHIS2_data_key="PMTCT MSF_sdp",  # Key to fetch PMTCT MSF service delivery point dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns2  # Include service delivery point columns for tested results
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Fetch and prepare additional HIVST mode data
        df_main2 = prepare_and_convert_df(  # Fetch and prepare secondary DataFrame from DHIS2
            DHIS2_data_key="PMTCT MSF_sdp_pos",  # Key to fetch PMTCT MSF positive results dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns
            data_columns=df_columns2  # Include service delivery point columns for positive results
        )  # Returns processed DataFrame or None if failed
        if df_main2 is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 4: Merge datasets
        df_main = df_main.merge(  # Merge primary and secondary DataFrames
            df_main2,  # Secondary DataFrame with positive results
            on=MSF_hierarchy,  # Merge on MSF hierarchy columns
            how="left"  # Left join to keep all rows from primary DataFrame
        )  # Updates df_main with merged data

        # -- Step 5: Rename columns for consistency
        df_main = df_main.rename(columns={  # Rename merged columns to match df_columns
            f'ANC_x': f"{df_columns[0]}",  # Rename ANC tested to ANC tested results
            'L&D_x': f"{df_columns[1]}",  # Rename L&D tested to L&D tested results
            '<72hrs Post Partum_x': f"{df_columns[2]}",  # Rename <72hrs Post Partum to postpartum tested
            'ANC_y': f"{df_columns[3]}",  # Rename ANC positives to ANC positive results
            'L&D_y': f"{df_columns[4]}",  # Rename L&D positives to L&D positive results
            '<72hrs Post Partum_y': f"{df_columns[5]}"  # Rename <72hrs Post Partum to postpartum positive
        })  # Apply renamed columns to DataFrame

        # -- Step 6: Clean and format data
        df_main.sort_values(by=MSF_hierarchy, inplace=True, ignore_index=True)  # Sort by hierarchy columns
        df_main = df_main.fillna(0)  # Replace NaN values with 0
        float_columns = df_main.select_dtypes(include=['float64']).columns  # Identify float-type columns
        for col in float_columns:  # Convert float columns to integers
            df_main[col] = df_main[col].astype(int)  # Apply integer conversion

        # -- Step 7: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate ANC positive gap (requires numpy as np)
            df_main[df_columns[3]] > df_main[df_columns[0]],  # Check if ANC positives exceed ANC tested
            df_main[df_columns[3]] - df_main[df_columns[0]],  # Compute gap (positives - tested)
            0  # Set to 0 if no gap
        )  # Adds ANC positive gap column

        df_main[gap_columns[1]] = np.where(  # Calculate L&D positive gap
            df_main[df_columns[4]] > df_main[df_columns[1]],  # Check if L&D positives exceed L&D tested
            df_main[df_columns[4]] - df_main[df_columns[1]],  # Compute gap (positives - tested)
            0  # Set to 0 if no gap
        )  # Adds L&D positive gap column

        df_main[gap_columns[2]] = np.where(  # Calculate <72hrs Post Partum positive gap
            df_main[df_columns[5]] > df_main[df_columns[2]],  # Check if postpartum positives exceed postpartum tested
            df_main[df_columns[5]] - df_main[df_columns[2]],  # Compute gap (positives - tested)
            0  # Set to 0 if no gap
        )  # Adds <72hrs Post Partum positive gap column

        # -- Step 8: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 9: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_PMTCT_Positive_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_PMTCT_Positive_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_PMTCT_Positive_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 10: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gaps
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_PMTCT_Positive_gap, 'cached_style'):  # Check for cached style
                del process_PMTCT_Positive_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_PMTCT_Positive_gap, 'cached_shape'):  # Check for cached shape
                del process_PMTCT_Positive_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 11: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 12: Cache styled DataFrame and shape
        process_PMTCT_Positive_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_PMTCT_Positive_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 13: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 14: Create descriptions for Word document
        report_description = []  # Initialize empty list for descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero ANC gaps
            report_description.append(  # Add description for ANC gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[3]}\n"
                f"should not be greater than {df_columns[0]}"  # Describe expected ANC constraint
            )  # Append ANC description
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero L&D gaps
            report_description.append(  # Add description for L&D gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[4]}\n"
                f"should not be greater than {df_columns[1]}"  # Describe expected L&D constraint
            )  # Append L&D description
        if (df_main_gap[gap_columns_wrap[2]] != 0).any():  # Check for non-zero <72hrs Post Partum gaps
            report_description.append(  # Add description for postpartum gap
                f"Report Name: {gap_columns[2]}\n"
                f"{df_columns[5]}\n"
                f"should not be greater than {df_columns[2]}"  # Describe expected postpartum constraint
            )  # Append postpartum description
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

        # -- Step 15: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 16: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_PMTCT_Positive_gap, 'cached_style'):  # Check for cached style
            del process_PMTCT_Positive_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_PMTCT_Positive_gap, 'cached_shape'):  # Check for cached shape
            del process_PMTCT_Positive_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PMTCT Previously Known gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PMTCT Previously Known gap
def process_PMTCT_PK_gap(display_output=None):
    """
    Process PMTCT Previously Known gap, comparing known HIV-positive pregnant women
    to those already on ART prior to pregnancy. Exports results as image, Excel, and
    Word files. Caches the styled DataFrame and displays it on subsequent calls if
    data shape is unchanged.

    Args:
        display_output (bool, optional): If True, displays the DataFrame for gaps.
            Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PMTCT Previously Known metrics
            "Number of pregnant women with previously known HIV positive infection",  # Known HIV positives
            "Number of HIV positive pregnant women already on ART prior to this pregnancy"  # ART prior to pregnancy
        ]  # Defines columns for PMTCT Previously Known gap analysis
        name = "PMTCT Previously Known Gap"  # Base name for report
        gap_columns = ["PMTCT previously known gap"]  # Name for the calculated gap column
        report_name = f"{name}30"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="PMTCT MSF",  # Key to fetch PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PMTCT Previously Known columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate PMTCT Previously Known gap (requires numpy as np)
            df_main[df_columns[1]] != df_main[df_columns[0]],  # Check if ART counts differ from known positives
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Compute gap (ART - known positives)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for Previously Known

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column name for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_PMTCT_PK_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_PMTCT_PK_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_PMTCT_PK_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_PMTCT_PK_gap, 'cached_style'):  # Check for cached style
                del process_PMTCT_PK_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_PMTCT_PK_gap, 'cached_shape'):  # Check for cached shape
                del process_PMTCT_PK_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight gap in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_PMTCT_PK_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_PMTCT_PK_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = ""  # Initialize empty description string
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero Previously Known gaps
            report_description = (  # Create description for gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"  # Describe expected equality
            )  # Define Word document description

        # -- Step 11: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

        # -- Step 13: Handle errors
    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_PMTCT_PK_gap, 'cached_style'):  # Check for cached style
            del process_PMTCT_PK_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_PMTCT_PK_gap, 'cached_shape'):  # Check for cached shape
            del process_PMTCT_PK_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PMTCT Linkage gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PMTCT Positive Linkage gap
def process_PMTCT_Positive_Linkage_gap(display_output=None):
    """
    Process PMTCT Positive Linkage gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PMTCT Positive Linkage metrics
            "Number of pregnant women tested HIV positive",  # Total HIV positives
            "Number of HIV positive pregnant women newly started on ART during ANC <36wks of pregnancy",  # ART initiation ANC <36wks
            "Number of HIV positive pregnant women newly started on ART during ANC >36wks of pregnancy",  # ART initiation ANC >36wks
            "Number of HIV positive pregnant women newly started on ART during Labour",  # ART initiation during Labour
            "Number of HIV positive pregnant women newly started on ART during Post Partum (<72 hrs)",  # ART initiation Postpartum <72hrs
            "Number of HIV positive pregnant women newly started on ART during Post Partum (>72 hrs - < 6 months)",  # ART initiation Postpartum >72hrs-<6 months
            "Number of HIV positive pregnant women newly started on ART during Post Partum (>6 - 12 months)"  # ART initiation Postpartum >6-12 months
        ]  # Defines columns for PMTCT Positive Linkage gap analysis
        name = "PMTCT Positive Linkage Known Gap"  # Base name for report (misnamed, preserved as-is)
        gap_columns = ["PMTCT positive linkage gap"]  # Name for the calculated gap column
        report_name = f"{name}31"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="PMTCT MSF",  # Key to fetch PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PMTCT Positive Linkage columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate PMTCT Positive Linkage gap (requires numpy as np)
            df_main[df_columns[1:7]].sum(axis=1) != df_main[df_columns[0]],  # Check if sum of ART initiations differs from total positives
            df_main[df_columns[1:7]].sum(axis=1) - df_main[df_columns[0]],  # Compute gap (ART sum - positives)
            0  # Set to 0 if no gap (values equal)
        )  # Adds gap column for Positive Linkage

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column name for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_PMTCT_Positive_Linkage_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_PMTCT_Positive_Linkage_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_PMTCT_Positive_Linkage_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap
            msg=No_gap_msg,  # Message to display if no gaps are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_PMTCT_Positive_Linkage_gap, 'cached_style'):  # Check for cached style
                del process_PMTCT_Positive_Linkage_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_PMTCT_Positive_Linkage_gap, 'cached_shape'):  # Check for cached shape
                del process_PMTCT_Positive_Linkage_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight gap in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_PMTCT_Positive_Linkage_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_PMTCT_Positive_Linkage_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero Positive Linkage gaps
            report_description = (  # Create description for gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\npluse {df_columns[2]}\npluse {df_columns[3]}\npluse {df_columns[4]}"  # Describe ART initiation sum (typo preserved)
                f"\npluse {df_columns[5]}\npluse {df_columns[6]}\n"
                f"should be equal to {df_columns[0]}"  # Specify expected equality
            )  # Define Word document description

        # -- Step 11: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_PMTCT_Positive_Linkage_gap, 'cached_style'):  # Check for cached style
            del process_PMTCT_Positive_Linkage_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_PMTCT_Positive_Linkage_gap, 'cached_shape'):  # Check for cached shape
            del process_PMTCT_Positive_Linkage_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PMTCT Seroconversion gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PMTCT Seroconversion gap
def process_PMTCT_Seroconversion_gap(display_output=None):
    """
    Process PMTCT Seroconversion gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PMTCT Seroconversion metrics
            "Number of pregnant women retested after initial HIV negative test",  # Women retested after initial negative
            "Number of pregnant women retested who seroconverted to HIV positive after initial HIV negative test"  # Women who seroconverted
        ]  # Defines columns for PMTCT Seroconversion analysis
        name = "PMTCT Seroconversion Gap"  # Base name for report
        gap_columns = ["PMTCT seroconversion gap"]  # Name for the calculated check column
        report_name = f"{name}32"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no seroconversions are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="PMTCT MSF",  # Key to fetch PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PMTCT Seroconversion columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate seroconversion checks
        df_main[gap_columns[0]] = np.where(  # Calculate PMTCT Seroconversion check (requires numpy as np)
            df_main[df_columns[1]] > 0,  # Check if any seroconversions occurred
            df_main[df_columns[1]],  # Set check value to number of seroconversions
            0  # Set to 0 if no seroconversions
        )  # Adds seroconversion check column (flags positive values rather than a true gap)

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column name for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_PMTCT_Seroconversion_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_PMTCT_Seroconversion_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_PMTCT_Seroconversion_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate checks
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero seroconversion checks
            df=df_main,  # Input DataFrame with check column
            msg=No_gap_msg,  # Message to display if no seroconversions are found
            opNonZero=gap_columns_wrap,  # Filter for non-zero check values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no seroconversions
        if df_main_gap is None:  # Check if no seroconversions were found
            if hasattr(process_PMTCT_Seroconversion_gap, 'cached_style'):  # Check for cached style
                del process_PMTCT_Seroconversion_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_PMTCT_Seroconversion_gap, 'cached_shape'):  # Check for cached shape
                del process_PMTCT_Seroconversion_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no seroconversions

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_yellow, subset=gap_columns_wrap)  # Highlight non-zero checks in yellow (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_PMTCT_Seroconversion_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_PMTCT_Seroconversion_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero Seroconversion checks
            report_description = (  # Create description for seroconversion
                f"Report Name: {gap_columns[0]}\n"
                f"Note: {df_columns[1]} is a quality indicator and should be reviewed thoroughly."  # Highlight seroconversion as quality indicator
            )  # Define Word document description

        # -- Step 11: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline check column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

        # -- Step 13: Handle errors
    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_PMTCT_Seroconversion_gap, 'cached_style'):  # Check for cached style
            del process_PMTCT_Seroconversion_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_PMTCT_Seroconversion_gap, 'cached_shape'):  # Check for cached shape
            del process_PMTCT_Seroconversion_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PMTCT Coinfection (syphilis) gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PMTCT Syphilis Test gap
def process_PMTCT_Syphilis_Test_gap(display_output=None):
    """
    Process PMTCT Syphilis Test gap, exporting results as image, Excel, and Word files.
    Iterates over each cluster, caches styled DataFrames, and displays them on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PMTCT Syphilis Test metrics
            "Number of new ANC Clients",  # New ANC clients count
            "Number of new ANC Clients tested for syphilis - Total",  # Clients tested for syphilis
            "Number of new ANC Clients tested positive for syphilis - Total",  # Clients tested positive
            "Number of the ANC Clients treated for Syphilis - Total"  # Clients treated for syphilis
        ]  # Defines columns for PMTCT Syphilis Test analysis
        name = "PMTCT Syphilis Test Gap"  # Base name for report
        gap_columns = ["PMTCT new ANC syphilis test gap",  # Gap for testing vs. new clients
                       "PMTCT syphilis positive gap",  # Gap for positives vs. tested
                       "PMTCT syphilis positive treatment gap"]  # Gap for treated vs. positives
        report_name = f"{name}33"  # Report name with unique suffix

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="PMTCT MSF",  # Key to fetch PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PMTCT Syphilis Test columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate PMTCT New ANC Syphilis Test gap (requires numpy as np)
            df_main[df_columns[1]] != df_main[df_columns[0]],  # Check if tested differs from new ANC clients
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for gap
            0  # Set to 0 if no gap
        )  # Adds test gap column
        
        df_main[gap_columns[1]] = np.where(  # Calculate PMTCT Syphilis Positive gap
            df_main[df_columns[2]] > df_main[df_columns[1]],  # Check if positives exceed tested
            df_main[df_columns[2]] - df_main[df_columns[1]],  # Calculate difference for gap
            0  # Set to 0 if no gap
        )  # Adds positive gap column
        
        df_main[gap_columns[2]] = np.where(  # Calculate PMTCT Syphilis Positive Treatment gap
            df_main[df_columns[3]] != df_main[df_columns[2]],  # Check if treated differs from positives
            df_main[df_columns[3]] - df_main[df_columns[2]],  # Calculate difference for gap
            0  # Set to 0 if no gap
        )  # Adds treatment gap column

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrames
        if display_output:  # Check if display output is requested
            if hasattr(process_PMTCT_Syphilis_Test_gap, 'cached_styles'):  # Check if cached styled DataFrames exist
                cached_shape = getattr(process_PMTCT_Syphilis_Test_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    for cluster, style in process_PMTCT_Syphilis_Test_gap.cached_styles.items():  # Iterate over cached cluster styles
                        display_name = f"✔️ Displaying {cluster} {report_name}"  # Formatted display name for output
                        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
                        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display 
                        print(print_display_name)  # Print display name with separators
                        widget_display_df(style)  # Display cached styled DataFrame (assumed widget function)
                    return  # Exit to avoid reprocessing

        # -- Step 6: Initialize cache
        if not hasattr(process_PMTCT_Syphilis_Test_gap, 'cached_styles'):  # Check if cache attribute exists
            process_PMTCT_Syphilis_Test_gap.cached_styles = {}  # Initialize empty dictionary for caching styles

        # -- Step 7: Identify unique clusters
        cluster_list = pd.Series(df_main['Cluster'].unique())  # Extract unique cluster names (requires pandas as pd)

        # -- Step 8: Process each cluster
        for current_cluster in cluster_list:  # Iterate over each unique cluster
            cluster_filtered = df_main[df_main['Cluster'] == current_cluster]  # Filter DataFrame for current cluster
            
            PMTCT_Syphilis_msg = f"No {current_cluster} {report_name}"  # Message if no gaps found for cluster
            display_name = f"✔️ Displaying {current_cluster} {report_name}"  # Formatted display name for output
            display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
            print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

            cluster_filtered_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with gaps
                df=cluster_filtered,  # Input cluster-filtered DataFrame
                msg=PMTCT_Syphilis_msg,  # Message to display if no gaps
                opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
                opNeg=None,  # No negative value filter
                opPos=None,  # No positive value filter
                opZero=None,  # No zero value filter
                opLT100=None  # No less-than-100 filter
            )  # Returns filtered DataFrame or None if no gaps

            if cluster_filtered_gap is None:  # Check if no gaps were found
                if current_cluster in process_PMTCT_Syphilis_Test_gap.cached_styles:  # Check if cluster in cache
                    del process_PMTCT_Syphilis_Test_gap.cached_styles[current_cluster]  # Remove cluster from cache
                continue  # Skip to next cluster

            cluster_filtered_style = (  # Apply styling to filtered DataFrame
                cluster_filtered_gap.style  # Create style object from filtered DataFrame
                .hide(axis='index')  # Hide row index for cleaner display
                .map(outlier_yellow, subset=gap_columns_wrap[0])  # Highlight test gap in yellow (assumed function)
                .map(outlier_red, subset=gap_columns_wrap[1:3])  # Highlight positive/treatment gaps in red
            )  # Creates styled DataFrame for display/export

            process_PMTCT_Syphilis_Test_gap.cached_styles[current_cluster] = cluster_filtered_style  # Cache styled DataFrame for cluster

            # -- Step 9: Define export variables
            report_name_cluster = f"{current_cluster}_{report_name}"  # Create cluster-specific report name
            report_month = cluster_filtered_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
            report_image_name = f"{report_month}_{report_name_cluster}.png"  # Create image file name with report month
            report_sheet_name = f"{current_cluster}_{report_name}"  # Define Excel sheet name with cluster

            # -- Step 10: Create descriptions for Word document
            report_description = []  # Initialize list for report descriptions
            if (cluster_filtered_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero test gaps
                report_description.append(  # Add description for test gap
                    f"Report Name: {gap_columns[0]}\n"
                    f"{df_columns[1]}\n"
                    f"should be equal to {df_columns[0]}\n"
                    f"Note: Where this report is correct, please ignore the gap - only review."
                )  # Describe test gap issue
            if (cluster_filtered_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero positive gaps
                report_description.append(  # Add description for positive gap
                    f"Report Name: {gap_columns[1]}\n"
                    f"{df_columns[2]}\n"
                    f"should not be greater than {df_columns[1]}"
                )  # Describe positive gap issue
            if (cluster_filtered_gap[gap_columns_wrap[2]] != 0).any():  # Check for non-zero treatment gaps
                report_description.append(  # Add description for treatment gap
                    f"Report Name: {gap_columns[2]}\n"
                    f"{df_columns[3]}\n"
                    f"should be equal to {df_columns[2]}\n"
                    f"Note: Where this report is correct, please ignore the gap - only review."
                )  # Describe treatment gap issue
            report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

            # -- Step 11: Export results to multiple formats
            if not display_output:  # Check if user requested to export results
                export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                    report_name=report_name_cluster,  # Cluster-specific report name
                    df_style=cluster_filtered_style,  # Styled DataFrame for export
                    img_file_name=report_image_name,  # Image file name
                    img_file_path=sub_folder2_image_file_msf_outlier,  # Image file path (assumed defined)
                    doc_description=report_description,  # Word document description
                    doc_indicators_to_italicize=df_columns,  # Italicize input columns
                    doc_indicators_to_underline=gap_columns,  # Underline gap columns
                    xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                    xlm_sheet_name=report_sheet_name  # Excel sheet name
                )  # Exports to specified formats

            if display_output:  # Check if display is requested
                print(print_display_name)  # Print display name with separators
                widget_display_df(cluster_filtered_style)  # Display styled DataFrame (assumed widget function)

        # -- Step 12: Cache overall unfiltered DataFrame shape
        process_PMTCT_Syphilis_Test_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_PMTCT_Syphilis_Test_gap, 'cached_styles'):  # Check for cached styles
            process_PMTCT_Syphilis_Test_gap.cached_styles.clear()  # Clear cached styled DataFrames
        if hasattr(process_PMTCT_Syphilis_Test_gap, 'cached_shape'):  # Check for cached shape
            del process_PMTCT_Syphilis_Test_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PMTCT Coinfection (HBV, HCV) gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PMTCT Hepatitis Test gap
def process_PMTCT_Hepatitis_Test_gap(display_output=None):
    """
    Process PMTCT Hepatitis Test gap, exporting results as image, Excel, and Word files.
    Iterates over each cluster, caches styled DataFrames, and displays them on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PMTCT Hepatitis Test metrics
            "Number of new ANC Clients",  # New ANC clients count
            "Number of new ANC Clients tested for HBV ( ANC, L&D, <72hrs Post Partum)",  # Clients tested for Hepatitis B
            "Number of new ANC Clients tested for HCV ( ANC, L&D, < 72hrs Post Partum)"  # Clients tested for Hepatitis C
        ]  # Defines columns for PMTCT Hepatitis Test analysis
        name = "PMTCT Hepatitis Test Gap"  # Base name for report
        gap_columns = [  # Names for calculated gap columns
            "PMTCT hepatitis B test gap",  # Gap for HBV testing vs. new clients
            "PMTCT hepatitis C positive gap"  # Gap for HCV testing vs. new clients
        ]  # Defines gap column names
        report_name = f"{name}34"  # Report name with unique suffix
        
        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="PMTCT MSF",  # Key to fetch PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PMTCT Hepatitis Test columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate PMTCT Hepatitis B Test gap (requires numpy as np)
            df_main[df_columns[1]] != df_main[df_columns[0]],  # Check if HBV tested differs from new ANC clients
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for HBV gap
            0  # Set to 0 if no gap
        )  # Adds HBV test gap column
        
        df_main[gap_columns[1]] = np.where(  # Calculate PMTCT Hepatitis C Test gap
            df_main[df_columns[2]] != df_main[df_columns[0]],  # Check if HCV tested differs from new ANC clients
            df_main[df_columns[2]] - df_main[df_columns[0]],  # Calculate difference for HCV gap
            0  # Set to 0 if no gap
        )  # Adds HCV test gap column

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrames
        if display_output:  # Check if display output is requested
            if hasattr(process_PMTCT_Hepatitis_Test_gap, 'cached_styles'):  # Check if cached styled DataFrames exist
                cached_shape = getattr(process_PMTCT_Hepatitis_Test_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    for cluster, style in process_PMTCT_Hepatitis_Test_gap.cached_styles.items():  # Iterate over cached cluster styles
                        display_name = f"✔️ Displaying {cluster} {report_name}"  # Formatted display name for output
                        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
                        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header
                        print(print_display_name)  # Print display name with separators
                        widget_display_df(style)  # Display cached styled DataFrame (assumed widget function)
                    return  # Exit to avoid reprocessing

        # -- Step 6: Initialize cache
        if not hasattr(process_PMTCT_Hepatitis_Test_gap, 'cached_styles'):  # Check if cache attribute exists
            process_PMTCT_Hepatitis_Test_gap.cached_styles = {}  # Initialize empty dictionary for caching styles

        # -- Step 7: Identify unique clusters
        cluster_list = pd.Series(df_main['Cluster'].unique())  # Extract unique cluster names (requires pandas as pd)

        # -- Step 8: Process each cluster
        for current_cluster in cluster_list:  # Iterate over each unique cluster
            cluster_filtered = df_main[df_main['Cluster'] == current_cluster]  # Filter DataFrame for current cluster
            
            PMTCT_Hepatitis_msg = f"No {current_cluster} {report_name}"  # Message if no gaps found for cluster
            display_name = f"✔️ Displaying {current_cluster} {report_name}"  # Formatted display name for output
            display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
            print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

            cluster_filtered_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with gaps
                df=cluster_filtered,  # Input cluster-filtered DataFrame
                msg=PMTCT_Hepatitis_msg,  # Message to display if no gaps
                opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
                opNeg=None,  # No negative value filter
                opPos=None,  # No positive value filter
                opZero=None,  # No zero value filter
                opLT100=None  # No less-than-100 filter
            )  # Returns filtered DataFrame or None if no gaps

            if cluster_filtered_gap is None:  # Check if no gaps were found
                if current_cluster in process_PMTCT_Hepatitis_Test_gap.cached_styles:  # Check if cluster in cache
                    del process_PMTCT_Hepatitis_Test_gap.cached_styles[current_cluster]  # Remove cluster from cache
                continue  # Skip to next cluster

            cluster_filtered_style = (  # Apply styling to filtered DataFrame
                cluster_filtered_gap.style  # Create style object from filtered DataFrame
                .hide(axis='index')  # Hide row index for cleaner display
                .map(outlier_yellow, subset=gap_columns_wrap)  # Highlight non-zero gaps in yellow (assumed function)
            )  # Creates styled DataFrame for display/export

            process_PMTCT_Hepatitis_Test_gap.cached_styles[current_cluster] = cluster_filtered_style  # Cache styled DataFrame for cluster

            # -- Step 9: Define export variables
            report_name_cluster = f"{current_cluster}_{report_name}"  # Create cluster-specific report name
            report_month = cluster_filtered_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
            report_image_name = f"{report_month}_{report_name_cluster}.png"  # Create image file name with report month
            report_sheet_name = f"{current_cluster}_{report_name}"  # Define Excel sheet name with cluster

            # -- Step 10: Create descriptions for Word document
            report_description = []  # Initialize list for report descriptions
            if (cluster_filtered_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero Hepatitis B test gaps
                report_description.append(  # Add description for HBV test gap
                    f"Report Name: {gap_columns[0]}\n"
                    f"{df_columns[1]}\n"
                    f"should be equal to {df_columns[0]}\n"
                    f"Note: Where this report is correct, please ignore the gap - only review."
                )  # Describe HBV test gap issue
            if (cluster_filtered_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero Hepatitis C test gaps
                report_description.append(  # Add description for HCV test gap
                    f"Report Name: {gap_columns[1]}\n"
                    f"{df_columns[2]}\n"
                    f"should be equal to {df_columns[0]}\n"
                    f"Note: Where this report is correct, please ignore the gap - only review."
                )  # Describe HCV test gap issue
            report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

            # -- Step 11: Export results to multiple formats
            if not display_output:  # Check if user requested to export results
                export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                    report_name=report_name_cluster,  # Cluster-specific report name
                    df_style=cluster_filtered_style,  # Styled DataFrame for export
                    img_file_name=report_image_name,  # Image file name
                    img_file_path=sub_folder2_image_file_msf_outlier,  # Image file path (assumed defined)
                    doc_description=report_description,  # Word document description
                    doc_indicators_to_italicize=df_columns,  # Italicize input columns
                    doc_indicators_to_underline=gap_columns,  # Underline gap columns
                    xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                    xlm_sheet_name=report_sheet_name  # Excel sheet name
                )  # Exports to specified formats

            if display_output:  # Check if display is requested
                print(print_display_name)  # Print display name with separators
                widget_display_df(cluster_filtered_style)  # Display styled DataFrame (assumed widget function)

        # -- Step 12: Cache overall unfiltered DataFrame shape
        process_PMTCT_Hepatitis_Test_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_PMTCT_Hepatitis_Test_gap, 'cached_styles'):  # Check for cached styles
            process_PMTCT_Hepatitis_Test_gap.cached_styles.clear()  # Clear cached styled DataFrames
        if hasattr(process_PMTCT_Hepatitis_Test_gap, 'cached_shape'):  # Check for cached shape
            del process_PMTCT_Hepatitis_Test_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PMTCT Labour and Delivery gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PMTCT Labour and Delivery gap
def process_PMTCT_Labour_Delivery_gap(display_output=None):
    """
    Process PMTCT Labour and Delivery gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PMTCT Labour and Delivery metrics
            "Total deliveries at facility (booked and unbooked pregnant women) - Total",  # Total facility deliveries
            "Number of booked HIV positive pregnant women who delivered at facility - Total",  # Booked HIV+ women delivered
            "Number of unbooked HIV positive pregnant women who delivered at the facility - Total",  # Unbooked HIV+ women delivered
            "Number of live births by HIV positive women who delivered at the facility - Total"  # Live births by HIV+ women
        ]  # Defines columns for PMTCT Labour and Delivery analysis
        name = "PMTCT Labour and Delivery Gap"  # Base name for report
        gap_columns = [  # Names for calculated gap columns
            "PMTCT facility delivery by PPW gap",  # Gap for booked/unbooked vs. total deliveries
            "PMTCT facility Llvebirth by PPW gap"  # Gap for live births vs. booked/unbooked
        ]  # Defines gap column names
        report_name = f"{name}35"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="PMTCT MSF",  # Key to fetch PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PMTCT Labour and Delivery columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate PMTCT Facility Delivery by PPW gap (requires numpy as np)
            df_main[df_columns[1:3]].sum(axis=1) > df_main[df_columns[0]],  # Check if sum of booked/unbooked exceeds total deliveries
            df_main[df_columns[1:3]].sum(axis=1) - df_main[df_columns[0]],  # Calculate difference for delivery gap
            0  # Set to 0 if no gap
        )  # Adds delivery gap column
        
        df_main[gap_columns[1]] = np.where(  # Calculate PMTCT Facility Livebirth by PPW gap
            df_main[df_columns[3]] < df_main[df_columns[1:3]].sum(axis=1),  # Check if live births are less than sum of booked/unbooked
            df_main[df_columns[3]] - df_main[df_columns[1:3]].sum(axis=1),  # Calculate difference for livebirth gap
            0  # Set to 0 if no gap
        )  # Adds livebirth gap column

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_PMTCT_Labour_Delivery_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_PMTCT_Labour_Delivery_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_PMTCT_Labour_Delivery_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_PMTCT_Labour_Delivery_gap, 'cached_style'):  # Check for cached style
                del process_PMTCT_Labour_Delivery_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_PMTCT_Labour_Delivery_gap, 'cached_shape'):  # Check for cached shape
                del process_PMTCT_Labour_Delivery_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_PMTCT_Labour_Delivery_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_PMTCT_Labour_Delivery_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero delivery gaps
            report_description.append(  # Add description for delivery gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\nplus {df_columns[2]}\n"
                f"should not be greater than {df_columns[0]}"  # Describe expected delivery equality
            )  # Describe delivery gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero livebirth gaps
            report_description.append(  # Add description for livebirth gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[3]}\n"
                f"should not be less than {df_columns[1]}\nplus {df_columns[2]}"  # Describe expected livebirth relation
                f"Note: Where this PMTCT livebirth gap is true, please ignore the outlier."
            )  # Describe livebirth gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

        # -- Step 11: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_PMTCT_Labour_Delivery_gap, 'cached_style'):  # Check for cached style
            del process_PMTCT_Labour_Delivery_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_PMTCT_Labour_Delivery_gap, 'cached_shape'):  # Check for cached shape
            del process_PMTCT_Labour_Delivery_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PMTCT Facility HEI ARVs gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PMTCT Facility HEI ARVs gap
def process_PMTCT_Facility_HEI_ARVs_gap(display_output=None):
    """
    Process PMTCT Facility HEI ARVs gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PMTCT Facility HEI ARVs metrics
            "Number of live births by HIV positive women who delivered at the facility - Total",  # Total live births by HIV+ women
            "Number of HIV-exposed infants born to HIV positive women who received ARV prophylaxis within 72 hrs of delivery",  # Infants receiving ARV within 72hrs
            "Number of HIV-exposed infants born to HIV positive women who received ARV prophylaxis after 72 hrs of delivery"  # Infants receiving ARV after 72hrs
        ]  # Defines columns for PMTCT Facility HEI ARVs analysis
        df_columns2 = MSF_hierarchy + ['Facility']  # Combine MSF hierarchy and Facility column (assumed defined elsewhere)
        columns_to_keep = MSF_hierarchy + [df_columns[0]] + [f"{df_columns[1]} (within the Facility)"] + [f"{df_columns[2]} (within the Facility)"]  # Combine hierarchy and renamed data columns
        name = "PMTCT Facility HEI ARVs Gap"  # Base name for report
        gap_columns = ["PMTCT Facility HEI ARVs gap"]  # Name for calculated gap column
        report_name = f"{name}36"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header``

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="PMTCT MSF",  # Key to fetch PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns
            data_columns=df_columns  # Include PMTCT HEI ARVs columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Rename and prepare ARV within 72hrs data
        df_main2 = DHIS2_data['PMTCT MSF_sd<72_in-outside'][df_columns2].rename(  # Select and rename columns from <72hrs dataset
            columns={"Facility": f"{df_columns[1]} (within the Facility)"}  # Rename Facility to ARV within 72hrs
        )  # Store renamed DataFrame for ARV within 72hrs

        # -- Step 4: Rename and prepare ARV after 72hrs data
        if not 'Facility' in DHIS2_data['PMTCT MSF_sd>72_in-outside'].columns:  # Check if Facility column exists in >72hrs dataset
            df_main3 = pd.DataFrame(columns=df_columns2).rename(  # Create empty DataFrame with renamed columns (requires pandas as pd)
                columns={"Facility": f"{df_columns[2]} (within the Facility)"}  # Rename Facility to ARV after 72hrs
            )  # Store empty renamed DataFrame
        else:
            df_main3 = DHIS2_data['PMTCT MSF_sd>72_in-outside'][df_columns2].rename(  # Select and rename columns from >72hrs dataset
                columns={"Facility": f"{df_columns[2]} (within the Facility)"}  # Rename Facility to ARV after 72hrs
            )  # Store renamed DataFrame for ARV after 72hrs

        # -- Step 5: Merge primary data with ARV data
        df_main = df_main.merge(df_main2, on=MSF_hierarchy, how='left')  # Merge primary data with ARV within 72hrs data
        df_main = df_main.merge(df_main3, on=MSF_hierarchy, how='left')  # Merge primary data with ARV after 72hrs data

        # -- Step 6: Retain specified columns
        df_main = df_main[columns_to_keep]  # Filter DataFrame to keep only specified columns

        # -- Step 7: Clean and convert data types
        df_main[df_columns[0]] = pd.to_numeric(df_main[df_columns[0]], errors='coerce')  # Convert total live births to numeric
        df_main[f"{df_columns[1]} (within the Facility)"] = pd.to_numeric(df_main[f"{df_columns[1]} (within the Facility)"], errors='coerce')  # Convert ARV within 72hrs to numeric
        df_main[f"{df_columns[2]} (within the Facility)"] = pd.to_numeric(df_main[f"{df_columns[2]} (within the Facility)"], errors='coerce')  # Convert ARV after 72hrs to numeric
        df_main = df_main.fillna(0)  # Replace NaN values with 0
        float_columns = df_main.select_dtypes(include=['float64', 'float32']).columns  # Identify float columns in DataFrame
        for col in float_columns:  # Iterate over float columns
            df_main[col] = df_main[col].astype(int)  # Convert float columns to integers

        # -- Step 8: Calculate PMTCT Facility HEI ARVs gap
        df_main[gap_columns[0]] = np.where(  # Calculate PMTCT ARV prophylaxis gap (requires numpy as np)
            df_main[[f"{df_columns[1]} (within the Facility)", f"{df_columns[2]} (within the Facility)"]].sum(axis=1) != df_main[df_columns[0]],  # Check if sum of ARV prophylaxis differs from total live births
            df_main[[f"{df_columns[1]} (within the Facility)", f"{df_columns[2]} (within the Facility)"]].sum(axis=1) - df_main[df_columns[0]],  # Calculate gap as sum minus total live births
            0  # Set to 0 if no gap
        )  # Adds ARV prophylaxis gap column

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_PMTCT_Facility_HEI_ARVs_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_PMTCT_Facility_HEI_ARVs_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_PMTCT_Facility_HEI_ARVs_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 11: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap column
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_PMTCT_Facility_HEI_ARVs_gap, 'cached_style'):  # Check for cached style
                del process_PMTCT_Facility_HEI_ARVs_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_PMTCT_Facility_HEI_ARVs_gap, 'cached_shape'):  # Check for cached shape
                del process_PMTCT_Facility_HEI_ARVs_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 12: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 13: Cache styled DataFrame and shape
        process_PMTCT_Facility_HEI_ARVs_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_PMTCT_Facility_HEI_ARVs_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 14: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 15: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero ARV prophylaxis gaps
            report_description = (  # Define description for ARV gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\nplus {df_columns[2]}\n"
                f"should be equal to {df_columns[0]}"  
                f"REPORT ONLY HEI ARVs FOR 2025 LIVE BIRTHS BY PPW."         
                f"Note: Where this PMTCT livebirth gap is true, please ignore the outlier."
            )  # Describe ARV prophylaxis gap issue

        # -- Step 16: Export results to multiple formats
        if not display_output:  # Check if user requested to export results``
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 17: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_PMTCT_Facility_HEI_ARVs_gap, 'cached_style'):  # Check for cached style
            del process_PMTCT_Facility_HEI_ARVs_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_PMTCT_Facility_HEI_ARVs_gap, 'cached_shape'):  # Check for cached shape
            del process_PMTCT_Facility_HEI_ARVs_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PMTCT EID PCR Test gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PMTCT EID PCR Test gap
def process_PMTCT_EID_PCR_Test_gap(display_output=None):
    """
    Process PMTCT EID PCR Test gap, exporting results as image, Excel, and Word files.
    Iterates over each cluster, caches styled DataFrames, and displays them on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PMTCT EID PCR Test metrics
            "Number of live births by HIV positive women who delivered at the facility - Total",  # Total live births by HIV+ women
            "Number of HIV-exposed infants born to HIV positive women who received ARV prophylaxis within 72 hrs of delivery",  # Infants receiving ARV within 72hrs
            "Number of HIV-exposed infants born to HIV positive women who received ARV prophylaxis after 72 hrs of delivery",  # Infants receiving ARV after 72hrs
            "Number of Infants born to HIV positive women whose blood samples were taken for DNA PCR test within 72 hrs of birth",  # Infants tested for PCR within 72hrs
            "Number of Infants born to HIV positive women whose blood samples were taken for DNA PCR test between >72 hrs - < 2 months of birth"  # Infants tested for PCR after 72hrs
        ]  # Defines columns for PMTCT EID PCR Test analysis
        df_columns_spec = [  # List of column names including outside facility metrics
            "Number of live births by HIV positive women who delivered at the facility - Total",  # Total live births by HIV+ women
            "Number of HIV-exposed infants born to HIV positive women who received ARV prophylaxis within 72 hrs of delivery (outside the Facility)",  # Infants receiving ARV within 72hrs outside facility
            "Number of HIV-exposed infants born to HIV positive women who received ARV prophylaxis after 72 hrs of delivery (outside the Facility)",  # Infants receiving ARV after 72hrs outside facility
            "Number of Infants born to HIV positive women whose blood samples were taken for DNA PCR test within 72 hrs of birth",  # Infants tested for PCR within 72hrs
            "Number of Infants born to HIV positive women whose blood samples were taken for DNA PCR test between >72 hrs - < 2 months of birth"  # Infants tested for PCR after 72hrs
        ]  # Defines columns including outside facility data
        df_columns2 = MSF_hierarchy + ['Outside Facility']  # Combine MSF hierarchy and Outside Facility column (assumed defined elsewhere)
        columns_to_keep = (
            MSF_hierarchy + [df_columns_spec[0]] + [df_columns_spec[1]] + 
            [df_columns_spec[2]] + [df_columns_spec[3]] + [df_columns_spec[4]]
        )  # Combine hierarchy and specified data columns
        name = "PMTCT EID PCR Test Gap"  # Base name for report
        gap_columns = ["PMTCT EID PCR Test gap"]  # Name for calculated gap column
        report_name = f"{name}37"  # Report name with unique suffix

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="PMTCT MSF",  # Key to fetch PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns
            data_columns=df_columns  # Include PMTCT EID PCR Test columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Rename and prepare ARV within 72hrs data
        if 'Outside Facility' not in DHIS2_data['PMTCT MSF_sd<72_in-outside'].columns:  # Check if Outside Facility column exists in <72hrs dataset
            df_main2 = pd.DataFrame(columns=df_columns2).rename(  # Create empty DataFrame with renamed columns (requires pandas as pd)
                columns={"Outside Facility": df_columns_spec[1]}  # Rename Outside Facility to ARV within 72hrs outside
            )  # Store empty renamed DataFrame
        else:
            df_main2 = DHIS2_data['PMTCT MSF_sd<72_in-outside'][df_columns2].rename(  # Select and rename columns from <72hrs dataset
                columns={"Outside Facility": df_columns_spec[1]}  # Rename Outside Facility to ARV within 72hrs outside
            )  # Store renamed DataFrame for ARV within 72hrs

        # -- Step 4: Rename and prepare ARV after 72hrs data
        if 'Outside Facility' not in DHIS2_data['PMTCT MSF_sd>72_in-outside'].columns:  # Check if Outside Facility column exists in >72hrs dataset
            df_main3 = pd.DataFrame(columns=df_columns2).rename(  # Create empty DataFrame with renamed columns
                columns={"Outside Facility": df_columns_spec[2]}  # Rename Outside Facility to ARV after 72hrs outside
            )  # Store empty renamed DataFrame
        else:
            df_main3 = DHIS2_data['PMTCT MSF_sd>72_in-outside'][df_columns2].rename(  # Select and rename columns from >72hrs dataset
                columns={"Outside Facility": df_columns_spec[2]}  # Rename Outside Facility to ARV after 72hrs outside
            )  # Store renamed DataFrame for ARV after 72hrs

        # -- Step 5: Merge primary data with ARV data
        df_main = df_main.merge(df_main2, on=MSF_hierarchy, how='left')  # Merge primary data with ARV within 72hrs data
        df_main = df_main.merge(df_main3, on=MSF_hierarchy, how='left')  # Merge primary data with ARV after 72hrs data

        # -- Step 6: Retain specified columns
        df_main = df_main[columns_to_keep]  # Filter DataFrame to keep only specified columns

        # -- Step 7: Clean and convert data types
        df_main = df_main.fillna(0)  # Replace NaN values with 0
        # Explicitly convert relevant columns to numeric, coercing errors to 0
        for col in df_columns_spec:  # Iterate over specified columns
            df_main[col] = pd.to_numeric(df_main[col], errors='coerce').fillna(0).astype(int)  # Convert to numeric and then to integer

        # -- Step 8: Calculate PMTCT EID PCR Test gap
        df_main[gap_columns[0]] = np.where(  # Calculate PMTCT EID PCR Test gap (requires numpy as np)
            df_main[df_columns_spec[3:5]].sum(axis=1) != df_main[df_columns_spec[0:3]].sum(axis=1),  # Check if sum of PCR tests differs from sum of live births and ARVs
            df_main[df_columns_spec[3:5]].sum(axis=1) - df_main[df_columns_spec[0:3]].sum(axis=1),  # Calculate gap as PCR tests minus live births and ARVs
            0  # Set to 0 if no gap
        ).astype(int)  # Ensure integer type for gap column

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 10: Check and display cached styled DataFrames
        if display_output:  # Check if display output is requested
            if hasattr(process_PMTCT_EID_PCR_Test_gap, 'cached_styles'):  # Check if cached styled DataFrames exist
                cached_shape = getattr(process_PMTCT_EID_PCR_Test_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    for cluster, style in process_PMTCT_EID_PCR_Test_gap.cached_styles.items():  # Iterate over cached cluster styles
                        display_name = f"✔️ Displaying {cluster} {report_name}"  # Formatted display name for output
                        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
                        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header
                        print(print_display_name)  # Print display name with separators
                        widget_display_df(style)  # Display cached styled DataFrame (assumed widget function)
                    return  # Exit to avoid reprocessing

        # -- Step 11: Initialize cache
        if not hasattr(process_PMTCT_EID_PCR_Test_gap, 'cached_styles'):  # Check if cache attribute exists
            process_PMTCT_EID_PCR_Test_gap.cached_styles = {}  # Initialize empty dictionary for caching styles

        # -- Step 12: Identify unique clusters
        cluster_list = pd.Series(df_main['Cluster'].unique())  # Extract unique cluster names (requires pandas as pd)

        # -- Step 13: Process each cluster
        for current_cluster in cluster_list:  # Iterate over each unique cluster
            cluster_filtered = df_main[df_main['Cluster'] == current_cluster]  # Filter DataFrame for current cluster
            
            PMTCT_EID_msg = f"No {current_cluster} {report_name}"  # Message if no gaps found for cluster
            display_name = f"✔️ Displaying {current_cluster} {report_name}"  # Formatted display name for output
            display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
            print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

            cluster_filtered_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with gaps
                df=cluster_filtered,  # Input cluster-filtered DataFrame
                msg=PMTCT_EID_msg,  # Message to display if no gaps
                opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
                opNeg=None,  # No negative value filter
                opPos=None,  # No positive value filter
                opZero=None,  # No zero value filter
                opLT100=None  # No less-than-100 filter
            )  # Returns filtered DataFrame or None if no gaps

            if cluster_filtered_gap is None:  # Check if no gaps were found
                if current_cluster in process_PMTCT_EID_PCR_Test_gap.cached_styles:  # Check if cluster in cache
                    del process_PMTCT_EID_PCR_Test_gap.cached_styles[current_cluster]  # Remove cluster from cache
                continue  # Skip to next cluster

            cluster_filtered_style = (  # Apply styling to filtered DataFrame
                cluster_filtered_gap.style  # Create style object from filtered DataFrame
                .hide(axis='index')  # Hide row index for cleaner display
                .map(outlier_yellow, subset=gap_columns_wrap)  # Highlight non-zero gaps in yellow (assumed function)
            )  # Creates styled DataFrame for display/export

            process_PMTCT_EID_PCR_Test_gap.cached_styles[current_cluster] = cluster_filtered_style  # Cache styled DataFrame for cluster

            # -- Step 14: Define export variables
            report_name_cluster = f"{current_cluster}_{report_name}"  # Create cluster-specific report name
            report_month = cluster_filtered_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
            report_image_name = f"{report_month}_{report_name_cluster}.png"  # Create image file name with report month
            report_sheet_name = f"{current_cluster}_{report_name}"  # Define Excel sheet name with cluster

            # -- Step 15: Create descriptions for Word document
            report_description = []  # Initialize list for report descriptions
            if (cluster_filtered_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero EID PCR Test gaps
                report_description.append(  # Add description for EID PCR Test gap
                    f"Report Name: {gap_columns[0]}\n"
                    f"{df_columns_spec[0]}\nplus {df_columns_spec[1]}\nplus {df_columns_spec[2]}\n"
                    f"should be equal to {df_columns[3]}\nplus {df_columns_spec[4]}\n"
                    f"REPORT ONLY EID PCR TEST FOR 2025 LIVE BIRTHS BY PPW.\n"
                    f"Note: Where this PMTCT EID PCR Test gap is true, please ignore the outlier."
                )  # Describe EID PCR Test gap issue
            report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

            # -- Step 16: Export results to multiple formats
            if not display_output:  # Check if user requested to export results
                export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                    report_name=report_name_cluster,  # Cluster-specific report name
                    df_style=cluster_filtered_style,  # Styled DataFrame for export
                    img_file_name=report_image_name,  # Image file name
                    img_file_path=sub_folder2_image_file_msf_outlier,  # Image file path (assumed defined)
                    doc_description=report_description,  # Word document description
                    doc_indicators_to_italicize=df_columns,  # Italicize input columns
                    doc_indicators_to_underline=gap_columns,  # Underline gap column
                    xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                    xlm_sheet_name=report_sheet_name  # Excel sheet name
                )  # Exports to specified formats

            if display_output:  # Check if display is requested
                print(print_display_name)  # Print display name with separators
                widget_display_df(cluster_filtered_style)  # Display styled DataFrame (assumed widget function)

        # -- Step 17: Cache overall unfiltered DataFrame shape
        process_PMTCT_EID_PCR_Test_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_PMTCT_EID_PCR_Test_gap, 'cached_styles'):  # Check for cached styles
            process_PMTCT_EID_PCR_Test_gap.cached_styles.clear()  # Clear cached styled DataFrames
        if hasattr(process_PMTCT_EID_PCR_Test_gap, 'cached_shape'):  # Check for cached shape
            del process_PMTCT_EID_PCR_Test_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PMTCT EID PCR Test Result gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PMTCT EID PCR Test Result gap
def process_PMTCT_EID_PCR_Test_Result_gap(display_output=None):
    """
    Process PMTCT EID PCR Test Result gap, exporting results as image, Excel, and Word files.
    Iterates over each cluster, caches styled DataFrames, and displays them on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PMTCT EID PCR Test Result metrics
            "Number of Infants born to HIV positive women whose blood samples were taken for DNA PCR test within 72 hrs of birth",  # Infants tested for PCR within 72hrs
            "Number of Infants born to HIV positive women whose blood samples were taken for DNA PCR test between >72 hrs - < 2 months of birth",  # Infants tested for PCR after 72hrs
            "Number of HIV PCR results received for babies whose samples were taken within 72 hrs of birth",  # PCR results received within 72hrs
            "Number of HIV PCR results received for babies whose samples were taken between >72 hrs - < 2 months of birth"  # PCR results received after 72hrs
        ]  # Defines columns for PMTCT EID PCR Test Result analysis
        name = "PMTCT EID PCR Test Result Gap"  # Base name for report
        gap_columns = ["PMTCT EID PCR test result gap"]  # Name for calculated gap column
        report_name = f"{name}38"  # Report name with unique suffix

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="PMTCT MSF",  # Key to fetch PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PMTCT EID PCR Test Result columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Clean and convert data types
        df_main = df_main.fillna(0)  # Replace NaN values with 0
        # Explicitly convert relevant columns to numeric, coercing errors to 0
        for col in df_columns:  # Iterate over specified columns
            df_main[col] = pd.to_numeric(df_main[col], errors='coerce').fillna(0).astype(int)  # Convert to numeric and then to integer

        # -- Step 4: Calculate PMTCT EID PCR Test Result gap
        df_main[gap_columns[0]] = np.where(  # Calculate PMTCT EID PCR Test Result gap (requires numpy as np)
            df_main[df_columns[0:2]].sum(axis=1) != df_main[df_columns[2:4]].sum(axis=1),  # Check if sum of samples taken differs from sum of results received
            df_main[df_columns[0:2]].sum(axis=1) - df_main[df_columns[2:4]].sum(axis=1),  # Calculate gap as samples minus results
            0  # Set to 0 if no gap
        ).astype(int)  # Ensure integer type for gap column

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 6: Check and display cached styled DataFrames
        if display_output:  # Check if display output is requested
            if hasattr(process_PMTCT_EID_PCR_Test_Result_gap, 'cached_styles'):  # Check if cached styled DataFrames exist
                cached_shape = getattr(process_PMTCT_EID_PCR_Test_Result_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    for cluster, style in process_PMTCT_EID_PCR_Test_Result_gap.cached_styles.items():  # Iterate over cached cluster styles
                        display_name = f"✔️ Displaying {cluster} {report_name}"  # Formatted display name for output
                        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
                        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header
                        print(print_display_name)  # Print display name with separators
                        widget_display_df(style)  # Display cached styled DataFrame (assumed widget function)
                    return  # Exit to avoid reprocessing

        # -- Step 7: Initialize cache
        if not hasattr(process_PMTCT_EID_PCR_Test_Result_gap, 'cached_styles'):  # Check if cache attribute exists
            process_PMTCT_EID_PCR_Test_Result_gap.cached_styles = {}  # Initialize empty dictionary for caching styles

        # -- Step 8: Identify unique clusters
        cluster_list = pd.Series(df_main['Cluster'].unique())  # Extract unique cluster names (requires pandas as pd)

        # -- Step 9: Process each cluster
        for current_cluster in cluster_list:  # Iterate over each unique cluster
            cluster_filtered = df_main[df_main['Cluster'] == current_cluster]  # Filter DataFrame for current cluster
            
            PMTCT_EID_msg = f"No {current_cluster} {report_name}"  # Message if no gaps found for cluster
            display_name = f"✔️ Displaying {current_cluster} {report_name}"  # Formatted display name for output
            display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
            print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

            cluster_filtered_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with gaps
                df=cluster_filtered,  # Input cluster-filtered DataFrame
                msg=PMTCT_EID_msg,  # Message to display if no gaps
                opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
                opNeg=None,  # No negative value filter
                opPos=None,  # No positive value filter
                opZero=None,  # No zero value filter
                opLT100=None  # No less-than-100 filter
            )  # Returns filtered DataFrame or None if no gaps

            if cluster_filtered_gap is None:  # Check if no gaps were found
                if current_cluster in process_PMTCT_EID_PCR_Test_Result_gap.cached_styles:  # Check if cluster in cache
                    del process_PMTCT_EID_PCR_Test_Result_gap.cached_styles[current_cluster]  # Remove cluster from cache
                continue  # Skip to next cluster

            cluster_filtered_style = (  # Apply styling to filtered DataFrame
                cluster_filtered_gap.style  # Create style object from filtered DataFrame
                .hide(axis='index')  # Hide row index for cleaner display
                .map(outlier_yellow, subset=gap_columns_wrap)  # Highlight non-zero gaps in yellow (assumed function)
            )  # Creates styled DataFrame for display/export

            process_PMTCT_EID_PCR_Test_Result_gap.cached_styles[current_cluster] = cluster_filtered_style  # Cache styled DataFrame for cluster

            # -- Step 10: Define export variables
            report_name_cluster = f"{current_cluster}_{report_name}"  # Create cluster-specific report name
            report_month = cluster_filtered_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
            report_image_name = f"{report_month}_{report_name_cluster}.png"  # Create image file name with report month
            report_sheet_name = f"{current_cluster}_{report_name}"  # Define Excel sheet name with cluster

            # -- Step 11: Create descriptions for Word document
            report_description = []  # Initialize list for report descriptions
            if (cluster_filtered_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero EID PCR Test Result gaps
                report_description.append(  # Add description for EID PCR Test Result gap
                    f"Report Name: {gap_columns[0]}\n"
                    f"{df_columns[0]}\nplus {df_columns[1]}\n"
                    f"should be equal to {df_columns[2]}\nplus {df_columns[3]}\n"
                    f"REPORT ONLY EID PCR TEST RESULT FOR 2025 LIVE BIRTHS BY PPW.\n"
                    f"Note: Where this {name} is true, please ignore the outlier."
                )  # Describe EID PCR Test Result gap issue
            report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

            # -- Step 12: Export results to multiple formats
            if not display_output:  # Check if user requested to export results
                export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                    report_name=report_name_cluster,  # Cluster-specific report name
                    df_style=cluster_filtered_style,  # Styled DataFrame for export
                    img_file_name=report_image_name,  # Image file name
                    img_file_path=sub_folder2_image_file_msf_outlier,  # Image file path (assumed defined)
                    doc_description=report_description,  # Word document description
                    doc_indicators_to_italicize=df_columns,  # Italicize input columns
                    doc_indicators_to_underline=gap_columns,  # Underline gap column
                    xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                    xlm_sheet_name=report_sheet_name  # Excel sheet name
                )  # Exports to specified formats

            if display_output:  # Check if display is requested
                print(print_display_name)  # Print display name with separators
                widget_display_df(cluster_filtered_style)  # Display styled DataFrame (assumed widget function)
        
        # -- Step 13: Cache overall unfiltered DataFrame shape
        process_PMTCT_EID_PCR_Test_Result_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_PMTCT_EID_PCR_Test_Result_gap, 'cached_styles'):  # Check for cached styles
            process_PMTCT_EID_PCR_Test_Result_gap.cached_styles.clear()  # Clear cached styled DataFrames
        if hasattr(process_PMTCT_EID_PCR_Test_Result_gap, 'cached_shape'):  # Check for cached shape
            del process_PMTCT_EID_PCR_Test_Result_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

## NSP MSF
### - PWID Newly Recriuted gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process NSP Newly Recruited gap
def process_NSP_Newly_Recruited_gap(display_output=None):
    """
    Process NSP Newly Recruited gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for NSP Newly Recruited metrics
            "Number of PWID Newly recruited into the program within the reporting month",  # Newly recruited PWID count
            "Number of PWID New and old recruited into the program within the reporting month"  # Total new and old PWID count
        ]  # Defines columns for NSP Newly Recruited analysis
        name = "PWID Newly Recruited Gap"  # Base name for report
        gap_columns = ["PWID newly recruited gap"]  # Name for calculated gap column
        report_name = f"{name}39"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="NSP MSF",  # Key to fetch NSP MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include NSP Newly Recruited columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gap for newly recruited PWID
        df_main[gap_columns[0]] = np.where(  # Calculate NSP Newly Recruited gap (requires numpy as np)
            df_main[df_columns[0]] > df_main[df_columns[1]],  # Check if newly recruited exceeds total new and old recruited
            df_main[df_columns[0]] - df_main[df_columns[1]],  # Calculate difference for gap
            0  # Set to 0 if no gap
        ).astype(int)  # Ensure integer type for gap column

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        #df_columns_spec_wrap = wrap_column_headers2(df_columns_spec)  # Commented out in original code, preserved as is
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_NSP_Newly_Recruited_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_NSP_Newly_Recruited_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_NSP_Newly_Recruited_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap column
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_NSP_Newly_Recruited_gap, 'cached_style'):  # Check for cached style
                del process_NSP_Newly_Recruited_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_NSP_Newly_Recruited_gap, 'cached_shape'):  # Check for cached shape
                del process_NSP_Newly_Recruited_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_NSP_Newly_Recruited_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_NSP_Newly_Recruited_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero newly recruited gaps
            report_description = (  # Define description for gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[0]}\n"
                f"should not be greater than {df_columns[1]}"  # Describe expected relation
            )  # Describe newly recruited gap issue

        # -- Step 11: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap column
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_NSP_Newly_Recruited_gap, 'cached_style'):  # Check for cached style
            del process_NSP_Newly_Recruited_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_NSP_Newly_Recruited_gap, 'cached_shape'):  # Check for cached shape
            del process_NSP_Newly_Recruited_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PWID HTS Positive Linkage gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PWID HTS Positive gap
def process_NSP_HTS_Positive_Linkage_gap(display_output=None):
    """
    Process PWID HTS Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PWID HTS Positive and Linkage metrics
            "Number of PWID tested for HIV and received their test result",  # PWID tested and received results
            "Number of PWID tested for HIV Positive during the reporting month",  # PWID tested HIV positive
            "Number of PWID tested for HIV Positive and were linked to ART during the reporting month"  # PWID linked to ART
        ]  # Defines columns for PWID HTS Positive and Linkage analysis
        name = "PWID HTS Positive Gap"  # Base name for report
        gap_columns = ["PWID HTS positive gap", "PWID positive linkage gap"]  # Names for calculated gap columns
        report_name = f"{name}40"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header
        

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="NSP MSF",  # Key to fetch NSP MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PWID HTS Positive and Linkage columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate PWID HTS Positive gap (requires numpy as np)
            df_main[df_columns[1]] > df_main[df_columns[0]],  # Check if HIV positive tests exceed total tested
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for positive gap
            0  # Set to 0 if no gap
        )  # Adds HTS Positive gap column

        df_main[gap_columns[1]] = np.where(  # Calculate PWID Positive Linkage gap
            df_main[df_columns[2]] != df_main[df_columns[1]],  # Check if ART linkage differs from HIV positive tests
            df_main[df_columns[2]] - df_main[df_columns[1]],  # Calculate difference for linkage gap
            0  # Set to 0 if no gap
        )  # Adds Positive Linkage gap column

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_NSP_HTS_Positive_Linkage_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_NSP_HTS_Positive_Linkage_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_NSP_HTS_Positive_Linkage_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_NSP_HTS_Positive_Linkage_gap, 'cached_style'):  # Check for cached style
                del process_NSP_HTS_Positive_Linkage_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_NSP_HTS_Positive_Linkage_gap, 'cached_shape'):  # Check for cached shape
                del process_NSP_HTS_Positive_Linkage_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_NSP_HTS_Positive_Linkage_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_NSP_HTS_Positive_Linkage_gap.cached_shape = df_main.shape  # Store original DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero HTS Positive gaps
            report_description.append(  # Add description for HTS Positive gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be greater than {df_columns[0]}"  # Describe expected relation for HTS Positive
            )  # Describe HTS Positive gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero Positive Linkage gaps
            report_description.append(  # Add description for Positive Linkage gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[2]}\n"
                f"should be equal to {df_columns[1]}"  # Describe expected equality for linkage
            )  # Describe Positive Linkage gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

        # -- Step 11: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=df_columns, # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_NSP_HTS_Positive_Linkage_gap, 'cached_style'):  # Check for cached style
            del process_NSP_HTS_Positive_Linkage_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_NSP_HTS_Positive_Linkage_gap, 'cached_shape'):  # Check for cached shape
            del process_NSP_HTS_Positive_Linkage_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PWID Coinfection gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PWID Coinfection gap
def process_NSP_Coinfection_gap(display_output=None):
    """
    Process PWID Coinfection gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PWID Coinfection metrics
            "Number of PWID Screened for STI and scored 1+ within the reporting month",  # PWID screened for STI
            "Number of PWID Screened for STI and scored 1+ who were referred for STI treatment",  # PWID referred for STI treatment
            "Number of PWID Screened for TB and scored 1+ within reporting month",  # PWID screened for TB
            "Number of PWID Screened for TB and scored 1+ who were referred for TB treatment"  # PWID referred for TB treatment
        ]  # Defines columns for PWID Coinfection analysis
        name = "PWID Coinfection Gap"  # Base name for report
        gap_columns = [
            "PWID coinfection treatment - STI gap",
            "PWID coinfection treatment - TB gap"
        ]  # Names for calculated gap columns
        report_name = f"{name}41"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="NSP MSF",  # Key to fetch NSP MSF
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PWID Coinfection columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate PWID Coinfection Treatment - STI gap (requires numpy as np)
            df_main[df_columns[1]] != df_main[df_columns[0]],  # Check if STI referrals differ from STI screened
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for STI gap
            0  # Set to 0 if no gap
        )  # Adds STI Coinfection gap column

        df_main[gap_columns[1]] = np.where(  # Calculate PWID Coinfection Treatment - TB gap
            df_main[df_columns[3]] != df_main[df_columns[2]],  # Check if TB referrals differ from TB screened
            df_main[df_columns[3]] - df_main[df_columns[2]],  # Calculate difference for TB gap
            0  # Set to 0 if no gap
        )  # Adds TB Coinfection gap column

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_NSP_Coinfection_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_NSP_Coinfection_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_NSP_Coinfection_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_NSP_Coinfection_gap, 'cached_style'):  # Check for cached style
                del process_NSP_Coinfection_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_NSP_Coinfection_gap, 'cached_shape'):  # Check for cached shape
                del process_NSP_Coinfection_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_NSP_Coinfection_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_NSP_Coinfection_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero STI Coinfection gaps
            report_description.append(  # Add description for STI Coinfection gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"  # Describe expected equality for STI
            )  # Describe STI Coinfection gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero TB Coinfection gaps
            report_description.append(  # Add description for TB Coinfection gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[3]}\n"
                f"should be equal to {df_columns[2]}"  # Describe expected equality for TB
            )  # Describe TB Coinfection gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

        # -- Step 11: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_NSP_Coinfection_gap, 'cached_style'):  # Check for cached style
            del process_NSP_Coinfection_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_NSP_Coinfection_gap, 'cached_shape'):  # Check for cached shape
            del process_NSP_Coinfection_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PWID Substance Abuse gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PWID Substance Abuse gap
def process_NSP_Substance_Abuse_gap(display_output=None):
    """
    Process PWID Coinfection gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PWID Substance Abuse metrics
            "Number of PWID that developed an Injection abscess within the reporting month",  # PWID with injection abscess
            "Number of PWID treated for Injection abscess within the reporting month",  # PWID treated for injection abscess
            "Number of PWID with Opiod overdose within the reporting month",  # PWID with opioid overdose
            "Number of PWID with Opiod overdose who were resuscitated medically within the reporting month"  # PWID resuscitated for opioid overdose
        ]  # Defines columns for PWID Substance Abuse analysis
        name = "PWID Substance Abuse Gap"  # Base name for report
        gap_columns = [
            "PWID injection abuse gap",
            "PWID Opid overdose gap"  # Note: Typo 'Opid' preserved as in original
        ]  # Names for calculated gap columns
        report_name = f"{name}42"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="NSP MSF",  # Key to fetch NSP MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PWID Substance Abuse columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate PWID Injection Abuse gap (requires numpy as np)
            df_main[df_columns[1]] != df_main[df_columns[0]],  # Check if abscess treatments differ from abscess cases
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for injection abuse gap
            0  # Set to 0 if no gap
        )  # Adds Injection Abuse gap column

        df_main[gap_columns[1]] = np.where(  # Calculate PWID Opioid Overdose gap
            df_main[df_columns[3]] != df_main[df_columns[2]],  # Check if overdose resuscitations differ from overdose cases
            df_main[df_columns[3]] - df_main[df_columns[2]],  # Calculate difference for opioid overdose gap
            0  # Set to 0 if no gap
        )  # Adds Opioid Overdose gap column

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_NSP_Substance_Abuse_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_NSP_Substance_Abuse_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_NSP_Substance_Abuse_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_NSP_Substance_Abuse_gap, 'cached_style'):  # Check for cached style
                del process_NSP_Substance_Abuse_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_NSP_Substance_Abuse_gap, 'cached_shape'):  # Check for cached shape
                del process_NSP_Substance_Abuse_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_NSP_Substance_Abuse_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_NSP_Substance_Abuse_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero Injection Abuse gaps
            report_description.append(  # Add description for Injection Abuse gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"  # Describe expected equality for injection abscess
            )  # Describe Injection Abuse gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero Opioid Overdose gaps
            report_description.append(  # Add description for Opioid Overdose gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[3]}\n"
                f"should be equal to {df_columns[2]}"  # Describe expected equality for opioid overdose
            )  # Describe Opioid Overdose gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

        # -- Step 11: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_NSP_Substance_Abuse_gap, 'cached_style'):  # Check for cached style
            del process_NSP_Substance_Abuse_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_NSP_Substance_Abuse_gap, 'cached_shape'):  # Check for cached shape
            del process_NSP_Substance_Abuse_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - PWID MAT Referral gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process PWID MAT Referral gap
def process_NSP_MAT_Referral_gap(display_output=None):
    """
    Process PWID Coinfection gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PWID MAT Referral metrics
            "Number of PWID eligible for medication-assisted therapy (MAT)",  # PWID eligible for MAT
            "Number of PWID referred for medication-assisted therapy"  # PWID referred for MAT
        ]  # Defines columns for PWID MAT Referral analysis
        name = "PWID Medication Assisted Theraphy Gap"  # Base name for report (typo: 'Theraphy' preserved)
        gap_columns = ["PWID MAT referral gap"]  # Names for calculated gap columns (typo: 'Refferal' preserved)
        report_name = f"{name}43"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="NSP MSF",  # Key to fetch NSP MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PWID MAT Referral columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate PWID MAT Referral gap (requires numpy as np)
            df_main[df_columns[1]] != df_main[df_columns[0]],  # Check if MAT referrals differ from eligible PWID
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for MAT referral gap
            0  # Set to 0 if no gap
        )  # Adds MAT Referral gap column

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_NSP_MAT_Referral_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_NSP_MAT_Referral_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_NSP_MAT_Referral_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_NSP_MAT_Referral_gap, 'cached_style'):  # Check for cached style
                del process_NSP_MAT_Referral_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_NSP_MAT_Referral_gap, 'cached_shape'):  # Check for cached shape
                del process_NSP_MAT_Referral_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_NSP_MAT_Referral_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_NSP_MAT_Referral_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero MAT Referral gaps
            report_description = (  # Define description for MAT Referral gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"  # Describe expected equality for MAT referrals
            )  # Describe MAT Referral gap issue

        # -- Step 11: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_NSP_MAT_Referral_gap, 'cached_style'):  # Check for cached style
            del process_NSP_MAT_Referral_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_NSP_MAT_Referral_gap, 'cached_shape'):  # Check for cached shape
            del process_NSP_MAT_Referral_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

## KP Prev MSF
### - KP-Prev MSH gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP MHS and Psychosocial Support gap
def process_KP_MHS_gap(display_output=None):
    """
    Process KP MHS and Psychosocial Support gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP MHS and Psychosocial Support metrics
            "NTHRIP-Number of KPs who were screened for Mental Health and Psychosocial support during the reporting period",  # KPs screened for MHS
            "NTHRIP-Number of KPs who were diagnosed with Mental Health and Psychosocial issues during the reporting month",  # KPs diagnosed with MHS issues
            "NTHRIP-Number of KPs who were diagnosed with Mental Health and Psychosocial issues and were offered support services during the reporting month"  # KPs offered MHS support
        ]  # Defines columns for KP MHS analysis
        name = "KP Prev MHS and Psychosocial Support Gap"  # Base name for report
        gap_columns = [  # Names for calculated gap columns
            "KP Prev-MHS and psychological dignosed gap",  # Typo: 'dignosed' preserved
            "KP Prev-MHS and psychological support gap"  # Typo: 'psychological' preserved
        ]  # Defines gap columns for MHS metrics
        report_name = f"{name}44"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP MHS columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate MHS diagnosis gap (requires numpy as np)
            df_main[df_columns[1]] > df_main[df_columns[0]],  # Check if diagnosed exceeds screened
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for diagnosis gap
            0  # Set to 0 if no gap
        )  # Adds MHS diagnosis gap column
        df_main[gap_columns[1]] = np.where(  # Calculate MHS support gap
            df_main[df_columns[2]] < df_main[df_columns[1]],  # Check if support is less than diagnosed
            df_main[df_columns[2]] - df_main[df_columns[1]],  # Calculate difference for support gap
            0  # Set to 0 if no gap
        )  # Adds MHS support gap column
        
        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_MHS_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_MHS_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_MHS_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_MHS_gap, 'cached_style'):  # Check for cached style
                del process_KP_MHS_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_MHS_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_MHS_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_MHS_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_KP_MHS_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero MHS diagnosis gaps
            report_description.append(  # Add description for MHS diagnosis gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be greater than {df_columns[0]}"  # Describe expected relation for diagnosis
            )  # Describe MHS diagnosis gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero MHS support gaps
            report_description.append(  # Add description for MHS support gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[2]}\n"
                f"should not be less than {df_columns[1]}"  # Describe expected relation for support
            )  # Describe MHS support gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

        # -- Step 11: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_MHS_gap, 'cached_style'):  # Check for cached style
            del process_KP_MHS_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_MHS_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_MHS_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-Prev MSH by Type gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP MHS and Psychosocial Support gap
def process_KP_MHS_Access_Type_gap(display_output=None):
    """
    Process KP KP MHS and Psychosocial Support gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP MHS and Psychosocial Support metrics
            "NTHRIP-Number of KPs who were screened for Mental Health and Psychosocial support during the reporting period",  # KPs screened for MHS
            "NTHRIP-Number of KPs who were diagnosed with Mental Health and Psychosocial issues during the reporting month",  # KPs diagnosed with MHS issues
            "NTHRIP-Number of KPs who were diagnosed with Mental Health and Psychosocial issues and were offered support services during the reporting month"  # KPs offered MHS support
        ]  # Defines columns for KP MHS analysis
        df_columns_spec = [  # List of specified columns including aggregated access types
            df_columns[0], f"{df_columns[0]} (Walk-In and Community)",  # Screened columns
            df_columns[1], f"{df_columns[1]} (Walk-In and Community)",  # Diagnosed columns
            df_columns[2], f"{df_columns[2]} (Walk-In and Community)"   # Support columns
        ]  # Defines columns for access type comparisons
        df_columns2 = ["Community", "Walk-In"]  # Access type columns for aggregation
        name = "KP Prev_Prev MHS and Psychosocial Support by Access Type Gap"  # Base name for report
        gap_columns = [  # Names for calculated gap columns
            "KP Prev-MHS and psychosocial support by access type (Walk-In and Community) gap",
            "KP Prev-MHS and psychosocial diagnosis by access type (Walk-In and Community) gap",
            "KP Prev-MHS and psychosocial issues offered support services by access type (Walk-In and Community) gap"
        ]  # Defines gap columns for MHS metrics
        report_name = f"{name}45"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare main DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP MHS columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        df_main2 = DHIS2_data['KP Prev MSF_access_type_mhs'][MSF_hierarchy + df_columns2].copy()  # Copy MHS access type data
        df_main2[df_columns2] = df_main2[df_columns2].apply(pd.to_numeric, errors='coerce').fillna(0)  # Convert access type columns to numeric, fill NaN with 0
        df_main2[df_columns_spec[1]] = df_main2[df_columns2].sum(axis=1)  # Sum Community and Walk-In for MHS screening
        df_main2.drop(columns=df_columns2, inplace=True)  # Drop original access type columns

        df_main3 = DHIS2_data['KP Prev MSF_access_type_msh_diagnose'][MSF_hierarchy + df_columns2].copy()  # Copy MHS diagnosis access type data (typo: 'msh' preserved)
        df_main3[df_columns2] = df_main3[df_columns2].apply(pd.to_numeric, errors='coerce').fillna(0)  # Convert access type columns to numeric, fill NaN with 0
        df_main3[df_columns_spec[3]] = df_main3[df_columns2].sum(axis=1)  # Sum Community and Walk-In for MHS diagnosis
        df_main3.drop(columns=df_columns2, inplace=True)  # Drop original access type columns

        df_main4 = DHIS2_data['KP Prev MSF_access_type_msh_support'][MSF_hierarchy + df_columns2].copy()  # Copy MHS support access type data (typo: 'msh' preserved)
        df_main4[df_columns2] = df_main4[df_columns2].apply(pd.to_numeric, errors='coerce').fillna(0)  # Convert access type columns to numeric, fill NaN with 0
        df_main4[df_columns_spec[5]] = df_main4[df_columns2].sum(axis=1)  # Sum Community and Walk-In for MHS support
        df_main4.drop(columns=df_columns2, inplace=True)  # Drop original access type columns

        df_main = df_main.merge(df_main2, on=MSF_hierarchy, how='left')  # Merge main DataFrame with MHS screening access type data
        df_main = df_main.merge(df_main3, on=MSF_hierarchy, how='left')  # Merge with MHS diagnosis access type data
        df_main = df_main.merge(df_main4, on=MSF_hierarchy, how='left')  # Merge with MHS support access type data

        df_main[df_columns_spec] = df_main[df_columns_spec].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int) # Convert specified columns to numeric, fill NaN with 0, and cast to int

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate MHS screening gap (requires numpy as np)
            df_main[df_columns_spec[1]] != df_main[df_columns_spec[0]],  # Check if access type sum differs from total screened
            df_main[df_columns_spec[1]] - df_main[df_columns_spec[0]],  # Calculate difference for screening gap
            0  # Set to 0 if no gap
        )  # Adds MHS screening gap column
        df_main[gap_columns[1]] = np.where(  # Calculate MHS diagnosis gap
            df_main[df_columns_spec[3]] != df_main[df_columns_spec[2]],  # Check if access type sum differs from total diagnosed
            df_main[df_columns_spec[3]] - df_main[df_columns_spec[2]],  # Calculate difference for diagnosis gap
            0  # Set to 0 if no gap
        )  # Adds MHS diagnosis gap column
        df_main[gap_columns[2]] = np.where(  # Calculate MHS support gap
            df_main[df_columns_spec[5]] != df_main[df_columns_spec[4]],  # Check if access type sum differs from total supported
            df_main[df_columns_spec[5]] - df_main[df_columns_spec[4]],  # Calculate difference for support gap
            0  # Set to 0 if no gap
        )  # Adds MHS support gap column
        reorder_columns = (  # Define column order for DataFrame
            MSF_hierarchy +
            [
                df_columns[0],  # Total MHS screened
                df_columns_spec[1],  # MHS screened (Walk-In and Community)
                gap_columns[0],  # MHS screening gap
                df_columns[1],  # Total MHS diagnosed
                df_columns_spec[3],  # MHS diagnosed (Walk-In and Community)
                gap_columns[1],  # MHS diagnosis gap
                df_columns[2],  # Total MHS supported
                df_columns_spec[5],  # MHS supported (Walk-In and Community)
                gap_columns[2],  # MHS support gap
            ]
        )  # Specifies ordered columns for clarity
        df_main = df_main[reorder_columns]  # Reorder DataFrame columns to match specified order
        df_main = df_main.reset_index(drop=True)  # Reset index to ensure clean DataFrame structure

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_MHS_Access_Type_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_MHS_Access_Type_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_MHS_Access_Type_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_MHS_Access_Type_gap, 'cached_style'):  # Check for cached style
                del process_KP_MHS_Access_Type_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_MHS_Access_Type_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_MHS_Access_Type_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_MHS_Access_Type_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_KP_MHS_Access_Type_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero MHS screening gaps
            report_description.append(  # Add description for MHS screening gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns_spec[1]}\n"
                f"should be equal to {df_columns_spec[0]}"  # Describe expected equality for screening
            )  # Describe MHS screening gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero MHS diagnosis gaps
            report_description.append(  # Add description for MHS diagnosis gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns_spec[3]}\n"
                f"should be equal to {df_columns_spec[2]}"  # Describe expected equality for diagnosis
            )  # Describe MHS diagnosis gap issue
        if (df_main_gap[gap_columns_wrap[2]] != 0).any():  # Check for non-zero MHS support gaps
            report_description.append(  # Add description for MHS support gap
                f"Report Name: {gap_columns[2]}\n"
                f"{df_columns_spec[5]}\n"
                f"should be equal to {df_columns_spec[4]}"  # Describe expected equality for support
            )  # Describe MHS support gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

        # -- Step 11: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns_spec,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_MHS_Access_Type_gap, 'cached_style'):  # Check for cached style
            del process_KP_MHS_Access_Type_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_MHS_Access_Type_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_MHS_Access_Type_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-GBV CSR (Counselled, Screened & Referred) gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP GBV CSR gap
def process_KP_GBV_CSR_gap(display_output=None):
    """
    Process KP GBV CSR gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP GBV CSR metrics
            "NTHRIP-Number of PLHIV Counselled on gender norms",  # PLHIV counselled on gender norms
            "NTHRIP-Number of PLHIV screened for GBV",  # PLHIV screened for GBV
            "NTHRIP-Number of PLHIV referred for post GBV care"  # PLHIV referred for post-GBV care
        ]  # Defines columns for KP GBV CSR analysis
        name = "KP Prev_GBV CSR Gap"  # Base name for report
        gap_columns = [  # Names for calculated gap columns
            "KP Prev-GBV screening gap",
            "KP Prev-GBV referral gap"
        ]  # Defines gap columns for GBV metrics
        report_name = f"{name}46"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP GBV CSR columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate GBV screening gap (requires numpy as np)
            df_main[df_columns[1]] > df_main[df_columns[0]],  # Check if screened is greater than counselled
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for screening gap
            0  # Set to 0 if no gap
        )  # Adds GBV screening gap column
        df_main[gap_columns[1]] = np.where(  # Calculate GBV referral gap
            df_main[df_columns[2]] != df_main[df_columns[1]],  # Check if referrals differ from screened
            df_main[df_columns[2]] - df_main[df_columns[1]],  # Calculate difference for referral gap
            0  # Set to 0 if no gap
        )  # Adds GBV referral gap column

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_GBV_CSR_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_GBV_CSR_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_GBV_CSR_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)  # Print display name with separators
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_GBV_CSR_gap, 'cached_style'):  # Check for cached style
                del process_KP_GBV_CSR_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_GBV_CSR_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_GBV_CSR_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_GBV_CSR_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_KP_GBV_CSR_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero GBV screening gaps
            report_description.append(  # Add description for GBV screening gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be greater than {df_columns[0]}"  # Describe expected relation for screening
            )  # Describe GBV screening gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero GBV referral gaps
            report_description.append(  # Add description for GBV referral gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[3]}\n"  # Note: Error, df_columns[3] does not exist, preserved as is
                f"should be equal to {df_columns[2]}"  # Describe expected equality for referral
            )  # Describe GBV referral gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

        # -- Step 11: Export results to multiple formats
        if not display_output:  # Check if user requested to export results
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)  # Print display name with separators
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_GBV_CSR_gap, 'cached_style'):  # Check for cached style
            del process_KP_GBV_CSR_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_GBV_CSR_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_GBV_CSR_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-GBV Post GBV Sexual Violence PEP gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP Post GBV Sexual Violence gap
def process_KP_Post_GBV_SV_PEP_gap(display_output=None):
    """
    Process KP Post GBV Sexual Violence gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP Post GBV Sexual Violence metrics
            "NTHRIP-Number of people receiving post-GBV care for Sexual Violence (post-rape care)",  # People receiving post-GBV care
            "NTHRIP-Number of people receiving Post-Exposure Prophylaxis (PEP) services"  # People receiving PEP services
        ]  # Defines columns for KP Post GBV Sexual Violence analysis
        name = "KP Prev_GBV Post GBV SV PEP Gap"  # Base name for report
        gap_columns = ["KP Prev-GBV Post-Exposure Prophylaxis (PEP) services gap"]  # Names for calculated gap columns
        report_name = f"{name}47"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP Post GBV Sexual Violence columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate PEP services gap (requires numpy as np)
            df_main[df_columns[1]] < df_main[df_columns[0]],  # Check if PEP services are less than post-GBV care
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for PEP services gap
            0  # Set to 0 if no gap
        )  # Adds PEP services gap column

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_Post_GBV_SV_PEP_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_Post_GBV_SV_PEP_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_Post_GBV_SV_PEP_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)   
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_Post_GBV_SV_PEP_gap, 'cached_style'):  # Check for cached style
                del process_KP_Post_GBV_SV_PEP_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_Post_GBV_SV_PEP_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_Post_GBV_SV_PEP_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_Post_GBV_SV_PEP_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_Post_GBV_SV_PEP_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero PEP services gaps
            report_description = (  # Define description for PEP services gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be less than {df_columns[0]}"  # Describe expected relation for PEP services
            )  # Describe PEP services gap issue

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)   
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_Post_GBV_SV_PEP_gap, 'cached_style'):  # Check for cached style (original had error: process_KP_Post_GBV_SV)
            del process_KP_Post_GBV_SV_PEP_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_Post_GBV_SV_PEP_gap, 'cached_shape'):  # Check for cached shape (original had error: process_KP_Post_GBV_SV_PEP_gap)
            del process_KP_Post_GBV_SV_PEP_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-KP_GBV Incidence of Violence or Abuse gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP GBV Incidence of Violence or Abuse gap
def process_KP_GBV_IVA_gap(display_output=None):
    """
    Process KP GBV Incidence of Violence or Abuse gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP GBV Incidence of Violence or Abuse metrics
            "Number of KPs reporting incidence of Violence or Abuse during the reporting month",  # KPs reporting violence or abuse
            "Number of KPs receiving Post GBV Clinical Care during the reporting month"  # KPs receiving post-GBV clinical care
        ]  # Defines columns for KP GBV Incidence of Violence or Abuse analysis
        name = "KP Prev_KP-GBV IVA Gap"  # Base name for report
        gap_columns = ["KP Prev-KP_GBV receiving post GBV clinical care gap"]  # Names for calculated gap columns
        report_name = f"{name}48"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP GBV Incidence columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate post-GBV clinical care gap (requires numpy as np)
            df_main[df_columns[1]] > df_main[df_columns[0]],  # Check if clinical care exceeds reported incidents
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for clinical care gap
            0  # Set to 0 if no gap
        )  # Adds post-GBV clinical care gap column

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_GBV_IVA_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_GBV_IVA_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_GBV_IVA_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_GBV_IVA_gap, 'cached_style'):  # Check for cached style
                del process_KP_GBV_IVA_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_GBV_IVA_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_GBV_IVA_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_GBV_IVA_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_GBV_IVA_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero post-GBV clinical care gaps
            report_description = (  # Define description for post-GBV clinical care gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be greater than {df_columns[0]}"  # Describe expected relation for clinical care
            )  # Describe post-GBV clinical care gap issue

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_GBV_IVA_gap, 'cached_style'):  # Check for cached style
            del process_KP_GBV_IVA_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_GBV_IVA_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_GBV_IVA_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-KP_GBV Legal Support gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP GBV Legal Support gap
def process_KP_GBV_Legal_Support_gap(display_output=None):
    """
    Process KP GBV Legal Support gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP GBV Legal Support metrics
            "Number of KPs referred for Legal support or advice services during the reporting month",  # KPs referred for legal support
            "Number of KPs received Legal support or services during the reporting month"  # KPs receiving legal support (preserves grammatical error: 'received')
        ]  # Defines columns for KP GBV Legal Support analysis
        name = "KP Prev_KP-GBV Legal Support Gap"  # Base name for report
        gap_columns = ["KP Prev-KP_GBV legal support gap"]  # Names for calculated gap columns
        report_name = f"{name}49"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP GBV Legal Support columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate legal support gap (requires numpy as np)
            df_main[df_columns[1]] != df_main[df_columns[0]],  # Check if received legal support differs from referrals
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for legal support gap
            0  # Set to 0 if no gap
        )  # Adds legal support gap column

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_GBV_Legal_Support_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_GBV_Legal_Support_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_GBV_Legal_Support_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_GBV_Legal_Support_gap, 'cached_style'):  # Check for cached style
                del process_KP_GBV_Legal_Support_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_GBV_Legal_Support_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_GBV_Legal_Support_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_GBV_Legal_Support_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_GBV_Legal_Support_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero legal support gaps
            report_description = (  # Define description for legal support gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"  # Describe expected equality for legal support
            )  # Describe legal support gap issue

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_GBV_Legal_Support_gap, 'cached_style'):  # Check for cached style
            del process_KP_GBV_Legal_Support_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_GBV_Legal_Support_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_GBV_Legal_Support_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-PEP PEP gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP PEP gap
def process_KP_PEP_gap(display_output=None):
    """
    Process KP PEP gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP PEP metrics
            "NTHRIP-Number of reported HIV exposures during the reporting month (excluding HIV-exposed babies)",  # Reported HIV exposures
            "NTHRIP-Number of persons provided with post-exposure prophylaxis"  # Persons provided with PEP
        ]  # Defines columns for KP PEP analysis
        name = "KP Prev_PEP PEP Gap"  # Base name for report
        gap_columns = ["KP Prev-PEP gap"]  # Names for calculated gap columns
        report_name = f"{name}50"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP PEP columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate PEP gap (requires numpy as np)
            df_main[df_columns[1]] != df_main[df_columns[0]],  # Check if PEP provision differs from reported exposures
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for PEP gap
            0  # Set to 0 if no gap
        )  # Adds PEP gap column (original comment incorrectly references legal support gap)

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_PEP_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_PEP_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_PEP_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)   
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_PEP_gap, 'cached_style'):  # Check for cached style
                del process_KP_PEP_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_PEP_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_PEP_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_PEP_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_PEP_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero PEP gaps
            report_description = (  # Define description for PEP gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"  # Describe expected equality for PEP provision
            )  # Describe PEP gap issue

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_PEP_gap, 'cached_style'):  # Check for cached style
            del process_KP_PEP_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_PEP_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_PEP_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-PrEP_KP PrEP Product Recieved MSM gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP PrEP Product Received MSM gap
def process_KP_PrEP_Product_Recieved_MSM_gap(display_output=None):
    """
    Process KP PrEP Product Received MSM gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize         
        df_columns = [  # List of column names for MSM PrEP metrics
            "NTHRIP-KP-6a Number of MSM who received any PrEP product at least once during the reporting period",  # Total MSM receiving PrEP
            "NTHRIP-KP-6a Number of MSM who received any PrEP product at least once during the reporting period: PrEP Type",  # PrEP receipt by type
            "NTHRIP-KP-6a Number of MSM who received any PrEP product at least once during the reporting period: PrEP Distribution"  # PrEP receipt by distribution channel
        ]  # Defines columns for MSM PrEP gap analysis
        df_name_distribution = ['Community', 'Facility']  # Distribution channels for PrEP
        df_name_type = ['Injectable', 'Oral', 'Others']  # Types of PrEP products
        df_columns_spec = [  # Specific columns for gap analysis
            df_columns[0],  # Total MSM receiving PrEP
            f"{df_columns[0]} ({df_name_type[0]}, {df_name_type[1]}, {df_name_type[2]})",  # Aggregated PrEP types
            f"{df_columns[0]} ({df_name_distribution[0]}, {df_name_distribution[1]})"  # Aggregated distribution channels
        ]  # Defines specific columns for MSM PrEP gap calculations
        name = "KP Prev_KP-PrEP Product Received MSM Gap"  # Base name for report
        gap_columns = [  # Names for calculated gap columns
            "KP Prev-KP_PrEP product received by type MSM gap",  # Gap by PrEP type
            "KP Prev-KP_PrEP product received by distribution MSM gap"  # Gap by distribution channel
        ]  # Defines gap columns for TG PrEP metrics
        report_name = f"{name}51"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare primary DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include MSM PrEP column
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        df_main = df_main.rename(  # Rename columns for clarity
            columns=dict(zip(  # Create a mapping of old to new column names
                df_columns,  # Original column names
                df_columns_spec  # New column names for gap analysis
            ))  # Maps original columns to new names
        )  # Renames columns for consistency with gap analysis

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate PrEP type gap (requires numpy as np)
            df_main[df_columns_spec[1]] != df_main[df_columns_spec[0]],  # Check if sum by PrEP type differs from total
            df_main[df_columns_spec[1]] - df_main[df_columns_spec[0]],  # Calculate difference for PrEP type gap
            0  # Set to 0 if no gap
        )  # Adds PrEP type gap column (original comment incorrectly references PEP gap)
        df_main[gap_columns[1]] = np.where(  # Calculate PrEP distribution gap
            df_main[df_columns_spec[2]] != df_main[df_columns_spec[0]],  # Check if sum by distribution differs from total
            df_main[df_columns_spec[2]] - df_main[df_columns_spec[0]],  # Calculate difference for PrEP distribution gap
            0  # Set to 0 if no gap
        )  # Adds PrEP distribution gap column (original comment incorrectly references PEP gap)

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_PrEP_Product_Recieved_MSM_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_PrEP_Product_Recieved_MSM_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_PrEP_Product_Recieved_MSM_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_PrEP_Product_Recieved_MSM_gap, 'cached_style'):  # Check for cached style
                del process_KP_PrEP_Product_Recieved_MSM_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_PrEP_Product_Recieved_MSM_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_PrEP_Product_Recieved_MSM_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_PrEP_Product_Recieved_MSM_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_PrEP_Product_Recieved_MSM_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero PrEP type gaps
            report_description.append(  # Add description for PrEP type gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns_spec[1]}\n"
                f"should be equal to {df_columns_spec[0]}"  # Describe expected equality for PrEP type
            )  # Describe PrEP type gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero PrEP distribution gaps
            report_description.append(  # Add description for PrEP distribution gap
                f"Report Name: {gap_columns[1]}\n"  # Note: Original incorrectly uses gap_columns[0]
                f"{df_columns_spec[2]}\n"
                f"should be equal to {df_columns_spec[0]}"  # Describe expected equality for PrEP distribution
            )  # Describe PrEP distribution gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines for clarity

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns_spec,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_PrEP_Product_Recieved_MSM_gap, 'cached_style'):  # Check for cached style
            del process_KP_PrEP_Product_Recieved_MSM_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_PrEP_Product_Recieved_MSM_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_PrEP_Product_Recieved_MSM_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-PrEP_KP PrEP Product Recieved TG gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP PrEP Product Received TG gap
def process_KP_PrEP_Product_Recieved_TG_gap(display_output=None):
    """
    Process KP PrEP Product Received TG gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for TG PrEP metrics
            "NTHRIP-KP-6b Number of transgender (TG) people who received any PrEP product at least once during the reporting period",  # Total TG receiving PrEP
            "NTHRIP-KP-6b Number of transgender (TG) people who received any PrEP product at least once during the reporting period: Pregnancy/breastfeeding status",  # PrEP receipt by pregnancy/breastfeeding status
            "NTHRIP-KP-6b Number of transgender (TG) people who received any PrEP product at least once during the reporting period: PrEP Type",  # PrEP receipt by type
            "NTHRIP-KP-6b Number of transgender (TG) people who received any PrEP product at least once during the reporting period: PrEP Distribution"  # PrEP receipt by distribution channel
        ]  # Defines columns for TG PrEP gap analysis (original comment incorrectly references MSM PrEP metrics)
        df_name_preg_breastfeeding = ['Pregnant', 'Breastfeeding']  # Pregnancy/breastfeeding status categories
        df_name_distribution = ['Community', 'Facility']  # Distribution channels for PrEP
        df_name_type = ['Injectable', 'Oral', 'Others']  # Types of PrEP products
        df_columns_spec = [  # Specific columns for gap analysis
            df_columns[0],  # Total TG receiving PrEP (original comment incorrectly references MSM)
            f"{df_columns[0]} ({df_name_type[0]}, {df_name_type[1]}, {df_name_type[2]})",  # Aggregated PrEP types
            f"{df_columns[0]} ({df_name_distribution[0]}, {df_name_distribution[1]})",  # Aggregated distribution channels
            f"{df_columns[0]} ({df_name_preg_breastfeeding[0]}, {df_name_preg_breastfeeding[1]})"  # Aggregated pregnancy/breastfeeding status
        ]  # Defines specific columns for TG PrEP gap calculations
        name = "KP Prev_KP-PrEP Product Received TG Gap"  # Base name for report
        gap_columns = [  # Names for calculated gap columns
            "KP Prev-KP_PrEP Product received by pregnancy/breastfeeding status TG gap",  # Gap by pregnancy/breastfeeding status
            "KP Prev-KP_PrEP Product received by type TG gap",  # Gap by PrEP type
            "KP Prev-KP_PrEP Product received by distribution TG gap"  # Gap by distribution channel
        ]  # Defines gap columns for TG PrEP metrics
        report_name = f"{name}52"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare primary DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include TG PrEP columns
        )  # Returns processed DataFrame or None if failed (original comment incorrectly references MSM PrEP column)
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        df_main = df_main.rename(  # Rename columns for clarity
            columns=dict(zip(  # Create a mapping of old to new column names
                df_columns,  # Original column names
                df_columns_spec  # New column names for gap analysis
            ))  # Maps original columns to new names
        )  # Renames columns for consistency with gap analysis

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate pregnancy/breastfeeding gap (requires numpy as np)
            df_main[df_columns_spec[3]] > df_main[df_columns_spec[0]],  # Check if pregnancy/breastfeeding sum exceeds total
            df_main[df_columns_spec[3]] - df_main[df_columns_spec[0]],  # Calculate difference for pregnancy/breastfeeding gap
            0  # Set to 0 if no gap
        )  # Adds pregnancy/breastfeeding gap column (original comment incorrectly references PEP gap and PrEP type gap)
        df_main[gap_columns[1]] = np.where(  # Calculate PrEP type gap
            df_main[df_columns_spec[1]] != df_main[df_columns_spec[0]],  # Check if PrEP type sum differs from total
            df_main[df_columns_spec[1]] - df_main[df_columns_spec[0]],  # Calculate difference for PrEP type gap
            0  # Set to 0 if no gap
        )  # Adds PrEP type gap column (original comment incorrectly references PEP gap and PrEP distribution gap)
        df_main[gap_columns[2]] = np.where(  # Calculate distribution gap
            df_main[df_columns_spec[2]] != df_main[df_columns_spec[0]],  # Check if distribution sum differs from total
            df_main[df_columns_spec[2]] - df_main[df_columns_spec[0]],  # Calculate difference for distribution gap
            0  # Set to 0 if no gap
        )  # Adds distribution gap column (original comment incorrectly references PEP gap and PrEP pregnancy/breastfeeding gap)

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_PrEP_Product_Recieved_TG_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_PrEP_Product_Recieved_TG_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_PrEP_Product_Recieved_TG_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_PrEP_Product_Recieved_TG_gap, 'cached_style'):  # Check for cached style
                del process_KP_PrEP_Product_Recieved_TG_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_PrEP_Product_Recieved_TG_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_PrEP_Product_Recieved_TG_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_PrEP_Product_Recieved_TG_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_PrEP_Product_Recieved_TG_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero pregnancy/breastfeeding gaps
            report_description.append(  # Add description for pregnancy/breastfeeding gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns_spec[3]}\n"
                f"should not be greater than {df_columns_spec[0]}"  # Describe expected relation for pregnancy/breastfeeding
            )  # Describe pregnancy/breastfeeding gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero PrEP type gaps
            report_description.append(  # Add description for PrEP type gap
                f"Report Name: {gap_columns[1]}\n"  # Preserves correct use of gap_columns[1]
                f"{df_columns_spec[1]}\n"
                f"should be equal to {df_columns_spec[0]}"  # Describe expected equality for PrEP type
            )  # Describe PrEP type gap issue
        if (df_main_gap[gap_columns_wrap[2]] != 0).any():  # Check for non-zero distribution gaps
            report_description.append(  # Add description for distribution gap
                f"Report Name: {gap_columns[2]}\n"  # Preserves correct use of gap_columns[2]
                f"{df_columns_spec[2]}\n"
                f"should be equal to {df_columns_spec[0]}"  # Describe expected equality for distribution
            )  # Describe distribution gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines for clarity

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns_spec,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_PrEP_Product_Recieved_TG_gap, 'cached_style'):  # Check for cached style
            del process_KP_PrEP_Product_Recieved_TG_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_PrEP_Product_Recieved_TG_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_PrEP_Product_Recieved_TG_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-PrEP_KP PrEP Product Recieved SW gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP PrEP Product Received SW gap
def process_KP_PrEP_Product_Recieved_SW_gap(display_output=None):
    """
    Process KP PrEP Product Received SW gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for SW PrEP metrics
            "NTHRIP-KP-6c Number of sex workers who received any PrEP product at least once during the reporting period",  # Total SW receiving PrEP
            "NTHRIP-KP-6c Number of sex workers who received any PrEP product at least once during the reporting period: Pregnancy/breastfeeding status",  # PrEP receipt by pregnancy/breastfeeding status
            "NTHRIP-KP-6c Number of sex workers who received any PrEP product at least once during the reporting period: PrEP Type",  # PrEP receipt by type
            "NTHRIP-KP-6c Number of sex workers who received any PrEP product at least once during the reporting period: PrEP Distribution"  # PrEP receipt by distribution channel
        ]  # Defines columns for SW PrEP gap analysis
        df_name_preg_breastfeeding = ['Pregnant', 'Breastfeeding']  # Pregnancy/breastfeeding status categories
        df_name_distribution = ['Community', 'Facility']  # Distribution channels for PrEP
        df_name_type = ['Injectable', 'Oral', 'Others']  # Types of PrEP products
        df_columns_spec = [  # Specific columns for gap analysis
            df_columns[0],  # Total SW receiving PrEP
            f"{df_columns[0]} ({df_name_preg_breastfeeding[0]}, {df_name_preg_breastfeeding[1]})",  # Aggregated pregnancy/breastfeeding status
            f"{df_columns[0]} ({df_name_type[0]}, {df_name_type[1]}, {df_name_type[2]})",  # Aggregated PrEP types
            f"{df_columns[0]} ({df_name_distribution[0]}, {df_name_distribution[1]})"  # Aggregated distribution channels
        ]  # Defines specific columns for SW PrEP gap calculations
        name = "KP Prev_KP-PrEP Product Received SW Gap"  # Base name for report
        gap_columns = [  # Names for calculated gap columns
            "KP Prev-KP_PrEP Product received by pregnancy/breastfeeding status SW gap",  # Gap by pregnancy/breastfeeding status
            "KP Prev-KP_PrEP Product received by type SW gap",  # Gap by PrEP type
            "KP Prev-KP_PrEP Product received by distribution SW gap"  # Gap by distribution channel
        ]  # Defines gap columns for SW PrEP metrics
        report_name = f"{name}53"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare primary DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include SW PrEP columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        df_main = df_main.rename(  # Rename columns for clarity
            columns=dict(zip(  # Create a mapping of old to new column names
                df_columns,  # Original column names
                df_columns_spec  # New column names for gap analysis
            ))  # Maps original columns to new names
        )  # Renames columns for consistency with gap analysis

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate pregnancy/breastfeeding gap (requires numpy as np)
            df_main[df_columns_spec[1]] > df_main[df_columns_spec[0]],  # Check if pregnancy/breastfeeding sum exceeds total
            df_main[df_columns_spec[1]] - df_main[df_columns_spec[0]],  # Calculate difference for pregnancy/breastfeeding gap
            0  # Set to 0 if no gap
        )  # Adds pregnancy/breastfeeding gap column (original comment incorrectly references PEP gap)
        df_main[gap_columns[1]] = np.where(  # Calculate PrEP type gap
            df_main[df_columns_spec[2]] != df_main[df_columns_spec[0]],  # Check if PrEP type sum differs from total
            df_main[df_columns_spec[2]] - df_main[df_columns_spec[0]],  # Calculate difference for PrEP type gap
            0  # Set to 0 if no gap
        )  # Adds PrEP type gap column (original comment incorrectly references PEP gap)
        df_main[gap_columns[2]] = np.where(  # Calculate distribution gap
            df_main[df_columns_spec[3]] != df_main[df_columns_spec[0]],  # Check if distribution sum differs from total
            df_main[df_columns_spec[3]] - df_main[df_columns_spec[0]],  # Calculate difference for distribution gap
            0  # Set to 0 if no gap
        )  # Adds distribution gap column (original comment incorrectly references PEP gap)

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_PrEP_Product_Recieved_SW_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_PrEP_Product_Recieved_SW_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_PrEP_Product_Recieved_SW_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_PrEP_Product_Recieved_SW_gap, 'cached_style'):  # Check for cached style
                del process_KP_PrEP_Product_Recieved_SW_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_PrEP_Product_Recieved_SW_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_PrEP_Product_Recieved_SW_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_PrEP_Product_Recieved_SW_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_PrEP_Product_Recieved_SW_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero pregnancy/breastfeeding gaps
            report_description.append(  # Add description for pregnancy/breastfeeding gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns_spec[1]}\n"
                f"should not be greater than {df_columns_spec[0]}"  # Describe expected relation for pregnancy/breastfeeding
            )  # Describe pregnancy/breastfeeding gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero PrEP type gaps
            report_description.append(  # Add description for PrEP type gap
                f"Report Name: {gap_columns[1]}\n"  # Preserves correct use of gap_columns[1]
                f"{df_columns_spec[2]}\n"
                f"should be equal to {df_columns_spec[0]}"  # Describe expected equality for PrEP type
            )  # Describe PrEP type gap issue
        if (df_main_gap[gap_columns_wrap[2]] != 0).any():  # Check for non-zero distribution gaps
            report_description.append(  # Add description for distribution gap
                f"Report Name: {gap_columns[2]}\n"  # Preserves correct use of gap_columns[2]
                f"{df_columns_spec[3]}\n"
                f"should be equal to {df_columns_spec[0]}"  # Describe expected equality for distribution
            )  # Describe distribution gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines for clarity

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns_spec,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_PrEP_Product_Recieved_SW_gap, 'cached_style'):  # Check for cached style
            del process_KP_PrEP_Product_Recieved_SW_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_PrEP_Product_Recieved_SW_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_PrEP_Product_Recieved_SW_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-PrEP_KP PrEP Product Recieved PWID gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP PrEP Product Received PWID gap
def process_KP_PrEP_Product_Recieved_PWID_gap(display_output=None):
    """
    Process KP PrEP Product Received PWID gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PWID PrEP metrics
            "NTHRIP-KP-6d Number of PWID who received any PrEP product at least once during the reporting period",  # Total PWID receiving PrEP
            "NTHRIP-KP-6d Number of PWID who received any PrEP product at least once during the reporting period: Pregnancy/Breastfeeding Status",  # PrEP receipt by pregnancy/breastfeeding status
            "NTHRIP-KP-6d Number of PWID who received any PrEP product at least once during the reporting period: PrePType",  # PrEP receipt by type (preserves typo: 'PrePType')
            "NTHRIP-KP-6d Number of PWID who received any PrEP product at least once during the reporting period: PrEPDistribution"  # PrEP receipt by distribution channel (preserves typo: 'PrEPDistribution')
        ]  # Defines columns for PWID PrEP gap analysis
        df_name_preg_breastfeeding = ['Pregnant', 'Breastfeeding']  # Pregnancy/breastfeeding status categories
        df_name_distribution = ['Community', 'Facility']  # Distribution channels for PrEP
        df_name_type = ['Injectable', 'Oral', 'Others']  # Types of PrEP products
        df_columns_spec = [  # Specific columns for gap analysis
            df_columns[0],  # Total PWID receiving PrEP
            f"{df_columns[0]} ({df_name_preg_breastfeeding[0]}, {df_name_preg_breastfeeding[1]})",  # Aggregated pregnancy/breastfeeding status
            f"{df_columns[0]} ({df_name_type[0]}, {df_name_type[1]}, {df_name_type[2]})",  # Aggregated PrEP types
            f"{df_columns[0]} ({df_name_distribution[0]}, {df_name_distribution[1]})"  # Aggregated distribution channels
        ]  # Defines specific columns for PWID PrEP gap calculations
        name = "KP Prev_KP-PrEP Product received PWID Gap"  # Base name for report
        gap_columns = [  # Names for calculated gap columns
            "KP Prev-KP_PrEP Product received by pregnancy/breastfeeding status PWID gap",  # Gap by pregnancy/breastfeeding status
            "KP Prev-KP_PrEP Product received by type PWID gap",  # Gap by PrEP type
            "KP Prev-KP_PrEP Product received by distribution PWID gap"  # Gap by distribution channel
        ]  # Defines gap columns for PWID PrEP metrics
        report_name = f"{name}54"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare primary DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PWID PrEP columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        df_main = df_main.rename(  # Rename columns for clarity
            columns=dict(zip(  # Create a mapping of old to new column names
                df_columns,  # Original column names
                df_columns_spec  # New column names for gap analysis
            ))  # Maps original columns to new names
        )  # Renames columns for consistency with gap analysis

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate pregnancy/breastfeeding gap (requires numpy as np)
            df_main[df_columns_spec[1]] > df_main[df_columns_spec[0]],  # Check if pregnancy/breastfeeding sum exceeds total
            df_main[df_columns_spec[1]] - df_main[df_columns_spec[0]],  # Calculate difference for pregnancy/breastfeeding gap
            0  # Set to 0 if no gap
        )  # Adds pregnancy/breastfeeding gap column (original comment incorrectly references PEP gap)
        df_main[gap_columns[1]] = np.where(  # Calculate PrEP type gap
            df_main[df_columns_spec[2]] != df_main[df_columns_spec[0]],  # Check if PrEP type sum differs from total
            df_main[df_columns_spec[2]] - df_main[df_columns_spec[0]],  # Calculate difference for PrEP type gap
            0  # Set to 0 if no gap
        )  # Adds PrEP type gap column (original comment incorrectly references PEP gap)
        df_main[gap_columns[2]] = np.where(  # Calculate distribution gap
            df_main[df_columns_spec[3]] != df_main[df_columns_spec[0]],  # Check if distribution sum differs from total
            df_main[df_columns_spec[3]] - df_main[df_columns_spec[0]],  # Calculate difference for distribution gap
            0  # Set to 0 if no gap
        )  # Adds distribution gap column (original comment incorrectly references PEP gap)

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_PrEP_Product_Recieved_PWID_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_PrEP_Product_Recieved_PWID_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_PrEP_Product_Recieved_PWID_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_PrEP_Product_Recieved_PWID_gap, 'cached_style'):  # Check for cached style
                del process_KP_PrEP_Product_Recieved_PWID_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_PrEP_Product_Recieved_PWID_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_PrEP_Product_Recieved_PWID_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_PrEP_Product_Recieved_PWID_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_PrEP_Product_Recieved_PWID_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero pregnancy/breastfeeding gaps
            report_description.append(  # Add description for pregnancy/breastfeeding gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns_spec[1]}\n"
                f"should not be greater than {df_columns_spec[0]}"  # Describe expected relation for pregnancy/breastfeeding
            )  # Describe pregnancy/breastfeeding gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero PrEP type gaps
            report_description.append(  # Add description for PrEP type gap
                f"Report Name: {gap_columns[1]}\n"  # Preserves correct use of gap_columns[1]
                f"{df_columns_spec[2]}\n"
                f"should be equal to {df_columns_spec[0]}"  # Describe expected equality for PrEP type
            )  # Describe PrEP type gap issue
        if (df_main_gap[gap_columns_wrap[2]] != 0).any():  # Check for non-zero distribution gaps
            report_description.append(  # Add description for distribution gap
                f"Report Name: {gap_columns[2]}\n"  # Preserves correct use of gap_columns[2]
                f"{df_columns_spec[3]}\n"
                f"should be equal to {df_columns_spec[0]}"  # Describe expected equality for distribution
            )  # Describe distribution gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines for clarity

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns_spec,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_PrEP_Product_Recieved_PWID_gap, 'cached_style'):  # Check for cached style
            del process_KP_PrEP_Product_Recieved_PWID_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_PrEP_Product_Recieved_PWID_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_PrEP_Product_Recieved_PWID_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-KP_Condom Distribution gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP Condom Distribution gap
def process_KP_Condom_Distribution_gap(display_output=None):
    """
    Process KP Condom Distribution gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for condom distribution metrics
            "NTHRIP-Number of Condoms distributed during the reporting period",  # Total condoms distributed
            "NTHRIP-Number of Condoms distributed during the reporting period (Total Male Condoms)",  # Male condoms distributed
            "NTHRIP-Number of Condoms distributed during the reporting period (Total Female Condoms)"  # Female condoms distributed
        ]  # Defines columns for condom distribution analysis
        name = "KP Prev_KP-Condom Distribution Gap"  # Base name for report
        gap_columns = ["KP Prev-KP_Condom Distribution by male and female gap"]  # Names for calculated gap columns (preserves typo: 'femalegap')
        report_name = f"{name}55"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include condom distribution columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate condom distribution gap (requires numpy as np)
            df_main[df_columns[1:3]].sum(axis=1) != df_main[df_columns[0]],  # Check if sum of male and female condoms differs from total
            df_main[df_columns[1:3]].sum(axis=1) - df_main[df_columns[0]],  # Calculate difference for condom distribution gap
            0  # Set to 0 if no gap
        )  # Adds condom distribution gap column (original comment incorrectly references PrEP type gap)

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_Condom_Distribution_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_Condom_Distribution_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_Condom_Distribution_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_Condom_Distribution_gap, 'cached_style'):  # Check for cached style
                del process_KP_Condom_Distribution_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_Condom_Distribution_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_Condom_Distribution_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_Condom_Distribution_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_Condom_Distribution_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero condom distribution gaps
            report_description = (  # Define description for condom distribution gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\nplus {df_columns[2]}\n"
                f"should be equal to {df_columns[0]}"  # Describe expected equality for condom distribution
            )  # Describe condom distribution gap issue (original comment incorrectly references PrEP type gap)

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_Condom_Distribution_gap, 'cached_style'):  # Check for cached style
            del process_KP_Condom_Distribution_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_Condom_Distribution_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_Condom_Distribution_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-KP_Condom Lubricants Distribution gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP Lubricants Distribution gap
def process_KP_Lubricants_Distribution_gap(display_output=None):
    """
    Process KP Lubricants Distribution gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for lubricant distribution metrics
            "NTHRIP-Number of Lubricants distributed during the reporting period",  # Total lubricants distributed
            "NTHRIP-Number of Lubricants distributed during the reporting period (Total Lubricants Distributed Male)",  # Male-targeted lubricants distributed
            "NTHRIP-Number of Lubricants distributed during the reporting period(Total Lubricants Distributed Female)"  # Female-targeted lubricants distributed (preserves missing space)
        ]  # Defines columns for lubricant distribution analysis
        name = "KP Prev_KP-Condom Lubricants Distribution Gap"  # Base name for report
        gap_columns = ["KP Prev-KP_Condom Lubricants distribution by male and female gap"]  # Names for calculated gap columns (preserves typo: 'femalegap')
        report_name = f"{name}56"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include lubricant distribution columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate lubricant distribution gap (requires numpy as np)
            df_main[df_columns[1:3]].sum(axis=1) != df_main[df_columns[0]],  # Check if sum of male and female lubricants differs from total
            df_main[df_columns[1:3]].sum(axis=1) - df_main[df_columns[0]],  # Calculate difference for lubricant distribution gap
            0  # Set to 0 if no gap
        )  # Adds lubricant distribution gap column (original comment incorrectly references condom distribution gap)

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_Lubricants_Distribution_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_Lubricants_Distribution_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_Lubricants_Distribution_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_Lubricants_Distribution_gap, 'cached_style'):  # Check for cached style
                del process_KP_Lubricants_Distribution_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_Lubricants_Distribution_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_Lubricants_Distribution_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_Lubricants_Distribution_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_Lubricants_Distribution_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero lubricant distribution gaps
            report_description = (  # Define description for lubricant distribution gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\nplus {df_columns[2]}\n"
                f"should be equal to {df_columns[0]}"  # Describe expected equality for lubricant distribution
            )  # Describe lubricant distribution gap issue 

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_Lubricants_Distribution_gap, 'cached_style'):  # Check for cached style
            del process_KP_Lubricants_Distribution_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_Lubricants_Distribution_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_Lubricants_Distribution_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-PrEP_GP PrEP Enrolment gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP PrEP Enrolment gap
def process_KP_PrEP_Enrolment_gap(display_output=None):
    """
    Process KP PrEP Enrolment gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PrEP enrolment metrics
            "NTHRIP-Number of individuals who were newly enrolled on pre-exposure prophylaxis (PrEP) to prevent HIV infection in the reporting period",  # Total individuals newly enrolled on PrEP
            "NTHRIP-Number of individuals who were newly enrolled on pre-exposure prophylaxis (PrEP) to prevent HIV infection in the reporting period: Pregnancy/breastfeeding status",  # PrEP enrolment by pregnancy/breastfeeding status
            "NTHRIP-Number of individuals who were newly enrolled on pre-exposure prophylaxis (PrEP) to prevent HIV infection in the reporting period: PrEP Type",  # PrEP enrolment by type
            "NTHRIP-Number of individuals who were newly enrolled on pre-exposure prophylaxis (PrEP) to prevent HIV infection in the reporting period: PrEP Distribution"  # PrEP enrolment by distribution channel
        ]  # Defines columns for PrEP enrolment analysis
        df_name_preg_breastfeeding = ['Pregnant', 'Breastfeeding']  # Pregnancy/breastfeeding status categories
        df_name_type = ['Injectable', 'Oral', 'Others']  # Types of PrEP products
        df_name_distribution = ['Community', 'Facility']  # Distribution channels for PrEP
        df_columns_spec = [  # Specific columns for gap analysis
            df_columns[0],  # Total PrEP enrolment
            f"{df_columns[1]} ({df_name_preg_breastfeeding[0]}, {df_name_preg_breastfeeding[1]})",  # Aggregated pregnancy/breastfeeding status
            f"{df_columns[2]} ({df_name_type[0]}, {df_name_type[1]}, {df_name_type[2]})",  # Aggregated PrEP types
            f"{df_columns[3]} ({df_name_distribution[0]}, {df_name_distribution[1]})"  # Aggregated distribution channels
        ]  # Defines specific columns for gap calculations
        name = "KP Prev_PrEP-GP Enrolment Gap"  # Base name for report (preserves typo: 'GP')
        gap_columns = [  # Names for calculated gap columns
            "KP Prev-PrEP_GP PrEP enrolment by pregnancy/breastfeeding status gap",  # Gap by pregnancy/breastfeeding status (preserves typo: 'GP')
            "KP Prev-PrEP_GP PrEP enrolment by type gap",  # Gap by PrEP type (preserves typo: 'GP')
            "KP Prev-PrEP-GP PrEP enrolment by distribution gap"  # Gap by distribution channel
        ]  # Defines gap columns for PrEP enrolment metrics
        report_name = f"{name}57"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PrEP enrolment columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate pregnancy/breastfeeding gap (requires numpy as np)
            df_main[df_columns[1]] > df_main[df_columns[0]],  # Check if pregnancy/breastfeeding enrolment exceeds total
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for pregnancy/breastfeeding gap
            0  # Set to 0 if no gap
        )  # Adds pregnancy/breastfeeding gap column (original comment incorrectly references lubricant distribution gap)
        df_main[gap_columns[1]] = np.where(  # Calculate PrEP type gap
            df_main[df_columns[2]] != df_main[df_columns[0]],  # Check if PrEP type enrolment differs from total
            df_main[df_columns[2]] - df_main[df_columns[0]],  # Calculate difference for PrEP type gap
            0  # Set to 0 if no gap
        )  # Adds PrEP type gap column (original comment incorrectly references lubricant distribution gap)
        df_main[gap_columns[2]] = np.where(  # Calculate distribution gap
            df_main[df_columns[3]] != df_main[df_columns[0]],  # Check if distribution enrolment differs from total
            df_main[df_columns[3]] - df_main[df_columns[0]],  # Calculate difference for distribution gap
            0  # Set to 0 if no gap
        )  # Adds distribution gap column (original comment incorrectly references lubricant distribution gap)

        df_main = df_main.rename(  # Rename columns for clarity
            columns=dict(zip(df_columns, df_columns_spec))  # Map original columns to specific names
        )  # Renames columns for better readability

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_PrEP_Enrolment_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_PrEP_Enrolment_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_PrEP_Enrolment_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_PrEP_Enrolment_gap, 'cached_style'):  # Check for cached style
                del process_KP_PrEP_Enrolment_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_PrEP_Enrolment_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_PrEP_Enrolment_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_PrEP_Enrolment_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_PrEP_Enrolment_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero pregnancy/breastfeeding gaps
            report_description.append(  # Add description for pregnancy/breastfeeding gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns_spec[1]}\n"
                f"should not be greater than {df_columns_spec[0]}"  # Describe expected relation for pregnancy/breastfeeding
            )  # Describe pregnancy/breastfeeding gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero PrEP type gaps
            report_description.append(  # Add description for PrEP type gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns_spec[2]}\n"
                f"should be equal to {df_columns_spec[0]}"  # Describe expected equality for PrEP type
            )  # Describe PrEP type gap issue
        if (df_main_gap[gap_columns_wrap[2]] != 0).any():  # Check for non-zero distribution gaps
            report_description.append(  # Add description for distribution gap
                f"Report Name: {gap_columns[2]}\n"
                f"{df_columns_spec[3]}\n"
                f"should be equal to {df_columns_spec[0]}"  # Describe expected equality for distribution
            )  # Describe distribution gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines for clarity

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns_spec,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_PrEP_Enrolment_gap, 'cached_style'):  # Check for cached style
            del process_KP_PrEP_Enrolment_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_PrEP_Enrolment_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_PrEP_Enrolment_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-PrEP_GP PrEP Restart gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP PrEP Restart gap
def process_KP_PrEP_Restart_gap(display_output=None):
    """
    Process KP PrEP Restart gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for PrEP restart metrics
            "NTHRIP-Number of individuals that returned for a follow-up or re-initiation visit to receive PrEP during the reporting period",  # Total individuals restarting or following up on PrEP
            "NTHRIP-Number of individuals that returned for a follow-up or re-initiation visit to receive PrEP during the reporting period: Pregnancy/breastfeeding status",  # PrEP restart by pregnancy/breastfeeding status
            "NTHRIP-Number of individuals that returned for a follow-up or re-initiation visit to receive PrEP during the reporting period: PrEP Type",  # PrEP restart by type
            "NTHRIP-Number of individuals that returned for a follow-up or re-initiation visit to receive PrEP during the reporting period: PrEP Distribution"  # PrEP restart by distribution channel
        ]  # Defines columns for PrEP restart analysis
        df_name_preg_breastfeeding = ['Pregnant', 'Breastfeeding']  # Pregnancy/breastfeeding status categories
        df_name_type = ['Injectable', 'Oral', 'Others']  # Types of PrEP products
        df_name_distribution = ['Community', 'Facility']  # Distribution channels for PrEP
        df_columns_spec = [  # Specific columns for gap analysis
            df_columns[0],  # Total PrEP restart
            f"{df_columns[1]} ({df_name_preg_breastfeeding[0]}, {df_name_preg_breastfeeding[1]})",  # Aggregated pregnancy/breastfeeding status
            f"{df_columns[2]} ({df_name_type[0]}, {df_name_type[1]}, {df_name_type[2]})",  # Aggregated PrEP types
            f"{df_columns[3]} ({df_name_distribution[0]}, {df_name_distribution[1]})"  # Aggregated distribution channels
        ]  # Defines specific columns for gap calculations
        name = "KP Prev_PrEP-GP Restart Gap"  # Base name for report (preserves typo: 'GP')
        gap_columns = [  # Names for calculated gap columns
            "KP Prev-PrEP_GP PrEP Restart by pregnancy/breastfeeding status gap",  # Gap by pregnancy/breastfeeding status (preserves typo: 'GP')
            "KP Prev-PrEP_GP PrEP Restart by type gap",  # Gap by PrEP type (preserves typo: 'GP')
            "KP Prev-PrEP-GP PrEP Restart by distribution gap"  # Gap by distribution channel
        ]  # Defines gap columns for PrEP restart metrics
        report_name = f"{name}58"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PrEP restart columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate pregnancy/breastfeeding gap (requires numpy as np)
            df_main[df_columns[1]] > df_main[df_columns[0]],  # Check if pregnancy/breastfeeding restart exceeds total
            df_main[df_columns[1]] - df_main[df_columns[0]],  # Calculate difference for pregnancy/breastfeeding gap
            0  # Set to 0 if no gap
        )  # Adds pregnancy/breastfeeding gap column (original comment incorrectly references lubricant distribution gap)
        df_main[gap_columns[1]] = np.where(  # Calculate PrEP type gap
            df_main[df_columns[2]] != df_main[df_columns[0]],  # Check if PrEP type restart differs from total
            df_main[df_columns[2]] - df_main[df_columns[0]],  # Calculate difference for PrEP type gap
            0  # Set to 0 if no gap
        )  # Adds PrEP type gap column (original comment incorrectly references lubricant distribution gap)
        df_main[gap_columns[2]] = np.where(  # Calculate distribution gap
            df_main[df_columns[3]] != df_main[df_columns[0]],  # Check if distribution restart differs from total
            df_main[df_columns[3]] - df_main[df_columns[0]],  # Calculate difference for distribution gap
            0  # Set to 0 if no gap
        )  # Adds distribution gap column (original comment incorrectly references lubricant distribution gap)

        df_main = df_main.rename(  # Rename columns for clarity
            columns=dict(zip(df_columns, df_columns_spec))  # Map original columns to specific names
        )  # Renames columns for better readability

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_PrEP_Restart_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_PrEP_Restart_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_PrEP_Restart_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_PrEP_Restart_gap, 'cached_style'):  # Check for cached style
                del process_KP_PrEP_Restart_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_PrEP_Restart_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_PrEP_Restart_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_PrEP_Restart_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_PrEP_Restart_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero pregnancy/breastfeeding gaps
            report_description.append(  # Add description for pregnancy/breastfeeding gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns_spec[1]}\n"
                f"should not be greater than {df_columns_spec[0]}"  # Describe expected relation for pregnancy/breastfeeding
            )  # Describe pregnancy/breastfeeding gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero PrEP type gaps
            report_description.append(  # Add description for PrEP type gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns_spec[2]}\n"
                f"should be equal to {df_columns_spec[0]}"  # Describe expected equality for PrEP type
            )  # Describe PrEP type gap issue
        if (df_main_gap[gap_columns_wrap[2]] != 0).any():  # Check for non-zero distribution gaps
            report_description.append(  # Add description for distribution gap
                f"Report Name: {gap_columns[2]}\n"
                f"{df_columns_spec[3]}\n"
                f"should be equal to {df_columns_spec[0]}"  # Describe expected equality for distribution
            )  # Describe distribution gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines for clarity

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns_spec,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_PrEP_Restart_gap, 'cached_style'):  # Check for cached style
            del process_KP_PrEP_Restart_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_PrEP_Restart_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_PrEP_Restart_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-PrEP_MSF PrEP Eligible gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP PrEP Eligible gap
def process_KP_PrEP_Eligible_gap(display_output=None):
    """
    Process KP PrEP Eligible gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = ['No. of individuals who were eligible and started PrEP in the reporting month']  # Column for total eligible individuals starting PrEP
        high_risk_kp = [  # List of high-risk key population categories
            'Serodiscordant Couples (SDC)', 'SW', 'Partners of Sex Workers', 'Injecting Drug Users',
            'Individuals who engage in anal sex on a prolonged and regular basis', 
            'Exposed adolescents and young people', 'Transgender People', 'Other population'
        ]  # Defines key populations for PrEP eligibility analysis
        name = "KP Prev_PrEP-MSF Eligible Gap"  # Base name for report (preserves typo: 'MSF')
        gap_columns = ["KP Prev-PrEP_MSF PrEP eligible by KP risk population gap"]  # Name for calculated gap column (preserves typo: 'MSF')
        report_name = f"{name}59"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PrEP eligibility column
        )  # Returns processed DataFrame or None if failed (original comment incorrectly references PrEP restart columns)
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        df_main2 = DHIS2_data['KP Prev MSF_PrEP_eligible_for_pk_at_risk'][MSF_hierarchy + high_risk_kp].copy()  # Copy DataFrame for high-risk KP disaggregation
        df_main = df_main.merge(  # Merge primary and high-risk KP DataFrames
            df_main2, 
            on=MSF_hierarchy, 
            how='left'  # Left join to retain all records from df_main
        )  # Combines total eligibility with disaggregated KP data

        df_main[high_risk_kp] = df_main[high_risk_kp].apply(pd.to_numeric, errors='coerce').fillna(0)  # Convert KP columns to numeric, replacing non-numeric with 0
        df_main[gap_columns[0]] = np.where(  # Calculate gap by KP risk population (requires numpy as np)
            df_main[high_risk_kp].sum(axis=1) != df_main[df_columns[0]],  # Check if sum of KP categories differs from total
            df_main[high_risk_kp].sum(axis=1) - df_main[df_columns[0]],  # Calculate difference for KP eligibility gap
            0  # Set to 0 if no gap
        )  # Adds gap column for KP risk population

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_PrEP_Eligible_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_PrEP_Eligible_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_PrEP_Eligible_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_PrEP_Eligible_gap, 'cached_style'):  # Check for cached style
                del process_KP_PrEP_Eligible_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_PrEP_Eligible_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_PrEP_Eligible_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_PrEP_Eligible_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_PrEP_Eligible_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero KP risk population gaps
            report_description = (  # Add description for KP risk population gap
                f"Report Name: {gap_columns[0]}\n"
                f"A sum of these KP high risk population {high_risk_kp}\n"
                f"should be equal to {df_columns[0]}"  # Describe expected equality for KP risk population
            )  # Describe KP risk population gap issue (original comment incorrectly references pregnancy/breastfeeding gaps)

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns + high_risk_kp,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_PrEP_Eligible_gap, 'cached_style'):  # Check for cached style
            del process_KP_PrEP_Eligible_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_PrEP_Eligible_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_PrEP_Eligible_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-PrEP_MSF PrEP Received gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP PrEP Received gap
def process_KP_PrEP_Received_gap(display_output=None):
    """
    Process KP PrEP Received gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = ["No. of individuals who received PrEP in the reporting month"]  # Column for PrEP received
        high_risk_kp = [  # List of high-risk key population categories
            'Serodiscordant Couples (SDC)', 'SW', 'Partners of Sex Workers', 'Injecting Drug Users',
            'Individuals who engage in anal sex on a prolonged and regular basis', 
            'Exposed adolescents and young people', 'Transgender People', 'Other population'
        ]  # Defines key populations for PrEP eligibility analysis
        name = "KP Prev_PrEP-MSF Received Gap"  # Base name for report 
        gap_columns = ["KP Prev-PrEP_MSF PrEP received by KP risk population gap"]  # Name for calculated gap column
        report_name = f"{name}60"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header


        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PrEP eligibility column
        )  # Returns processed DataFrame or None if failed (original comment incorrectly references PrEP restart columns)
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        df_main2 = DHIS2_data['KP Prev MSF_PrEP_received_for_pk_at_risk'][MSF_hierarchy + high_risk_kp].copy()  # Copy DataFrame for high-risk KP disaggregation
        df_main = df_main.merge(  # Merge primary and high-risk KP DataFrames
            df_main2, 
            on=MSF_hierarchy, 
            how='left'  # Left join to retain all records from df_main
        )  # Combines total eligibility with disaggregated KP data

        df_main[high_risk_kp] = df_main[high_risk_kp].apply(pd.to_numeric, errors='coerce').fillna(0)  # Convert KP columns to numeric, replacing non-numeric with 0
        df_main[gap_columns[0]] = np.where(  # Calculate gap by KP risk population (requires numpy as np)
            df_main[high_risk_kp].sum(axis=1) != df_main[df_columns[0]],  # Check if sum of KP categories differs from total
            df_main[high_risk_kp].sum(axis=1) - df_main[df_columns[0]],  # Calculate difference for KP eligibility gap
            0  # Set to 0 if no gap
        )  # Adds gap column for KP risk population

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_PrEP_Received_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_PrEP_Received_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_PrEP_Received_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_PrEP_Received_gap, 'cached_style'):  # Check for cached style
                del process_KP_PrEP_Received_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_PrEP_Received_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_PrEP_Received_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_PrEP_Received_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_PrEP_Received_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero KP risk population gaps
            report_description = (  # Add description for KP risk population gap
                f"Report Name: {gap_columns[0]}\n"
                f"A sum of these KP high risk population {high_risk_kp}\n"
                f"should be equal to {df_columns[0]}"  # Describe expected equality for KP risk population
            )  # Describe KP risk population gap issue (original comment incorrectly references pregnancy/breastfeeding gaps)

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns + high_risk_kp,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_PrEP_Received_gap, 'cached_style'):  # Check for cached style
            del process_KP_PrEP_Received_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_PrEP_Received_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_PrEP_Received_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-PrEP_MSF PrEP Returned and Retested Negetive gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP PrEP Retest Negative gap
def process_KP_PrEP_Retest_Negative_gap(display_output=None):
    """
    Process KP PrEP Retest Negative gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = ["No. of individuals who returning for PrEP who received repeat HIV testing in the reporting month: HIV Negative"]  # Column for individuals retested HIV negative for PrEP
        high_risk_kp = [  # List of high-risk key population categories
            'Serodiscordant Couples (SDC)', 'SW', 'Partners of Sex Workers', 'Injecting Drug Users',
            'Individuals who engage in anal sex on a prolonged and regular basis', 
            'Exposed adolescents and young people', 'Transgender People', 'Other population'
        ]  # Defines key populations for PrEP retesting analysis
        name = "KP Prev_PrEP-MSF RR Negative Gap"  # Base name for report (preserves incorrect reference to 'Received')
        gap_columns = ["KP Prev-PrEP_MSF PrEP retested  with a negative report gap"]  # Name for calculated gap column (preserves incorrect reference to 'Received')
        report_name = f"{name}61"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PrEP retest negative column
        )  # Returns processed DataFrame or None if failed (original comment incorrectly references PrEP eligibility column)
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        df_main2 = DHIS2_data['KP Prev MSF_PrEP_returned_with_retesting_negetive_for_pk_at_risk'][MSF_hierarchy + high_risk_kp].copy()  # Copy DataFrame for high-risk KP disaggregation (preserves typo: 'negetive')
        df_main = df_main.merge(  # Merge primary and high-risk KP DataFrames
            df_main2, 
            on=MSF_hierarchy, 
            how='left'  # Left join to retain all records from df_main
        )  # Combines total retest negative data with disaggregated KP data

        df_main[high_risk_kp] = df_main[high_risk_kp].apply(pd.to_numeric, errors='coerce').fillna(0)  # Convert KP columns to numeric, replacing non-numeric with 0
        df_main[gap_columns[0]] = np.where(  # Calculate gap by KP risk population (requires numpy as np)
            df_main[high_risk_kp].sum(axis=1) != df_main[df_columns[0]],  # Check if sum of KP categories differs from total
            df_main[high_risk_kp].sum(axis=1) - df_main[df_columns[0]],  # Calculate difference for KP retest negative gap
            0  # Set to 0 if no gap
        )  # Adds gap column for KP risk population

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_PrEP_Retest_Negative_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_PrEP_Retest_Negative_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_PrEP_Retest_Negative_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_PrEP_Retest_Negative_gap, 'cached_style'):  # Check for cached style
                del process_KP_PrEP_Retest_Negative_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_PrEP_Retest_Negative_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_PrEP_Retest_Negative_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_PrEP_Retest_Negative_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_PrEP_Retest_Negative_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero KP risk population gaps
            report_description = (  # Add description for KP risk population gap
                f"Report Name: {gap_columns[0]}\n"
                f"A sum of these KP high risk population {high_risk_kp}\n"
                f"should be equal to {df_columns[0]}"  # Describe expected equality for KP risk population
            )  # Describe KP risk population gap issue (original comment incorrectly references pregnancy/breastfeeding gaps)

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns + high_risk_kp,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_PrEP_Retest_Negative_gap, 'cached_style'):  # Check for cached style
            del process_KP_PrEP_Retest_Negative_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_PrEP_Retest_Negative_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_PrEP_Retest_Negative_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-PrEP_MSF PrEP Returned and Retested Positive gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP PrEP Retest Positive gap
def process_KP_PrEP_Retest_Positive_gap(display_output=None):
    """
    Process KP PrEP Retest Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = ["No. of individuals who returning for PrEP who received repeat HIV testing in the reporting month: HIV Positive"]  # Column for individuals retested HIV negative for PrEP
        high_risk_kp = [  # List of high-risk key population categories
            'Serodiscordant Couples (SDC)', 'SW', 'Partners of Sex Workers', 'Injecting Drug Users',
            'Individuals who engage in anal sex on a prolonged and regular basis', 
            'Exposed adolescents and young people', 'Transgender People', 'Other population'
        ]  # Defines key populations for PrEP retesting analysis
        name = "KP Prev_PrEP-MSF RR Positive Gap"  # Base name for report (preserves incorrect reference to 'Received')
        gap_columns = ["KP Prev-PrEP_MSF PrEP retested with a positive report gap"]  # Name for calculated gap column (preserves incorrect reference to 'Received')
        report_name = f"{name}62"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PrEP retest negative column
        )  # Returns processed DataFrame or None if failed (original comment incorrectly references PrEP eligibility column)
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        df_main2 = DHIS2_data['KP Prev MSF_PrEP_returned_with_retesting_positive_for_pk_at_risk'][MSF_hierarchy + high_risk_kp].copy()  # Copy DataFrame for high-risk KP disaggregation (preserves typo: 'negetive')
        df_main = df_main.merge(  # Merge primary and high-risk KP DataFrames
            df_main2, 
            on=MSF_hierarchy, 
            how='left'  # Left join to retain all records from df_main
        )  # Combines total retest negative data with disaggregated KP data

        df_main[high_risk_kp] = df_main[high_risk_kp].apply(pd.to_numeric, errors='coerce').fillna(0)  # Convert KP columns to numeric, replacing non-numeric with 0
        df_main[gap_columns[0]] = np.where(  # Calculate gap by KP risk population (requires numpy as np)
            df_main[high_risk_kp].sum(axis=1) != df_main[df_columns[0]],  # Check if sum of KP categories differs from total
            df_main[high_risk_kp].sum(axis=1) - df_main[df_columns[0]],  # Calculate difference for KP retest negative gap
            0  # Set to 0 if no gap
        )  # Adds gap column for KP risk population

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_PrEP_Retest_Positive_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_PrEP_Retest_Positive_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_PrEP_Retest_Positive_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_PrEP_Retest_Positive_gap, 'cached_style'):  # Check for cached style
                del process_KP_PrEP_Retest_Positive_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_PrEP_Retest_Positive_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_PrEP_Retest_Positive_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_PrEP_Retest_Positive_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_PrEP_Retest_Positive_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero KP risk population gaps
            report_description = (  # Add description for KP risk population gap
                f"Report Name: {gap_columns[0]}\n"
                f"A sum of these KP high risk population {high_risk_kp}\n"
                f"should be equal to {df_columns[0]}"  # Describe expected equality for KP risk population
            )  # Describe KP risk population gap issue (original comment incorrectly references pregnancy/breastfeeding gaps)

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns + high_risk_kp,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_PrEP_Retest_Positive_gap, 'cached_style'):  # Check for cached style
            del process_KP_PrEP_Retest_Positive_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_PrEP_Retest_Positive_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_PrEP_Retest_Positive_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-PrEP_MSF PrEP Discontinue gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP PrEP Retest Positive gap
def process_KP_PrEP_Discontinued_gap(display_output=None):
    """
    Process KP PrEP Retest Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = ["No. of individuals who discontinued PrEP"]  # Column for individuals retested HIV negative for PrEP
        high_risk_kp = [  # List of high-risk key population categories
            'Serodiscordant Couples (SDC)', 'SW', 'Partners of Sex Workers', 'Injecting Drug Users',
            'Individuals who engage in anal sex on a prolonged and regular basis', 
            'Exposed adolescents and young people', 'Transgender People', 'Other population'
        ]  # Defines key populations for analysis
        name = "KP Prev_PrEP-MSF PrEP Discontined Gap"  # Base name for report
        gap_columns = ["KP Prev-PrEP_MSF PrEP discontinued gap"]  # Name for calculated gap column
        report_name = f"{name}63"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include PrEP retest negative column
        )  # Returns processed DataFrame or None if failed (original comment incorrectly references PrEP eligibility column)
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data

        df_main2 = DHIS2_data['KP Prev MSF_PrEP_discountined_for_pk_at_risk'][MSF_hierarchy + high_risk_kp].copy()  # Copy DataFrame for high-risk KP disaggregation (preserves typo: 'negetive')
        df_main = df_main.merge(  # Merge primary and high-risk KP DataFrames
            df_main2, 
            on=MSF_hierarchy, 
            how='left'  # Left join to retain all records from df_main
        )  # Combines total retest negative data with disaggregated KP data

        df_main[high_risk_kp] = df_main[high_risk_kp].apply(pd.to_numeric, errors='coerce').fillna(0)  # Convert KP columns to numeric, replacing non-numeric with 0
        df_main[gap_columns[0]] = np.where(  # Calculate gap by KP risk population (requires numpy as np)
            df_main[high_risk_kp].sum(axis=1) != df_main[df_columns[0]],  # Check if sum of KP categories differs from total
            df_main[high_risk_kp].sum(axis=1) - df_main[df_columns[0]],  # Calculate difference for KP retest negative gap
            0  # Set to 0 if no gap
        )  # Adds gap column for KP risk population

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_PrEP_Discontinued_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_PrEP_Discontinued_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_PrEP_Discontinued_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_PrEP_Discontinued_gap, 'cached_style'):  # Check for cached style
                del process_KP_PrEP_Discontinued_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_PrEP_Discontinued_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_PrEP_Discontinued_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_PrEP_Discontinued_gap.cached_style = df_main_gap_style  # Store styled DataFrame in cache
        process_KP_PrEP_Discontinued_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape in cache

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero KP risk population gaps
            report_description = (  # Add description for KP risk population gap
                f"Report Name: {gap_columns[0]}\n"
                f"A sum of these KP high risk population {high_risk_kp}\n"
                f"should be equal to {df_columns[0]}"  # Describe expected equality for KP risk population
            )  # Describe KP risk population gap issue (original comment incorrectly references pregnancy/breastfeeding gaps)

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns + high_risk_kp,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display output is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_PrEP_Discontinued_gap, 'cached_style'):  # Check for cached style
            del process_KP_PrEP_Discontinued_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_PrEP_Discontinued_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_PrEP_Discontinued_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-HTS HTS TST gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP HTS and Positive gap
def process_KP_HTS_Positive_gap(display_output=None):
    """
    Process KP HTS and Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP HTS and Positive metrics
            "Number of key population receiving HIV prevention services - (defined packages of services by topology)",  # KP receiving HIV prevention services
            "Number of KPs that received HIS Test during the reporting period and know their HIV status - Total Walk-In",  # KPs testing positive via walk-in
            "Number of KPs that received HIS Test during the reporting period and know their HIV status - Total Community",  # KPs testing positive via community
            "Number of KPs who tested HIV Positive during the reporting period - Total Walk-In",    # KPs testing positive via walk-in
            "Number of KPs who tested HIV Positive during the reporting period - Total Community"   # KPs testing positive via community
        ]  # Defines columns for KP HTS and Positive analysis
        name = "KP Prev_HTS HTS and Positive Gap"  # Base name for report
        gap_columns = [
            "KP Prev HTS (Walk-In and Community) gap",
            "KP Prev HTS positive (Walk-In and Community) gap"
        ]  # Names for calculated gap columns
        report_name = f"{name}64"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP HTS and Positive columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate KP HTS gap (requires numpy as np)
            df_main[df_columns[1:3]].sum(axis=1) > df_main[df_columns[0]],  # Check if total tested (walk-in + community) exceeds prevention services
            df_main[df_columns[1:3]].sum(axis=1) - df_main[df_columns[0]],  # Calculate difference for HTS gap
            0  # Set to 0 if no gap
        )  # Adds HTS gap column
        df_main[gap_columns[1]] = np.where(  # Calculate KP HTS Positive gap
            df_main[df_columns[3:5]].sum(axis=1) > df_main[df_columns[1:3]].sum(axis=1),  # Check if total positive (walk-in + community) exceeds total tested
            df_main[df_columns[3:5]].sum(axis=1) - df_main[df_columns[1:3]].sum(axis=1),  # Calculate difference for Positive gap
            0  # Set to 0 if no gap
        )  # Adds Positive gap column

        order_columns = MSF_hierarchy + [  # Define column order for DataFrame
            df_columns[0],  # KP receiving HIV prevention services
            df_columns[1],  # KP Walk-In HTS
            df_columns[2],  # KP Community HTS
            gap_columns[0],  # KP HTS gap
            df_columns[3],  # KP Walk-In Positive
            df_columns[4],  # KP Community Positive
            gap_columns[1]  # KP Positive gap
        ]  # Specifies ordered columns for clarity

        df_main = df_main[order_columns]  # Reorder DataFrame columns to match specified order
        df_main = df_main.reset_index(drop=True)  # Reset index to ensure clean DataFrame structure

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_HTS_Positive_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_HTS_Positive_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_HTS_Positive_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_HTS_Positive_gap, 'cached_style'):  # Check for cached style
                del process_KP_HTS_Positive_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_HTS_Positive_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_HTS_Positive_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_HTS_Positive_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_KP_HTS_Positive_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        report_description = []  # Initialize list for report descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero HTS gaps
            report_description.append(  # Add description for HTS gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\nplus {df_columns[2]}\n"
                f"should not be greater than {df_columns[0]}"  # Describe expected relation for HTS
            )  # Describe HTS gap issue
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():  # Check for non-zero Positive gaps
            report_description.append(  # Add description for Positive gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[3]}\nplus {df_columns[4]}\n"
                f"should not be greater than {df_columns[1]} plus {df_columns[2]}"  # Describe expected relation for Positive
            )  # Describe Positive gap issue
        report_description = "\n\n".join(report_description)  # Join descriptions with double newlines

        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_HTS_Positive_gap, 'cached_style'):  # Check for cached style
            del process_KP_HTS_Positive_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_HTS_Positive_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_HTS_Positive_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-HTS HTS-3b (TG) Positive gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP HTS 3b and Positive gap
def process_KP_HTS_3b_Positive_gap(display_output=None):
    """
    Process KP HTS 3b and Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP HTS and Positive metrics
            "HTS-3b Number of TG that have received an HIV test during the reporting period in KP-specific programs and know their results",
            "HTS-3b Number of TG that have received an HIV test during the reporting period in KP-specific programs and HIV positive results"
        ]  # Defines columns for KP HTS and Positive analysis
        name = "KP Prev_HTS 3b (TG) Positive Gap"  # Base name for report
        gap_columns = ["KP Prev HTS 3b positive gap"]  # Names for calculated gap columns
        report_name = f"{name}65"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP HTS and Positive columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate KP HTS gap (requires numpy as np)
            df_main[df_columns[1]] > df_main[df_columns[0]],  
            df_main[df_columns[1]] - df_main[df_columns[0]],  
            0  # Set to 0 if no gap
        )  

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_HTS_3b_Positive_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_HTS_3b_Positive_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_HTS_3b_Positive_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_HTS_3b_Positive_gap, 'cached_style'):  # Check for cached style
                del process_KP_HTS_3b_Positive_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_HTS_3b_Positive_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_HTS_3b_Positive_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_HTS_3b_Positive_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_KP_HTS_3b_Positive_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero HTS gaps
            report_description = (  # Add description for HTS gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be greater than {df_columns[0]}"  # Describe expected relation for HTS
            ) 
        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_HTS_3b_Positive_gap, 'cached_style'):  # Check for cached style
            del process_KP_HTS_3b_Positive_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_HTS_3b_Positive_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_HTS_3b_Positive_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-HTS HTS-3c (SW) Positive gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP HTS 3c and Positive gap
def process_KP_HTS_3c_Positive_gap(display_output=None):
    """
    Process KP HTS 3c and Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP HTS and Positive metrics
            "HTS-3c Number of sex workers that have received an HIV test during the reporting period in KP-specific programs and know their results",
            "HTS-3c Number of sex workers that have received an HIV test during the reporting period in KP-specific programs and received HIV-positive results"   
        ]  # Defines columns for KP HTS and Positive analysis
        name = "KP Prev_HTS 3c (SW) Positive Gap"  # Base name for report
        gap_columns = ["KP Prev HTS 3c positive gap"]  # Names for calculated gap columns
        report_name = f"{name}66"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP HTS and Positive columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate KP HTS gap (requires numpy as np)
            df_main[df_columns[1]] > df_main[df_columns[0]],  
            df_main[df_columns[1]] - df_main[df_columns[0]],  
            0  # Set to 0 if no gap
        )  

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_HTS_3c_Positive_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_HTS_3c_Positive_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_HTS_3c_Positive_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_HTS_3c_Positive_gap, 'cached_style'):  # Check for cached style
                del process_KP_HTS_3c_Positive_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_HTS_3c_Positive_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_HTS_3c_Positive_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_HTS_3c_Positive_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_KP_HTS_3c_Positive_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero HTS gaps
            report_description = (  # Add description for HTS gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be greater than {df_columns[0]}"  # Describe expected relation for HTS
            ) 
        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_HTS_3c_Positive_gap, 'cached_style'):  # Check for cached style
            del process_KP_HTS_3c_Positive_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_HTS_3c_Positive_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_HTS_3c_Positive_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-HTS HTS-3d (PWID) Positive gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP HTS 3d and Positive gap
def process_KP_HTS_3d_Positive_gap(display_output=None):
    """
    Process KP HTS 3d and Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP HTS and Positive metrics
            "HTS-3d Number of people who inject drugs (PWID) that have received an HIV test during the reporting period in KP-specific programs and know their results",
            "HTS-3d Number of people who inject drugs (PWID) that have received an HIV test during the reporting period in KP-specific programs and received HIV positive results"   
        ]  # Defines columns for KP HTS and Positive analysis
        name = "KP Prev_HTS 3d (PWID) Positive Gap"  # Base name for report
        gap_columns = ["KP Prev HTS 3d positive gap"]  # Names for calculated gap columns
        report_name = f"{name}67"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP HTS and Positive columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate KP HTS gap (requires numpy as np)
            df_main[df_columns[1]] > df_main[df_columns[0]],  
            df_main[df_columns[1]] - df_main[df_columns[0]],  
            0  # Set to 0 if no gap
        )  

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_HTS_3d_Positive_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_HTS_3d_Positive_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_HTS_3d_Positive_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_HTS_3d_Positive_gap, 'cached_style'):  # Check for cached style
                del process_KP_HTS_3d_Positive_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_HTS_3d_Positive_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_HTS_3d_Positive_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_HTS_3d_Positive_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_KP_HTS_3d_Positive_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero HTS gaps
            report_description = (  # Add description for HTS gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be greater than {df_columns[0]}"  # Describe expected relation for HTS
            ) 
        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_HTS_3d_Positive_gap, 'cached_style'):  # Check for cached style
            del process_KP_HTS_3d_Positive_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_HTS_3d_Positive_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_HTS_3d_Positive_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-HTS HTS-3e (OVC) Positive gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP HTS 3e and Positive gap
def process_KP_HTS_3e_Positive_gap(display_output=None):
    """
    Process KP HTS 3e and Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP HTS and Positive metrics
            "HTS-3e Number of other vulnerable populations (OVP) that have received an HIV test during the reporting period and know their results",
            "HTS-3e Number of other vulnerable populations (OVP) that have received an HIV test during the reporting period and received HIV-positive results"
        ]  # Defines columns for KP HTS and Positive analysis
        name = "KP Prev_HTS 3e (OVC) Positive Gap"  # Base name for report
        gap_columns = ["KP Prev HTS 3e positive gap"]  # Names for calculated gap columns
        report_name = f"{name}68"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP HTS and Positive columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate KP HTS gap (requires numpy as np)
            df_main[df_columns[1]] > df_main[df_columns[0]],  
            df_main[df_columns[1]] - df_main[df_columns[0]],  
            0  # Set to 0 if no gap
        )  

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_HTS_3e_Positive_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_HTS_3e_Positive_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_HTS_3e_Positive_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_HTS_3e_Positive_gap, 'cached_style'):  # Check for cached style
                del process_KP_HTS_3e_Positive_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_HTS_3e_Positive_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_HTS_3e_Positive_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_HTS_3e_Positive_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_KP_HTS_3e_Positive_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero HTS gaps
            report_description = (  # Add description for HTS gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be greater than {df_columns[0]}"  # Describe expected relation for HTS
            ) 
        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_HTS_3e_Positive_gap, 'cached_style'):  # Check for cached style
            del process_KP_HTS_3e_Positive_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_HTS_3e_Positive_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_HTS_3e_Positive_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-HTS HTS-3f (Prinsons) Positive gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP HTS 3f and Positive gap
def process_KP_HTS_3f_Positive_gap(display_output=None):
    """
    Process KP HTS 3f and Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP HTS and Positive metrics
            "HTS-3f Number of people in prisons and other closed settings that have received an HIV test during the reporting period and know their results ",
            "HTS-3f Number of people in prisons and other closed settings that have received an HIV test during the reporting period and received HIV-positive results"
        ]  # Defines columns for KP HTS and Positive analysis
        name = "KP Prev_HTS 3f (Prisons) Positive Gap"  # Base name for report
        gap_columns = ["KP Prev HTS 3f positive gap"]  # Names for calculated gap columns
        report_name = f"{name}69"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP HTS and Positive columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate KP HTS gap (requires numpy as np)
            df_main[df_columns[1]] > df_main[df_columns[0]],  
            df_main[df_columns[1]] - df_main[df_columns[0]],  
            0  # Set to 0 if no gap
        )  

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_HTS_3f_Positive_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_HTS_3f_Positive_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_HTS_3f_Positive_gap.cached_style  # Retrieve cached styled DataFrame
                    print(print_display_name)
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_HTS_3f_Positive_gap, 'cached_style'):  # Check for cached style
                del process_KP_HTS_3f_Positive_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_HTS_3f_Positive_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_HTS_3f_Positive_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_HTS_3f_Positive_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_KP_HTS_3f_Positive_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero HTS gaps
            report_description = (  # Add description for HTS gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be greater than {df_columns[0]}"  # Describe expected relation for HTS
            ) 
        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_HTS_3f_Positive_gap, 'cached_style'):  # Check for cached style
            del process_KP_HTS_3f_Positive_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_HTS_3f_Positive_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_HTS_3f_Positive_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

### - KP-HTS HTS-2 (AGYQ) Positive gap
# -----------------------------------------------------------------------------------------
# -- Define the main function to process KP HTS 2 and Positive gap
def process_KP_HTS_2_Positive_gap(display_output=None):
    """
    Process KP HTS 2 and Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:  # Begin exception handling for robust error management
        # -- Step 1: Initialize constants
        df_columns = [  # List of column names for KP HTS and Positive metrics
            "HTS-2 Percentage of high risk AGYW that have received an HIV test during the reporting period in AGYW programs",
            "HTS-2 Percentage of high-risk AGYW that have received an HIV test during the reporting period in AGYW programs that tested positive for HIV"
        ]  # Defines columns for KP HTS and Positive analysis
        name = "KP Prev_HTS 2 (AGYW) Positive Gap"  # Base name for report
        gap_columns = ["KP Prev HTS 2 positive gap"]  # Names for calculated gap columns
        report_name = f"{name}70"  # Report name with unique suffix
        No_gap_msg = f"No {report_name}"  # Message to display if no gaps are found
        display_name = f"✔️ Displaying {report_name}"  # Formatted display name for output
        display_line = "-" * (len(display_name) + 1)  # Create separator line based on display name length          
        print_display_name = f"{display_line}\n{display_name}\n{display_line}"  # Combined string for display header

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(  # Fetch and prepare DataFrame from DHIS2
            DHIS2_data_key="KP Prev MSF",  # Key to fetch KP Prev MSF dataset
            hierarchy_columns=MSF_hierarchy,  # Predefined MSF hierarchy columns (assumed defined elsewhere)
            data_columns=df_columns  # Include KP HTS and Positive columns
        )  # Returns processed DataFrame or None if failed
        if df_main is None:  # Check if data preparation failed or returned empty
            return  # Exit function if no valid data
        
        # -- Step 3: Calculate gaps
        df_main[gap_columns[0]] = np.where(  # Calculate KP HTS gap (requires numpy as np)
            df_main[df_columns[1]] > df_main[df_columns[0]],  
            df_main[df_columns[1]] - df_main[df_columns[0]],  
            0  # Set to 0 if no gap
        )  

        # -- Step 4: Wrap column headers for better readability
        wrap_column_headers(df_main)  # Format DataFrame column headers (assumed function)
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # Wrap gap column names for display

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:  # Check if display output is requested
            if hasattr(process_KP_HTS_2_Positive_gap, 'cached_style'):  # Check if cached styled DataFrame exists
                cached_shape = getattr(process_KP_HTS_2_Positive_gap, 'cached_shape', None)  # Retrieve cached DataFrame shape
                current_shape = df_main.shape  # Get current unfiltered DataFrame shape
                if cached_shape == current_shape:  # Compare shapes to use cached version
                    display = process_KP_HTS_2_Positive_gap.cached_style  # Retrieve cached styled DataFrame
                    widget_display_df(display)  # Display cached styled DataFrame (assumed widget function)
                return  # Exit to avoid reprocessing

        # -- Step 6: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(  # Filter DataFrame for rows with non-zero gaps
            df=df_main,  # Input DataFrame with gap columns
            msg=No_gap_msg,  # Message to display if no gaps
            opNonZero=gap_columns_wrap,  # Filter for non-zero gap values
            opNeg=None,  # No negative value filter
            opPos=None,  # No positive value filter
            opZero=None,  # No zero value filter
            opLT100=None  # No less-than-100 filter
        )  # Returns filtered DataFrame or None if no gaps
        if df_main_gap is None:  # Check if no gaps were found
            if hasattr(process_KP_HTS_2_Positive_gap, 'cached_style'):  # Check for cached style
                del process_KP_HTS_2_Positive_gap.cached_style  # Clear cached styled DataFrame
            if hasattr(process_KP_HTS_2_Positive_gap, 'cached_shape'):  # Check for cached shape
                del process_KP_HTS_2_Positive_gap.cached_shape  # Clear cached DataFrame shape
            return  # Exit function if no gaps

        # -- Step 7: Style the filtered DataFrame
        df_main_gap_style = (  # Apply styling to filtered DataFrame
            df_main_gap.style  # Create style object from filtered DataFrame
            .hide(axis='index')  # Hide row index for cleaner display
            .map(outlier_red, subset=gap_columns_wrap)  # Highlight non-zero gaps in red (assumed function)
        )  # Creates styled DataFrame for display/export

        # -- Step 8: Cache styled DataFrame and shape
        process_KP_HTS_2_Positive_gap.cached_style = df_main_gap_style  # Store styled DataFrame
        process_KP_HTS_2_Positive_gap.cached_shape = df_main.shape  # Store original unfiltered DataFrame shape

        # -- Step 9: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]  # Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # Create image file name with report month
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # Define image file path (assumed defined)
        report_sheet_name = report_name  # Define Excel sheet name same as report name

        # -- Step 10: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():  # Check for non-zero HTS gaps
            report_description = (  # Add description for HTS gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be greater than {df_columns[0]}"  # Describe expected relation for HTS
            ) 
        # -- Step 11: Export results to multiple formats
        if not display_output:
            export_df_to_doc_image_excel(  # Export styled DataFrame to image, Excel, and Word
                report_name=report_name,  # Report name for file naming
                df_style=df_main_gap_style,  # Styled DataFrame for export
                img_file_name=report_image_name,  # Image file name
                img_file_path=report_image_path,  # Image file path
                doc_description=report_description,  # Word document description
                doc_indicators_to_italicize=df_columns,  # Italicize input columns
                doc_indicators_to_underline=gap_columns,  # Underline gap columns
                xlm_file_path=doc_file_msf_outlier_xlsx,  # Excel file path (assumed defined)
                xlm_sheet_name=report_sheet_name  # Excel sheet name
            )  # Exports to specified formats

        # -- Step 12: Display styled DataFrame if requested
        if display_output:  # Check if display is requested
            print(print_display_name)
            widget_display_df(df_main_gap_style)  # Display styled DataFrame (assumed widget function)

    except Exception as e:  # Catch any processing errors
        print(f"⦸ Error processing {report_name}: {str(e)}")  # Print error with report name
        if hasattr(process_KP_HTS_2_Positive_gap, 'cached_style'):  # Check for cached style
            del process_KP_HTS_2_Positive_gap.cached_style  # Clear cached styled DataFrame
        if hasattr(process_KP_HTS_2_Positive_gap, 'cached_shape'):  # Check for cached shape
            del process_KP_HTS_2_Positive_gap.cached_shape  # Clear cached DataFrame shape
        return  # Exit on error
# End of the function ---------------------------------------------------------------------

In [ ]:
# -- List of function descriptions for display
function_description_name = [
    "Get Data",  # Description for LGA report rate gap
    "ANSO Report Rate",  # Description for LGA report rate gap
    "LGA Report Rate",  # Description for facility report rate gap
    "AGYW HTS",  # Description for AGYW HTS gap
    "AGYW Pos",  # Description for AGYW positive gap
    "AGYW Pos Linkage",  # Description for AGYW positive linkage gap
    "AGYW TB Screening",  # Description for AGYW TB screening gap
    "ART PosEnrolment",  # Description for ART positive enrolment gap
    "ART-3 subsets",  # Description for ART regimen line, MMD and DSD
    "ART Tx_New TB Screening",  # Description for ART Tx_New TB screening gap
    "ART TB Presumptive Test",  # Description for ART TB presumptive test gap
    "ART TB Treatment",  # Description for ART TB treatment gap
    "ART Viral Load Suppression",  # Description for ART viral load suppression gap
    "HTS New Positive",  # Description for HTS new positive gap
    "HTS TB Screening",  # Description for HTS TB screening gap
    "HTS Enrolment",  # Description for HTS enrolment
    "HTS Discordant Couples Test",  # Description for HTS discordant couple testing gap
    "HTS CD4 Test",  # Description for HTS CD4 test result gap
    "HIVST Mode",  # Description for HIVST distribution mode gap
    "HIVST Test Freq.",  # Description for HIVST distribution mode gap
    "HIVST Result",  # Description for HIVST result gap
    "HIVST Reactive Link",  # Description for HIVST reactive, confirmation and linkage gap
    "HIVST Prevention Service",  # Description for HIVST prevention service and linkage gap
    "HIVST Partner Screening",  # Description for HIVST partner screening gap
    "ICT Index Acceptance",  # Description for ICT index acceptance gap
    "ICT Contact",  # Description for ICT contact gap
    "ICT HTS",  # Description for ICT HTS gap
    "ICT Positive Linkage",  # Description for ICT positive linkage gap
    "PMTCT New ANC HTS",  # Description for PMTCT new ANC gap
    "PMTCT Positive",  # Description for PMTCT positive gap
    "PMTCT Previously Known",  # Description for PMTCT previously known gap
    "PMTCT Positive Linkage",  # Description for PMTCT positive linkage
    "PMTCT Seroconversion",  # Description for PMTCT seroconversion gap
    "PMTCT Syphilis Test",  # Description for PMTCT syphilis test gap
    "PMTCT Hepatitis Test",  # Description for PMTCT hepatitis test gap
    "PMTCT Labour and Delivery",  # Description for PMTCT labour and delivery gap
    "PMTCT Facility HEI ARVs",  # Description for PMTCT facility HEI ARVs gap
    "PMTCT EID PCR Test",  # Description for PMTCT EID PCR test gap
    "PMTCT EID PCR Test Result",  # Description for PMTCT EID PCR test result gap
    "NSP Newly Recruited",  # Description for NSP Newly recruited gap
    "NSP HTS Positive Linkage",  # Description for NSP HTS positive linkage gap
    "NSP Coinfection Treatment",  # Description for NSP coinfection treatment gap
    "NSP Substance Abuse",  # Description for NSP substance abuse gap
    "NSP MAT Referral",  # Description for NSP MAT referral gap
    "KP Prev MHS",  # Description for KP Prev MHS gap
    "KP Prev MSH Access Type",  # Description for KP Prev MSH access type gap
    "KP GBV CSR",  # Description for KP Prev GBV CSR gap
    "KP GBV Post GBV",  # Description for KP Prev GBV post GBV gap
    "KP KP_GBV IVA",  # Description for KP Prev GBV incident of voilence or abuse
    "KP KP_GBV Legal Support",  # Description for KP Prev GBV legal support gap
    "KP PEP",  # Description for KP PEP gap
    "KP PrEP_KP Product (MSM)",  # Description for KP PrEP product MSM gap
    "KP PrEP_KP Product (TG)",  # Description for KP PrEP product TG gap
    "KP PrEP_KP Product (SW)",  # Description for KP PrEP product SW gap
    "KP PrEP_KP Product (PWID)",  # Description for KP PrEP product PWID gap
    "KP KP_Condom Distribution",  # Description for KP condom distribution gap
    "KP KP_Condom Lubricants",  # Description for KP condom lubricants gap
    "KP PrEP_GP Enrolment",  # Description for KP PrEP enrolment gap
    "KP PrEP_GP Restart",  # Description for KP PrEP restart gap
    "KP PrEP_MSF PrEP Eligible",  # Description for KP PrEP eligible gap
    "KP PrEP_MSF PrEP Received", # Description for KP PrEP received gap
    "KP KP_PrEP R&R Negetive",  # Description for KP PrEP returned and restested negative gap
    "KP KP_PrEP R&R Positive",  # Description for KP PrEP returned and restested positive gap
    "KP KP_PrEP Discounted",  # Description for KP PrEP discounted gap 
    "KP HTS TST",  # Description for KP HTS TST gap
    "KP HTS 3b (TG) Positive",  # Description for KP HTS 3b (TG) positive gap
    "KP HTS 3c (SW) Positive",  # Description for KP HTS 3c (SW) positive gap
    "KP HTS 3d (PWID) Positive",  # Description for KP HTS 3d (PWID) positive gap
    "KP HTS 3e (OVC) Positive",  # Description for KP HTS 3e (OVC) positive gap
    "KP HTS 3f (Prison) Positive",  # Description for KP HTS 3f (Prison) positive gap
    "KP HTS 2 (AGYW) Positive"  # Description for KP HTS 2 (AGYW) positive gap
]

# -- Define constants for UI elements
ui_title = "ANSO IHVN DHIS2 MSF Console"  # Report title to be displayed in bold
author = "Reuben Edidiong"  # Author name, kept plain due to terminal italic limitation
version = "msf.vlr v1.0"  # Version identifier for the report
ui_sepperator_line = 150  # Length of separator lines (adjustable; 80 for cleaner look)
bold = "\033[1m"  # ANSI code for bold text
reset = "\033[0m"  # ANSI code to reset formatting

# -- Core components for separators
header = f"{bold}{ui_title}{reset} {f'© {author} {version}':>122}"  # -- Header with bold title, right-aligned copyright
top_line = f"{'-' * ui_sepperator_line}\n"      # -- Top separator line
bottom_line = f"\n{'-' * ui_sepperator_line}"    # -- Bottom separator line
spacing = "\n" * 15                              # -- Empty line gap in ui_separator_clear

# -- Separator definitions
ui_separator_top = f"{header}\n{top_line}"       # -- Top separator: header followed by a single line
ui_separator_bottom = f"{bottom_line}"           # -- Bottom separator: just a single line
ui_separator_clear = (                           # -- Full clear separator: header, top line, spacing, bottom line
    f"{ui_separator_top}\n"
    f"{spacing}"
    f"{bottom_line}"
)

def run_jupyter_mode():
    """
    Runs an interactive Jupyter interface with buttons to execute report processing functions.
    Group names are displayed with a maximum of 6 buttons per line, breaking to a new line for additional buttons.
    Sub-buttons are smaller in width than main buttons and appear when a group name is clicked.
    
    Args:
        None
    
    Returns:
        None 
    """

    # -- Step 1: Create buttons with descriptive labels
    botton0 = widgets.Button(description=f"{function_description_name[0]}")
    botton1 = widgets.Button(description=f"{function_description_name[1]}")
    botton2 = widgets.Button(description=f"{function_description_name[2]}")
    botton3 = widgets.Button(description=f"{function_description_name[3]}")
    botton4 = widgets.Button(description=f"{function_description_name[4]}")
    botton5 = widgets.Button(description=f"{function_description_name[5]}")
    botton6 = widgets.Button(description=f"{function_description_name[6]}")
    botton7 = widgets.Button(description=f"{function_description_name[7]}")
    botton8 = widgets.Button(description=f"{function_description_name[8]}")
    botton9 = widgets.Button(description=f"{function_description_name[9]}")
    botton10 = widgets.Button(description=f"{function_description_name[10]}")
    botton11 = widgets.Button(description=f"{function_description_name[11]}")
    botton12 = widgets.Button(description=f"{function_description_name[12]}")
    botton13 = widgets.Button(description=f"{function_description_name[13]}")
    botton14 = widgets.Button(description=f"{function_description_name[14]}")
    botton15 = widgets.Button(description=f"{function_description_name[15]}")
    botton16 = widgets.Button(description=f"{function_description_name[16]}")
    botton17 = widgets.Button(description=f"{function_description_name[17]}")
    botton18 = widgets.Button(description=f"{function_description_name[18]}")
    botton19 = widgets.Button(description=f"{function_description_name[19]}")
    botton20 = widgets.Button(description=f"{function_description_name[20]}")
    botton21 = widgets.Button(description=f"{function_description_name[21]}")
    botton22 = widgets.Button(description=f"{function_description_name[22]}")
    botton23 = widgets.Button(description=f"{function_description_name[23]}")
    botton24 = widgets.Button(description=f"{function_description_name[24]}")
    botton25 = widgets.Button(description=f"{function_description_name[25]}")
    botton26 = widgets.Button(description=f"{function_description_name[26]}")
    botton27 = widgets.Button(description=f"{function_description_name[27]}")
    botton28 = widgets.Button(description=f"{function_description_name[28]}")
    botton29 = widgets.Button(description=f"{function_description_name[29]}")
    botton30 = widgets.Button(description=f"{function_description_name[30]}")
    botton31 = widgets.Button(description=f"{function_description_name[31]}")
    botton32 = widgets.Button(description=f"{function_description_name[32]}")
    botton33 = widgets.Button(description=f"{function_description_name[33]}") 
    botton34 = widgets.Button(description=f"{function_description_name[34]}")
    botton35 = widgets.Button(description=f"{function_description_name[35]}") 
    botton36 = widgets.Button(description=f"{function_description_name[36]}")
    botton37 = widgets.Button(description=f"{function_description_name[37]}")
    botton38 = widgets.Button(description=f"{function_description_name[38]}")
    botton39 = widgets.Button(description=f"{function_description_name[39]}")
    botton40 = widgets.Button(description=f"{function_description_name[40]}")
    botton41 = widgets.Button(description=f"{function_description_name[41]}")
    botton42 = widgets.Button(description=f"{function_description_name[42]}")
    botton43 = widgets.Button(description=f"{function_description_name[43]}")
    botton44 = widgets.Button(description=f"{function_description_name[44]}")
    botton45 = widgets.Button(description=f"{function_description_name[45]}")
    botton46 = widgets.Button(description=f"{function_description_name[46]}")
    botton47 = widgets.Button(description=f"{function_description_name[47]}")
    botton48 = widgets.Button(description=f"{function_description_name[48]}")
    botton49 = widgets.Button(description=f"{function_description_name[49]}")
    botton50 = widgets.Button(description=f"{function_description_name[50]}")
    botton51 = widgets.Button(description=f"{function_description_name[51]}")
    botton52 = widgets.Button(description=f"{function_description_name[52]}")
    botton53 = widgets.Button(description=f"{function_description_name[53]}")
    botton54 = widgets.Button(description=f"{function_description_name[54]}")
    botton55 = widgets.Button(description=f"{function_description_name[55]}")
    botton56 = widgets.Button(description=f"{function_description_name[56]}")
    botton57 = widgets.Button(description=f"{function_description_name[57]}")
    botton58 = widgets.Button(description=f"{function_description_name[58]}")
    botton59 = widgets.Button(description=f"{function_description_name[59]}")
    botton60 = widgets.Button(description=f"{function_description_name[60]}")
    botton61 = widgets.Button(description=f"{function_description_name[61]}")
    botton62 = widgets.Button(description=f"{function_description_name[62]}")
    botton63 = widgets.Button(description=f"{function_description_name[63]}")
    botton64 = widgets.Button(description=f"{function_description_name[64]}")
    botton65 = widgets.Button(description=f"{function_description_name[65]}")
    botton66 = widgets.Button(description=f"{function_description_name[66]}")
    botton67 = widgets.Button(description=f"{function_description_name[67]}")
    botton68 = widgets.Button(description=f"{function_description_name[68]}")
    botton69 = widgets.Button(description=f"{function_description_name[69]}")
    botton70 = widgets.Button(description=f"{function_description_name[70]}")
    generate_report = widgets.Button(description="Export Report") 
    clear_button = widgets.Button(description="Clear screen")
    output = widgets.Output()

    # -- Step 2: Define button handlers 
    def on_botton0_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            fetch_dhis2_data_interactive_jupyter_mode()
            print(ui_separator_bottom)
            
    def on_botton1_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_lga_report_rate_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton2_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_facility_report_rate_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton3_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_AGYW_HTS_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton4_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_AGYW_Positive_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton5_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_AGYW_Positive_Linkage_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton6_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_AGYW_TB_Screening_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton7_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ART_PosEnrolment_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton8_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ART_RegimentLine_MMD_DSD_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton9_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ART_TB_Screening_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton10_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ART_TB_Presumptive_Test_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton11_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ART_TB_Treatment_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton12_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ART_Viral_Load_Suppression_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton13_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HTS_New_Positive_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton14_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HTS_TB_Screening_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton15_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HTS_Enrolment_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton16_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HTS_Couple_Counselling_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton17_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HTS_CD4_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton18_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HIVST_Distr_Mode_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton19_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HIVST_Test_Freq_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton20_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HIVST_Result_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton21_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HIVST_Reactive_Link_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton22_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HIVST_Prevention_Serv_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton23_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HIVST_Partner_Screening_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton24_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ICT_Index_Acceptance_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton25_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ICT_Contact_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton26_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ICT_HTS_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton27_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ICT_Positive_Link_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton28_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_ANC_Optmz_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton29_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Positive_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton30_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_PK_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton31_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Positive_Linkage_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton32_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Seroconversion_gap(display_output=True)
            print(ui_separator_bottom) 
    
    def on_botton33_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Syphilis_Test_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton34_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Hepatitis_Test_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton35_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Labour_Delivery_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton36_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Facility_HEI_ARVs_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton37_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_EID_PCR_Test_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton38_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_EID_PCR_Test_Result_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton39_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_NSP_Newly_Recruited_gap(display_output=True)
            print(ui_separator_bottom)      
    
    def on_botton40_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_NSP_HTS_Positive_Linkage_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton41_click(b):   
        with output:
            clear_output()
            print(ui_separator_top)
            process_NSP_Coinfection_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton42_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_NSP_Substance_Abuse_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton43_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_NSP_MAT_Referral_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton44_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_MHS_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton45_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_MHS_Access_Type_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton46_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_GBV_CSR_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton47_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_Post_GBV_SV_PEP_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton48_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_GBV_IVA_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton49_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_GBV_Legal_Support_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton50_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_PEP_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton51_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_PrEP_Product_Recieved_MSM_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton52_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_PrEP_Product_Recieved_TG_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton53_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_PrEP_Product_Recieved_SW_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton54_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_PrEP_Product_Recieved_PWID_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton55_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_Condom_Distribution_gap(display_output=True)
            print(ui_separator_bottom)    
    
    def on_botton56_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_Lubricants_Distribution_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton57_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_PrEP_Enrolment_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton58_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_PrEP_Restart_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton59_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_PrEP_Eligible_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton60_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_PrEP_Received_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton61_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_PrEP_Retest_Negative_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton62_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_PrEP_Retest_Positive_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton63_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_PrEP_Discontinued_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton64_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_HTS_Positive_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton65_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_HTS_3b_Positive_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton66_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_HTS_3c_Positive_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton67_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_HTS_3d_Positive_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton68_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_HTS_3e_Positive_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton69_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_HTS_3f_Positive_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton70_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_KP_HTS_2_Positive_gap(display_output=True)
            print(ui_separator_bottom)

    def on_clear_button_click(b):
        with output:
            clear_output()
            print(ui_separator_clear)

    # List of gap functions with their descriptions for progress tracking
    gap_functions = [
        ('process_lga_report_rate_gap', 'LGA Report Rate Gap'),
        ('process_facility_report_rate_gap', 'Facility Report Rate Gap'),
        ('process_AGYW_HTS_gap', 'AGYW HTS Gap'),
        ('process_AGYW_Positive_gap', 'AGYW Positive Gap'),
        ('process_AGYW_Positive_Linkage_gap', 'AGYW Positive Linkage Gap'),
        ('process_AGYW_TB_Screening_gap', 'AGYW TB Screening Gap'),
        ('process_ART_PosEnrolment_gap', 'ART Positive-Enrolment Gap'),
        ('process_ART_RegimentLine_MMD_DSD_gap', 'ART Regimen-Line MMD DSD Gap'),
        ('process_ART_TB_Screening_gap', 'ART TB Screening Gap'),
        ('process_ART_TB_Presumptive_Test_gap', 'ART TB Presumptive Test Gap'),
        ('process_ART_TB_Treatment_gap', 'ART TB Treatment Gap'),
        ('process_ART_Viral_Load_Suppression_gap', 'ART Viral Load Suppression Gap'),
        ('process_HTS_New_Positive_gap', 'HTS New Positive Gap'),
        ('process_HTS_TB_Screening_gap', 'HTS TB Screening Gap'),
        ('process_HTS_Enrolment_gap', 'HTS Enrolment Gap'),
        ('process_HTS_Couple_Counselling_gap', 'HTS Couple Counselling Gap'),
        ('process_HTS_CD4_gap', 'HTS CD4 Gap'),
        ('process_HIVST_Distr_Mode_gap', 'HIVST Distribution Mode Gap'),
        ('process_HIVST_Test_Freq_gap', 'HIVST Testing Frequency Gap'),
        ('process_HIVST_Result_gap', 'HIVST Result Gap'),
        ('process_HIVST_Reactive_Link_gap', 'HIVST Reactive and Linkage Gap'),
        ('process_HIVST_Prevention_Serv_gap', 'HIVST Prevention Service Gap'),
        ('process_HIVST_Partner_Screening_gap', 'HIVST Partner Screening Gap'),
        ('process_ICT_Index_Acceptance_gap', 'ICT Index Acceptance Gap'),
        ('process_ICT_Contact_gap', 'ICT Contact Gap'),
        ('process_ICT_HTS_gap', 'ICT HTS Gap'),
        ('process_ICT_Positive_Link_gap', 'ICT Positive Linkage Gap'),
        ('process_PMTCT_ANC_Optmz_gap', 'PMTCT New ANC HTS Optimization Gap'),
        ('process_PMTCT_Positive_gap', 'PMTCT Positive Gap'),
        ('process_PMTCT_PK_gap', 'PMTCT Previously Known Gap'),
        ('process_PMTCT_Positive_Linkage_gap', 'PMTCT Positive Linkage Gap'),
        ('process_PMTCT_Seroconversion_gap', 'PMTCT Seroconversion Gap'),
        ('process_PMTCT_Syphilis_Test_gap', 'PMTCT Syphilis Test Gap'),
        ('process_PMTCT_Hepatitis_Test_gap', 'PMTCT Hepatitis Test Gap'),
        ('process_PMTCT_Labour_Delivery_gap', 'PMTCT Labour and Delivery Gap'),
        ('process_PMTCT_Facility_HEI_ARVs_gap', 'PMTCT Facility HEI ARVs Gap'),
        ('process_PMTCT_EID_PCR_Test_gap', 'PMTCT EID PCR Test Gap'),
        ('process_PMTCT_EID_PCR_Test_Result_gap', 'PMTCT EID PCR Test Result Gap'),
        ('process_NSP_Newly_Recruited_gap', 'NSP Newly Recruited Gap'),
        ('process_NSP_HTS_Positive_Linkage_gap', 'NSP HTS Positive and Linkage Gap'),
        ('process_NSP_Coinfection_gap', 'NSP Coinfection Gap'),
        ('process_NSP_Substance_Abuse_gap', 'NSP Substance Abuse Gap'),
        ('process_NSP_MAT_Referral_gap', 'NSP MAT Referral Gap'),
        ('process_KP_MHS_gap', 'KP MHS and Psychosocial Support Gap'),
        ('process_KP_MHS_Access_Type_gap', 'KP MHS and Psychosocial Support by Access Type Gap'),
        ('process_KP_GBV_CSR_gap', 'KP GBV CSR Gap'),
        ('process_KP_Post_GBV_SV_PEP_gap', 'KP Post GBV Sexual Violence Gap'),
        ('process_KP_GBV_IVA_gap', 'KP Prev_KP-GBV IVA Gap'),
        ('process_KP_GBV_Legal_Support_gap', 'KP Prev_KP-GBV Legal Support Gap'),
        ('process_KP_PEP_gap', 'KP Prev_PEP PEP Gap'),
        ('process_KP_PrEP_Product_Recieved_MSM_gap', 'KP Prev_KP-PrEP Product Received MSM Gap'),
        ('process_KP_PrEP_Product_Recieved_TG_gap', 'KP Prev_KP-PrEP Product Received TG Gap'),
        ('process_KP_PrEP_Product_Recieved_SW_gap', 'KP Prev_KP-PrEP Product Received SW Gap'),
        ('process_KP_PrEP_Product_Recieved_PWID_gap', 'KP Prev_KP-PrEP Product Received PWID Gap'),
        ('process_KP_Condom_Distribution_gap', 'KP Prev_KP-Condom Distribution Gap'),
        ('process_KP_Lubricants_Distribution_gap', 'KP Prev_KP-Condom Lubricants Distribution Gap'),
        ('process_KP_PrEP_Enrolment_gap', 'KP Prev_PrEP-GP Enrolment Gap'),
        ('process_KP_PrEP_Restart_gap', 'KP Prev_PrEP-GP Restart Gap'),
        ('process_KP_PrEP_Eligible_gap', 'KP Prev_PrEP-MSF Eligible Gap'),
        ('process_KP_PrEP_Received_gap', 'KP Prev_PrEP-MSF Received Gap'),
        ('process_KP_PrEP_Retest_Negative_gap', 'KP Prev_PrEP-MSF RR Negative Gap'),
        ('process_KP_PrEP_Retest_Positive_gap', 'KP Prev_PrEP-MSF RR Positive Gap'),
        ('process_KP_PrEP_Discontinued_gap', 'KP Prev_PrEP-MSF PrEP Discontinued Gap'),
        ('process_KP_HTS_Positive_gap', 'KP Prev_HTS HTS and Positive Gap'),
        ('process_KP_HTS_3b_Positive_gap', 'KP Prev_HTS 3b (TG) Positive Gap'),
        ('process_KP_HTS_3c_Positive_gap', 'KP Prev_HTS 3c (SW) Positive Gap'),
        ('process_KP_HTS_3d_Positive_gap', 'KP Prev_HTS 3d (PWID) Positive Gap'),
        ('process_KP_HTS_3e_Positive_gap', 'KP Prev_HTS 3e (OVC) Positive Gap'),
        ('process_KP_HTS_3f_Positive_gap', 'KP Prev_HTS 3f (Prisons) Positive Gap'),
        ('process_KP_HTS_2_Positive_gap', 'KP Prev_HTS 2 (AGYW) Positive Gap')
    ]

    def on_generate_report_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            # Initialize progress bar
            total_gaps = len(gap_functions)
            progress = widgets.IntProgress(
                value=0,
                min=0,
                max=total_gaps,
                bar_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                style={'description_width': 'initial'},
                layout={'width': '500px'}
            )
            # Initialize progress label
            progress_label = widgets.HTML(
                value=f'Preparing report: Processing gap 0 of {total_gaps}'
            )
            # Display UI header, progress bar, and label
            display(progress)
            display(progress_label)

            # Process each gap function
            global idx, func_name, description
            for idx, (func_name, description) in enumerate(gap_functions, 1):
                clear_output(wait=True)  # Clear previous function output
                # Update progress bar and label
                progress.value = idx
                progress_label.value = f'Processing {description} ({idx}/{total_gaps})'
                # Re-display UI elements
                print(ui_separator_top)
                display(progress)
                display(progress_label)
                # Execute the gap function
                func = globals()[func_name]  # Get function reference
                func(display_output=False)
                time.sleep(2.5)  # Brief pause for visual smoothness

            clear_output(wait=True)
            print(ui_separator_top)
            display(progress)
            display(HTML(f"✔️ <b>Report generation completed! {idx}/{total_gaps}</b>"))
            file_link = create_report_zip()
            if file_link:
                display(HTML('<style>.download-link a { color: green; font-size: 10px; }</style>'))
                display(file_link)
            else:
                display(HTML('<b><font color="red">Failed to create ZIP file.</font></b>'))

            # Display final separator
            print(ui_separator_bottom)

    # -- Step 3: Link buttons to their handlers
    botton0.on_click(on_botton0_click)
    botton1.on_click(on_botton1_click)
    botton2.on_click(on_botton2_click)
    botton3.on_click(on_botton3_click)
    botton4.on_click(on_botton4_click)
    botton5.on_click(on_botton5_click)
    botton6.on_click(on_botton6_click)
    botton7.on_click(on_botton7_click)
    botton8.on_click(on_botton8_click)
    botton9.on_click(on_botton9_click)
    botton10.on_click(on_botton10_click)
    botton11.on_click(on_botton11_click)
    botton12.on_click(on_botton12_click)
    botton13.on_click(on_botton13_click)
    botton14.on_click(on_botton14_click)
    botton15.on_click(on_botton15_click)
    botton16.on_click(on_botton16_click)
    botton17.on_click(on_botton17_click)
    botton18.on_click(on_botton18_click)
    botton19.on_click(on_botton19_click)
    botton20.on_click(on_botton20_click)
    botton21.on_click(on_botton21_click)
    botton22.on_click(on_botton22_click)
    botton23.on_click(on_botton23_click)
    botton24.on_click(on_botton24_click)
    botton25.on_click(on_botton25_click)
    botton26.on_click(on_botton26_click)
    botton27.on_click(on_botton27_click)
    botton28.on_click(on_botton28_click)
    botton29.on_click(on_botton29_click)
    botton30.on_click(on_botton30_click)
    botton31.on_click(on_botton31_click)
    botton32.on_click(on_botton32_click)
    botton33.on_click(on_botton33_click)
    botton34.on_click(on_botton34_click)
    botton35.on_click(on_botton35_click)
    botton36.on_click(on_botton36_click)
    botton37.on_click(on_botton37_click)
    botton38.on_click(on_botton38_click)
    botton39.on_click(on_botton39_click)
    botton40.on_click(on_botton40_click)
    botton41.on_click(on_botton41_click)
    botton42.on_click(on_botton42_click)
    botton43.on_click(on_botton43_click)
    botton44.on_click(on_botton44_click)
    botton45.on_click(on_botton45_click)
    botton46.on_click(on_botton46_click)
    botton47.on_click(on_botton47_click)
    botton48.on_click(on_botton48_click)
    botton49.on_click(on_botton49_click)
    botton50.on_click(on_botton50_click)
    botton51.on_click(on_botton51_click)
    botton52.on_click(on_botton52_click)
    botton53.on_click(on_botton53_click)
    botton54.on_click(on_botton54_click)
    botton55.on_click(on_botton55_click)
    botton56.on_click(on_botton56_click)
    botton57.on_click(on_botton57_click)
    botton58.on_click(on_botton58_click)
    botton59.on_click(on_botton59_click)
    botton60.on_click(on_botton60_click)
    botton61.on_click(on_botton61_click)
    botton62.on_click(on_botton62_click)
    botton63.on_click(on_botton63_click)
    botton64.on_click(on_botton64_click)
    botton65.on_click(on_botton65_click)
    botton66.on_click(on_botton66_click)
    botton67.on_click(on_botton67_click)
    botton68.on_click(on_botton68_click)
    botton69.on_click(on_botton69_click)
    botton70.on_click(on_botton70_click)
    generate_report.on_click(on_generate_report_click)
    clear_button.on_click(on_clear_button_click)

    # -- Step 4: Create group buttons with dropdown arrow, bold text, and larger font
    # -- Define a consistent layout for main group buttons
    group_button_layout = widgets.Layout(
        width='150px',
        border='1.5px solid #999'  # Thin visible border
    )  # Main buttons width: 150px
    group_button_style = {'font_family': 'Calibri', 'font_weight': 'bold', 'font_size': '12px'}

    # -- Define a layout for child buttons (sub-buttons) with a smaller width, font size, border, and reduced height
    child_button_layout = widgets.Layout(
        width='150px',
        height='25px',  # Reduced height for child buttons
        border='1px solid #999'  # Thin visible border
    )
    child_button_style = {'font_family': 'Calibri', 'font_size': '10px', 'button_color': '#ffffff'}  # Font size for child buttons
    
    # Apply the child button layout and style to all child buttons
    for btn in [
        botton0, botton1, botton2, botton3, botton4, botton5, botton6, botton7, botton8, botton9, botton10, 
        botton11, botton12, botton13, botton14, botton15, botton16, botton17, botton18, botton19, botton20, 
        botton21, botton22, botton23, botton24, botton25, botton26, botton27, botton28, botton29, botton30, 
        botton31, botton32, botton33, botton34, botton35, botton36, botton37, botton38, botton39, botton40, 
        botton41, botton42, botton43, botton44, botton45, botton46, botton47, botton48, botton49, botton50, 
        botton51, botton52, botton53, botton54, botton55, botton56, botton57, botton58, botton59, botton60, 
        botton61, botton62, botton63, botton64, botton65, botton66, botton67, botton68, botton69, botton70,
        clear_button, generate_report
    ]:
        btn.layout = child_button_layout
        btn.style = child_button_style

    main_button_description = [
        'Main Actions', 'Report Rate', 'AGYW Report', 'ART Report', 'HTS Report', 
        'HIVST Report', 'ICT Report', 'PMTCT Report', 'NSP Report', 'KP Report'
    ]                                                 # -- Main button description

    # Find the length of the longest description
    max_length = len(max(main_button_description, key=len)) 

    general_button = widgets.Button(
        description=f"{main_button_description[0]:<{max_length}} ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    report_rate_button = widgets.Button(
        description=f"{main_button_description[1]:<{max_length}} ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    agyw_button = widgets.Button(
        description=f"{main_button_description[2]:<{max_length}} ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    art_button = widgets.Button(
        description=f"{main_button_description[3]:<{max_length}} ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    hts_button = widgets.Button(
        description=f"{main_button_description[4]:<{max_length}} ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    hivst_button = widgets.Button(
        description=f"{main_button_description[5]:<{max_length}} ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    ict_button = widgets.Button(
        description=f"{main_button_description[6]:<{max_length}} ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    pmtct_button = widgets.Button(
        description=f"{main_button_description[7]:<{max_length}} ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    nsp_button = widgets.Button(
        description=f"{main_button_description[8]:<{max_length}} ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    kp_button = widgets.Button(
        description=f"{main_button_description[9]:<{max_length}} ▼",
        layout=group_button_layout,
        style=group_button_style
    )

    # -- Step 5: Define sub-button containers with row breaking for more than 5 buttons
    def create_sub_button_rows(buttons, max_per_row=5):
        # Split buttons into chunks of max_per_row
        button_rows = [buttons[i:i + max_per_row] for i in range(0, len(buttons), max_per_row)]
        # Create an HBox for each row with no spacing
        return [widgets.HBox(row, layout=widgets.Layout(margin='0px')) for row in button_rows]

    # Create sub-button containers with row breaking and a general border
    sub_button_container_layout = widgets.Layout(
        border='1px solid #999',  # General border around the group of child buttons
        padding='1px',  # Small padding inside the border for better appearance
    )
    general_sub_buttons = widgets.VBox(create_sub_button_rows([botton0, generate_report, clear_button]), layout=sub_button_container_layout)
    report_rate_sub_buttons = widgets.VBox(create_sub_button_rows([botton1, botton2, clear_button]), layout=sub_button_container_layout)
    agyw_sub_buttons = widgets.VBox(create_sub_button_rows([botton3, botton4, botton5, botton6, clear_button]), layout=sub_button_container_layout)
    art_sub_buttons = widgets.VBox(create_sub_button_rows([botton7, botton8, botton9, botton10, botton11, botton12, clear_button]), layout=sub_button_container_layout)
    hts_sub_buttons = widgets.VBox(create_sub_button_rows([botton13, botton14, botton15, botton16, botton17, clear_button]), layout=sub_button_container_layout)
    hivst_sub_buttons = widgets.VBox(create_sub_button_rows([botton18, botton19, botton20, botton21, botton22, botton23, botton24, clear_button]), layout=sub_button_container_layout)
    ict_sub_buttons = widgets.VBox(create_sub_button_rows([botton25, botton26, botton27, clear_button]), layout=sub_button_container_layout)
    pmtct_sub_buttons = widgets.VBox(create_sub_button_rows([botton28, botton29, botton30, botton31, botton32, botton33, botton34, 
                                                             botton35, botton36, botton37, botton38, clear_button]), layout=sub_button_container_layout)
    nsp_sub_buttons = widgets.VBox(create_sub_button_rows([botton39, botton40, botton41, botton42, botton43, clear_button]), layout=sub_button_container_layout)
    kp_sub_buttons = widgets.VBox(create_sub_button_rows([botton44, botton45, botton46, botton47, botton48, botton49, botton50, botton51, botton52, botton53, 
                                                          botton54, botton55, botton56, botton57, botton58, botton59, botton60, botton61, botton62, botton63,
                                                          botton64, botton65, botton66, botton67, botton68, botton69, botton70, clear_button]), layout=sub_button_container_layout)

    # Define custom CSS for background color for main botton area
    css_sub_botton = """
    <style>
    .custom-vbox {
        background-color: #e8e8e8f6 !important;
    }
    </style>
    """
    display(HTML(css_sub_botton))
    # Apply custom CSS class for background color
    for add in [general_sub_buttons, report_rate_sub_buttons, agyw_sub_buttons, art_sub_buttons,
                 hts_sub_buttons, hivst_sub_buttons, ict_sub_buttons, pmtct_sub_buttons,
                 nsp_sub_buttons, kp_sub_buttons]:
        add.add_class('custom-vbox')

    # -- Track the currently open group
    current_open = [None]
    sub_button_area = widgets.VBox([])

    # -- Step 6: Define group button handlers to toggle sub-buttons
    def update_button_descriptions(closed_button, opened_button):
        for btn in [general_button, report_rate_button, agyw_button, art_button, 
                    hts_button, hivst_button, ict_button, pmtct_button, nsp_button, kp_button]:
            if btn != opened_button and btn.description.endswith("▲"):
                btn.description = btn.description.replace("▲", "▼")
        if closed_button and closed_button.description.endswith("▲"):
            closed_button.description = closed_button.description.replace("▲", "▼")
        if opened_button and not opened_button.description.endswith("▲"):
            opened_button.description = opened_button.description.replace("▼", "▲")

    def on_general_button_click(b):
        if current_open[0] == general_button:
            sub_button_area.children = []
            update_button_descriptions(general_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [general_sub_buttons]
            update_button_descriptions(current_open[0], general_button)
            current_open[0] = general_button

    def on_report_rate_button_click(b):
        if current_open[0] == report_rate_button:
            sub_button_area.children = []
            update_button_descriptions(report_rate_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [report_rate_sub_buttons]
            update_button_descriptions(current_open[0], report_rate_button)
            current_open[0] = report_rate_button

    def on_agyw_button_click(b):
        if current_open[0] == agyw_button:
            sub_button_area.children = []
            update_button_descriptions(agyw_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [agyw_sub_buttons]
            update_button_descriptions(current_open[0], agyw_button)
            current_open[0] = agyw_button

    def on_art_button_click(b):
        if current_open[0] == art_button:
            sub_button_area.children = []
            update_button_descriptions(art_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [art_sub_buttons]
            update_button_descriptions(current_open[0], art_button)
            current_open[0] = art_button
    
    def on_hts_button_click(b):
        if current_open[0] == hts_button:
            sub_button_area.children = []
            update_button_descriptions(hts_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [hts_sub_buttons]
            update_button_descriptions(current_open[0], hts_button)
            current_open[0] = hts_button

    def on_hivst_button_click(b):
        if current_open[0] == hivst_button:
            sub_button_area.children = []
            update_button_descriptions(hivst_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [hivst_sub_buttons]
            update_button_descriptions(current_open[0], hivst_button)
            current_open[0] = hivst_button
    
    def on_ict_button_click(b):
        if current_open[0] == ict_button:
            sub_button_area.children = []
            update_button_descriptions(ict_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [ict_sub_buttons]
            update_button_descriptions(current_open[0], ict_button)
            current_open[0] = ict_button
    
    def on_pmtct_button_click(b):
        if current_open[0] == pmtct_button:
            sub_button_area.children = []
            update_button_descriptions(pmtct_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [pmtct_sub_buttons]
            update_button_descriptions(current_open[0], pmtct_button)
            current_open[0] = pmtct_button
    
    def on_nsp_button_click(b):
        if current_open[0] == nsp_button:
            sub_button_area.children = []
            update_button_descriptions(nsp_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [nsp_sub_buttons]
            update_button_descriptions(current_open[0], nsp_button)
            current_open[0] = nsp_button
    
    def on_kp_button_click(b):
        if current_open[0] == kp_button:
            sub_button_area.children = []
            update_button_descriptions(kp_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [kp_sub_buttons]
            update_button_descriptions(current_open[0], kp_button)
            current_open[0] = kp_button

    # -- Step 7: Link group buttons to their handlers
    general_button.on_click(on_general_button_click)
    report_rate_button.on_click(on_report_rate_button_click)
    agyw_button.on_click(on_agyw_button_click)
    art_button.on_click(on_art_button_click)
    hts_button.on_click(on_hts_button_click)
    hivst_button.on_click(on_hivst_button_click)
    ict_button.on_click(on_ict_button_click)
    pmtct_button.on_click(on_pmtct_button_click)
    nsp_button.on_click(on_nsp_button_click)
    kp_button.on_click(on_kp_button_click)

    # -- Step 8: Create a layout for group buttons with a maximum of 5 per row and no spacing between rows
    all_group_buttons = [
        general_button,
        report_rate_button,
        agyw_button,
        art_button,
        hts_button,
        hivst_button,
        ict_button,
        pmtct_button,
        nsp_button,
        kp_button
    ]
    # Split the buttons into chunks of 5
    max_buttons_per_row = 5
    group_button_rows = [
        all_group_buttons[i:i + max_buttons_per_row]
        for i in range(0, len(all_group_buttons), max_buttons_per_row)
    ]
    # Create HBox for each row with no margin or padding, and wrap them in a VBox with no spacing
    group_buttons_layout = widgets.VBox(
        [widgets.HBox(row, layout=widgets.Layout(align_items="flex-start", margin='0px', padding='0px'))
         for row in group_button_rows],
        layout=widgets.Layout(align_items="flex-start", margin='0px', padding='1px', border='1px solid #999')
    )
    # Define custom CSS for background color for main botton area
    css = """
    <style>
    .custom-vbox {
        background-color: #e8e8e8da !important;
    }
    </style>
    """
    display(HTML(css))
    # Apply custom CSS class for background color
    #group_buttons_layout.add_class('custom-vbox')

    # -- Step 9: Create the main layout
    output = widgets.Output()
    with output:  # Initialize output with ui_separator_clear
        print(ui_separator_clear)

    layout = widgets.VBox([
        group_buttons_layout,
        sub_button_area,
        output
    ], layout=widgets.Layout(
        align_items="flex-start",
        padding="10px"
    ))

    # -- Step 10: Display the interface
    display(layout)
  
# -- Ensure this is the last cell in your notebook
run_jupyter_mode()